# GAANN - Genetic Algorithm and Artificial Neural Network

This notebook tries to explain the details behind the implemented GAANN algorithm with an example.

## The basic idea

The GAANN is the combinaison of a [genetic algorithm](https://en.wikipedia.org/wiki/Genetic_algorithm) and an [artificial neural network](https://en.wikipedia.org/wiki/Multilayer_perceptron). It is a wrapper method since the output of the ANN, the score, is used to improve the classifier. As all the algorithms in this project, the classifier's score itself is not important but the features that give the best score are and the score is therefore used as an indicator.

Let's see how it works.

In [ ]:
def timeit(method):
    """
    Just a simple method to mesure the execution time of a method
    """
    import time

    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()

        print('%r %2.2f sec' % (method.__name__, te - ts))
        return result

    return timed

In [ ]:
from datasets.DatasetEncoder import DatasetEncoder
from datasets.DatasetSplitter import DatasetSplitter
from datasets.Golub99.GolubDataset import GolubDataset
from algorithms.GAANNAlgorithm import GAANAAlgorithm

from matplotlib import pyplot as plt
%matplotlib inline

ds = GolubDataset()

# encode Dataset string classes into numbers
ds_encoder = DatasetEncoder(ds)
ds = ds_encoder.encode()
ds = DatasetSplitter(ds, test_size=0.4)

# with verbose=True, we can see what are the slowest methods.
gaanaa = GAANAAlgorithm(ds, n=1000, verbose=True)

# accessing to a private member is generally not a good idea, but see: https://mail.python.org/pipermail/tutor/2003-October/025932.html
ga = gaanaa._ga

### Crossover method

The crossover method is about combining two individuals, called male and female in the code, to create an other individual, called child in the code.

The child creation consists of theses steps:
1. take the features shared by both parents
2. fill the remaining feature space using random unique features from the parents

As we can see on the plot below, early generations take more time than the latest generations because there is less shared features between the male and the female at the beginning.


In [ ]:
plt.plot(ga._bench_times["_crossover"])
plt.title("Evolution of the execution time for $crossover$")
plt.xlabel("Generations")
plt.ylabel("Seconds")
plt.grid(True)

### Mutation method

### Score evolution

In [ ]:
scores = [fh[1] for fh in ga._fitness_history]
plt.plot(scores)
plt.title("Evolution of the score \nobtained by the best individual at each generation")
plt.ylim(0, 1.1)
plt.xlabel("Generations")
plt.ylabel("Score")
plt.grid(True)

As we can see above, the evolution of the score does not increase as much as expected.

In [ ]:
best_f = max(ga._fitness_history, key=lambda x:x[1])
print("Best score (%.2f) with this list of features by GAANAA %s...." % (best_f[1], best_f[0][:3]))

from itertools import islice, izip_longest

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

def jaccard(a, b):
    return len(a.intersection(b))/float(len(a.union(b)))

history_similarity = list()

for n, m in window(ga._fitness_history):
    features_n = set(n[0])
    features_m = set(m[0])
    history_similarity.append(jaccard(features_n, features_m))
    
plt.plot(history_similarity)
plt.ylim(0, 1.1)
plt.title("Jaccard similarity between the best lists of gen N and N+1")



    

We can see that the curve is increasing, this means that the lists are getting similars. 

In [ ]:
#print([ft[0] for ft in ga._fitness_history])

In [ ]:
#yolo = [[2934, 3918, 5366, 3334, 132, 1768, 1650, 5228, 3294, 2170, 2630, 1608, 4461, 5600, 2931, 2038, 3047, 5778, 3510, 4400, 1820, 3213, 5802, 3108, 5633, 4420, 4511, 2863, 997, 1936, 2416, 4942, 2268, 6796, 3703, 6268, 376, 3220, 6677, 3159, 4077, 3445, 1024, 6474, 105, 924, 5774, 4989, 2283, 7067, 1904, 5413, 4743, 4898, 4011, 4410, 2717, 5580, 6031, 542, 749, 6324, 2574, 1725, 3446, 2520, 3469, 1578, 123, 1108, 2842, 2755, 5836, 6210, 1031, 3450, 5884, 1925, 1532, 6155, 65, 958, 541, 4309, 1664, 3331, 2628, 6572, 2738, 3057, 1050, 3123, 1362, 3145, 6060, 1062, 5163, 3930, 3536, 3973, 3487, 2415, 5244, 4359, 12, 11, 4871, 3196, 2865, 6647, 3936, 469, 1289, 2510, 3855, 3641, 4811, 184, 5226, 307, 3376, 1596, 5270, 6681, 3201, 4447, 3361, 2129, 2061, 6727, 1171, 4672, 3523, 6742, 2412, 3532, 7119, 1354, 6423, 5168, 6413, 3262, 2272, 2086, 5738, 2903, 2401, 2815, 358, 1672, 4768, 7043, 6848, 378, 6865, 1882, 1880, 6728, 5702, 4650, 7112, 5234, 5506, 2145, 164, 457, 6442, 1008, 808, 6467, 3112, 723, 6687, 5459, 3634, 2791, 6585, 4773, 6012, 4025, 2530, 5258, 7068, 2899, 7046, 204, 5934, 5155, 7114, 4987, 2220, 2454, 2920, 5585, 727, 6708, 1573, 3846, 3684, 5911, 6433, 5260, 1127, 1686, 547, 3970, 5097, 5203, 5555, 6509, 6456, 795, 2500, 7084, 1170, 4543, 6561, 928, 4787, 4398, 4991, 390, 2030, 5876, 3083, 3772, 5676, 3425, 6532, 4998, 5351, 5222, 745, 5170, 5587, 5521, 2582, 1016, 7008, 3824, 4572, 6240, 2908, 5894, 6857, 3040, 4353, 3484, 450, 6404, 405, 5164, 3441, 6788, 995, 1537, 1670, 4841, 4403, 2923, 3662, 5634, 2619, 3625, 3895, 2495, 2705, 5792, 4600, 4546, 1689, 5686, 864, 2723, 5080, 516, 4954, 4324, 1563, 3979, 5516, 4664, 4673, 1156, 2567, 5466, 2393, 4982, 3533, 4112, 2821, 570, 4931, 3448, 3311, 5292, 1075, 638, 1007, 4847, 901, 4116, 6040, 475, 290, 7013, 7024, 1544, 2296, 6732, 2300, 6683, 2999, 5358, 1751, 3696, 4257, 6517, 214, 4209, 3806, 3104, 6405, 4834, 6239, 4526, 4507, 3037, 507, 4051, 4819, 2773, 3771, 2483, 6841, 6893, 1002, 5828, 2854, 6396, 2291, 4972, 1270, 6797, 7122, 4409, 1704, 4902, 3713, 2521, 601, 2298, 3246, 1938, 3341, 2346, 3863, 531, 4709, 6992, 5957, 1972, 3559, 5423, 4282, 4589, 5300, 3010, 4452, 1781, 3360, 2343, 6438, 6098, 1329, 5748, 6323, 2893, 1070, 4822, 55, 1627, 2792, 4613, 1365, 4317, 2850, 1293, 3902, 94, 784, 527, 2861, 120, 2900, 2299, 2748, 4121, 5801, 834, 3171, 6291, 2732, 4810, 1838, 5996, 6208, 3562, 2541, 4010, 2586, 2864, 1874, 7060, 2199, 3673, 3874, 1023, 1106, 5976, 6843, 3649, 2730, 4274, 5906, 1690, 375, 6949, 4249, 763, 5401, 2035, 2720, 6737, 3579, 6238, 3599, 6150, 980, 2719, 5768, 6422, 2651, 3977, 1927, 172, 2898, 4203, 4478, 6254, 5094, 6296, 5912, 3861, 4800, 1484, 57, 6043, 4228, 4332, 3056, 3827, 5440, 3873, 690, 1049, 1794, 2884, 6550, 39, 3422, 4083, 4761, 150, 1875, 3724, 2165, 5326, 3042, 3516, 2380, 2869, 3217, 5532, 5194, 1332, 2012, 5667, 6653, 4524, 5327, 508, 3769, 4358, 6311, 4171, 4460, 1092, 3999, 1351, 2846, 5402, 4200, 1791, 2034, 4574, 3028, 6302, 3416, 569, 2045, 6502, 6598, 5870, 104, 4006, 6707, 2164, 5022, 3345, 4055, 409, 4702, 738, 2506, 4111, 3938, 3800, 4519, 2250, 2060, 3050, 2527, 536, 6437, 3848, 5574, 6406, 3513, 1539, 927, 6769, 6557, 6819, 875, 6539, 261, 5389, 4126, 580, 4923, 951, 1377, 6689, 178, 6776, 2392, 6562, 1502, 815, 6074, 5827, 1566, 600, 5507, 6124, 6817, 3423, 2069, 5247, 4306, 2606, 4731, 173, 4561, 4937, 2851, 330, 4020, 5059, 29, 3628, 6920, 5138, 155, 5299, 1233, 5013, 7091, 6100, 899, 6524, 4891, 5716, 2121, 58, 1897, 6137, 3493, 66, 1478, 4013, 3419, 5275, 5663, 3185, 4708, 1150, 5659, 3743, 1275, 2259, 5636, 3206, 3248, 1071, 3912, 1355, 6724, 6403, 3075, 6350, 6274, 6491, 3710, 3357, 1446, 486, 4183, 2464, 5609, 1769, 2014, 6775, 2824, 5497, 5037, 682, 4657, 142, 978, 523, 6564, 4510, 3985, 2818, 3497, 2189, 1463, 5356, 3121, 622, 3437, 2766, 522, 4864, 1615, 804, 2819, 571, 2734, 4265, 1839, 5639, 2847, 2789, 3499, 6472, 1479, 5152, 6501, 2867, 5767, 4046, 2037, 1495, 4530, 3731, 2209, 185, 2116, 2099, 970, 6221, 349, 6823, 119, 2765, 264, 3834, 4141, 3074, 6384, 1982, 5245, 6904, 3292, 641, 2807, 6998, 5766, 4964, 3274, 3557, 530, 2786, 4430, 3725, 1924, 989, 5098, 999, 914, 5341, 3962, 1599, 4271, 5157, 191, 1516, 6995, 2210, 286, 5305, 4247, 1846, 2754, 4903, 5371, 4638, 3941, 1082, 1552, 443, 7012, 3238, 5993, 4944, 2888, 3779, 1204, 6497, 2545, 3208, 3843, 109, 5429, 3792, 2826, 4536, 6334, 5033, 695, 268, 1580, 1207, 6758, 2831, 2466, 1348, 5176, 2595, 2996, 5626, 6574, 6463, 6910, 5806, 6365, 5839, 647, 5719, 6229, 850, 2103, 4910, 1017, 3176, 4167, 2071, 6211, 3150, 1785, 7004, 3517, 1623, 4896, 1201, 4102, 6542, 963, 6528, 6626, 2615, 3790, 2697, 4608, 6521, 3019, 3994, 2794, 4204, 5680, 1941, 1247, 5075, 4950, 4869, 5009, 6605, 639, 5658, 2546, 4745, 2714, 3974, 1988, 2187, 817, 4238, 1163, 2552, 1760, 6326, 2616, 1709, 230, 2991, 3626, 1322, 6308, 2203, 4489, 4550, 4551, 76, 3013, 4634, 3333, 6868, 1776, 5485, 5056, 3907, 5005, 6294, 5696, 3678, 1683, 6076, 1338, 4598, 3362, 1625, 2711, 2252, 3183, 1013, 1545, 4957, 710, 182, 153, 1407, 1065, 882, 243, 3325, 559, 6715, 4229, 6900, 3471, 2374, 3062, 2656, 5751, 3894, 2405, 3754, 3690, 159, 1439, 128, 160, 5760, 7121, 5339, 5343, 5476, 5146, 56, 1012, 2635, 930, 5677, 6036, 1394, 2446, 741, 406, 1954, 5399, 4122, 829, 810, 3989, 423, 4344, 6746, 2032, 496, 2637, 843, 869, 6870, 6267, 6050, 6792, 4585, 6588, 3849, 2444, 6008, 2580, 3374, 4043, 6853, 1797, 3453, 2638, 578, 3958, 670, 3546, 242, 213, 3099, 1837, 5833, 3506, 5281, 1431, 655, 139, 2336, 821, 4418, 5076, 3507, 5318, 1490, 5938, 1583, 5687, 4492, 6684, 3433, 3211, 1180, 674, 5322, 819, 75, 5378, 3087, 4031, 6976, 6224, 1630, 1567, 5250, 4341, 5765, 1823, 444, 6791, 3627, 2104, 6374, 4255, 4882, 1109, 5822, 303, 809, 6425, 5179, 5368, 5190, 6160, 3961, 4502, 969, 6232, 4472, 957, 6046, 1621, 1388], [1028, 3079, 2568, 4617, 5642, 533, 5655, 5144, 2564, 6686, 3620, 3626, 4651, 1582, 2100, 1592, 4106, 3137, 2116, 5190, 1804, 5200, 85, 4184, 6233, 1626, 6748, 3681, 4707, 5820, 4209, 5566, 2682, 2689, 5255, 6280, 3211, 4240, 6291, 2709, 151, 3225, 5279, 673, 3748, 4262, 1704, 5813, 1311, 3772, 4299, 196, 5834, 203, 1740, 3789, 6866, 1747, 5336, 2777, 2266, 4315, 2783, 4320, 2273, 2277, 233, 6894, 2799, 4336, 1784, 1788, 5888, 5381, 3340, 3853, 2838, 3864, 1822, 2847, 4386, 803, 1318, 6962, 6455, 1336, 3392, 1675, 3402, 4431, 338, 3779, 1374, 7007, 3936, 4452, 4966, 5009, 360, 2929, 3780, 3960, 7039, 320, 7042, 3462, 3464, 1930, 5516, 5006, 399, 4496, 7057, 7059, 3478, 3737, 1946, 780, 6563, 7078, 936, 5037, 1974, 4023, 4542, 1479, 3528, 4558, 5072, 5073, 6098, 6103, 4056, 6618, 3551, 6113, 5605, 3922, 6638, 6639, 2545, 3575, 3290, 2902, 6681, 2617, 2778, 5934, 6698, 5754, 6406, 2317, 894, 890, 1843, 3815, 3094, 6147, 1055, 5393, 3523, 3624, 1537, 508, 636, 5283, 6288, 3665, 7112, 2503, 468, 2529, 4130, 3844, 2424, 3067, 3934, 3439, 2585, 6823, 2505, 7009, 1433, 5189, 1581, 6705, 3297, 914, 5251, 4753, 4730, 6746, 4716, 4300, 4812, 5673, 3655, 3377, 5135, 5330, 2901, 5308, 997, 1555, 1611, 2332, 2900, 1139, 96, 4420, 3945, 4167, 3023, 1914, 4876, 4662, 6120, 4473, 3180, 2354, 6994, 5350, 19, 3563, 5992, 4072, 6628, 1257, 6562, 2963, 1732, 5367, 1832, 538, 5803, 1480, 3318, 4218, 5115, 3365, 5848, 1691, 6075, 1321, 5643, 4332, 5268, 396, 1823, 4990, 1310, 2775, 3300, 6068, 4193, 488, 6360, 6855, 5419, 5533, 5800, 3601, 4924, 5607, 4836, 6800, 1349, 3777, 7027, 5493, 1326, 4620, 6602, 6595, 4991, 2818, 2541, 1927, 4046, 498, 2825, 506, 2001, 6796, 2045, 2464, 5871, 330, 3133, 1653, 192, 2802, 1004, 4619, 232, 2641, 2271, 3346, 3529, 152, 2475, 3006, 2809, 366, 3443, 1432, 40, 3444, 4779, 6696, 4429, 2250, 3741, 1387, 5725, 65, 5258, 5629, 3678, 1013, 6645, 2824, 6688, 1273, 2283, 5041, 618, 2083, 3980, 3435, 4402, 3988, 4652, 3932, 270, 4643, 5068, 3332, 4808, 5578, 2578, 3852, 4316, 1793, 4323, 2814, 4398, 4380, 4245, 4647, 2766, 5636, 6412, 4038, 708, 764, 2680, 6545, 699, 520, 3476, 989, 1596, 6014, 179, 4689, 3698, 2787, 516, 4095, 225, 3965, 5169, 6168, 1765, 6305, 4904, 2606, 5023, 5509, 217, 5372, 738, 1426, 5174, 1074, 5368, 4259, 5900, 2189, 4788, 24, 6320, 5772, 368, 5192, 721, 2649, 5252, 6192, 4505, 4135, 2527, 4001, 1174, 2763, 4081, 4830, 6219, 6435, 4826, 4639, 4059, 1962, 3590, 6766, 6788, 3944, 3830, 1764, 73, 2395, 2995, 1875, 5893, 1218, 2589, 6820, 2141, 4533, 417, 1045, 552, 4699, 1758, 1149, 669, 5053, 3904, 529, 612, 3178, 264, 6561, 5315, 771, 2438, 3359, 986, 4964, 2747, 6760, 742, 3485, 20, 2360, 2592, 1412, 6007, 6627, 845, 5130, 1030, 2117, 3610, 5654, 31, 6344, 4075, 2898, 5025, 5843, 3553, 6830, 6105, 4706, 1920, 4197, 5887, 2534, 5547, 3000, 4120, 6860, 2655, 1498, 1757, 2525, 3619, 573, 2817, 4014, 480, 2410, 1506, 157, 991, 2605, 5018, 5765, 6751, 133, 2945, 1604, 5299, 1872, 4144, 1147, 6948, 5757, 2261, 4932, 6556, 78, 3517, 5598, 3033, 4442, 5564, 3821, 4446, 4829, 5389, 6849, 4076, 2405, 5074, 6695, 4810, 2982, 6755, 2869, 3694, 5715, 3677, 1737, 1768, 6574, 906, 4352, 5561, 6678, 5151, 3153, 6977, 6575, 5380, 6888, 2958, 6743, 4472, 2743, 5126, 5707, 1209, 4824, 132, 3502, 3333, 5338, 1952, 3872, 346, 6177, 957, 463, 2459, 5545, 1225, 5959, 4743, 2050, 1038, 3173, 3558, 6379, 1258, 2144, 2160, 1857, 1359, 2430, 2080, 191, 5487, 6296, 5606, 7026, 1702, 2295, 6436, 2771, 6370, 291, 3281, 2170, 60, 5324, 268, 5580, 1567, 2903, 5103, 6781, 1164, 5560, 824, 80, 2081, 1693, 760, 2850, 6619, 3189, 4058, 4054, 648, 1006, 5686, 2923, 3882, 4507, 6013, 2980, 174, 4171, 3467, 2359, 6181, 1769, 415, 4568, 4581, 82, 6452, 5344, 4147, 100, 6316, 6668, 4903, 3666, 6569, 739, 5208, 1723, 5924, 4305, 4399, 4664, 6775, 5312, 135, 6153, 5587, 4840, 5259, 6395, 3684, 2740, 167, 4296, 3246, 7011, 2244, 1955, 430, 1628, 4642, 3705, 1381, 246, 3022, 6740, 2129, 4910, 1668, 3531, 5359, 3430, 3423, 6509, 4426, 493, 3434, 4638, 1271, 3261, 6514, 2013, 5872, 5223, 237, 2894, 4747, 1926, 2770, 736, 3270, 1027, 719, 672, 5123, 2376, 4042, 5695, 1734, 1965, 2482, 6480, 248, 4199, 2718, 4424, 3549, 6121, 325, 311, 4024, 397, 407, 4913, 1554, 2218, 4877, 6006, 5983, 595, 3708, 6782, 5691, 2684, 2242, 5224, 6978, 2924, 1203, 2581, 2611, 4453, 448, 755, 1990, 6044, 5997, 2920, 5798, 859, 2735, 5763, 1523, 4962, 3146, 4878, 6676, 1041, 1777, 1370, 3061, 374, 5889, 1193, 4160, 557, 3824, 6459, 3236, 958, 2640, 2060, 6283, 1169, 3386, 3972, 2229, 6528, 4074, 6352, 1365, 704, 1501, 9, 1154, 1762, 6497, 5612, 1102, 2106, 522, 7093, 6264, 5220, 5309, 3308, 6850, 4986, 5718, 2374, 5250, 6090, 6550, 3751, 1228, 259, 3380, 3284, 551, 1560, 5281, 5451, 6813, 6986, 4953, 2663, 3494, 3313, 923, 5331, 5858, 1527, 1061, 6339, 2225, 2993, 2251, 6208, 2642, 4333, 5842, 4775, 2011, 1275, 4267, 4288, 5188, 5436, 2821, 2382, 1435, 4137, 495, 5932, 2002, 2856, 5672, 355, 627, 6993, 114, 427, 2953, 2331, 547, 6253, 5706, 485, 1202, 7053, 6831, 1313, 6382, 103, 2306, 4511, 5542, 2318, 5166, 3191, 3758, 645, 4969, 120, 6845, 5432, 793, 1277, 471, 2299, 2912, 5168, 188, 1558, 3849, 6871, 4886, 2342, 6139, 6570, 4401, 3670, 670, 2187, 825, 4439, 4216, 2813, 1749, 3307, 2919, 348, 5919, 1785, 4114, 1700, 2334, 5478, 3715, 6853, 104, 3024, 3396, 1778, 3900, 689, 4945, 4532, 5177, 3379, 2163, 3838, 6141, 3686, 4285, 2364, 122, 784, 3855, 5611, 510, 1508, 6465, 5439, 5838, 5909, 3446, 6547, 6900, 3161, 3474, 377, 5114, 869, 6918, 2591, 4523, 5414, 965, 5319, 5680, 4797, 294, 5136, 4345, 5993, 4586, 6841, 679, 6031, 946, 7043, 5351, 3413, 2939, 1237, 4989, 3103, 4164, 6600, 4856, 2168, 2575, 5622, 2079, 6221, 5460, 577, 3389, 1247, 3828, 1119, 664, 5874, 1457, 2380, 6460, 306, 4202, 3206, 3756, 6819, 4290, 2715, 6683, 3649, 4796, 6414, 5844, 5825, 3963, 6466], [2560, 4609, 2565, 6662, 4097, 6165, 5125, 2596, 262, 2104, 4665, 5178, 3650, 6212, 2121, 6220, 5717, 6241, 616, 111, 3184, 4725, 2175, 4737, 1154, 2696, 3209, 1162, 4749, 5775, 148, 3734, 1177, 5278, 1477, 2209, 1186, 5289, 6087, 1198, 1720, 2233, 3264, 1219, 2244, 1234, 5328, 4817, 722, 5334, 6871, 2264, 6180, 2267, 6365, 2782, 5841, 4840, 1259, 4845, 1262, 6383, 3312, 2294, 5367, 3835, 1789, 4863, 1792, 6405, 4358, 5383, 6411, 3852, 759, 273, 5407, 4899, 3878, 2857, 3373, 816, 4403, 1333, 2877, 1869, 6478, 2899, 3931, 6492, 7007, 1376, 5990, 4327, 2420, 7034, 5500, 7045, 5002, 1423, 1940, 2969, 2462, 417, 5550, 1456, 1459, 3514, 4031, 4033, 3010, 6084, 4037, 1478, 7111, 456, 1015, 3029, 1497, 5598, 3042, 484, 6118, 4804, 6634, 5614, 1523, 2039, 1564, 5102, 2488, 1178, 2274, 3589, 609, 5197, 3999, 4938, 6928, 712, 4053, 6354, 2991, 6756, 1489, 4925, 6230, 2982, 940, 6899, 5350, 6719, 6853, 5767, 214, 6651, 5245, 105, 332, 6615, 4209, 2518, 2035, 979, 4167, 2845, 758, 4954, 6559, 476, 4827, 4535, 4028, 2404, 3065, 7025, 3386, 6650, 5054, 6254, 5524, 3403, 2936, 3500, 904, 2814, 5190, 1248, 4112, 2385, 188, 6770, 706, 6885, 4361, 879, 1767, 1729, 1222, 6190, 1804, 2907, 892, 5150, 2065, 1078, 2689, 7017, 1593, 1785, 3784, 80, 1080, 2766, 3752, 6608, 6967, 3242, 3154, 186, 5161, 6322, 3425, 4812, 884, 2334, 2083, 2781, 1358, 1685, 4279, 887, 4635, 6750, 5210, 5945, 5567, 3964, 2995, 4952, 2451, 1927, 1844, 3183, 6392, 3646, 41, 4686, 6936, 3845, 5354, 2189, 3410, 5904, 3738, 6687, 3343, 5843, 1265, 5084, 173, 5593, 6140, 5675, 5462, 987, 1475, 1444, 4519, 341, 6707, 2419, 2544, 5461, 1866, 839, 3480, 6817, 823, 5606, 1288, 3247, 1737, 3831, 4244, 327, 1312, 4729, 3411, 2248, 1239, 4246, 442, 3756, 6530, 4248, 2496, 88, 2032, 843, 3653, 5582, 6902, 2131, 4607, 2748, 4917, 6182, 3594, 6429, 2564, 2187, 1898, 5533, 5001, 252, 731, 260, 6996, 2693, 4490, 5227, 3519, 2421, 501, 750, 286, 1033, 5917, 3524, 883, 2008, 5435, 6949, 3146, 1224, 5876, 2161, 2366, 4983, 3804, 1256, 5745, 3677, 5386, 6961, 936, 4267, 1657, 6914, 1315, 5736, 3748, 1956, 4673, 4380, 6355, 4168, 6229, 7041, 4537, 1902, 3261, 269, 5312, 1253, 2772, 6816, 4294, 337, 117, 3773, 4127, 1350, 2716, 6018, 1464, 4771, 1660, 7021, 2678, 6455, 5837, 1815, 1752, 2409, 6041, 6352, 5184, 166, 4115, 2119, 3006, 5702, 5030, 4256, 1753, 4147, 1589, 592, 6923, 3692, 2414, 2774, 4107, 235, 2195, 2839, 1290, 1559, 2874, 4511, 2754, 3509, 782, 3259, 1769, 3805, 5319, 2919, 1530, 830, 5988, 6528, 6246, 5534, 4533, 594, 6646, 3441, 2471, 1193, 5653, 2695, 4269, 1577, 69, 4330, 3708, 169, 4364, 6187, 441, 1169, 1432, 1861, 6574, 1499, 5966, 479, 6642, 4517, 643, 4620, 4381, 4183, 2115, 251, 25, 1623, 5872, 4133, 119, 3603, 2502, 4961, 3651, 6676, 1140, 3314, 922, 5, 5553, 320, 3367, 2178, 520, 4141, 5077, 6774, 3884, 3965, 4890, 3725, 6752, 335, 3889, 4009, 5864, 5401, 281, 3871, 7067, 1252, 4277, 1377, 3657, 774, 3491, 1069, 1458, 288, 4307, 177, 4592, 5052, 6558, 4866, 660, 6497, 2833, 4543, 6203, 3622, 6958, 2573, 2515, 268, 1985, 2236, 220, 3775, 2687, 1407, 4067, 6785, 5714, 2330, 4007, 3984, 6068, 1787, 4700, 4111, 2557, 3900, 5684, 5091, 5420, 1718, 1344, 4552, 6622, 2521, 7100, 1596, 4032, 1504, 1896, 6569, 3336, 1144, 4034, 903, 5704, 2787, 662, 5536, 2997, 2201, 4875, 1935, 427, 642, 1372, 1838, 5948, 2594, 6467, 2967, 5840, 6096, 82, 6482, 4177, 4059, 190, 4857, 5499, 6679, 1213, 1124, 992, 1397, 4083, 4743, 6302, 2791, 6969, 933, 1206, 3283, 647, 2739, 4106, 1745, 5112, 3017, 5060, 2638, 5801, 901, 3838, 4847, 2749, 4132, 3087, 3636, 4628, 6643, 5584, 3260, 1429, 935, 2387, 3642, 4285, 1661, 1313, 1783, 4767, 5201, 2356, 6701, 2239, 2504, 808, 2160, 4108, 3531, 3624, 129, 1827, 3933, 818, 4180, 5712, 123, 3475, 4867, 1601, 6675, 4987, 6773, 1598, 1679, 179, 5596, 4012, 296, 3097, 1107, 1137, 911, 1413, 3854, 2610, 6382, 2460, 3440, 5727, 3062, 4978, 3354, 5725, 4838, 4450, 6588, 1388, 3073, 2098, 2339, 2167, 3389, 1919, 248, 3039, 5465, 858, 4304, 6464, 388, 5333, 2275, 3094, 967, 1766, 1746, 1676, 3038, 3153, 790, 5134, 5274, 4306, 1104, 5980, 6909, 3827, 4617, 1047, 6447, 5866, 5173, 79, 356, 2668, 1223, 6891, 6908, 5818, 1624, 164, 6465, 5554, 4957, 5720, 2676, 1858, 6367, 6763, 753, 964, 6565, 2122, 1965, 7071, 928, 4818, 4861, 1725, 4529, 6933, 5649, 3861, 3320, 6359, 142, 6490, 5835, 2261, 6418, 3207, 2509, 6480, 431, 5258, 5563, 2371, 3541, 3008, 1411, 5664, 6831, 149, 2112, 1231, 3192, 6685, 411, 5157, 6628, 6183, 4522, 681, 4137, 6766, 6133, 854, 3906, 3150, 4854, 4546, 355, 2843, 3327, 1467, 4664, 6730, 5085, 1409, 2755, 5452, 4056, 4709, 3517, 6619, 3801, 710, 2732, 78, 1275, 1218, 4768, 3808, 1438, 6300, 5899, 428, 4298, 4345, 62, 4292, 2337, 6920, 1247, 5382, 781, 6399, 4998, 1707, 1472, 73, 836, 5205, 1152, 2752, 2293, 1619, 6036, 5645, 5915, 6621, 6543, 2947, 6767, 3122, 2778, 1251, 1337, 1633, 3227, 5994, 3515, 1306, 735, 906, 283, 4489, 3423, 1606, 3050, 1917, 1565, 486, 2394, 2588, 6280, 136, 5339, 5130, 3463, 6782, 780, 4518, 3292, 4930, 6975, 4923, 2731, 5898, 4203, 6139, 5517, 6683, 1241, 949, 3291, 2984, 5746, 2627, 850, 6022, 3054, 3553, 4367, 6197, 1984, 6964, 2771, 3219, 3372, 3455, 6008, 3445, 4043, 4981, 191, 5389, 6453, 6601, 1153, 667, 1079, 1739, 3545, 1643, 807, 1556, 4501, 4321, 4219, 3577, 3361, 4191, 5422, 5331, 4329, 6304, 6282, 1331, 3485, 94, 1295, 2474, 6849, 6509, 5203, 3895, 5936, 570, 1916, 140, 5812, 6506, 2827, 1540, 6155, 2144, 5446, 232, 1700, 5905, 6170, 3330, 3483, 4750, 713, 3654, 4671, 2246, 913, 3371, 2559, 1607, 2336, 5737, 2553, 2875, 5453, 202, 6703, 4394, 5220, 1405, 3131, 6045, 2955, 2285, 3032, 6328, 3158, 7101, 6915, 4390, 3004, 1721, 4993, 4382, 1435, 3905, 6998, 7125, 1570, 3252, 1469, 5529, 3346, 6397, 5642, 6257, 6221, 2672, 1323, 6802, 2666, 4030, 1873, 4410, 5958, 2876, 225, 2683, 6883, 4433, 6714, 4569, 5633, 2304, 2255], [2570, 525, 1040, 5649, 4358, 6183, 5163, 1584, 6712, 6202, 3131, 6666, 62, 4160, 2626, 73, 75, 6732, 3150, 2573, 6235, 4189, 6752, 1633, 188, 4719, 2672, 1384, 5234, 1214, 1152, 3201, 6280, 3220, 1178, 667, 4256, 163, 4775, 3756, 5812, 6330, 2335, 6844, 3774, 6851, 4804, 6854, 5835, 4812, 5328, 1233, 5330, 6359, 4827, 3805, 4840, 4845, 2802, 3316, 5877, 3145, 759, 6909, 3330, 259, 1796, 5382, 264, 3852, 2008, 2840, 5915, 3871, 803, 3878, 4391, 3373, 1160, 2356, 5429, 822, 1337, 2874, 2877, 836, 1350, 3401, 843, 2907, 5690, 3934, 1376, 6499, 7016, 2409, 3439, 6008, 1917, 5508, 2957, 2958, 1423, 5018, 5274, 3486, 2984, 4522, 3501, 5550, 3032, 3509, 2121, 2494, 4032, 1355, 1478, 6087, 3026, 6734, 4056, 1497, 1499, 3039, 3042, 4600, 3577, 6276, 4578, 1134, 6151, 338, 2596, 1949, 262, 4213, 1331, 3194, 1210, 5304, 1960, 652, 3823, 3445, 5841, 283, 3530, 679, 7018, 3649, 4069, 2529, 6553, 4181, 220, 5239, 1628, 6479, 6382, 4726, 2526, 414, 4930, 5800, 2399, 5533, 6118, 658, 1009, 4854, 4524, 6729, 903, 225, 5893, 698, 512, 5203, 589, 5250, 5560, 1873, 6917, 7025, 4327, 2393, 4030, 6587, 5840, 2264, 266, 6558, 4940, 5119, 3900, 567, 1916, 6233, 5719, 1302, 4696, 6041, 5180, 6714, 5083, 2926, 1729, 232, 1223, 4410, 3171, 1192, 601, 3723, 2502, 3367, 6782, 5511, 5241, 2086, 4737, 4413, 4664, 3390, 2485, 3644, 1103, 4525, 5011, 1556, 2876, 1994, 1660, 3905, 2837, 3860, 4144, 6914, 6528, 7048, 2419, 2841, 3769, 1089, 1787, 1459, 427, 6375, 2544, 438, 1777, 3698, 3610, 5624, 3734, 2226, 2301, 3895, 4, 5519, 2562, 1529, 4364, 3652, 4191, 4918, 5545, 4448, 1047, 924, 5631, 5271, 753, 2194, 5048, 7057, 6896, 5553, 6068, 4403, 295, 1726, 6633, 3027, 851, 4435, 6412, 2715, 6115, 3386, 412, 2610, 441, 2198, 4018, 6569, 2871, 6496, 303, 420, 5502, 4068, 6560, 2666, 4610, 4143, 5232, 7024, 3166, 816, 2754, 3175, 6482, 783, 203, 1679, 4115, 2416, 4863, 1498, 4141, 2206, 6627, 6526, 4156, 6632, 5278, 5027, 5089, 6403, 2758, 5080, 5339, 1863, 177, 2451, 6258, 5020, 6200, 1222, 4906, 2865, 2083, 6045, 6902, 5986, 5599, 4211, 1486, 6228, 6112, 2899, 649, 341, 2873, 1522, 1130, 2787, 3597, 1541, 3891, 3074, 335, 4096, 1821, 1106, 1409, 916, 3807, 6400, 6700, 5312, 2936, 2234, 5988, 1421, 5140, 4116, 2372, 6093, 5866, 5488, 1048, 6164, 4926, 4439, 6157, 1557, 5030, 5356, 6134, 6972, 5876, 3104, 258, 818, 5359, 1975, 3106, 1574, 642, 2330, 4519, 6265, 5741, 3808, 5354, 5717, 4649, 6903, 1822, 2669, 1056, 3441, 912, 5205, 487, 5201, 1827, 6852, 5383, 2061, 2993, 1469, 1196, 343, 4026, 4648, 183, 7001, 4710, 7015, 5477, 4909, 6288, 3738, 4635, 94, 6207, 6880, 1776, 5422, 2647, 2261, 58, 2955, 2547, 5421, 2975, 6915, 4169, 1765, 2748, 1871, 2511, 5171, 1037, 1168, 4687, 762, 4764, 4493, 6923, 5370, 3154, 4834, 6165, 4482, 2456, 3720, 5143, 3737, 191, 3319, 2919, 2878, 6066, 5369, 686, 3531, 1263, 5964, 5202, 1456, 5777, 4007, 1700, 5614, 7009, 2204, 7004, 2623, 1462, 4392, 2439, 5600, 3370, 2930, 3008, 2472, 3831, 2351, 1148, 5638, 6885, 630, 470, 3705, 6678, 3455, 6687, 971, 3275, 5500, 5074, 6684, 3038, 6212, 3137, 2113, 6663, 6251, 6149, 4318, 3609, 4553, 4228, 3004, 6806, 6384, 1797, 3281, 1283, 2254, 6232, 1181, 6047, 5001, 2839, 750, 5386, 5748, 4296, 479, 2532, 6750, 169, 5522, 2115, 7003, 4837, 76, 5764, 1154, 1407, 4133, 3647, 4890, 3204, 4423, 5226, 2432, 1504, 3231, 11, 6434, 538, 2011, 1461, 3252, 429, 1590, 3822, 3169, 2469, 742, 5752, 627, 4016, 6817, 5575, 4767, 6404, 1603, 2879, 2070, 2569, 4187, 1133, 2065, 6213, 1502, 5598, 7110, 3031, 2291, 4555, 1745, 2771, 355, 774, 3091, 945, 3044, 1523, 6122, 5647, 263, 5128, 5845, 4609, 2071, 4053, 6580, 4253, 2428, 4582, 3835, 4805, 508, 1333, 6600, 2182, 4151, 217, 6881, 3312, 647, 2324, 4209, 527, 906, 473, 3881, 6440, 3219, 6275, 5683, 5268, 5004, 158, 892, 20, 2027, 1854, 1096, 2521, 4561, 2131, 5645, 2462, 7020, 5768, 3554, 372, 1265, 6368, 3978, 2286, 2470, 4233, 4028, 1102, 1587, 2845, 4042, 5072, 6956, 3889, 625, 4915, 757, 2043, 4124, 5948, 1751, 7111, 5481, 1205, 6709, 3646, 2782, 4145, 4502, 717, 6019, 4529, 1758, 430, 2766, 2146, 1401, 1905, 1540, 7041, 2039, 195, 6975, 888, 3910, 97, 4997, 6256, 2593, 3548, 1668, 5652, 6411, 6890, 2982, 4216, 4137, 4518, 5331, 1789, 2707, 1247, 4867, 3838, 1844, 4279, 3518, 6248, 733, 6295, 385, 2662, 6952, 4697, 3505, 809, 5399, 3725, 3936, 1489, 2175, 5327, 148, 2781, 2245, 6701, 4460, 230, 1643, 5091, 6418, 6891, 5340, 1881, 1720, 308, 2640, 2668, 4945, 4772, 5549, 6353, 5875, 5803, 4119, 3611, 501, 1151, 4175, 1547, 6076, 6702, 1869, 2106, 4125, 110, 3924, 6468, 643, 2460, 6996, 7021, 4627, 4709, 858, 732, 6532, 3583, 1641, 2195, 2209, 1177, 5834, 6536, 4771, 5886, 6577, 5907, 1219, 1740, 5470, 5107, 980, 1662, 548, 3261, 5060, 2783, 4031, 710, 5116, 92, 3488, 6117, 1721, 2018, 922, 3320, 5471, 6474, 5381, 4457, 4986, 1137, 4390, 5994, 3299, 6724, 2114, 1689, 829, 3999, 2321, 2118, 2685, 4109, 5225, 4291, 484, 1626, 1912, 4952, 6303, 6766, 1306, 7036, 5104, 3683, 2407, 4746, 3520, 4058, 4075, 456, 5190, 964, 5757, 6871, 1275, 3079, 1366, 6187, 6650, 5657, 3059, 5120, 1918, 4504, 4619, 3711, 1476, 2860, 337, 1501, 2970, 5248, 1691, 4485, 6077, 2110, 623, 5184, 2415, 3483, 1848, 2262, 4665, 82, 6845, 4625, 3146, 3173, 4008, 6489, 4267, 616, 243, 1846, 5195, 3182, 2294, 3496, 4325, 444, 2434, 5933, 331, 1147, 6775, 3334, 2519, 6150, 3184, 3076, 1474, 6220, 2454, 1166, 2938, 6352, 1991, 2484, 4509, 1808, 2711, 4219, 1259, 481, 624, 2956, 7090, 2267, 4162, 5583, 3560, 6545, 1736, 4324, 1218, 4905, 4218, 111, 3545, 268, 5767, 7013, 3658, 1392, 4965, 3855, 5955, 6478, 7098, 5157, 958, 6294, 3199, 1440, 2152, 3368, 2618, 63, 1576, 5811, 4436, 5358, 5977, 3657, 2824, 6950, 4725, 3622, 1162, 3775, 320, 6770, 149, 424, 1199, 3863, 5737, 3511, 6388, 5621, 5289, 556, 3313, 1509, 6344, 4193, 5095, 970, 614, 5990, 3710, 5212, 6383, 480, 607, 561, 1606, 4902, 2252, 5142, 6302], [5126, 9, 5643, 4620, 1552, 5650, 1043, 4120, 3097, 6683, 540, 3616, 3172, 2609, 4146, 6198, 3133, 3936, 3650, 11, 5190, 72, 5707, 6739, 5719, 1113, 611, 5732, 2356, 4711, 2160, 6770, 6675, 119, 6270, 3199, 2180, 1157, 6792, 1388, 4239, 5780, 2711, 6815, 4260, 6309, 5286, 3242, 1195, 626, 177, 697, 3770, 6844, 1214, 191, 1736, 3273, 5328, 1233, 6355, 4308, 2262, 228, 4838, 3303, 4840, 5948, 6891, 4849, 761, 1151, 1788, 1322, 2815, 1792, 5430, 4362, 3852, 3855, 1306, 2845, 1829, 2346, 2525, 2868, 2358, 3895, 3897, 827, 3900, 4413, 4928, 5442, 324, 1846, 823, 2383, 6480, 1365, 4439, 1880, 6496, 7010, 5475, 742, 360, 4924, 7018, 2791, 4972, 5488, 404, 5952, 66, 1422, 6036, 6045, 4000, 1955, 422, 6572, 1611, 5062, 1479, 3528, 6090, 6093, 2004, 3541, 2008, 1497, 4572, 5085, 3551, 2529, 4069, 6118, 5608, 6125, 2032, 5629, 6143, 648, 4940, 4841, 6524, 797, 6671, 7022, 4917, 3399, 1457, 5569, 5777, 3883, 3288, 6667, 694, 1007, 2920, 1194, 453, 3168, 1564, 2333, 4660, 1750, 664, 4172, 2390, 5148, 3370, 3330, 290, 6857, 6766, 5110, 6362, 1321, 1279, 843, 6752, 819, 1695, 6664, 581, 1686, 7016, 310, 6721, 5083, 2102, 493, 332, 297, 303, 3150, 1874, 2389, 1746, 1481, 640, 3251, 1811, 2794, 1563, 1328, 5018, 1228, 68, 871, 1149, 6028, 2637, 806, 1318, 2168, 1763, 4915, 3692, 4636, 4340, 167, 6658, 2881, 2374, 2274, 3589, 507, 5087, 3258, 2518, 5986, 6296, 3171, 5914, 3837, 6908, 6734, 5750, 4818, 4244, 1173, 6446, 2670, 3829, 5757, 1834, 2937, 6492, 6041, 4406, 3429, 3831, 4826, 6015, 288, 5980, 6645, 2476, 4457, 3696, 4517, 3204, 5876, 3459, 179, 364, 1129, 299, 4505, 5407, 1014, 794, 426, 2449, 6228, 796, 4859, 2746, 6410, 6180, 6800, 6133, 5971, 6801, 786, 4433, 3917, 4753, 4768, 6432, 2904, 4398, 2470, 3114, 4026, 1027, 995, 1719, 2106, 5752, 2628, 6349, 5603, 5983, 5456, 1127, 2938, 1546, 3726, 1238, 1213, 341, 3901, 632, 906, 4493, 3773, 5819, 6623, 5573, 628, 3151, 3054, 5367, 4206, 1456, 1038, 4590, 3752, 4130, 3821, 6294, 38, 2565, 3955, 782, 6554, 1273, 658, 7058, 6806, 4785, 5928, 6302, 5170, 1718, 669, 2018, 6016, 5678, 6096, 5596, 3142, 6421, 987, 1424, 428, 1124, 1797, 1654, 7089, 460, 6852, 3620, 4198, 2323, 2296, 1148, 5173, 1902, 3529, 3531, 2480, 5785, 1095, 896, 3407, 6618, 1998, 1115, 215, 5431, 6887, 6883, 6091, 4215, 2161, 3340, 4125, 3019, 3515, 2495, 3446, 5974, 2318, 3972, 5898, 3096, 1821, 1981, 4664, 867, 1575, 2112, 4187, 4276, 5246, 6175, 287, 1031, 5606, 3796, 5614, 5193, 5169, 5072, 734, 2067, 5033, 2960, 1825, 6712, 769, 5311, 3032, 3283, 2277, 5166, 7007, 245, 85, 6529, 6700, 1275, 6208, 1387, 2890, 2007, 2097, 1033, 6293, 5891, 4020, 5428, 2907, 5787, 5222, 4869, 6451, 5679, 2003, 300, 5355, 4015, 6267, 5210, 1532, 6894, 1865, 3010, 6594, 2059, 1423, 2542, 3100, 2605, 4550, 1021, 5025, 4129, 6061, 2017, 4219, 6264, 5586, 5215, 4707, 566, 2603, 3231, 1435, 6965, 6259, 3556, 1287, 5528, 7047, 1137, 6746, 535, 3083, 631, 3695, 6240, 2733, 6775, 5053, 4563, 4633, 5902, 4723, 5070, 42, 6005, 2597, 6303, 4487, 3341, 903, 4135, 5620, 2802, 6477, 4351, 5401, 6124, 847, 5450, 4012, 5975, 5372, 4167, 5404, 6703, 3622, 624, 2672, 5364, 7051, 4307, 5610, 5189, 6840, 4649, 4740, 5550, 5343, 2413, 4291, 3647, 7008, 1005, 1494, 5035, 2212, 94, 1272, 6164, 4147, 7081, 704, 3500, 5396, 6642, 205, 5562, 6343, 7041, 6359, 6280, 2117, 2404, 2852, 4418, 1432, 3609, 6804, 5760, 6081, 2772, 1162, 3544, 4666, 4722, 2589, 1776, 6945, 2814, 6078, 3447, 5851, 6029, 4391, 5211, 3655, 5831, 2961, 495, 7026, 6381, 4816, 4119, 6064, 3073, 3300, 4989, 3464, 1915, 1160, 2600, 2898, 2902, 2326, 1921, 2532, 1577, 5265, 365, 3365, 2504, 4873, 4302, 7036, 5325, 673, 362, 5547, 1411, 5612, 4894, 3789, 2724, 873, 6676, 5940, 1413, 4081, 6518, 4156, 5489, 217, 5632, 6928, 28, 3043, 4469, 206, 2085, 7098, 4344, 3441, 996, 2022, 3136, 4844, 3603, 4324, 1332, 6483, 1649, 810, 166, 5878, 3402, 6922, 424, 1598, 5141, 2062, 4317, 958, 5466, 3927, 6071, 4296, 752, 2697, 1783, 5225, 6900, 2490, 2195, 3820, 1506, 3665, 6724, 87, 3313, 869, 5354, 6696, 762, 1711, 1883, 775, 6588, 6988, 6214, 589, 417, 5702, 3322, 1734, 4271, 4799, 2626, 1700, 1911, 4612, 3863, 524, 1462, 3001, 6952, 146, 5731, 820, 1116, 862, 6749, 5691, 1099, 5736, 4904, 4524, 3312, 3145, 378, 3493, 5094, 4025, 480, 1, 2647, 3930, 2041, 5946, 5010, 5041, 6051, 5593, 248, 4342, 5919, 4671, 5616, 4281, 6462, 415, 5687, 5930, 2762, 4261, 6211, 2375, 4751, 4052, 6312, 7100, 4669, 1164, 1490, 5536, 2999, 4426, 5147, 2301, 3987, 4371, 4110, 4097, 4551, 6286, 372, 5439, 2530, 2510, 3393, 4414, 2963, 2131, 6730, 2444, 3376, 3338, 3683, 1576, 5243, 501, 2929, 3127, 511, 1072, 2214, 4436, 6507, 285, 4376, 5381, 902, 1343, 755, 7112, 3980, 4839, 2681, 1585, 2133, 452, 90, 3522, 3388, 2873, 2206, 2305, 3708, 140, 3113, 6833, 1668, 5942, 3487, 1069, 2984, 6052, 7126, 1612, 5520, 5769, 2606, 3818, 3717, 1962, 6608, 5497, 2909, 920, 2766, 4033, 5391, 1377, 301, 4074, 1745, 561, 223, 2293, 5613, 156, 5945, 4933, 584, 4150, 225, 6600, 2740, 3003, 860, 845, 4038, 5308, 1516, 4386, 5004, 3457, 2197, 2430, 5255, 1265, 2427, 4213, 254, 3730, 2570, 454, 3781, 3304, 990, 4056, 2679, 659, 4706, 1550, 2276, 1198, 438, 4574, 3828, 1669, 3044, 1239, 3011, 2539, 5425, 1504, 3486, 5376, 5904, 6488, 5197, 3963, 3505, 4695, 3642, 6684, 3532, 4872, 1918, 3724, 5781, 6513, 1420, 7052, 3202, 5872, 6352, 6174, 2788, 5358, 5648, 73, 2993, 3684, 1395, 5534, 4176, 2775, 3725, 2913, 268, 3106, 2455, 2593, 2664, 3640, 3566, 3675, 3929, 4984, 7095, 4866, 1266, 3861, 3754, 132, 2728, 915, 1808, 2244, 4087, 4299, 1817, 5028, 4488, 4348, 6580, 2866, 1393, 6672, 5909, 5107, 6747, 1626, 2083, 4754, 1762, 3057, 4258, 246, 2203, 186, 4953, 2979, 5336, 2677, 2194, 4310, 6614, 1633, 6936, 2342, 1289, 4489, 6257, 5855, 6378, 4885, 4374, 1881, 2612, 2023, 639, 5231, 21, 4107, 4617, 5129, 4842, 1470, 1662, 909, 3594, 4111, 283, 831, 3584, 4032], [5632, 1026, 2564, 4103, 3080, 2569, 11, 526, 2063, 856, 1043, 1557, 3094, 2583, 6681, 1563, 4125, 2590, 2591, 6688, 4016, 547, 4133, 3622, 4648, 2603, 557, 1629, 1584, 5169, 1587, 6071, 3126, 3127, 1592, 6202, 5691, 6206, 6837, 3136, 2625, 66, 580, 950, 5135, 4279, 6732, 5198, 2639, 3665, 2131, 2133, 4299, 5723, 6749, 6751, 608, 2145, 5203, 3684, 2663, 1640, 6249, 3694, 4207, 2160, 3176, 1139, 2494, 6266, 2684, 2173, 1151, 4224, 4739, 1668, 5255, 364, 5259, 5775, 6288, 823, 6291, 1686, 2711, 4760, 3225, 5274, 1391, 4662, 670, 624, 5283, 1701, 4262, 4775, 3756, 5018, 200, 6323, 6686, 2230, 183, 5816, 4796, 4799, 2763, 3275, 1224, 1740, 2251, 4300, 290, 4968, 4307, 3022, 6874, 4316, 3194, 223, 6368, 2273, 2786, 2788, 1765, 5350, 3818, 5355, 238, 2600, 3316, 245, 2294, 3831, 3320, 3322, 4859, 764, 3837, 2815, 2817, 3330, 3883, 260, 6918, 2532, 2318, 3885, 6929, 4882, 2324, 1302, 280, 3865, 283, 6940, 1242, 2352, 4386, 3876, 3877, 5419, 301, 27, 2865, 6450, 5428, 478, 3977, 1848, 5433, 1851, 6972, 4413, 1343, 5440, 322, 6283, 1078, 4426, 4919, 2894, 1051, 4436, 6030, 2902, 1368, 1881, 5978, 4239, 3935, 5477, 2406, 360, 4972, 1902, 367, 4172, 6513, 7059, 462, 7036, 1258, 1918, 2945, 2027, 5508, 6533, 1927, 3464, 4489, 1422, 3983, 4496, 2962, 2963, 2453, 5785, 3140, 922, 2543, 1949, 1872, 2466, 1955, 2984, 1946, 2475, 6574, 2973, 6064, 6066, 3507, 2718, 438, 1259, 3001, 4026, 1979, 5053, 958, 6664, 3017, 7114, 3531, 2760, 4046, 595, 4563, 3541, 3032, 473, 4687, 3151, 2014, 2880, 3041, 483, 6628, 4069, 6123, 6124, 493, 6639, 1009, 6739, 2900, 1533, 3582, 4095, 6188, 6738, 5933, 3762, 3826, 6091, 4333, 6233, 6294, 3527, 6682, 6139, 4354, 3276, 2461, 5878, 2136, 773, 3853, 6770, 84, 6677, 3063, 7052, 6229, 4588, 1832, 6872, 1023, 5365, 3921, 5436, 2510, 2322, 6620, 5140, 771, 803, 4812, 6198, 3638, 6309, 2628, 2380, 7020, 2607, 3020, 4007, 6990, 6113, 6762, 5080, 1509, 2840, 5328, 6445, 2495, 5986, 3145, 790, 6703, 660, 3948, 5431, 5885, 5476, 3626, 5763, 3376, 1099, 4738, 5643, 7101, 4072, 4156, 3863, 807, 5418, 888, 5787, 2646, 3953, 6663, 6305, 1028, 3030, 5160, 3901, 2799, 4332, 5758, 6800, 4157, 4429, 2619, 157, 4130, 1787, 6660, 2503, 62, 6656, 5866, 25, 94, 708, 971, 5714, 24, 3167, 426, 1446, 5527, 1722, 3896, 2845, 3581, 4541, 108, 1578, 4443, 881, 337, 536, 3952, 3416, 6915, 6389, 2335, 5839, 2416, 3494, 5070, 3362, 6700, 4921, 4730, 1880, 6495, 1736, 569, 1237, 2868, 2937, 5154, 3515, 259, 2045, 3239, 2203, 5189, 81, 3350, 5443, 274, 5120, 1598, 5010, 3379, 880, 1788, 5256, 3796, 4976, 3613, 736, 2051, 3447, 3396, 3692, 5968, 3732, 4720, 2468, 6684, 6868, 3765, 4490, 299, 775, 2505, 5667, 2727, 6320, 849, 2042, 5974, 6657, 5032, 6218, 2805, 6360, 6956, 7108, 675, 6270, 2439, 1233, 583, 4856, 3530, 6154, 6931, 2434, 6705, 3702, 2609, 308, 3751, 5107, 5501, 3261, 1652, 1154, 5504, 4741, 2573, 6539, 237, 4044, 6543, 87, 1039, 3781, 4991, 649, 1849, 317, 3278, 4092, 3106, 2856, 1597, 6545, 424, 5928, 2776, 6801, 1210, 4182, 1838, 306, 1168, 929, 6683, 4234, 292, 5290, 6273, 1527, 1159, 3543, 2296, 3758, 2770, 3133, 5950, 2750, 1911, 78, 1193, 6671, 203, 3097, 6340, 1749, 6666, 5781, 7009, 124, 2106, 2964, 4120, 6757, 3142, 6100, 4826, 3273, 6777, 2556, 3763, 3236, 3501, 5209, 734, 6616, 614, 876, 6763, 1477, 20, 4313, 6630, 1552, 1176, 854, 4420, 970, 3363, 585, 398, 1143, 3650, 7123, 4958, 5725, 177, 7047, 4827, 6045, 5771, 346, 3743, 6898, 2653, 4666, 6452, 4946, 964, 69, 5416, 7098, 4240, 2473, 320, 5183, 913, 403, 281, 3897, 2547, 4987, 6103, 4568, 2211, 645, 5837, 5126, 4318, 3288, 522, 1532, 2365, 5772, 1405, 6578, 6111, 502, 212, 6547, 3852, 6820, 5788, 3713, 1697, 2077, 5147, 3434, 5538, 915, 5640, 3552, 2103, 1376, 5687, 1065, 6434, 4895, 3726, 5100, 2771, 4908, 2641, 5935, 2767, 6335, 4197, 689, 6906, 6559, 3252, 1538, 3649, 1488, 4309, 5364, 6807, 3459, 5081, 5942, 5893, 2637, 39, 5819, 3966, 4392, 1792, 4379, 5304, 227, 7024, 6003, 2697, 1893, 4691, 3174, 3556, 859, 3619, 3153, 3526, 452, 5731, 6439, 3855, 3735, 241, 220, 2023, 2511, 4584, 6690, 1359, 4259, 2651, 1463, 35, 5282, 2195, 2150, 1369, 4340, 6520, 323, 7042, 2465, 2498, 2276, 6463, 6280, 6917, 191, 1088, 492, 5162, 3864, 6729, 1215, 2193, 5487, 6473, 3654, 3859, 2480, 6106, 309, 5606, 3092, 210, 5648, 3035, 2226, 6581, 1013, 1440, 6051, 5042, 5468, 4507, 935, 4629, 996, 6347, 2568, 909, 6397, 4028, 1102, 2304, 4381, 7021, 1917, 2998, 6443, 3269, 2146, 2137, 6883, 5546, 3620, 2873, 648, 5130, 2995, 5146, 5472, 4772, 1160, 1401, 7048, 698, 4406, 3211, 1116, 5800, 1306, 345, 1460, 4164, 6208, 6286, 1680, 6962, 218, 5562, 3674, 7045, 858, 2844, 4789, 4198, 3917, 5344, 6314, 2642, 6949, 1653, 6643, 658, 3632, 3168, 1162, 6412, 1571, 808, 2688, 5661, 1915, 5492, 3231, 2092, 3717, 6357, 1253, 3115, 6446, 6838, 973, 2035, 732, 2044, 6975, 2787, 4020, 7044, 233, 4424, 1755, 721, 4219, 6947, 4932, 4904, 2514, 5706, 2167, 3575, 6752, 2072, 2112, 5299, 5096, 1096, 1171, 343, 4538, 2356, 2579, 2838, 6720, 4291, 5769, 5458, 2570, 4032, 6852, 6745, 2139, 6057, 4119, 4642, 6723, 626, 2476, 6870, 2431, 6194, 368, 1330, 4493, 2762, 6024, 3873, 5556, 527, 6712, 2020, 1147, 1995, 843, 5358, 6505, 1000, 4713, 2120, 366, 5578, 710, 3025, 6740, 529, 2225, 6735, 2541, 6977, 2202, 1530, 1387, 4096, 1125, 2341, 1953, 1590, 3647, 5746, 287, 1464, 7115, 119, 6465, 2189, 3338, 510, 1758, 4264, 222, 3000, 1913, 4764, 646, 2529, 4256, 2292, 4087, 2677, 893, 251, 2753, 5773, 6811, 1288, 2405, 1601, 4025, 5732, 611, 4249, 597, 4187, 5698, 1583, 5417, 4223, 6480, 5381, 6488, 40, 4338, 5088, 2309, 1052, 2930, 4852, 5057, 3645, 1395, 3283, 3161, 1812, 5340, 2375, 2317, 3429, 4918, 2313, 1540, 4924, 1316, 1030, 3370, 6986, 4600, 995, 3196, 667, 1965, 6853, 1180, 5659, 4334, 6816, 3099, 4644, 6846, 3091, 4652, 3096, 3426, 1318, 1585, 1384, 507, 6430, 622, 1334, 2617, 6572, 3332, 2360, 4791], [3076, 1033, 2571, 2578, 1044, 5124, 5147, 1052, 5466, 552, 3628, 3337, 1080, 5177, 3423, 2110, 1383, 4689, 2642, 6740, 5717, 105, 1629, 5731, 4197, 5223, 5736, 1641, 7015, 873, 3698, 5748, 4726, 5239, 6780, 1672, 5772, 6800, 657, 2707, 1174, 196, 4763, 4771, 4773, 5288, 1193, 1721, 4797, 3779, 708, 6853, 4299, 208, 4201, 4825, 6588, 2781, 1252, 5866, 6379, 2088, 6902, 1787, 5374, 5377, 5382, 264, 4652, 778, 2837, 1814, 2841, 3867, 295, 2344, 2347, 3372, 4912, 2039, 6969, 1872, 2898, 855, 7001, 2394, 2399, 4960, 3429, 4455, 2408, 2409, 4977, 2939, 7038, 6529, 1771, 7048, 396, 2445, 6030, 3565, 6033, 4511, 2980, 6569, 6061, 4537, 956, 5053, 5566, 1984, 1482, 503, 7122, 475, 5599, 6395, 5714, 6128, 2473, 5116, 2045, 31, 5364, 3710, 5505, 1592, 6567, 6592, 6763, 5248, 2385, 4343, 5672, 439, 811, 6556, 2882, 5504, 4534, 152, 1682, 1168, 6777, 871, 5353, 2872, 1490, 1009, 3934, 5926, 673, 6054, 4158, 938, 4639, 6743, 5869, 3595, 1871, 7109, 3562, 219, 2895, 2182, 139, 85, 4701, 2625, 5813, 534, 2287, 5545, 1823, 1028, 6834, 5825, 5155, 2403, 2723, 4439, 3852, 5044, 603, 210, 3222, 786, 6764, 3676, 5909, 7078, 916, 1201, 492, 162, 4870, 2081, 6668, 4468, 6681, 5307, 3887, 2811, 5003, 2117, 2075, 5125, 4345, 2603, 5938, 327, 5119, 4404, 4496, 1601, 1215, 2329, 361, 1414, 1268, 6760, 6374, 5675, 3612, 6072, 1244, 6428, 5919, 4421, 5367, 1424, 1436, 4211, 5073, 5735, 5236, 6120, 3658, 3368, 5788, 58, 582, 1151, 1669, 498, 5078, 3697, 3651, 4622, 5560, 2532, 1775, 2178, 6390, 1420, 6734, 2365, 5940, 5145, 2021, 1583, 2969, 1637, 291, 4488, 4361, 3831, 4126, 1594, 1898, 3447, 1873, 3906, 2779, 1965, 5132, 2008, 3079, 2330, 4806, 29, 4581, 224, 3734, 556, 619, 1726, 4599, 1235, 277, 3649, 1113, 6372, 3033, 5260, 2481, 4725, 1704, 5569, 2468, 2850, 2695, 3370, 578, 7071, 1471, 5, 5130, 2269, 3173, 4399, 2824, 1065, 2716, 793, 2202, 5214, 5824, 7003, 3738, 1635, 6547, 1658, 906, 6910, 2756, 355, 3425, 1961, 287, 4857, 4866, 447, 1623, 1632, 2346, 774, 1206, 2801, 5804, 2460, 6643, 3751, 3654, 1035, 3401, 312, 6411, 6267, 7058, 6847, 3000, 1431, 3504, 2313, 2107, 3226, 357, 926, 5038, 5095, 4481, 555, 6874, 4288, 3342, 7036, 4172, 5013, 5107, 2290, 5226, 2579, 4625, 1537, 3960, 3515, 4656, 4354, 6386, 6273, 2294, 6402, 96, 5715, 7008, 3189, 5148, 5368, 3219, 2926, 6235, 5066, 1096, 5216, 6266, 2059, 2979, 2651, 157, 330, 4644, 1210, 110, 2315, 128, 3176, 186, 635, 3179, 3374, 1867, 3373, 993, 792, 4102, 5524, 4425, 6470, 1337, 653, 2679, 2916, 3002, 5025, 3063, 1701, 2350, 75, 2934, 1796, 1705, 5010, 3527, 787, 6045, 2320, 1213, 3735, 5669, 3773, 6295, 966, 3804, 7090, 6583, 3534, 4132, 3841, 5273, 3688, 4553, 4303, 6806, 4833, 1269, 3073, 2740, 4095, 3239, 1545, 338, 2064, 3885, 6192, 2173, 5493, 6098, 3118, 6322, 5151, 6291, 5154, 1587, 1589, 4210, 5881, 3920, 2693, 1274, 2189, 5117, 4013, 6575, 2655, 4856, 3774, 821, 6202, 5549, 4412, 680, 3382, 6978, 1155, 5820, 6816, 4749, 2991, 5456, 1234, 5029, 3929, 1106, 1947, 2343, 3502, 6713, 3700, 3905, 4965, 3900, 3242, 1709, 2213, 2277, 3585, 5905, 5126, 1387, 3663, 2498, 4225, 1400, 911, 880, 6974, 3091, 6297, 538, 6096, 749, 6495, 315, 5858, 5269, 112, 2698, 64, 5826, 2591, 3620, 964, 566, 6211, 3019, 1768, 6842, 5345, 3294, 6238, 5017, 2167, 5864, 5338, 875, 2892, 2037, 485, 3517, 754, 6391, 1331, 3501, 3261, 2666, 2704, 5839, 4852, 6077, 1487, 2094, 6839, 5222, 4254, 6449, 214, 301, 1550, 4072, 713, 2904, 1225, 2086, 6887, 5241, 146, 7034, 1777, 860, 5614, 3224, 253, 6840, 136, 1784, 3001, 4304, 5161, 6180, 5189, 4708, 6019, 1912, 5000, 403, 3051, 1192, 5978, 4364, 3520, 6057, 2415, 1259, 6311, 525, 4231, 3776, 3295, 3202, 5775, 1812, 4504, 4458, 3788, 5928, 5681, 3320, 171, 7007, 5693, 3845, 6265, 2896, 7104, 1815, 6712, 6109, 837, 1467, 3477, 2876, 876, 6589, 4143, 5910, 2616, 1407, 5630, 594, 6699, 762, 6280, 2019, 5508, 6783, 4477, 3786, 4082, 3288, 3282, 954, 4918, 3758, 5695, 3204, 2711, 3466, 3884, 425, 164, 2410, 6486, 6549, 2727, 5946, 6327, 9, 2874, 4530, 6767, 4535, 2131, 1552, 6343, 5799, 2083, 4508, 6229, 463, 910, 5958, 5346, 2592, 6796, 2816, 4958, 3837, 4795, 6772, 7011, 5746, 504, 310, 5270, 241, 5068, 5533, 17, 3436, 2566, 1216, 3670, 2580, 3396, 3741, 167, 4285, 3362, 4463, 1729, 205, 6103, 318, 5205, 514, 6481, 4808, 3992, 501, 380, 2118, 1608, 1008, 6046, 5952, 331, 6818, 4115, 1130, 5347, 6187, 6651, 5372, 4125, 6619, 2565, 1576, 4730, 247, 2427, 6545, 7051, 4642, 6736, 3240, 2299, 4274, 4878, 2966, 345, 6316, 4906, 6268, 687, 4380, 2738, 529, 1900, 5667, 5880, 5540, 2119, 5264, 4037, 1198, 2941, 3322, 6935, 6641, 6230, 6687, 750, 2098, 2878, 4196, 1054, 3496, 3638, 4383, 4004, 6653, 6708, 2103, 4529, 353, 5655, 2825, 2650, 3488, 263, 3164, 5097, 348, 4915, 3489, 7116, 4381, 1233, 1792, 3555, 4985, 4921, 3353, 4654, 91, 4817, 4286, 3846, 5903, 6793, 2935, 3702, 3334, 2588, 2261, 5837, 3276, 3536, 3899, 6396, 4154, 6086, 394, 5580, 659, 2552, 1970, 4147, 6813, 2511, 5232, 445, 2206, 4995, 1529, 3480, 2154, 917, 817, 4992, 3133, 33, 2504, 3994, 6801, 2028, 1913, 3539, 6739, 7077, 5776, 4945, 3032, 4628, 7115, 6848, 90, 1423, 3931, 5917, 1868, 1451, 2046, 2894, 4280, 1375, 1808, 1356, 4322, 3508, 3264, 4824, 5511, 6819, 527, 74, 5798, 4151, 1940, 4077, 1058, 693, 3675, 4853, 5297, 4203, 5462, 4796, 4309, 5142, 6474, 2254, 526, 892, 2596, 3685, 4409, 2076, 1129, 2335, 3350, 4942, 4339, 2230, 1088, 2491, 4273, 1740, 4953, 6861, 5434, 1907, 5591, 1624, 6820, 1228, 3827, 1802, 6894, 418, 2222, 6770, 6014, 1376, 4745, 1015, 1496, 6698, 2537, 314, 4769, 857, 5339, 6683, 6312, 2383, 521, 4904, 6553, 2340, 6983, 1143, 6290, 6989, 4145, 4711, 5831, 4042, 4951, 790, 7025, 5043, 4133, 4259, 2998, 936, 536, 62, 2334, 1350, 6071, 7111, 6185, 207, 5606, 745, 6644, 3720, 6787, 1679, 522, 5947, 5616, 5696, 236, 4461, 3703, 1365, 1720, 1718, 1149, 3259, 1263, 1053, 3947, 1930, 2611, 6461], [7, 2060, 4114, 20, 24, 27, 2076, 31, 5126, 4775, 2106, 4157, 62, 2116, 4162, 4164, 69, 6218, 4171, 4176, 6227, 85, 2136, 2137, 6236, 2141, 6241, 2146, 4197, 2150, 103, 104, 6253, 6266, 2459, 126, 4225, 133, 6280, 4234, 2187, 2189, 143, 6288, 2072, 6291, 151, 153, 157, 6305, 4259, 4262, 6314, 6320, 2225, 4277, 183, 186, 6334, 196, 6343, 4812, 4299, 4300, 1741, 4305, 210, 212, 2261, 2263, 220, 223, 2273, 6370, 227, 233, 6379, 4332, 4333, 238, 5259, 2294, 4346, 6395, 6396, 6397, 4354, 2313, 2318, 4377, 4826, 2334, 4386, 5851, 6436, 295, 2347, 306, 316, 2365, 320, 6465, 322, 323, 4420, 4426, 4429, 338, 4439, 346, 2395, 348, 4445, 6495, 4452, 360, 6505, 4460, 6510, 368, 2430, 6208, 396, 2445, 6543, 4496, 6545, 6547, 6550, 4505, 4507, 6556, 2461, 4511, 6563, 2475, 6574, 6578, 4532, 4538, 4541, 2498, 454, 2503, 2505, 4554, 461, 462, 463, 4563, 472, 2527, 2532, 6630, 2541, 495, 2545, 6643, 4596, 510, 6656, 513, 6660, 6664, 522, 2571, 4622, 529, 2579, 3161, 4633, 538, 6686, 2591, 4642, 547, 549, 552, 4651, 4652, 557, 6705, 4662, 4663, 573, 6720, 2625, 6723, 2628, 4193, 585, 4043, 2639, 4688, 2642, 6740, 6745, 6749, 6751, 4707, 2663, 2667, 4730, 2684, 2688, 2689, 2691, 4741, 6791, 4749, 6800, 2709, 2718, 6816, 675, 6820, 2727, 3527, 689, 2740, 4789, 4791, 699, 4796, 704, 2753, 707, 708, 6853, 6855, 4808, 2763, 6860, 2766, 2767, 721, 6866, 2771, 2510, 6870, 6872, 6874, 4829, 736, 738, 2787, 6885, 748, 2799, 4904, 6898, 4852, 4853, 4856, 6913, 771, 6917, 4870, 4876, 4878, 4568, 2851, 2856, 4908, 6962, 6233, 4921, 4922, 6975, 6283, 4933, 2894, 6993, 2900, 2901, 2902, 856, 859, 4240, 4962, 7011, 4975, 2929, 7027, 2938, 2939, 7036, 4989, 4991, 7042, 7044, 2949, 5006, 913, 6918, 7059, 6168, 5018, 6639, 5023, 7078, 7079, 4253, 645, 5042, 5046, 3001, 5053, 958, 964, 1527, 3020, 3024, 5073, 5081, 7057, 5096, 3050, 1004, 1006, 1013, 5114, 1028, 1030, 3079, 5130, 5135, 3088, 1045, 3094, 2564, 1055, 1065, 5162, 3115, 5169, 5174, 1080, 5177, 3133, 1088, 3138, 523, 3140, 5190, 3153, 1108, 5209, 3162, 3165, 3168, 5219, 5223, 1139, 1143, 3192, 3194, 1154, 5255, 3211, 5269, 1174, 6681, 3225, 3226, 5281, 3236, 1193, 5304, 1567, 1215, 2251, 3270, 1224, 1225, 3276, 5330, 3619, 1237, 3288, 5338, 1247, 5347, 5350, 1258, 5356, 2770, 4308, 3322, 3332, 3333, 1288, 5388, 1298, 3362, 1318, 4316, 5418, 5419, 1330, 3379, 5428, 5436, 3396, 2568, 3402, 1359, 1360, 3410, 5460, 5689, 3429, 5009, 3434, 1391, 2963, 5493, 3460, 3462, 3464, 5525, 1432, 1433, 1435, 5533, 5538, 3494, 1460, 1479, 5578, 3531, 1488, 3538, 5606, 1523, 5620, 5621, 3575, 6666, 5642, 3596, 1557, 5655, 1560, 5381, 3616, 1571, 3620, 3626, 1582, 1587, 3638, 1592, 803, 1597, 3649, 3340, 3665, 3674, 3681, 3685, 3005, 5746, 1652, 1653, 3709, 5763, 1668, 1670, 1675, 5772, 3735, 1690, 1697, 5798, 3751, 5800, 3758, 4381, 5816, 3772, 5825, 5827, 1740, 5837, 5841, 1746, 1747, 1749, 5508, 1755, 3808, 5858, 1765, 5866, 5871, 3826, 5417, 3838, 1793, 4395, 4054, 3849, 1804, 3853, 3863, 3864, 1822, 6789, 1832, 3885, 5935, 5937, 1848, 3899, 2244, 1872, 5978, 3934, 5993, 5997, 1913, 6018, 1926, 1927, 6024, 6031, 1946, 6044, 1953, 1955, 1959, 1961, 1965, 6064, 6066, 670, 6071, 4024, 4026, 1984, 1985, 1986, 6340, 2112, 1995, 4046, 6098, 6819, 2005, 203, 6103, 6111, 6113, 2020, 4072, 6123, 6124, 2045, 1704, 2039, 6141, 4095, 2144, 5771, 1584, 5773, 6030, 4056, 3188, 1097, 2786, 4976, 3951, 4380, 5820, 4392, 2880, 6513, 5819, 3275, 7045, 6979, 2693, 1918, 84, 888, 1310, 2549, 6690, 2229, 3983, 3444, 6600, 1180, 4533, 997, 5166, 200, 3852, 2607, 3137, 4306, 1788, 823, 4833, 5394, 3136, 3846, 1251, 807, 366, 1849, 4941, 5723, 4958, 5160, 6327, 4431, 3348, 478, 3876, 3935, 4470, 6368, 7093, 673, 2865, 6618, 6638, 2332, 2507, 5440, 5616, 3392, 96, 3284, 60, 5200, 2995, 873, 367, 516, 1681, 4184, 1102, 1078, 957, 6455, 3563, 4347, 6739, 2592, 5834, 6719, 719, 5579, 739, 2004, 3041, 2583, 314, 6430, 2131, 1119, 364, 3622, 1259, 1838, 6620, 2063, 569, 2857, 1242, 3645, 4236, 6916, 1874, 5391, 7039, 2838, 3022, 4619, 2677, 382, 395, 7021, 280, 2777, 2218, 1161, 2665, 2611, 3763, 3178, 4064, 1116, 922, 525, 5037, 3670, 1873, 4223, 2077, 5597, 2958, 403, 6238, 5888, 2405, 3239, 2468, 6688, 5151, 6626, 2847, 5589, 6698, 2776, 1464, 2119, 4689, 5735, 2027, 483, 5147, 669, 4328, 5924, 4743, 1263, 398, 2100, 3426, 4081, 6261, 427, 6250, 2850, 1374, 880, 1784, 1446, 3937, 1902, 2698, 3447, 6882, 4859, 5725, 965, 7028, 1809, 5503, 3610, 4977, 3822, 1875, 3788, 1114, 2228, 6946, 4489, 5384, 2982, 1931, 2711, 3125, 2984, 6347, 2815, 2002, 245, 4014, 4863, 100, 1150, 438, 2473, 1426, 260, 3033, 3126, 1976, 4968, 2603, 6147, 6232, 2160, 3350, 3977, 2094, 1814, 4320, 1487, 1936, 1026, 1137, 3552, 4490, 1151, 1428, 4558, 3307, 840, 337, 5010, 624, 6763, 5909, 5051, 3590, 3936, 6296, 1509, 4613, 588, 4713, 1405, 1629, 6845, 825, 1172, 936, 7108, 5736, 7101, 218, 4336, 5038, 1052, 6075, 493, 5605, 5364, 3582, 950, 5337, 252, 6811, 1051, 2646, 4810, 1313, 2466, 290, 1949, 1222, 6628, 4293, 6459, 3248, 5125, 4209, 1370, 6297, 3295, 3948, 881, 2651, 4147, 2453, 7048, 90, 5173, 2657, 2828, 7123, 6188, 3122, 3960, 6290, 2778, 2649, 6697, 1558, 5718, 6390, 3338, 1338, 4638, 5788, 2352, 923, 404, 5619, 595, 430, 5848, 4481, 6409, 6760, 2361, 3189, 2869, 1591, 4339, 5810, 4001, 3103, 3613, 579, 3474, 4493, 4160, 2152, 6738, 3639, 6972, 1950, 2783, 5813, 3386, 3339, 7053, 3278, 3732, 241, 2014, 4436, 1439, 3741, 5566, 4560, 1267, 2478, 4338, 4103, 7104, 4137, 1771, 5144, 2915, 4105, 2080, 5344, 2962, 6682, 4023, 1943, 179, 6389, 5203, 5478, 2042, 480, 1422, 1046, 2335, 6940, 4910, 2217, 3251, 4799, 2617, 7115, 5136, 3779, 568, 4658, 2202, 4648, 2589, 4629, 1598, 2565, 1176, 578, 4102, 5123, 6206, 3541, 1952, 6575, 6841, 1640, 2482, 1552, 1563, 1856, 3352, 3828, 3549, 3061, 2983, 6559, 824, 6569, 2359, 1951, 6977, 1701, 2842, 2805, 309, 2600, 1369, 6883, 4972, 4882], [4096, 2563, 2564, 1033, 2059, 525, 6159, 6683, 4125, 2591, 2592, 33, 3106, 4529, 557, 6749, 1587, 5470, 1590, 9, 2617, 1035, 2118, 1097, 6732, 4026, 2131, 3674, 5050, 1633, 4708, 2663, 1641, 1130, 6763, 6770, 3189, 2167, 4729, 6266, 2173, 6782, 6787, 1669, 6280, 6806, 2711, 5273, 3737, 4463, 2716, 4255, 4256, 2209, 1701, 6311, 5288, 3759, 5812, 2244, 6855, 1225, 5839, 208, 1751, 219, 2780, 3805, 3294, 4833, 5346, 2277, 230, 1258, 238, 3312, 4853, 3831, 1787, 3838, 2815, 3845, 264, 1804, 3032, 5909, 2838, 1306, 2334, 2335, 4827, 3881, 4906, 5419, 3885, 2351, 2868, 310, 6969, 6978, 2375, 1420, 1872, 4945, 2756, 345, 7003, 4445, 4958, 4582, 1383, 1387, 367, 884, 6008, 5498, 1916, 6014, 6529, 5508, 5004, 1423, 6033, 5010, 6297, 3480, 5529, 167, 7077, 6061, 3505, 3507, 4021, 5046, 6072, 5562, 2494, 1984, 1985, 4035, 4037, 3527, 3017, 3531, 4046, 2511, 2008, 473, 6109, 995, 2532, 3852, 2027, 1522, 3061, 5116, 5910, 364, 2087, 6327, 121, 2779, 798, 5257, 4137, 3155, 735, 1823, 985, 4421, 5377, 4972, 232, 5109, 2905, 6041, 6185, 6223, 250, 2562, 5200, 5472, 1981, 1589, 3590, 6686, 5601, 6593, 1735, 1731, 4931, 7108, 1252, 3683, 713, 3171, 6051, 3316, 0, 1571, 2331, 3241, 1222, 4362, 2991, 4339, 1373, 7036, 3920, 6465, 6001, 4897, 2052, 303, 2434, 5731, 4318, 4616, 5848, 721, 2063, 1362, 2646, 577, 5606, 1763, 4657, 7122, 2456, 2588, 2066, 5917, 26, 6090, 3899, 6046, 445, 6466, 2473, 4187, 5597, 3091, 1313, 1069, 2549, 4025, 1911, 6392, 2727, 800, 5025, 4143, 6170, 2016, 7090, 677, 3034, 2636, 990, 4934, 3019, 3649, 3976, 2841, 756, 6714, 5096, 128, 3586, 3517, 3353, 3596, 4300, 752, 202, 335, 6399, 4154, 69, 4595, 1784, 5902, 4808, 3751, 5826, 253, 1930, 892, 4574, 6974, 561, 613, 4603, 3771, 65, 3365, 7015, 2221, 5745, 4732, 3043, 3514, 2753, 3259, 2970, 5189, 3698, 6682, 6904, 6390, 5723, 59, 6385, 1755, 4366, 7091, 2093, 2837, 3410, 301, 4455, 5488, 5921, 2251, 3344, 5577, 4239, 2813, 4775, 5878, 2585, 109, 5772, 6702, 1250, 5717, 4918, 4054, 5443, 2799, 3628, 214, 2745, 6804, 766, 5435, 2578, 2028, 137, 3429, 2117, 3734, 6316, 5390, 503, 4986, 3525, 1835, 318, 3774, 4379, 163, 510, 149, 5347, 3376, 6164, 3170, 1493, 6666, 5718, 2294, 24, 1859, 5833, 4056, 4258, 2254, 6535, 970, 686, 902, 996, 422, 4336, 6860, 967, 429, 3891, 774, 1839, 7014, 6379, 4147, 6148, 5102, 2132, 4689, 115, 4209, 1658, 1940, 5547, 1736, 5073, 2520, 190, 5937, 6212, 5429, 51, 715, 4495, 4069, 6874, 2203, 5897, 3541, 956, 2480, 1877, 6815, 2603, 1435, 2693, 1469, 5330, 7127, 5093, 394, 5066, 4184, 3347, 6130, 631, 2086, 3935, 2094, 6463, 5297, 3648, 876, 1181, 4115, 2995, 4222, 6155, 5888, 1557, 5080, 3396, 1624, 2609, 1354, 964, 3934, 6481, 5244, 6681, 1424, 251, 1795, 6052, 3372, 4439, 6076, 5407, 6775, 1368, 2723, 6468, 5616, 1080, 1459, 4514, 7018, 1219, 6527, 580, 2983, 2649, 3770, 5938, 1993, 6464, 2492, 1490, 2300, 177, 1467, 258, 164, 6584, 3846, 1044, 6703, 6080, 4028, 4538, 754, 1366, 5233, 1129, 7019, 5683, 5471, 3884, 1038, 5255, 2673, 2641, 3822, 3549, 3033, 3868, 2955, 3125, 673, 4584, 4668, 5119, 3185, 2126, 6792, 6633, 3859, 2876, 2103, 5339, 2832, 1062, 6977, 4341, 1819, 6563, 6641, 31, 5752, 4324, 4344, 2882, 330, 1421, 5234, 129, 654, 3051, 4460, 4639, 3082, 5522, 209, 1827, 3967, 2291, 697, 2229, 6837, 6840, 475, 7093, 4723, 521, 2155, 380, 2568, 2781, 4145, 179, 541, 4581, 6270, 4090, 5268, 803, 1482, 6573, 4250, 1048, 657, 2545, 5679, 792, 3076, 2039, 7024, 790, 3100, 6553, 3566, 1740, 687, 4293, 3127, 363, 6790, 3063, 778, 666, 6979, 4322, 64, 5456, 1535, 504, 689, 3944, 3339, 2704, 4682, 3694, 1295, 2468, 6902, 1846, 13, 4207, 1311, 327, 4612, 5454, 222, 1114, 1376, 4139, 4493, 4476, 3595, 3136, 4829, 5719, 3462, 417, 3191, 3079, 5367, 1008, 1814, 3781, 4728, 472, 1055, 3777, 750, 6287, 1498, 624, 4397, 237, 5537, 757, 2898, 2672, 2064, 6724, 3627, 708, 913, 3955, 210, 4353, 3657, 3555, 7048, 1947, 7054, 3332, 3350, 3001, 2679, 6838, 6698, 1487, 2653, 2900, 578, 870, 556, 4976, 5400, 5984, 5451, 90, 171, 2035, 4555, 5438, 6163, 1054, 4930, 3435, 37, 6715, 1381, 6950, 3580, 3360, 6531, 6852, 405, 2886, 5239, 4299, 6030, 2551, 2270, 3477, 157, 70, 4977, 1174, 6335, 4773, 2579, 2412, 6102, 6839, 5675, 6012, 2519, 1921, 5362, 4407, 6085, 6389, 3432, 419, 6066, 6290, 3768, 5053, 5803, 3994, 5695, 3338, 4940, 729, 3754, 4182, 5338, 4537, 6295, 3220, 6238, 1052, 558, 5446, 2980, 2347, 185, 5730, 2394, 5864, 6662, 888, 623, 5388, 5434, 133, 1043, 400, 3828, 4496, 2640, 5169, 4740, 4964, 809, 1771, 3295, 6975, 2659, 5356, 5768, 1592, 1242, 5693, 4262, 5414, 1640, 5940, 6421, 1367, 2700, 6575, 1209, 847, 1821, 1159, 3675, 4126, 81, 415, 4133, 911, 6518, 5178, 5781, 4211, 3929, 333, 4644, 5151, 966, 6554, 1202, 2011, 538, 4837, 6730, 1351, 2048, 880, 4648, 4752, 6396, 2183, 674, 2121, 481, 1377, 1206, 1268, 1679, 7004, 6931, 642, 4381, 3844, 4780, 1565, 6330, 2076, 2045, 4279, 5903, 4919, 1949, 2667, 6650, 3340, 6031, 3554, 3475, 2373, 5735, 228, 1583, 324, 856, 5619, 5222, 793, 1873, 5140, 4411, 1808, 6817, 3095, 5328, 2247, 6474, 3937, 5764, 2389, 5160, 5914, 2796, 3938, 3568, 6937, 1902, 2399, 5183, 495, 7052, 3206, 2684, 3821, 5300, 2329, 1213, 3575, 953, 3670, 343, 5081, 769, 704, 3094, 3300, 3489, 2082, 5640, 2415, 6572, 1233, 4443, 3515, 3, 2046, 2874, 5358, 1945, 2689, 6558, 6309, 2930, 5195, 4701, 4345, 3321, 1553, 3194, 5556, 2189, 5974, 5455, 6834, 4150, 3028, 195, 6846, 3710, 5147, 3180, 4071, 2014, 4008, 3408, 4171, 5263, 1350, 6514, 1028, 2778, 117, 1961, 5799, 5214, 1182, 1013, 4859, 5947, 5461, 6607, 6800, 3953, 4599, 1979, 1173, 2104, 5952, 2873, 3192, 4295, 3762, 110, 5402, 6202, 954, 7007, 2113, 1296, 4219, 930, 1154, 6924, 4960, 6723, 6282, 5728, 4420, 5038, 3423, 1328, 6788, 2712, 855, 5476, 5825, 1816, 2360, 7025, 2398, 2071, 5545, 582, 2577, 2948, 5, 2531, 6853, 6643, 6700, 3556, 3288, 2569, 3336, 4170], [5124, 1033, 5130, 2571, 2577, 2578, 6164, 4119, 5147, 2394, 5667, 552, 3628, 871, 3337, 5689, 2625, 5706, 4176, 3153, 5714, 5715, 6740, 87, 2651, 1629, 5736, 4201, 1383, 1134, 5053, 1137, 3698, 6776, 1044, 2837, 4225, 3720, 5772, 6801, 1174, 3176, 6305, 4264, 5813, 4797, 1216, 196, 4810, 2763, 3276, 3789, 6861, 4309, 4824, 6363, 2781, 1488, 2428, 1771, 1772, 749, 2600, 2088, 3834, 1151, 5372, 3837, 4354, 2347, 2825, 6743, 3800, 3861, 3867, 6436, 295, 3367, 5928, 6443, 301, 303, 4912, 7048, 6966, 5431, 1856, 6989, 6990, 1872, 2898, 5177, 855, 5466, 2399, 5984, 7010, 3429, 4455, 2408, 876, 2416, 2939, 7036, 3453, 2945, 5000, 395, 2445, 3471, 1424, 6033, 3478, 6045, 418, 2980, 938, 6071, 3003, 6218, 5566, 7104, 965, 1482, 2039, 3024, 3541, 3030, 6626, 6395, 492, 5107, 5620, 503, 5624, 2045, 4386, 5257, 3752, 4355, 5338, 2021, 2869, 5799, 6942, 3685, 5775, 1522, 2995, 3261, 1917, 4763, 5825, 856, 3776, 5023, 3366, 729, 1143, 5419, 3826, 6198, 5798, 4774, 3703, 4638, 5074, 3782, 4334, 5952, 579, 1410, 1623, 3318, 4804, 1940, 5340, 6482, 4947, 5569, 4857, 487, 6396, 801, 4622, 5292, 2650, 5300, 3355, 4904, 5456, 5377, 4421, 1959, 4952, 1815, 2838, 3020, 3827, 655, 2655, 791, 6641, 1597, 4903, 1637, 6525, 3660, 4399, 3900, 4876, 789, 2343, 4447, 1592, 1668, 3073, 4708, 1609, 3539, 715, 220, 3786, 2061, 3770, 161, 4426, 277, 5505, 1461, 2189, 6687, 4852, 922, 2929, 589, 1198, 6433, 1453, 6940, 309, 5116, 207, 5270, 3168, 6432, 1350, 4424, 1096, 6688, 77, 6331, 6324, 5280, 1619, 6840, 2663, 1331, 1693, 1838, 4096, 6267, 1583, 1258, 2813, 4628, 6886, 4773, 1413, 857, 6539, 2996, 228, 1228, 6885, 2284, 5128, 1125, 824, 4496, 4375, 1594, 3735, 4422, 6249, 110, 2884, 2895, 6357, 7115, 1545, 4883, 2850, 3645, 4775, 3306, 3654, 3822, 1274, 5476, 909, 5136, 2778, 5383, 1116, 1946, 6543, 787, 2044, 3920, 5029, 2304, 6874, 6057, 582, 4605, 4886, 343, 1870, 5217, 2460, 143, 5839, 5223, 119, 5434, 4404, 6816, 5328, 6300, 4769, 1976, 2417, 4474, 1129, 4488, 475, 6391, 6793, 2119, 2900, 6311, 251, 3399, 5119, 5622, 7008, 1479, 811, 2178, 1087, 5748, 4358, 396, 1721, 4261, 603, 6717, 7095, 7105, 2075, 4738, 6192, 3599, 919, 1907, 4696, 6229, 5745, 4315, 4466, 1970, 2786, 623, 99, 6790, 6187, 4553, 1739, 555, 6673, 6386, 5905, 3759, 5832, 6026, 1936, 2395, 6540, 748, 2915, 6832, 5489, 6655, 318, 6465, 6920, 1622, 976, 3107, 4873, 2936, 6796, 6505, 5833, 2667, 354, 1726, 5144, 1792, 6749, 146, 4817, 6447, 3864, 513, 1572, 4674, 6379, 4879, 5529, 4995, 463, 3674, 4448, 4650, 3658, 6495, 4158, 1063, 152, 3534, 5305, 5601, 3127, 3136, 1919, 2064, 3242, 547, 6370, 1422, 212, 6093, 1927, 4985, 4037, 1517, 5344, 2441, 4599, 2816, 1822, 4711, 5935, 5452, 6230, 2916, 5070, 312, 1249, 2081, 5140, 5746, 5869, 4799, 1102, 3935, 693, 185, 2261, 6842, 5938, 5228, 4537, 1682, 873, 6739, 5117, 353, 5368, 718, 4274, 4942, 6261, 3966, 7089, 6999, 3372, 2285, 2243, 236, 1054, 1110, 6407, 5524, 1462, 4652, 5205, 6294, 2732, 5555, 3494, 6068, 3224, 5847, 7064, 5009, 139, 4132, 3557, 1809, 3653, 5307, 942, 5781, 4265, 1052, 2695, 5043, 74, 3612, 3562, 338, 5936, 5068, 5428, 2217, 2510, 5856, 2313, 2998, 3426, 4741, 4742, 5907, 2756, 6668, 5533, 1786, 6971, 2287, 1192, 4352, 3268, 808, 2639, 3574, 4663, 5574, 5365, 3425, 4379, 5388, 38, 5211, 445, 2098, 7057, 345, 3683, 3050, 5841, 4656, 348, 6638, 1131, 1298, 5197, 5081, 5279, 3282, 1302, 3500, 792, 2409, 5829, 5698, 5095, 1397, 7078, 2046, 1585, 2202, 2260, 2248, 4934, 780, 5219, 6723, 2591, 2192, 4128, 1672, 3836, 4991, 5498, 1058, 523, 3585, 700, 507, 7011, 4634, 1898, 6125, 3849, 1621, 4988, 2464, 5713, 806, 94, 27, 2345, 4109, 334, 2070, 6143, 657, 2847, 1155, 4654, 893, 6588, 2346, 367, 3718, 6653, 1921, 4745, 5528, 5809, 6732, 783, 1601, 1487, 4713, 4588, 7003, 1097, 4932, 3204, 4504, 2811, 4849, 3157, 4254, 5386, 3937, 526, 6383, 1787, 1189, 4384, 3983, 1695, 267, 4343, 4502, 2589, 5148, 2085, 1374, 229, 2662, 3992, 1365, 553, 1003, 3906, 1105, 6708, 3447, 7122, 4921, 2723, 4878, 2131, 1750, 2137, 4688, 3353, 4667, 1252, 4725, 5946, 6574, 1091, 1814, 1974, 1949, 2293, 3499, 2935, 2375, 2519, 2267, 5511, 4581, 2872, 6797, 2904, 5684, 6053, 6428, 6109, 6970, 5597, 7098, 1641, 2724, 7009, 154, 6915, 6019, 5584, 5073, 289, 171, 6666, 5161, 245, 1496, 1777, 4829, 3692, 1337, 1850, 673, 4122, 5297, 771, 264, 2427, 2832, 1701, 6470, 5345, 2566, 6120, 5926, 3649, 6054, 6340, 1197, 2776, 5599, 4210, 4772, 7045, 5669, 1417, 6639, 4234, 427, 5184, 2059, 4666, 5717, 5831, 6734, 1537, 2310, 1113, 3758, 533, 4508, 72, 6773, 4889, 6892, 6792, 1915, 2012, 3143, 4273, 2263, 1259, 2035, 4413, 6231, 1851, 2817, 136, 1308, 6567, 3502, 2008, 3544, 2461, 5366, 6764, 2529, 527, 5481, 4166, 5843, 5650, 7018, 1427, 6359, 3643, 5819, 2938, 5993, 319, 3709, 178, 5562, 6762, 2618, 534, 2122, 1552, 3650, 566, 5816, 1168, 1224, 1225, 5066, 5082, 2646, 3520, 78, 21, 105, 3749, 1435, 5473, 1558, 5858, 5160, 1532, 3179, 5884, 4279, 2983, 1816, 4105, 1694, 5364, 3663, 6134, 192, 916, 4507, 2738, 2406, 4417, 2140, 1314, 6430, 2326, 4203, 5804, 3172, 3913, 6202, 2504, 1483, 3370, 1563, 2130, 2356, 227, 1504, 867, 255, 3841, 3348, 4425, 5578, 4860, 6914, 6570, 3131, 5438, 5262, 3702, 2941, 6860, 5588, 1690, 2152, 1827, 2552, 6291, 3344, 5492, 3472, 3397, 5179, 5418, 3386, 3032, 5132, 1849, 6191, 2579, 1590, 2155, 7109, 20, 1549, 1367, 4313, 4211, 6513, 361, 3098, 5269, 5287, 522, 6777, 578, 4568, 3248, 2902, 5374, 4025, 2812, 292, 1505, 5550, 4026, 6569, 6780, 4431, 4619, 1235, 4642, 2930, 6857, 708, 4339, 3004, 798, 3873, 35, 1414, 2887, 6979, 6784, 4787, 6807, 6208, 4870, 2154, 6124, 6713, 4303, 6806, 3888, 3592, 2467, 3191, 653, 6636, 4231, 3748, 91, 4798, 5881, 6477, 4004, 2845, 6086, 324, 6411, 5174, 2801, 371, 5937, 3552, 5640, 1124, 1731, 1873, 956, 2622, 6800, 1618, 875, 6211, 6736, 6683, 3488, 2583, 3680, 4280, 1389, 3569, 1014, 1738, 7058, 7111, 287, 912], [514, 6147, 5130, 2571, 1552, 6830, 1558, 1052, 2473, 5157, 552, 4652, 2611, 1080, 5177, 2399, 5693, 1608, 6218, 3147, 3153, 1106, 5715, 6740, 87, 1629, 5820, 4455, 1134, 5232, 3698, 4201, 6776, 5241, 6268, 4741, 2695, 5772, 143, 1168, 1174, 708, 3741, 4773, 5798, 5288, 2217, 3758, 3248, 2738, 3763, 5813, 4796, 196, 6853, 6343, 2763, 3789, 2261, 4824, 4825, 6379, 6384, 4851, 6395, 6396, 5689, 3367, 3330, 3337, 778, 1814, 6940, 806, 295, 2347, 4912, 1641, 6461, 1856, 3396, 6990, 2898, 6030, 855, 1374, 6495, 3429, 1383, 873, 4461, 3443, 3447, 1913, 2939, 7036, 6515, 5504, 4481, 7048, 395, 396, 2445, 7054, 6033, 5524, 3478, 922, 2460, 2461, 4511, 1955, 7078, 6569, 3485, 4339, 6071, 1976, 4537, 1439, 5053, 5566, 1482, 2039, 462, 1488, 1193, 485, 5372, 3565, 6128, 498, 503, 84, 2045, 3582, 6143, 4554, 6231, 4178, 6631, 6462, 1740, 4131, 2728, 3091, 6294, 6638, 3883, 1952, 2504, 4422, 2449, 4958, 2075, 2294, 7040, 301, 567, 3557, 5037, 797, 3372, 1734, 5987, 7050, 5824, 6626, 549, 2935, 3339, 1078, 2817, 5381, 1545, 1800, 2545, 2698, 2005, 1436, 2590, 5736, 2707, 5667, 4873, 6993, 1824, 1787, 1670, 4674, 1137, 5051, 237, 2781, 2983, 6968, 2411, 4288, 6432, 5368, 3449, 2060, 4315, 6026, 4299, 5725, 5574, 938, 1979, 4371, 2335, 3035, 749, 3154, 4111, 3094, 1668, 3243, 2160, 6556, 700, 1550, 768, 1091, 165, 5126, 270, 6280, 3399, 2427, 3948, 6514, 1089, 2901, 5124, 568, 1077, 5347, 1388, 2801, 2379, 1410, 439, 4128, 1532, 6842, 4420, 6290, 9, 3670, 6291, 5217, 2417, 3560, 2375, 527, 5804, 4613, 4102, 29, 5029, 3861, 2320, 3643, 2002, 4874, 2107, 566, 1881, 2805, 2841, 4054, 5528, 713, 1346, 3168, 1637, 6598, 2046, 6664, 569, 3818, 1792, 6086, 4941, 4303, 680, 318, 6090, 4082, 6558, 5283, 4985, 2732, 824, 3161, 3827, 5481, 6385, 6411, 6265, 3664, 992, 6775, 765, 5880, 4642, 7057, 5478, 4124, 229, 6091, 523, 5691, 2851, 529, 645, 871, 2510, 5624, 6297, 3180, 5270, 136, 5142, 454, 6801, 1135, 2904, 4307, 4096, 152, 1444, 6545, 1531, 4885, 5058, 1370, 330, 6579, 4361, 1597, 3894, 294, 3569, 1591, 5436, 1228, 1005, 2837, 91, 5569, 1601, 3536, 256, 2723, 3107, 6238, 3534, 85, 4425, 5114, 6955, 5440, 6820, 6620, 5549, 2880, 2661, 783, 3668, 6340, 2350, 6433, 3020, 4629, 827, 6748, 4064, 4992, 6886, 1837, 3357, 6061, 5525, 2961, 2181, 6687, 4942, 5374, 950, 6057, 364, 5140, 1549, 2428, 4013, 2299, 5353, 6999, 1765, 5000, 5366, 1224, 3355, 3126, 334, 1055, 4933, 6570, 5579, 2049, 5356, 3992, 5620, 5216, 5869, 1838, 4621, 210, 4399, 212, 2225, 5189, 3931, 7045, 4332, 3749, 7034, 2842, 3596, 477, 3720, 7008, 6481, 4172, 3382, 2995, 1772, 2353, 1796, 3555, 6724, 729, 587, 825, 3362, 5084, 3782, 1033, 547, 2811, 4004, 4014, 4599, 4584, 6053, 1808, 5009, 2936, 1784, 1705, 2900, 7038, 5802, 1619, 6972, 3309, 1253, 618, 1428, 1526, 4191, 2475, 1731, 3258, 4463, 4742, 6699, 4798, 1592, 811, 1487, 154, 4946, 66, 2580, 2907, 3140, 6164, 6874, 2872, 1771, 1872, 1930, 4277, 1946, 319, 6780, 2287, 5305, 3132, 6094, 2850, 126, 1336, 3017, 6720, 2766, 5260, 185, 3610, 1775, 5166, 786, 2012, 1330, 5866, 6755, 4553, 312, 6557, 3800, 3138, 3691, 5560, 2088, 5473, 2094, 6324, 4346, 3278, 495, 5144, 4625, 1414, 2242, 6019, 6333, 3694, 2385, 7011, 5117, 3599, 3993, 5068, 4442, 1331, 6422, 4882, 4240, 1151, 2119, 6588, 2416, 4358, 834, 6374, 1720, 3738, 6682, 2771, 3118, 6847, 2760, 5511, 3276, 2452, 653, 7095, 5928, 6666, 5799, 1479, 5816, 6288, 2568, 2895, 4918, 3142, 5341, 876, 1848, 2934, 146, 762, 4633, 3788, 3178, 4445, 1234, 4268, 361, 2495, 911, 2400, 731, 224, 6648, 5057, 3947, 510, 1198, 4265, 2615, 5448, 6120, 1215, 7079, 209, 2682, 6978, 5784, 4769, 1926, 4689, 4331, 1086, 5568, 105, 6283, 5831, 1542, 3887, 3471, 4075, 371, 6486, 3563, 2589, 5179, 4810, 2662, 4042, 6816, 2014, 3966, 1334, 583, 3634, 919, 238, 2213, 1985, 1758, 2725, 6653, 5281, 4109, 217, 492, 162, 5359, 287, 4386, 660, 3801, 5533, 6655, 6008, 789, 6813, 5884, 6513, 4402, 2033, 3845, 1269, 82, 1777, 2541, 4285, 3684, 4067, 4448, 4977, 787, 1424, 6452, 4300, 6913, 4654, 3551, 688, 3226, 7028, 4458, 4304, 3472, 1912, 484, 578, 2283, 5345, 4224, 2202, 4879, 5748, 2482, 1258, 807, 6054, 2334, 4310, 4243, 112, 4270, 447, 1726, 3793, 5881, 4274, 2815, 309, 4021, 6882, 2254, 4876, 3502, 6154, 3427, 5775, 6636, 6716, 3899, 6870, 3268, 5655, 69, 4210, 533, 4436, 4077, 7093, 4280, 6466, 5992, 5300, 3528, 5113, 5829, 1746, 4052, 5307, 3685, 2106, 6443, 4337, 3318, 1919, 3697, 5969, 3776, 4966, 5622, 6447, 5383, 6559, 3176, 6983, 3841, 5200, 2591, 2869, 4983, 6465, 4230, 5264, 910, 6539, 5859, 6096, 1102, 4466, 6819, 1044, 2218, 6772, 1453, 3221, 5304, 322, 5834, 3496, 6668, 6184, 4739, 4309, 3004, 3752, 4412, 636, 4663, 6970, 6684, 579, 5002, 3837, 4158, 110, 3001, 104, 2318, 802, 6249, 2020, 5776, 2718, 3444, 348, 2077, 192, 1928, 6981, 6325, 2741, 5074, 5614, 5827, 4475, 4546, 6719, 6770, 475, 245, 2824, 5228, 4023, 4044, 1318, 2666, 2405, 4596, 976, 3230, 316, 3261, 659, 1626, 2929, 7116, 1167, 5754, 5505, 3888, 6389, 5258, 2133, 3504, 277, 1524, 1701, 4477, 5706, 2394, 3195, 6230, 3453, 6113, 1435, 5471, 4409, 4572, 6574, 3838, 5344, 459, 2783, 5650, 3561, 4650, 4560, 2222, 1768, 2663, 401, 3765, 3867, 5287, 917, 2178, 1970, 7001, 4384, 5493, 1505, 6951, 6783, 5377, 2642, 1537, 2894, 3852, 3165, 3038, 2293, 3900, 1015, 55, 2605, 5731, 2646, 4785, 3674, 2182, 2778, 2273, 6589, 3511, 6641, 236, 208, 3306, 5919, 4791, 1426, 1709, 3179, 6974, 6910, 2230, 3395, 3612, 2748, 1366, 522, 1165, 1672, 2081, 1131, 3913, 6739, 1087, 3023, 2966, 3595, 6211, 707, 801, 315, 4795, 404, 4184, 6075, 7010, 227, 6766, 5082, 4947, 5255, 1585, 3433, 2483, 4713, 3397, 1907, 5800, 58, 5978, 7098, 2122, 4774, 5123, 5964, 6838, 4763, 4777, 4203, 4856, 3030, 2547, 1172, 1902, 2168, 2408, 1621, 4197, 555, 3660, 5466, 4429, 1871, 1483, 3402, 2750, 5935, 6470, 6442, 2145, 6482, 2874, 4235, 314, 1898, 3700, 1786, 2027, 5419, 2051, 3267], [4096, 6666, 6147, 3079, 2568, 9, 522, 523, 2060, 1550, 5135, 5081, 2579, 20, 1557, 3094, 6233, 260, 3610, 6008, 5800, 2590, 5893, 3616, 4131, 4133, 1065, 6699, 4652, 5798, 1584, 1585, 3634, 1587, 6238, 4663, 1592, 5177, 6716, 1597, 6719, 6720, 4808, 2571, 3140, 583, 3852, 3147, 4172, 2639, 6464, 3665, 1106, 6227, 6740, 85, 2646, 5561, 2137, 5381, 2063, 6748, 5725, 3678, 3173, 2144, 3170, 6757, 2663, 5820, 618, 3691, 3693, 3694, 4300, 1653, 873, 3192, 4730, 4799, 2684, 6783, 2688, 1668, 4630, 4230, 5255, 3948, 1675, 5772, 6091, 1168, 195, 1902, 1174, 151, 3225, 2718, 6853, 5281, 5283, 164, 4773, 4262, 4775, 5288, 1479, 2732, 3758, 824, 2225, 3443, 6837, 5816, 3258, 2748, 1215, 5827, 1626, 4325, 2649, 200, 713, 366, 2763, 1740, 3789, 5839, 4305, 6866, 3022, 3799, 1746, 5338, 5851, 4316, 3194, 2273, 5858, 739, 1765, 165, 3818, 210, 5359, 3824, 6898, 4339, 5259, 1781, 5878, 1784, 6395, 5372, 3236, 2817, 3330, 2327, 1796, 4054, 1801, 3339, 1804, 6071, 4568, 4370, 3863, 3865, 6874, 2335, 2340, 806, 295, 4904, 3883, 6444, 3885, 5935, 5937, 6452, 6557, 1338, 2266, 5440, 6465, 3395, 4420, 4933, 2375, 330, 1591, 2380, 738, 2894, 1873, 4946, 2767, 2901, 2902, 1368, 348, 1253, 2961, 7009, 3427, 4452, 4966, 6033, 4968, 5993, 364, 1390, 1391, 368, 6515, 6705, 4983, 4472, 2430, 2880, 3460, 6533, 1926, 2229, 398, 6545, 6723, 4105, 5525, 3478, 153, 5793, 922, 495, 6556, 1949, 6558, 6297, 5538, 1955, 1046, 7078, 7079, 6569, 3168, 2461, 6064, 2482, 2244, 4533, 585, 3001, 4026, 1979, 3517, 6749, 1985, 5380, 3527, 3152, 6090, 3531, 1686, 2004, 2005, 3542, 1701, 2741, 473, 6735, 3038, 3551, 6113, 4347, 485, 5606, 1788, 2727, 6124, 3154, 6639, 5617, 498, 6643, 1558, 5112, 1108, 5114, 5169, 1021, 2045, 6143, 2702, 5200, 6690, 4824, 707, 3756, 4231, 2334, 2025, 156, 5605, 2367, 5759, 4201, 6806, 992, 913, 6755, 16, 3269, 3785, 6811, 2304, 3095, 5840, 3632, 502, 1319, 825, 2396, 4429, 2277, 3343, 5162, 2382, 5578, 1683, 5750, 457, 5946, 1124, 2020, 4224, 6495, 3765, 865, 2776, 5516, 2603, 6044, 324, 309, 5590, 3226, 6686, 2014, 5579, 5300, 4346, 3162, 5620, 1604, 5429, 3831, 2217, 2359, 3243, 2695, 5024, 2532, 299, 675, 7026, 6111, 5037, 367, 5367, 2589, 624, 4148, 2136, 1531, 4541, 167, 218, 4804, 4534, 5566, 4095, 3023, 6024, 3126, 5834, 1931, 2625, 3611, 2565, 5355, 243, 2189, 7043, 196, 678, 1898, 5813, 3602, 5238, 4376, 2012, 1681, 669, 2545, 5691, 6956, 510, 7088, 2347, 6870, 4427, 4457, 2983, 2847, 5497, 2946, 2815, 2507, 4903, 6428, 6832, 3316, 6620, 5430, 6542, 395, 6181, 4866, 78, 2283, 1608, 5823, 2594, 688, 5681, 1432, 4578, 3698, 1935, 82, 5986, 2389, 6947, 4666, 2449, 6149, 2703, 3202, 5672, 6537, 710, 2485, 6784, 513, 6061, 1428, 3934, 3994, 5621, 2752, 4741, 1488, 2478, 1203, 3426, 5416, 4675, 997, 4308, 1306, 2035, 4738, 493, 902, 5388, 4975, 7010, 5290, 4556, 525, 5903, 6103, 850, 6218, 5203, 5219, 855, 7040, 38, 3145, 6878, 3951, 1490, 3309, 2061, 6236, 2318, 3050, 5659, 4240, 5884, 6977, 5693, 3525, 5347, 3887, 2242, 4756, 772, 3741, 259, 212, 6826, 4077, 3834, 1552, 2583, 2536, 1439, 2466, 4333, 4279, 6168, 5003, 396, 1747, 3528, 1237, 4563, 5602, 2445, 536, 5642, 1265, 1598, 4876, 3449, 2039, 1275, 5873, 4883, 6634, 2176, 6283, 5057, 133, 2081, 6660, 4941, 995, 3315, 2637, 5832, 407, 2354, 1848, 176, 5493, 5022, 284, 578, 2483, 7000, 5501, 6445, 5297, 1055, 5982, 322, 227, 3444, 4558, 4514, 5471, 3372, 2298, 6677, 4644, 4210, 2853, 6447, 272, 1533, 6266, 6553, 5719, 3976, 4187, 1120, 5508, 6075, 6194, 4183, 4796, 6002, 6123, 4743, 1089, 5212, 360, 4849, 3288, 4093, 4537, 1035, 5010, 842, 851, 2739, 1038, 1959, 2293, 1426, 2738, 4, 4639, 3463, 6325, 269, 849, 1053, 1155, 160, 6636, 6320, 5646, 2868, 1832, 626, 5293, 1224, 970, 4839, 1187, 1819, 1015, 5687, 4476, 1814, 6760, 6776, 1310, 3020, 2657, 404, 1923, 4146, 1933, 5268, 5224, 1482, 3628, 6268, 461, 4660, 1232, 4641, 645, 1517, 4647, 5533, 6917, 4810, 1960, 989, 1823, 5072, 5647, 1329, 4264, 6517, 3751, 4477, 6405, 3598, 233, 2161, 2929, 2294, 2766, 112, 2373, 7059, 915, 2960, 4426, 1838, 4157, 4252, 4042, 6396, 1359, 5319, 4307, 6031, 1872, 6179, 4694, 1481, 2094, 4162, 4181, 6463, 797, 1642, 3565, 6570, 5597, 1004, 2601, 4851, 1354, 3970, 2898, 2851, 6828, 5144, 4581, 4687, 4651, 3983, 1817, 7044, 5473, 1986, 3103, 7052, 4254, 5925, 3507, 4727, 4381, 2116, 586, 220, 4962, 5978, 4560, 964, 6112, 6107, 5356, 4977, 4713, 4126, 3096, 2928, 5550, 3794, 6563, 3896, 6513, 103, 1298, 6578, 6538, 3295, 4486, 5032, 5448, 2982, 3333, 1260, 5619, 399, 6630, 1641, 5728, 3581, 143, 1730, 2230, 3906, 6225, 5260, 4046, 4163, 2980, 2611, 2701, 5771, 1180, 4918, 90, 3929, 636, 1244, 2995, 241, 6993, 4072, 5304, 3849, 3624, 6270, 3577, 5447, 4490, 6514, 3368, 6461, 4035, 3367, 7036, 5164, 5454, 4470, 5187, 2267, 1451, 3603, 3647, 557, 6288, 4007, 5829, 6343, 4371, 2725, 2072, 5584, 2473, 3736, 4882, 7101, 4662, 463, 1639, 3138, 4455, 94, 6820, 3575, 3743, 925, 6154, 6618, 4679, 3049, 4040, 3530, 1005, 4596, 5247, 3467, 1080, 4182, 6037, 1043, 5053, 595, 3960, 6057, 780, 3365, 5232, 6030, 2558, 4261, 6359, 6752, 1615, 1951, 1009, 3430, 6188, 2648, 314, 5723, 1690, 84, 3035, 2106, 2495, 1824, 5969, 3410, 1697, 7007, 3030, 5846, 1540, 4322, 4386, 4829, 1040, 6684, 1800, 4259, 4458, 4089, 1927, 1424, 6872, 4850, 1720, 3580, 2907, 2577, 5775, 6499, 5312, 4949, 4310, 2864, 1050, 1915, 923, 4664, 6431, 3142, 245, 256, 597, 730, 6883, 1462, 5872, 2786, 4334, 2783, 7072, 2728, 3993, 6280, 3178, 316, 1618, 856, 2669, 1052, 237, 2119, 2893, 5369, 5859, 2150, 5655, 1757, 3133, 2778, 652, 5107, 7028, 4546, 4885, 4184, 6189, 238, 5640, 6579, 303, 6789, 4739, 5782, 6732, 6436, 389, 6978, 2411, 230, 6414, 5354, 6769, 4910, 3322, 1874, 6360, 954, 72, 2698, 213, 3429, 2049, 1930, 2945, 1793, 6994, 549, 823, 6013, 4117, 4986, 351, 5272, 3641, 5807, 5166, 2997, 3396, 6571, 1099, 4507, 5174, 1193, 5252, 4445, 846, 2453, 6984], [4096, 6666, 6147, 3079, 2568, 9, 522, 523, 2060, 1550, 5135, 5081, 2579, 20, 1557, 3094, 6233, 260, 3610, 6008, 5800, 2590, 5893, 3616, 4131, 4133, 1065, 6699, 4652, 5798, 1584, 1585, 3634, 1587, 6238, 4663, 1592, 5177, 6716, 1597, 6719, 6720, 4808, 2571, 3140, 583, 3852, 3147, 4172, 2639, 6464, 3665, 1106, 6227, 6740, 85, 2646, 5561, 2137, 5381, 2063, 6748, 5725, 3678, 3173, 2144, 3170, 6757, 2663, 5820, 618, 3691, 3693, 3694, 4300, 1653, 873, 3192, 4730, 4799, 2684, 6783, 2688, 1668, 4630, 4230, 5255, 3948, 1675, 5772, 6091, 1168, 195, 1902, 1174, 151, 3225, 2718, 6853, 5281, 5283, 164, 4773, 4262, 4775, 5288, 1479, 2732, 3758, 824, 2225, 3443, 6837, 5816, 3258, 2748, 1215, 5827, 1626, 4325, 2649, 200, 713, 366, 2763, 1740, 3789, 5839, 4305, 6866, 3022, 3799, 1746, 5338, 5851, 4316, 3194, 2273, 5858, 739, 1765, 165, 3818, 210, 5359, 3824, 6898, 4339, 5259, 1781, 5878, 1784, 6395, 5372, 3236, 2817, 3330, 2327, 1796, 4054, 1801, 3339, 1804, 6071, 4568, 4370, 3863, 3865, 6874, 2335, 2340, 806, 295, 4904, 3883, 6444, 3885, 5935, 5937, 6452, 6557, 1338, 2266, 5440, 6465, 3395, 4420, 4933, 2375, 330, 1591, 2380, 738, 2894, 1873, 4946, 2767, 2901, 2902, 1368, 348, 1253, 2961, 7009, 3427, 4452, 4966, 6033, 4968, 5993, 364, 1390, 1391, 368, 6515, 6705, 4983, 4472, 2430, 2880, 3460, 6533, 1926, 2229, 398, 6545, 6723, 4105, 5525, 3478, 153, 5793, 922, 495, 6556, 1949, 6558, 6297, 5538, 1955, 1046, 7078, 7079, 6569, 3168, 2461, 6064, 2482, 2244, 4533, 585, 3001, 4026, 1979, 3517, 6749, 1985, 5380, 3527, 3152, 6090, 3531, 1686, 2004, 2005, 3542, 1701, 2741, 473, 6735, 3038, 3551, 6113, 4347, 485, 5606, 1788, 2727, 6124, 3154, 6639, 5617, 498, 6643, 1558, 5112, 1108, 5114, 5169, 1021, 2045, 6143, 2702, 5200, 6690, 4824, 707, 3756, 4231, 2334, 2025, 156, 5605, 2367, 5759, 4201, 6806, 992, 913, 6755, 16, 3269, 3785, 6811, 2304, 3095, 5840, 3632, 502, 1319, 825, 2396, 4429, 2277, 3343, 5162, 2382, 5578, 1683, 5750, 457, 5946, 1124, 2020, 4224, 6495, 3765, 865, 2776, 5516, 2603, 6044, 324, 309, 5590, 3226, 6686, 2014, 5579, 5300, 4346, 3162, 5620, 1604, 5429, 3831, 2217, 2359, 3243, 2695, 5024, 2532, 299, 675, 7026, 6111, 5037, 367, 5367, 2589, 624, 4148, 2136, 1531, 4541, 167, 218, 4804, 4534, 5566, 4095, 3023, 6024, 3126, 5834, 1931, 2625, 3611, 2565, 5355, 243, 2189, 7043, 196, 678, 1898, 5813, 3602, 5238, 4376, 2012, 1681, 669, 2545, 5691, 6956, 510, 7088, 2347, 6870, 4427, 4457, 2983, 2847, 5497, 2946, 2815, 2507, 4903, 6428, 6832, 3316, 6620, 5430, 6542, 395, 6181, 4866, 78, 2283, 1608, 5823, 2594, 688, 5681, 1432, 4578, 3698, 1935, 82, 5986, 2389, 6947, 4666, 2449, 6149, 2703, 3202, 5672, 6537, 710, 2485, 6784, 513, 6061, 1428, 3934, 3994, 5621, 2752, 4741, 1488, 2478, 1203, 3426, 5416, 4675, 997, 4308, 1306, 2035, 4738, 493, 902, 5388, 4975, 7010, 5290, 4556, 525, 5903, 6103, 850, 6218, 5203, 5219, 855, 7040, 38, 3145, 6878, 3951, 1490, 3309, 2061, 6236, 2318, 3050, 5659, 4240, 5884, 6977, 5693, 3525, 5347, 3887, 2242, 4756, 772, 3741, 259, 212, 6826, 4077, 3834, 1552, 2583, 2536, 1439, 2466, 4333, 4279, 6168, 5003, 396, 1747, 3528, 1237, 4563, 5602, 2445, 536, 5642, 1265, 1598, 4876, 3449, 2039, 1275, 5873, 4883, 6634, 2176, 6283, 5057, 133, 2081, 6660, 4941, 995, 3315, 2637, 5832, 407, 2354, 1848, 176, 5493, 5022, 284, 578, 2483, 7000, 5501, 6445, 5297, 1055, 5982, 322, 227, 3444, 4558, 4514, 5471, 3372, 2298, 6677, 4644, 4210, 2853, 6447, 272, 1533, 6266, 6553, 5719, 3976, 4187, 1120, 5508, 6075, 6194, 4183, 4796, 6002, 6123, 4743, 1089, 5212, 360, 4849, 3288, 4093, 4537, 1035, 5010, 842, 851, 2739, 1038, 1959, 2293, 1426, 2738, 4, 4639, 3463, 6325, 269, 849, 1053, 1155, 160, 6636, 6320, 5646, 2868, 1832, 626, 5293, 1224, 970, 4839, 1187, 1819, 1015, 5687, 4476, 1814, 6760, 6776, 1310, 3020, 2657, 404, 1923, 4146, 1933, 5268, 5224, 1482, 3628, 6268, 461, 4660, 1232, 4641, 645, 1517, 4647, 5533, 6917, 4810, 1960, 989, 1823, 5072, 5647, 1329, 4264, 6517, 3751, 4477, 6405, 3598, 233, 2161, 2929, 2294, 2766, 112, 2373, 7059, 915, 2960, 4426, 1838, 4157, 4252, 4042, 6396, 1359, 5319, 4307, 6031, 1872, 6179, 4694, 1481, 2094, 4162, 4181, 6463, 797, 1642, 3565, 6570, 5597, 1004, 2601, 4851, 1354, 3970, 2898, 2851, 6828, 5144, 4581, 4687, 4651, 3983, 1817, 7044, 5473, 1986, 3103, 7052, 4254, 5925, 3507, 4727, 4381, 2116, 586, 220, 4962, 5978, 4560, 964, 6112, 6107, 5356, 4977, 4713, 4126, 3096, 2928, 5550, 3794, 6563, 3896, 6513, 103, 1298, 6578, 6538, 3295, 4486, 5032, 5448, 2982, 3333, 1260, 5619, 399, 6630, 1641, 5728, 3581, 143, 1730, 2230, 3906, 6225, 5260, 4046, 4163, 2980, 2611, 2701, 5771, 1180, 4918, 90, 3929, 636, 1244, 2995, 241, 6993, 4072, 5304, 3849, 3624, 6270, 3577, 5447, 4490, 6514, 3368, 6461, 4035, 3367, 7036, 5164, 5454, 4470, 5187, 2267, 1451, 3603, 3647, 557, 6288, 4007, 5829, 6343, 4371, 2725, 2072, 5584, 2473, 3736, 4882, 7101, 4662, 463, 1639, 3138, 4455, 94, 6820, 3575, 3743, 925, 6154, 6618, 4679, 3049, 4040, 3530, 1005, 4596, 5247, 3467, 1080, 4182, 6037, 1043, 5053, 595, 3960, 6057, 780, 3365, 5232, 6030, 2558, 4261, 6359, 6752, 1615, 1951, 1009, 3430, 6188, 2648, 314, 5723, 1690, 84, 3035, 2106, 2495, 1824, 5969, 3410, 1697, 7007, 3030, 5846, 1540, 4322, 4386, 4829, 1040, 6684, 1800, 4259, 4458, 4089, 1927, 1424, 6872, 4850, 1720, 3580, 2907, 2577, 5775, 6499, 5312, 4949, 4310, 2864, 1050, 1915, 923, 4664, 6431, 3142, 245, 256, 597, 730, 6883, 1462, 5872, 2786, 4334, 2783, 7072, 2728, 3993, 6280, 3178, 316, 1618, 856, 2669, 1052, 237, 2119, 2893, 5369, 5859, 2150, 5655, 1757, 3133, 2778, 652, 5107, 7028, 4546, 4885, 4184, 6189, 238, 5640, 6579, 303, 6789, 4739, 5782, 6732, 6436, 389, 6978, 2411, 230, 6414, 5354, 6769, 4910, 3322, 1874, 6360, 954, 72, 2698, 213, 3429, 2049, 1930, 2945, 1793, 6994, 549, 823, 6013, 4117, 4986, 351, 5272, 3641, 5807, 5166, 2997, 3396, 6571, 1099, 4507, 5174, 1193, 5252, 4445, 846, 2453, 6984], [2560, 1792, 1031, 5130, 4439, 12, 1560, 1570, 4187, 3620, 1576, 2092, 2609, 179, 5173, 5177, 3594, 1598, 3650, 6723, 2628, 3653, 5702, 72, 5707, 6732, 594, 2644, 85, 5817, 5732, 1115, 1116, 3770, 1124, 5222, 616, 6764, 4351, 3184, 4725, 6780, 6270, 2687, 4740, 5769, 5772, 660, 2330, 4254, 6304, 163, 3751, 4264, 6322, 3251, 4609, 1720, 3257, 186, 2235, 1213, 1214, 6857, 1740, 4817, 4309, 1752, 2781, 5855, 6883, 228, 5865, 1262, 3311, 4849, 5364, 2088, 2815, 2304, 774, 1287, 3336, 6413, 782, 1808, 1302, 3354, 1839, 2523, 3878, 2351, 2353, 2866, 4403, 1336, 5948, 2372, 4665, 3896, 2899, 5461, 343, 2395, 3420, 6496, 3937, 4962, 360, 2410, 3133, 6004, 3445, 3959, 1915, 7039, 896, 6529, 66, 1422, 7066, 415, 928, 417, 1955, 2469, 4518, 1962, 2991, 5554, 4537, 3522, 3525, 5575, 6220, 6090, 6093, 6096, 6616, 1497, 1499, 3036, 1957, 2529, 3042, 997, 6118, 4072, 6634, 5614, 1192, 2035, 1014, 507, 2747, 1668, 6642, 4239, 4600, 82, 4713, 3574, 3531, 4420, 3080, 5054, 2008, 2926, 6205, 3826, 6193, 6855, 5803, 6041, 6922, 332, 5997, 2495, 721, 6999, 3360, 3032, 4053, 5367, 67, 851, 1108, 119, 6376, 1553, 3549, 5248, 5311, 6673, 3429, 3710, 2415, 5102, 7118, 6446, 1913, 5843, 679, 1811, 2697, 2240, 1730, 5488, 4156, 2179, 6233, 4260, 304, 2906, 1272, 4872, 2246, 2472, 6661, 2414, 1006, 1827, 5801, 706, 6714, 3463, 3259, 1144, 2862, 1304, 3877, 5397, 1941, 4524, 4576, 1171, 6139, 177, 5354, 1956, 2807, 4108, 925, 7125, 2160, 6294, 4841, 1046, 5900, 3252, 6119, 2695, 3040, 5831, 2187, 2200, 5328, 5259, 905, 4136, 78, 2494, 5879, 1129, 5616, 2333, 4033, 4511, 2347, 6052, 1456, 4952, 484, 2201, 7041, 4607, 3801, 1898, 1464, 2302, 3837, 4555, 4732, 1107, 4577, 4126, 5646, 2938, 4899, 157, 5445, 628, 6045, 6815, 2626, 3471, 6690, 5812, 2802, 6084, 4552, 1055, 5672, 6478, 2144, 33, 761, 5347, 2900, 3, 3472, 6368, 2057, 6830, 5385, 3824, 1509, 5734, 1089, 2958, 4925, 525, 445, 1783, 6319, 4206, 6405, 5278, 5980, 6683, 1318, 6667, 6571, 3059, 4785, 495, 6865, 263, 2771, 5154, 681, 5813, 5965, 5992, 5715, 65, 2679, 1833, 5969, 5599, 820, 840, 4485, 2553, 2081, 6911, 4749, 1423, 1878, 5497, 2518, 6854, 285, 3838, 4111, 4877, 166, 510, 1869, 6361, 2587, 2045, 121, 4927, 2847, 4443, 3347, 2662, 348, 1800, 3644, 6993, 5910, 5391, 1310, 1242, 987, 6572, 3670, 4124, 4619, 3015, 1292, 6750, 5224, 1688, 5563, 5738, 3158, 6497, 1408, 5304, 2591, 6600, 1764, 391, 6032, 6811, 294, 2212, 2581, 35, 4671, 2593, 2868, 7091, 6626, 4878, 6266, 5905, 50, 426, 2790, 1322, 5002, 6409, 3510, 6681, 815, 1361, 1868, 3994, 6208, 3130, 5299, 1583, 2383, 1163, 3694, 2825, 7100, 1203, 4638, 4493, 4695, 6904, 4195, 4533, 3505, 1147, 3367, 3227, 6682, 4196, 3135, 856, 4519, 2285, 2059, 1550, 5269, 4505, 1951, 4954, 3134, 3819, 5952, 6311, 4115, 2166, 6402, 3292, 4928, 217, 1256, 2844, 5610, 6840, 2498, 3373, 3147, 4151, 144, 2716, 5765, 933, 4081, 112, 1815, 6640, 1458, 4982, 1253, 2032, 485, 1200, 4723, 1391, 1388, 2565, 6042, 6622, 3136, 5035, 2666, 4120, 6782, 5456, 738, 3871, 847, 6554, 2572, 3610, 5683, 3049, 6632, 2312, 2204, 6165, 3984, 1802, 6515, 4428, 4860, 6383, 3794, 3589, 1413, 224, 6355, 6558, 317, 1893, 1195, 1472, 1858, 871, 6697, 491, 1258, 3411, 4496, 3897, 1945, 4866, 197, 4112, 331, 6570, 1639, 1700, 6254, 4624, 3519, 1273, 6763, 1432, 3242, 4160, 2976, 966, 6249, 3748, 5475, 1912, 4031, 2576, 5898, 1043, 4244, 1591, 1223, 6500, 5790, 6434, 4216, 4107, 4318, 4501, 5384, 1205, 5928, 5215, 2674, 1397, 5826, 460, 2845, 2740, 1922, 1857, 6672, 4431, 6214, 6066, 826, 1825, 1212, 4442, 4129, 6621, 6958, 5852, 1169, 3735, 2711, 3129, 3692, 1027, 890, 2173, 4572, 2290, 448, 79, 3904, 4146, 4875, 1454, 806, 2889, 2226, 4543, 3586, 3247, 3938, 5085, 215, 6008, 5632, 4413, 4700, 3784, 2933, 3555, 1856, 6569, 4028, 283, 1866, 4338, 514, 6227, 3833, 6288, 1558, 524, 5835, 1411, 2250, 5712, 1332, 954, 1225, 4312, 1953, 1352, 6068, 2530, 4418, 5579, 1123, 4840, 3658, 2468, 2071, 2814, 3654, 4307, 3024, 3397, 6952, 6873, 1679, 1766, 1440, 3533, 1789, 6276, 3805, 3395, 1731, 6465, 90, 6283, 3500, 3541, 2203, 115, 6353, 2377, 427, 7029, 3010, 4818, 1153, 3683, 4542, 5313, 2346, 1646, 1180, 6313, 231, 6108, 3595, 4963, 4984, 2358, 1248, 4277, 2209, 6467, 7045, 5555, 4298, 3690, 4392, 5799, 1113, 1861, 7085, 6643, 1746, 4341, 1787, 6751, 5369, 3087, 3944, 5052, 5398, 2795, 2898, 5964, 9, 559, 2954, 6187, 654, 5203, 697, 3835, 3782, 5073, 6928, 5643, 5343, 961, 3752, 3491, 5998, 5615, 669, 5532, 253, 2041, 2579, 2276, 2336, 1380, 251, 4846, 6614, 5189, 6453, 1796, 5462, 2317, 1079, 349, 2152, 1577, 6716, 6675, 205, 4997, 2683, 5136, 3852, 6957, 6246, 1446, 4917, 1872, 2967, 3725, 2610, 4032, 1137, 3160, 4065, 4668, 3106, 326, 5394, 5891, 3333, 6354, 2618, 1763, 4001, 4285, 901, 1405, 225, 191, 2099, 1015, 1940, 2229, 4380, 5685, 1305, 5440, 2892, 6615, 926, 3076, 3362, 801, 142, 2168, 6612, 5000, 4095, 583, 3219, 3743, 3452, 2681, 5492, 5334, 2337, 1289, 5507, 2574, 2520, 6663, 3192, 277, 4272, 4248, 3501, 5131, 256, 1477, 1281, 2438, 1552, 1224, 4127, 867, 2233, 7003, 973, 6845, 547, 5466, 6480, 5804, 6977, 430, 861, 6418, 6581, 4147, 5236, 2185, 2843, 1162, 1809, 221, 3168, 1855, 1606, 73, 2831, 6608, 6817, 6588, 23, 1052, 7115, 970, 94, 2691, 1306, 148, 6219, 6767, 5557, 2713, 5019, 1592, 5856, 3905, 6885, 3518, 5567, 3737, 5286, 2259, 903, 888, 3236, 3607, 858, 1150, 1538, 4489, 6795, 631, 4812, 3902, 2678, 80, 5877, 2112, 3173, 57, 5851, 2456, 530, 6365, 1603, 6601, 4135, 642, 4207, 4490, 3929, 3446, 7004, 6036, 2189, 3451, 7017, 4810, 1425, 6228, 4876, 315, 2514, 366, 808, 1911, 4906, 3900, 3863, 695, 3478, 6367, 6875, 5330, 1197, 3180, 129, 3386, 5923, 1566, 4246, 1393, 7007, 860, 1619, 3696, 786, 2979, 6950, 2693, 6666, 6445, 3709, 135, 722, 1843, 4229, 456, 823, 3816, 3057, 5584, 2732, 739, 6747, 4487, 4770, 1136, 1672, 2786, 1786, 7034, 5678, 4558, 3140, 2752, 6779, 735], [2752, 7096, 1147, 5321, 4183, 2372, 6108, 165, 4635, 1000, 4440, 4770, 2808, 4248, 4273, 1979, 3667, 3610, 1886, 4412, 1953, 1898, 4041, 6053, 2190, 2913, 2929, 607, 5297, 3693, 4422, 574, 1604, 5728, 1469, 2161, 1684, 6333, 730, 3985, 3017, 1597, 1583, 1099, 3722, 4325, 6987, 2797, 4476, 4533, 5939, 4215, 5993, 483, 4282, 1205, 3825, 2265, 6852, 2423, 1550, 1059, 5584, 2757, 2594, 1361, 6745, 4457, 5858, 4567, 5312, 6901, 2220, 3524, 3042, 2549, 7028, 166, 3957, 6320, 5372, 3950, 6428, 1424, 509, 3152, 2739, 303, 2536, 851, 2678, 1878, 375, 7000, 4810, 2751, 3777, 5750, 1492, 4838, 1286, 5224, 4583, 2485, 3603, 6888, 1874, 6542, 6037, 134, 1717, 20, 5157, 5076, 4398, 2442, 5144, 5740, 2772, 3184, 5914, 5800, 4029, 1535, 2569, 4094, 4339, 1046, 2794, 243, 2008, 4534, 5373, 17, 4652, 2397, 2812, 1856, 5814, 206, 355, 5797, 5349, 6878, 7072, 1338, 1949, 6699, 6530, 5653, 913, 5219, 1602, 3831, 5716, 1129, 3101, 6445, 5134, 549, 3865, 3653, 4653, 1485, 1462, 6176, 1747, 1354, 1675, 1807, 1664, 1070, 164, 6697, 3757, 4146, 6673, 597, 3372, 6918, 1335, 2679, 1419, 5002, 2497, 5824, 6539, 5471, 1553, 871, 5135, 4410, 577, 565, 5293, 1219, 6767, 3161, 2349, 5846, 3626, 2780, 3551, 6342, 6054, 1049, 1155, 3137, 6447, 2693, 1524, 1751, 1014, 2460, 3694, 5185, 6538, 800, 5794, 6621, 6680, 3932, 3427, 3759, 239, 6928, 4101, 3657, 2921, 1801, 2770, 3867, 3638, 935, 2980, 3043, 780, 929, 5544, 4077, 2507, 5496, 2437, 605, 4949, 2999, 1596, 7117, 4842, 2469, 1495, 260, 3471, 1980, 2055, 4760, 5430, 3100, 3295, 3147, 6015, 6168, 4322, 4663, 4026, 2993, 910, 767, 3983, 6923, 6324, 2748, 4131, 387, 457, 4073, 6835, 4641, 3906, 791, 104, 4195, 4837, 847, 4581, 4001, 1996, 5354, 496, 6597, 5501, 2512, 3612, 6228, 4312, 3598, 405, 2860, 2216, 2016, 4271, 3736, 4272, 1869, 6757, 4308, 1910, 1926, 1048, 5242, 3691, 5272, 4111, 2241, 2468, 1316, 6410, 4405, 3201, 6149, 3585, 5607, 1088, 1746, 5440, 872, 1203, 5444, 6355, 1108, 2151, 828, 2004, 4743, 2137, 4458, 4438, 2843, 3890, 3994, 2107, 5529, 4447, 5793, 4, 4472, 6261, 5779, 713, 5702, 5950, 4611, 4933, 4983, 2771, 2383, 2125, 941, 1405, 6591, 2176, 6556, 4376, 389, 4334, 806, 87, 4427, 6143, 2201, 2320, 5415, 4483, 5899, 1043, 6686, 693, 6634, 1905, 857, 6961, 173, 983, 6181, 1797, 2312, 2351, 5164, 6864, 6502, 1840, 3129, 2461, 5876, 4037, 92, 3614, 4777, 6301, 3817, 3029, 2687, 195, 6838, 3948, 514, 5521, 4730, 1748, 3998, 6827, 3549, 1139, 5398, 5003, 5964, 3395, 4727, 602, 4946, 3213, 1790, 2270, 2316, 137, 6002, 4019, 3607, 284, 5839, 108, 830, 1291, 5636, 2362, 3785, 1935, 3315, 5661, 1983, 5467, 6347, 409, 1438, 297, 4477, 4133, 3953, 1273, 2410, 1857, 5022, 3146, 5363, 6189, 5796, 272, 3410, 735, 6414, 398, 1642, 1628, 2597, 3885, 4850, 3436, 2701, 1896, 4568, 6636, 5142, 2030, 1015, 6553, 230, 1456, 135, 5569, 1197, 1792, 1822, 5447, 3263, 1817, 6984, 476, 5497, 6158, 1092, 6633, 6405, 6465, 7015, 3399, 5482, 842, 5445, 2595, 612, 5840, 3665, 6094, 6828, 5252, 5986, 1956, 4313, 6032, 5402, 2702, 1740, 5703, 6233, 1619, 1223, 3297, 3616, 1548, 4660, 1698, 4448, 1902, 560, 652, 3030, 1277, 5897, 334, 5646, 4486, 6277, 5074, 481, 5045, 2997, 2014, 2646, 485, 1594, 4380, 7071, 3628, 1490, 2275, 4330, 5032, 7104, 1873, 5530, 716, 5733, 772, 719, 1699, 4920, 1918, 1232, 2880, 3834, 1930, 4905, 2928, 6334, 3587, 4903, 5172, 1686, 1201, 1771, 1708, 2367, 802, 6742, 6601, 1892, 6439, 556, 4385, 1603, 5300, 4694, 120, 2012, 4287, 2087, 873, 4181, 6193, 3641, 5807, 5775, 550, 91, 6579, 6136, 2961, 6524, 2082, 144, 1847, 5510, 4022, 4363, 4647, 55, 1275, 6948, 6028, 613, 5431, 3343, 4406, 2884, 6561, 5590, 4735, 5247, 3417, 2618, 2669, 3532, 4172, 702, 6769, 5102, 6756, 4093, 7, 1612, 2204, 1614, 3542, 4754, 3156, 3790, 997, 7013, 601, 3314, 4679, 96, 5722, 158, 3209, 5422, 4040, 1955, 4226, 5637, 3191, 6124, 6558, 5179, 4254, 1811, 6795, 3525, 6841, 674, 2774, 4571, 5873, 2526, 5174, 5133, 4970, 6705, 758, 131, 6156, 6799, 1613, 6369, 3737, 3921, 5508, 4989, 3203, 1390, 3546, 1888, 5229, 6854, 1214, 5580, 6185, 3550, 6817, 1933, 5367, 1113, 4830, 1415, 6356, 1587, 5647, 3173, 6596, 3698, 1618, 404, 424, 1782, 6046, 4709, 1631, 2135, 850, 3707, 2683, 2430, 739, 6092, 5220, 849, 2298, 726, 1215, 4756, 5953, 5340, 3038, 1281, 3883, 7088, 1757, 4177, 846, 7078, 4302, 1829, 7086, 3493, 6458, 793, 1124, 5451, 251, 4148, 1592, 4866, 6422, 4043, 2741, 5571, 5617, 1784, 705, 4923, 98, 3577, 500, 1192, 6083, 4630, 4578, 6537, 6274, 4722, 3220, 3887, 1683, 5338, 337, 6386, 6091, 2850, 1348, 6768, 2443, 2238, 6920, 4539, 7047, 2853, 2593, 7043, 3611, 1202, 3661, 3, 3366, 1893, 6965, 6616, 732, 4420, 4333, 5516, 4276, 2168, 524, 1921, 1730, 5238, 902, 3789, 924, 2463, 7037, 1510, 6162, 1168, 2864, 3520, 2340, 1872, 2773, 1360, 5582, 3398, 5700, 3796, 176, 4031, 6533, 6517, 3062, 7123, 1985, 6581, 4713, 3830, 4147, 5827, 2916, 498, 5604, 914, 5212, 6563, 5307, 6215, 5663, 5434, 1616, 4634, 2601, 5816, 4996, 2382, 3530, 1989, 79, 3446, 6626, 4299, 6113, 2885, 44, 3824, 7017, 5903, 4171, 5187, 2807, 3128, 3081, 1053, 1187, 4305, 380, 5342, 2511, 4020, 912, 5203, 2520, 4235, 3303, 4252, 3361, 5167, 2375, 583, 6963, 5148, 2058, 2114, 1089, 586, 2069, 6192, 3269, 1696, 2388, 3414, 3900, 1796, 4701, 1352, 1880, 4556, 2487, 3154, 3120, 5782, 1447, 3321, 81, 450, 171, 4117, 291, 4238, 3624, 542, 6196, 936, 5550, 4163, 5472, 6205, 3058, 3448, 3059, 2575, 74, 3970, 5454, 6033, 2673, 648, 5823, 5933, 5721, 6669, 269, 2345, 2954, 3678, 4666, 1133, 2648, 5460, 6464, 1373, 5125, 3420, 2219, 2389, 1303, 5734, 31, 4178, 4622, 5888, 2893, 897, 6902, 5925, 5937, 4968, 3109, 3781, 4564, 6716, 4675, 2728, 5289, 2722, 9, 213, 1418, 3001, 6344, 4975, 4230, 3094, 3134, 493, 4244, 4929, 5577, 5487, 2025, 3584, 2943, 6360, 750, 4205, 4664, 5319, 5627, 2574, 1971, 6515, 6290, 6080, 2268, 4358, 3430, 5288, 5418, 4461, 160, 6423, 3229, 2878], [4096, 2049, 3670, 3079, 6664, 9, 5130, 2571, 2353, 2063, 16, 529, 2579, 1557, 3094, 495, 4633, 6682, 4126, 2081, 4644, 3445, 552, 1065, 6699, 2094, 1629, 1584, 5169, 1587, 3126, 5689, 6716, 3133, 3649, 4162, 3140, 69, 583, 1097, 4171, 6732, 4687, 5200, 1618, 595, 1108, 4182, 2136, 6233, 3674, 5723, 4026, 3168, 3170, 3172, 4966, 2663, 6760, 618, 1131, 3180, 3694, 624, 6770, 6266, 6783, 4224, 4738, 1155, 4231, 6280, 2698, 6283, 533, 6288, 6290, 6291, 5780, 1902, 3735, 3225, 1690, 2715, 2718, 3743, 6818, 4259, 4262, 5799, 5288, 6450, 4210, 185, 3258, 4796, 4554, 1215, 6340, 200, 1225, 1740, 888, 210, 5848, 1758, 3295, 2273, 5347, 1253, 1258, 6379, 4851, 2294, 3831, 4856, 5884, 3838, 1793, 771, 3844, 6917, 3337, 778, 3202, 3885, 1809, 4882, 4883, 3352, 3865, 2842, 2334, 2335, 1953, 1827, 2340, 806, 295, 3373, 6960, 5937, 2354, 2868, 309, 4918, 1336, 2874, 1930, 6977, 522, 5259, 2568, 840, 2473, 330, 2380, 2894, 6993, 2900, 2901, 2902, 3929, 2688, 348, 3934, 4325, 3427, 7014, 2961, 4968, 1387, 1390, 4463, 7026, 5493, 6008, 1913, 7036, 6013, 2430, 3460, 6465, 4490, 1931, 66, 3565, 7057, 3395, 6297, 5528, 3993, 923, 6556, 3489, 1955, 5640, 6044, 2119, 430, 1459, 4533, 438, 1976, 954, 1979, 2492, 3517, 6749, 1985, 1986, 3147, 1788, 3527, 3528, 4172, 4042, 3020, 2510, 1488, 6819, 2005, 473, 3034, 3035, 5693, 992, 995, 2532, 5606, 6396, 4824, 3154, 6639, 498, 4596, 5621, 3575, 153, 5114, 508, 4760, 4610, 4001, 3151, 2276, 2133, 2225, 6816, 1032, 3756, 1048, 1172, 4014, 5051, 2563, 6768, 1541, 3905, 6293, 1544, 1343, 5561, 5784, 5859, 3582, 219, 2466, 1899, 6201, 1652, 6883, 2060, 6397, 2327, 2296, 802, 5578, 6584, 2131, 3318, 5339, 4604, 2281, 5135, 1054, 780, 1641, 57, 5370, 6202, 6452, 5854, 395, 936, 4612, 6723, 5344, 6649, 4193, 1106, 4378, 6273, 5103, 5279, 1517, 2277, 5725, 4558, 5144, 2630, 5112, 7054, 6821, 5341, 5547, 345, 3426, 6154, 1801, 6735, 114, 1757, 2461, 3960, 3602, 2845, 3632, 589, 3758, 5793, 2323, 1174, 5533, 2233, 4486, 256, 5602, 6671, 4684, 2653, 3171, 807, 4630, 6719, 1522, 5322, 4962, 5866, 2033, 6539, 6648, 966, 2583, 1952, 1608, 264, 670, 136, 922, 5503, 530, 2700, 1787, 2930, 6968, 3585, 5174, 3386, 3789, 3113, 3645, 1974, 1601, 6227, 713, 5448, 2946, 5771, 6652, 17, 396, 6841, 893, 632, 1384, 6792, 402, 2535, 4277, 6194, 6678, 6684, 4938, 5510, 1797, 5352, 6794, 4470, 4421, 2519, 1947, 3751, 4383, 5700, 78, 2242, 4044, 3762, 3788, 4949, 5154, 1873, 1084, 4469, 7012, 5698, 6739, 2230, 4279, 6882, 7104, 2147, 1251, 2020, 112, 1203, 797, 4574, 2748, 5088, 4445, 1709, 1330, 6495, 6609, 1837, 688, 217, 1383, 5006, 1999, 5146, 6705, 1548, 3230, 2697, 1479, 1838, 4225, 4124, 2534, 2449, 2411, 4420, 6369, 6511, 1881, 3542, 3675, 6766, 2217, 7055, 151, 2980, 2591, 4275, 5525, 4160, 5604, 5591, 2261, 1193, 1171, 3794, 3165, 4064, 1435, 1589, 1893, 2679, 525, 1701, 3103, 3909, 3239, 4483, 2963, 5797, 2533, 2991, 3277, 2304, 5915, 7077, 3545, 2622, 2445, 2564, 5950, 1720, 5893, 20, 2146, 6112, 6745, 4201, 4339, 5829, 715, 1263, 6832, 3236, 6464, 2997, 4515, 4371, 6721, 2984, 4184, 7101, 2292, 3845, 5941, 617, 4102, 2831, 4021, 5922, 4995, 1671, 4599, 3253, 2969, 6978, 6776, 526, 1370, 3119, 320, 4804, 1624, 4568, 5655, 2145, 316, 657, 6505, 582, 260, 5434, 4613, 5508, 5942, 4095, 2753, 5974, 3032, 2684, 209, 1432, 5443, 6759, 3610, 2515, 5046, 3248, 1487, 3138, 739, 94, 2682, 4200, 6106, 5679, 6975, 5348, 6752, 7007, 323, 5597, 2002, 5436, 7089, 2503, 3334, 3106, 3741, 7071, 830, 4813, 1780, 4627, 4876, 1028, 514, 7038, 3344, 157, 2216, 3824, 1179, 2313, 220, 1462, 2125, 5868, 5740, 2851, 2202, 1668, 2263, 4560, 195, 3000, 5355, 4307, 5566, 3781, 3379, 105, 3340, 405, 5992, 1592, 2206, 3266, 3793, 2605, 6520, 351, 6085, 634, 5025, 3153, 6755, 5903, 6990, 4316, 5440, 6974, 4758, 3394, 6474, 2045, 5524, 1766, 6860, 1649, 4622, 2781, 6563, 6031, 5798, 2039, 3600, 1052, 2703, 4429, 5813, 855, 3278, 4481, 5364, 4189, 4874, 2400, 5281, 1796, 3501, 4775, 3484, 7042, 2838, 3485, 6232, 87, 2478, 2783, 6102, 2898, 6296, 1779, 3397, 4638, 6139, 1038, 1262, 2695, 4054, 3883, 5203, 3672, 6630, 4904, 2813, 4115, 3447, 6853, 6533, 4704, 1832, 183, 5977, 1814, 4502, 3330, 7009, 2573, 4727, 1972, 2308, 1260, 2805, 133, 5478, 5737, 5759, 2817, 5356, 6807, 3529, 6030, 2865, 2541, 2960, 1208, 5381, 477, 238, 11, 4196, 6849, 5827, 3235, 1946, 3402, 2829, 687, 5083, 6236, 298, 6430, 4034, 3689, 3038, 594, 885, 4261, 36, 1693, 6943, 6549, 2124, 6820, 2167, 3226, 6903, 5559, 397, 4664, 660, 6771, 6680, 5498, 4917, 422, 4815, 5800, 2778, 5489, 4048, 5417, 1639, 5304, 1926, 4573, 3083, 4739, 6569, 5273, 3881, 228, 2590, 4452, 1408, 2558, 5715, 647, 5948, 2776, 4629, 1042, 3634, 569, 3396, 1590, 7035, 3936, 4584, 2652, 1849, 3368, 455, 4912, 6801, 1439, 5228, 566, 4133, 707, 578, 218, 1784, 6734, 3152, 2293, 222, 3399, 1851, 6618, 847, 6057, 5177, 5886, 4105, 1951, 4983, 1014, 1842, 1552, 1585, 6803, 3859, 6246, 6855, 5837, 5189, 5208, 5563, 4543, 2741, 5283, 5681, 6351, 5419, 1137, 1848, 1050, 858, 1686, 2687, 1168, 485, 5627, 2008, 1626, 1538, 1822, 6837, 4511, 5346, 708, 2657, 3994, 4976, 6014, 3092, 4402, 1161, 1489, 2625, 6184, 5935, 4056, 100, 2482, 4157, 2834, 1227, 3259, 2160, 1670, 2728, 2786, 3828, 5968, 4461, 1917, 4892, 3560, 5232, 2467, 4538, 5151, 3784, 5359, 2507, 6390, 3023, 2177, 4023, 6225, 4799, 3142, 3118, 3283, 5772, 3288, 62, 524, 2725, 3884, 5851, 5938, 3619, 4849, 1099, 6147, 3731, 2139, 925, 4496, 570, 212, 1765, 2017, 1222, 2452, 645, 3017, 5925, 6463, 675, 6683, 4007, 1091, 4808, 2789, 1781, 2252, 2945, 179, 126, 3463, 568, 6461, 6091, 3706, 805, 2116, 300, 6547, 419, 1008, 3948, 2399, 6219, 4347, 5986, 6093, 1316, 2514, 4741, 4046, 4662, 516, 2011, 128, 4066, 2760, 2639, 3913, 5719, 2983, 3691, 4310, 0, 346, 6866, 1371, 305, 1329, 7048, 1937, 1428, 1880, 3232, 3563, 727, 3476, 1023, 5815, 4580, 588, 5917, 6090, 1039, 1816, 303], [3076, 6666, 2571, 5136, 529, 2579, 2564, 5147, 5154, 2083, 3620, 6187, 4141, 4145, 3638, 1080, 5177, 3133, 1804, 4172, 2641, 3154, 85, 1114, 3683, 1130, 6763, 6253, 623, 5745, 5746, 1143, 3192, 6266, 6782, 2689, 5772, 2189, 6291, 3735, 4207, 6819, 164, 1192, 171, 6770, 687, 2740, 1740, 1225, 1215, 708, 713, 6860, 6352, 721, 5330, 547, 212, 6874, 4829, 4817, 1258, 210, 1263, 2290, 4856, 6395, 252, 253, 3845, 4876, 4878, 2837, 2330, 4380, 798, 1827, 295, 4904, 5419, 5937, 4918, 6969, 3386, 3899, 4490, 6975, 6465, 1861, 5958, 4426, 6712, 850, 5462, 2394, 348, 3934, 3427, 3429, 2939, 6474, 394, 2445, 2958, 1423, 1426, 403, 1940, 6556, 5533, 6558, 5025, 2715, 1350, 936, 431, 1459, 5046, 3514, 5053, 1985, 6085, 6732, 1493, 2532, 2533, 5606, 337, 2544, 3575, 2045, 7025, 911, 5992, 2173, 214, 2039, 2037, 5257, 3684, 105, 1701, 4096, 5260, 479, 6014, 6080, 1994, 6787, 3477, 521, 6391, 6683, 2779, 6974, 2151, 1641, 6311, 3226, 5499, 3531, 1669, 5382, 4046, 5016, 556, 4095, 501, 5013, 6767, 185, 3334, 1033, 2983, 24, 2294, 2607, 7007, 6816, 6971, 1590, 1930, 2771, 1155, 2563, 2334, 4804, 754, 3366, 13, 1490, 1193, 241, 1065, 4026, 5748, 2415, 4945, 873, 6619, 495, 3423, 6198, 4519, 3692, 2167, 2815, 251, 557, 1420, 5347, 3460, 2874, 3476, 6212, 4379, 1469, 2841, 1377, 5030, 5227, 112, 4143, 5151, 1400, 5498, 3338, 3051, 2353, 1129, 208, 4689, 793, 2379, 4537, 5698, 5119, 6280, 3259, 5050, 2969, 4639, 3372, 892, 1982, 1587, 5799, 6202, 1814, 2892, 5591, 4196, 3189, 5354, 4147, 4808, 6473, 2399, 1592, 6228, 5, 6093, 6811, 566, 4189, 503, 3902, 5445, 9, 330, 6366, 3312, 1837, 6180, 525, 2008, 4797, 5004, 865, 1209, 1009, 5602, 6705, 1373, 4209, 5038, 78, 3258, 310, 1784, 472, 179, 1186, 1690, 16, 2082, 475, 5434, 6851, 6235, 1872, 2603, 238, 2711, 2642, 5872, 3401, 954, 2667, 3368, 1815, 4042, 1218, 5809, 1110, 674, 5825, 4293, 4906, 5616, 3350, 4160, 4072, 1947, 4496, 2876, 3628, 522, 3786, 3173, 1571, 1984, 4708, 3751, 6197, 4455, 3033, 5346, 6699, 3527, 1796, 3505, 6751, 4035, 6297, 3000, 7014, 3953, 5470, 2005, 3017, 3700, 6644, 6229, 1318, 2351, 4299, 2617, 2209, 3219, 324, 578, 3943, 3395, 3828, 1550, 5297, 1589, 3439, 2813, 5764, 1729, 3340, 396, 4619, 3373, 4327, 2481, 3627, 4392, 1720, 5858, 5125, 4254, 7071, 5714, 4336, 1044, 5909, 5456, 4752, 5938, 4321, 4581, 5451, 2838, 5248, 5884, 2663, 3300, 5626, 5866, 3734, 1222, 4857, 230, 778, 2329, 3885, 5529, 5418, 165, 1801, 5239, 4495, 6568, 309, 3976, 5547, 5455, 814, 2118, 1787, 6351, 1252, 343, 3937, 417, 2152, 1467, 473, 7089, 2104, 5910, 2545, 5200, 2361, 5672, 6090, 5839, 5435, 928, 5508, 5695, 3651, 5545, 1206, 1052, 96, 6430, 6820, 2646, 6464, 746, 5388, 1986, 3935, 331, 2549, 5269, 6182, 2592, 4534, 2882, 4255, 6322, 2335, 2331, 657, 3106, 2492, 6509, 3264, 1213, 3936, 4652, 133, 2052, 4644, 5116, 2491, 1244, 3508, 3322, 5947, 2679, 3094, 504, 1288, 2786, 4976, 2878, 4154, 2155, 5377, 2551, 4395, 966, 90, 6148, 1896, 4667, 2202, 2980, 5438, 1336, 616, 5493, 4304, 1916, 6734, 1529, 156, 4421, 3788, 6327, 3374, 264, 6967, 7026, 4133, 1682, 582, 4007, 2631, 1353, 1431, 510, 2778, 6569, 3180, 4965, 1351, 1771, 6780, 855, 6185, 967, 6855, 4716, 5976, 6514, 821, 493, 2409, 2727, 5370, 6402, 6887, 5903, 6046, 1658, 913, 4259, 3710, 4657, 2094, 7122, 6302, 5917, 3554, 1224, 6529, 6295, 2780, 2206, 6853, 3501, 3549, 6698, 2365, 1482, 1979, 3905, 6946, 1362, 4853, 2473, 62, 2375, 1153, 5619, 17, 1219, 7115, 3282, 2568, 4885, 2408, 6740, 5528, 4322, 5394, 3127, 1330, 2781, 4037, 594, 6783, 4749, 2086, 1623, 6651, 380, 6563, 3884, 4745, 3552, 6390, 847, 3842, 3737, 2596, 345, 6553, 167, 6749, 1489, 3008, 2468, 3948, 3489, 6379, 6681, 2887, 6748, 5758, 5599, 1629, 7090, 3515, 232, 4300, 4223, 2588, 6061, 6800, 1961, 6760, 4535, 4420, 1268, 6682, 995, 4554, 3224, 4846, 5460, 2511, 33, 5784, 6220, 4021, 5073, 2028, 5826, 1028, 774, 5124, 4381, 1471, 5736, 4417, 2344, 4825, 263, 1624, 4460, 3029, 219, 3805, 1356, 3674, 6717, 2640, 6331, 4530, 4729, 5655, 2716, 3774, 1877, 2317, 3425, 2230, 4958, 4764, 3288, 1451, 3867, 790, 4445, 2494, 5810, 4504, 4827, 256, 2707, 1522, 4256, 3061, 4773, 2898, 2315, 6098, 6575, 6686, 6170, 5195, 3480, 3780, 6109, 5339, 4210, 5273, 3191, 7048, 3295, 4622, 4582, 1755, 429, 6937, 2870, 2868, 1053, 4833, 5731, 4912, 4740, 1233, 283, 4251, 926, 7003, 3838, 5466, 2700, 1679, 538, 7015, 4493, 4977, 3294, 2340, 3573, 3410, 3032, 2591, 3353, 6660, 1704, 6330, 5921, 4921, 3854, 4775, 3649, 4439, 1462, 3436, 5864, 860, 196, 2766, 5160, 4468, 2704, 4701, 4529, 31, 5010, 4126, 884, 1933, 1844, 5675, 7038, 2347, 1633, 5288, 5367, 5081, 1816, 6316, 4730, 6159, 1174, 5169, 5566, 4903, 69, 803, 2081, 2119, 2059, 4262, 4763, 3337, 735, 6238, 1635, 3525, 2638, 906, 1256, 6806, 327, 3994, 750, 6290, 6130, 1383, 3670, 3881, 1591, 4511, 5255, 5768, 5178, 3462, 5812, 3466, 5717, 1097, 1311, 1387, 6764, 2456, 4318, 5205, 4723, 7004, 5080, 5057, 202, 128, 7077, 6072, 2027, 2495, 1054, 2373, 3170, 1795, 1306, 2790, 5223, 792, 1035, 1721, 6662, 3685, 2110, 2313, 569, 753, 6547, 2277, 3001, 6902, 2756, 880, 6818, 6269, 3960, 577, 4726, 2011, 5735, 3507, 4960, 4463, 1800, 5222, 2076, 2991, 4261, 4771, 3846, 6392, 367, 4743, 883, 5833, 157, 956, 3836, 1873, 3125, 3831, 3202, 3002, 4609, 2565, 6481, 2383, 3675, 3929, 7109, 2131, 3619, 6950, 673, 4863, 4345, 2578, 1008, 6128, 964, 6723, 4538, 552, 2698, 2901, 1247, 583, 1672, 3804, 6042, 5562, 6545, 5718, 6839, 4102, 4171, 5681, 3019, 5066, 4397, 3698, 2894, 2753, 2570, 3759, 536, 3517, 4201, 5338, 2244, 6978, 2938, 1445, 4231, 2510, 5835, 1088, 3852, 3239, 6817, 4197, 1838, 1869, 1487, 6726, 4852, 5364, 3079, 1313, 3475, 247, 1823, 5978, 237, 64, 5374, 4056, 4125, 6588, 7001, 5214, 1583, 355, 6030, 3590, 3320, 2926, 993, 6008, 3344, 677, 4732, 3779, 5837, 5640, 4182, 6033, 2117, 6643, 729, 1376, 5848, 2916, 2088, 2425, 1751, 4115, 3565, 4001], [3076, 6666, 2571, 5136, 529, 2579, 2564, 5147, 5154, 2083, 3620, 6187, 4141, 4145, 3638, 1080, 5177, 3133, 1804, 4172, 2641, 3154, 85, 1114, 3683, 1130, 6763, 6253, 623, 5745, 5746, 1143, 3192, 6266, 6782, 2689, 5772, 2189, 6291, 3735, 4207, 6819, 164, 1192, 171, 6770, 687, 2740, 1740, 1225, 1215, 708, 713, 6860, 6352, 721, 5330, 547, 212, 6874, 4829, 4817, 1258, 210, 1263, 2290, 4856, 6395, 252, 253, 3845, 4876, 4878, 2837, 2330, 4380, 798, 1827, 295, 4904, 5419, 5937, 4918, 6969, 3386, 3899, 4490, 6975, 6465, 1861, 5958, 4426, 6712, 850, 5462, 2394, 348, 3934, 3427, 3429, 2939, 6474, 394, 2445, 2958, 1423, 1426, 403, 1940, 6556, 5533, 6558, 5025, 2715, 1350, 936, 431, 1459, 5046, 3514, 5053, 1985, 6085, 6732, 1493, 2532, 2533, 5606, 337, 2544, 3575, 2045, 7025, 911, 5992, 2173, 214, 2039, 2037, 5257, 3684, 105, 1701, 4096, 5260, 479, 6014, 6080, 1994, 6787, 3477, 521, 6391, 6683, 2779, 6974, 2151, 1641, 6311, 3226, 5499, 3531, 1669, 5382, 4046, 5016, 556, 4095, 501, 5013, 6767, 185, 3334, 1033, 2983, 24, 2294, 2607, 7007, 6816, 6971, 1590, 1930, 2771, 1155, 2563, 2334, 4804, 754, 3366, 13, 1490, 1193, 241, 1065, 4026, 5748, 2415, 4945, 873, 6619, 495, 3423, 6198, 4519, 3692, 2167, 2815, 251, 557, 1420, 5347, 3460, 2874, 3476, 6212, 4379, 1469, 2841, 1377, 5030, 5227, 112, 4143, 5151, 1400, 5498, 3338, 3051, 2353, 1129, 208, 4689, 793, 2379, 4537, 5698, 5119, 6280, 3259, 5050, 2969, 4639, 3372, 892, 1982, 1587, 5799, 6202, 1814, 2892, 5591, 4196, 3189, 5354, 4147, 4808, 6473, 2399, 1592, 6228, 5, 6093, 6811, 566, 4189, 503, 3902, 5445, 9, 330, 6366, 3312, 1837, 6180, 525, 2008, 4797, 5004, 865, 1209, 1009, 5602, 6705, 1373, 4209, 5038, 78, 3258, 310, 1784, 472, 179, 1186, 1690, 16, 2082, 475, 5434, 6851, 6235, 1872, 2603, 238, 2711, 2642, 5872, 3401, 954, 2667, 3368, 1815, 4042, 1218, 5809, 1110, 674, 5825, 4293, 4906, 5616, 3350, 4160, 4072, 1947, 4496, 2876, 3628, 522, 3786, 3173, 1571, 1984, 4708, 3751, 6197, 4455, 3033, 5346, 6699, 3527, 1796, 3505, 6751, 4035, 6297, 3000, 7014, 3953, 5470, 2005, 3017, 3700, 6644, 6229, 1318, 2351, 4299, 2617, 2209, 3219, 324, 578, 3943, 3395, 3828, 1550, 5297, 1589, 3439, 2813, 5764, 1729, 3340, 396, 4619, 3373, 4327, 2481, 3627, 4392, 1720, 5858, 5125, 4254, 7071, 5714, 4336, 1044, 5909, 5456, 4752, 5938, 4321, 4581, 5451, 2838, 5248, 5884, 2663, 3300, 5626, 5866, 3734, 1222, 4857, 230, 778, 2329, 3885, 5529, 5418, 165, 1801, 5239, 4495, 6568, 309, 3976, 5547, 5455, 814, 2118, 1787, 6351, 1252, 343, 3937, 417, 2152, 1467, 473, 7089, 2104, 5910, 2545, 5200, 2361, 5672, 6090, 5839, 5435, 928, 5508, 5695, 3651, 5545, 1206, 1052, 96, 6430, 6820, 2646, 6464, 746, 5388, 1986, 3935, 331, 2549, 5269, 6182, 2592, 4534, 2882, 4255, 6322, 2335, 2331, 657, 3106, 2492, 6509, 3264, 1213, 3936, 4652, 133, 2052, 4644, 5116, 2491, 1244, 3508, 3322, 5947, 2679, 3094, 504, 1288, 2786, 4976, 2878, 4154, 2155, 5377, 2551, 4395, 966, 90, 6148, 1896, 4667, 2202, 2980, 5438, 1336, 616, 5493, 4304, 1916, 6734, 1529, 156, 4421, 3788, 6327, 3374, 264, 6967, 7026, 4133, 1682, 582, 4007, 2631, 1353, 1431, 510, 2778, 6569, 3180, 4965, 1351, 1771, 6780, 855, 6185, 967, 6855, 4716, 5976, 6514, 821, 493, 2409, 2727, 5370, 6402, 6887, 5903, 6046, 1658, 913, 4259, 3710, 4657, 2094, 7122, 6302, 5917, 3554, 1224, 6529, 6295, 2780, 2206, 6853, 3501, 3549, 6698, 2365, 1482, 1979, 3905, 6946, 1362, 4853, 2473, 62, 2375, 1153, 5619, 17, 1219, 7115, 3282, 2568, 4885, 2408, 6740, 5528, 4322, 5394, 3127, 1330, 2781, 4037, 594, 6783, 4749, 2086, 1623, 6651, 380, 6563, 3884, 4745, 3552, 6390, 847, 3842, 3737, 2596, 345, 6553, 167, 6749, 1489, 3008, 2468, 3948, 3489, 6379, 6681, 2887, 6748, 5758, 5599, 1629, 7090, 3515, 232, 4300, 4223, 2588, 6061, 6800, 1961, 6760, 4535, 4420, 1268, 6682, 995, 4554, 3224, 4846, 5460, 2511, 33, 5784, 6220, 4021, 5073, 2028, 5826, 1028, 774, 5124, 4381, 1471, 5736, 4417, 2344, 4825, 263, 1624, 4460, 3029, 219, 3805, 1356, 3674, 6717, 2640, 6331, 4530, 4729, 5655, 2716, 3774, 1877, 2317, 3425, 2230, 4958, 4764, 3288, 1451, 3867, 790, 4445, 2494, 5810, 4504, 4827, 256, 2707, 1522, 4256, 3061, 4773, 2898, 2315, 6098, 6575, 6686, 6170, 5195, 3480, 3780, 6109, 5339, 4210, 5273, 3191, 7048, 3295, 4622, 4582, 1755, 429, 6937, 2870, 2868, 1053, 4833, 5731, 4912, 4740, 1233, 283, 4251, 926, 7003, 3838, 5466, 2700, 1679, 538, 7015, 4493, 4977, 3294, 2340, 3573, 3410, 3032, 2591, 3353, 6660, 1704, 6330, 5921, 4921, 3854, 4775, 3649, 4439, 1462, 3436, 5864, 860, 196, 2766, 5160, 4468, 2704, 4701, 4529, 31, 5010, 4126, 884, 1933, 1844, 5675, 7038, 2347, 1633, 5288, 5367, 5081, 1816, 6316, 4730, 6159, 1174, 5169, 5566, 4903, 69, 803, 2081, 2119, 2059, 4262, 4763, 3337, 735, 6238, 1635, 3525, 2638, 906, 1256, 6806, 327, 3994, 750, 6290, 6130, 1383, 3670, 3881, 1591, 4511, 5255, 5768, 5178, 3462, 5812, 3466, 5717, 1097, 1311, 1387, 6764, 2456, 4318, 5205, 4723, 7004, 5080, 5057, 202, 128, 7077, 6072, 2027, 2495, 1054, 2373, 3170, 1795, 1306, 2790, 5223, 792, 1035, 1721, 6662, 3685, 2110, 2313, 569, 753, 6547, 2277, 3001, 6902, 2756, 880, 6818, 6269, 3960, 577, 4726, 2011, 5735, 3507, 4960, 4463, 1800, 5222, 2076, 2991, 4261, 4771, 3846, 6392, 367, 4743, 883, 5833, 157, 956, 3836, 1873, 3125, 3831, 3202, 3002, 4609, 2565, 6481, 2383, 3675, 3929, 7109, 2131, 3619, 6950, 673, 4863, 4345, 2578, 1008, 6128, 964, 6723, 4538, 552, 2698, 2901, 1247, 583, 1672, 3804, 6042, 5562, 6545, 5718, 6839, 4102, 4171, 5681, 3019, 5066, 4397, 3698, 2894, 2753, 2570, 3759, 536, 3517, 4201, 5338, 2244, 6978, 2938, 1445, 4231, 2510, 5835, 1088, 3852, 3239, 6817, 4197, 1838, 1869, 1487, 6726, 4852, 5364, 3079, 1313, 3475, 247, 1823, 5978, 237, 64, 5374, 4056, 4125, 6588, 7001, 5214, 1583, 355, 6030, 3590, 3320, 2926, 993, 6008, 3344, 677, 4732, 3779, 5837, 5640, 4182, 6033, 2117, 6643, 729, 1376, 5848, 2916, 2088, 2425, 1751, 4115, 3565, 4001], [4096, 6666, 2563, 1110, 6662, 3079, 9, 522, 2571, 525, 1550, 16, 4115, 3094, 5655, 536, 6682, 4133, 4126, 2591, 3106, 547, 1740, 4519, 552, 6185, 4775, 2094, 4445, 5681, 1587, 1591, 1592, 5177, 3133, 4554, 3649, 578, 1035, 583, 4876, 4171, 5200, 5714, 6228, 1129, 3670, 90, 6749, 6238, 3685, 3170, 4196, 3173, 2663, 1641, 6763, 6764, 6770, 5698, 3189, 1143, 3192, 6266, 5248, 2689, 3202, 4740, 790, 1672, 3948, 5772, 2189, 6290, 3735, 5784, 2244, 2202, 6811, 156, 6816, 2209, 6818, 6820, 1701, 5799, 1192, 1193, 4210, 3288, 179, 2740, 2230, 713, 1720, 31, 6090, 1215, 3531, 708, 6855, 4808, 1225, 4300, 5837, 2766, 4304, 2771, 4046, 4201, 5848, 6874, 1244, 2938, 1247, 5347, 6128, 1258, 2088, 5364, 2294, 1065, 1784, 3322, 5884, 253, 2815, 2347, 774, 503, 1800, 3337, 778, 3852, 3885, 3180, 4885, 1814, 4381, 2335, 3692, 1827, 3575, 1318, 295, 3368, 5418, 5419, 6569, 3373, 4912, 2353, 4147, 2868, 309, 1336, 2874, 3899, 6465, 4420, 2375, 5260, 330, 2379, 1801, 2894, 1080, 2898, 569, 3929, 5978, 348, 5434, 4958, 2399, 5947, 3429, 7014, 4455, 873, 1387, 4460, 2926, 4463, 5493, 3960, 5498, 5508, 1837, 4490, 5004, 2445, 6030, 911, 1097, 6545, 6723, 5273, 6556, 1690, 6558, 3489, 2715, 4007, 2716, 4856, 2991, 2277, 4535, 3001, 4026, 1979, 2492, 4426, 5057, 1986, 3017, 4042, 3019, 2510, 1487, 1489, 2005, 3032, 5081, 6860, 3549, 165, 995, 6481, 2027, 3565, 954, 5616, 5288, 6643, 2983, 2039, 2045, 510, 85, 2837, 5764, 6977, 4633, 4853, 4995, 355, 2869, 4035, 4143, 5829, 1260, 5584, 1056, 3452, 6227, 6755, 1805, 4574, 5166, 2168, 323, 6950, 4439, 368, 5205, 2819, 5268, 752, 6705, 4833, 3484, 1052, 4328, 3162, 4336, 577, 5033, 6595, 3846, 5438, 264, 6033, 431, 4883, 2878, 6281, 82, 1781, 6712, 2063, 671, 4421, 6229, 5330, 2592, 3793, 5992, 5780, 4797, 834, 5343, 5010, 2704, 3057, 1517, 1154, 5304, 4774, 4251, 5455, 3780, 3401, 367, 4933, 5066, 6642, 3930, 883, 1675, 3278, 2753, 3507, 3842, 380, 892, 2642, 5080, 4386, 808, 6678, 6957, 4001, 4990, 1994, 2989, 5917, 41, 3334, 133, 3477, 6391, 7101, 675, 4125, 1388, 4992, 1267, 5169, 3033, 5735, 4511, 6379, 6085, 4665, 508, 1589, 2838, 1629, 6450, 4945, 3995, 5567, 3937, 4804, 6686, 5214, 5820, 2194, 624, 5154, 5534, 4817, 2241, 4624, 3480, 324, 556, 6351, 5717, 926, 195, 4207, 6660, 4261, 2216, 4044, 4613, 814, 5208, 485, 4725, 2411, 729, 1379, 3423, 3181, 4223, 1946, 1426, 673, 4413, 5334, 3616, 5822, 6075, 7045, 2865, 1972, 588, 5050, 1451, 5514, 4745, 3620, 2995, 5909, 3476, 5838, 3257, 238, 1848, 6282, 2133, 6803, 6061, 297, 462, 4904, 3838, 3844, 2980, 1493, 7001, 3054, 5554, 6461, 6699, 1878, 5032, 3366, 5626, 2605, 6588, 2503, 6819, 3448, 5147, 1646, 6402, 4255, 5969, 3824, 6344, 4533, 4310, 2916, 3696, 2817, 274, 2758, 3582, 4690, 2049, 4926, 1623, 3191, 1, 415, 3295, 472, 6974, 2551, 3634, 3816, 4986, 5864, 846, 6554, 251, 2549, 4417, 7003, 4459, 2579, 6837, 5958, 6071, 5672, 4054, 2456, 4267, 1669, 2117, 7090, 1444, 364, 6098, 5934, 6269, 7009, 3475, 5124, 2392, 521, 5591, 4184, 4827, 3128, 283, 5812, 5948, 4820, 4214, 1467, 6511, 2290, 4541, 2315, 3460, 3993, 1289, 2638, 557, 1329, 3786, 6374, 1033, 3781, 6574, 245, 2507, 3554, 6197, 1155, 6395, 4796, 1652, 1890, 1721, 1202, 4187, 464, 5369, 2780, 2425, 6978, 688, 2038, 2727, 5382, 4667, 4530, 3734, 404, 3239, 5619, 1633, 5835, 3833, 6568, 4197, 4299, 2509, 2489, 4635, 1474, 6288, 2180, 4018, 6648, 616, 4318, 3374, 6745, 4687, 1028, 2167, 343, 2110, 4102, 3420, 936, 1370, 6698, 4453, 825, 6070, 7068, 1462, 4743, 4534, 2495, 1356, 1586, 2718, 3915, 5354, 3969, 3596, 1915, 1005, 5399, 5606, 430, 2394, 1877, 6113, 5053, 2152, 1585, 6732, 1377, 6154, 5833, 5462, 2204, 1839, 5810, 5819, 1913, 501, 5975, 2975, 2735, 3951, 1896, 6182, 4481, 4136, 2365, 2802, 2116, 2361, 5222, 398, 1253, 2748, 6664, 394, 331, 3700, 6193, 4277, 1183, 5116, 3651, 1569, 5839, 5614, 2751, 4069, 5463, 4451, 1108, 2607, 4338, 396, 5338, 5816, 750, 145, 855, 3508, 3691, 5390, 6644, 337, 4701, 221, 2870, 5038, 7089, 4285, 3316, 5370, 5748, 3675, 4392, 3976, 4157, 6780, 1256, 1206, 2565, 105, 5746, 1069, 4468, 4657, 3277, 1947, 3226, 3505, 3236, 3574, 2166, 5215, 4418, 4581, 273, 3436, 1427, 351, 4708, 4231, 3689, 1985, 1961, 6883, 1054, 6845, 5281, 6566, 4568, 5825, 3125, 3398, 3683, 363, 686, 438, 1168, 5826, 2871, 806, 1553, 4203, 4064, 3386, 5223, 5473, 62, 1352, 2603, 2483, 300, 1755, 2570, 1219, 4429, 3350, 2781, 3130, 6684, 5146, 28, 1350, 4037, 5258, 3936, 6143, 3160, 6882, 3818, 1456, 6776, 4758, 4256, 4127, 3172, 4437, 1916, 884, 2617, 2707, 1423, 2409, 803, 2558, 3845, 6255, 913, 1390, 5921, 5804, 1984, 3883, 4913, 5344, 6388, 3265, 2059, 5533, 6783, 4395, 1982, 4868, 6198, 1008, 2457, 1497, 5777, 6667, 4663, 6734, 5827, 3779, 4355, 3944, 2176, 1931, 2596, 6910, 6870, 4072, 128, 3258, 4729, 880, 6064, 3061, 3153, 2494, 850, 4732, 4622, 2778, 1679, 3446, 6782, 2011, 2814, 6046, 3684, 4279, 66, 6295, 1228, 2646, 925, 3352, 5872, 3397, 5160, 5866, 2958, 5547, 6851, 3372, 5851, 5736, 6826, 6575, 956, 1930, 1583, 6302, 6509, 7040, 1940, 860, 499, 3552, 4589, 2969, 112, 2924, 1311, 2285, 2533, 5602, 7078, 4849, 5259, 2037, 3774, 1153, 3300, 212, 5910, 2621, 247, 3525, 6008, 6495, 479, 1177, 6473, 5302, 6596, 100, 2640, 3076, 1461, 4345, 96, 6093, 4538, 5236, 5465, 6711, 4505, 3051, 171, 6624, 2283, 280, 6717, 2544, 4983, 2430, 2020, 6651, 6114, 4596, 6767, 3905, 2776, 4432, 3002, 6618, 2002, 4655, 4609, 6866, 3122, 3000, 6042, 5695, 2564, 3751, 2467, 5640, 2455, 4260, 1214, 397, 6233, 2851, 5577, 1454, 1869, 1288, 6690, 1383, 6731, 7093, 1134, 3402, 3931, 5359, 4825, 1861, 3142, 2076, 1704, 5809, 4496, 739, 3765, 4782, 2876, 4325, 1856, 3095, 2014, 2545, 5883, 2900, 4771, 5683, 2741, 3353, 5692, 2118, 7044, 6614, 167, 1471, 6908, 7079, 4822, 3658, 3720, 29, 4316, 2173, 6044, 3392, 6004, 6859, 6072, 6438, 6316, 6080, 5195, 4921, 1263, 2862, 4512, 5976, 2984, 4436, 3427, 4543, 2351, 5858, 3632], [2752, 7096, 1147, 5321, 4183, 2372, 6108, 165, 4635, 1000, 4440, 4770, 2808, 4248, 4273, 1979, 3667, 3610, 1886, 4412, 1953, 1898, 4041, 6053, 2190, 2913, 2929, 607, 5297, 3693, 4422, 574, 1604, 5728, 1469, 2161, 1684, 6333, 730, 3985, 3017, 1597, 1583, 1099, 3722, 4325, 6987, 2797, 4476, 4533, 5939, 4215, 5993, 483, 4282, 1205, 3825, 2265, 6852, 2423, 1550, 1059, 5584, 2757, 2594, 1361, 6745, 4457, 5858, 4567, 5312, 6901, 2220, 3524, 3042, 2549, 7028, 166, 3957, 6320, 5372, 3950, 6428, 1424, 509, 3152, 2739, 303, 2536, 851, 2678, 1878, 375, 7000, 4810, 2751, 3777, 5750, 1492, 4838, 1286, 5224, 4583, 2485, 3603, 6888, 1874, 6542, 6037, 134, 1717, 20, 5157, 5076, 4398, 2442, 5144, 5740, 2772, 3184, 5914, 5800, 4029, 1535, 2569, 4094, 4339, 1046, 2794, 243, 2008, 4534, 5373, 17, 4652, 2397, 2812, 1856, 5814, 206, 355, 5797, 5349, 6878, 7072, 1338, 1949, 6699, 6530, 5653, 913, 5219, 1602, 3831, 5716, 1129, 3101, 6445, 5134, 549, 3865, 3653, 4653, 1485, 1462, 6176, 1747, 1354, 1675, 1807, 1664, 1070, 164, 6697, 3757, 4146, 6673, 597, 3372, 6918, 1335, 2679, 1419, 5002, 2497, 5824, 6539, 5471, 1553, 871, 5135, 4410, 577, 565, 5293, 1219, 6767, 3161, 2349, 5846, 3626, 2780, 3551, 6342, 6054, 1049, 1155, 3137, 6447, 2693, 1524, 1751, 1014, 2460, 3694, 5185, 6538, 800, 5794, 6621, 6680, 3932, 3427, 3759, 239, 6928, 4101, 3657, 2921, 1801, 2770, 3867, 3638, 935, 2980, 3043, 780, 929, 5544, 4077, 2507, 5496, 2437, 605, 4949, 2999, 1596, 7117, 4842, 2469, 1495, 260, 3471, 1980, 2055, 4760, 5430, 3100, 3295, 3147, 6015, 6168, 4322, 4663, 4026, 2993, 910, 767, 3983, 6923, 6324, 2748, 4131, 387, 457, 4073, 6835, 4641, 3906, 791, 104, 4195, 4837, 847, 4581, 4001, 1996, 5354, 496, 6597, 5501, 2512, 3612, 6228, 4312, 3598, 405, 2860, 2216, 2016, 4271, 3736, 4272, 1869, 6757, 4308, 1910, 1926, 1048, 5242, 3691, 5272, 4111, 2241, 2468, 1316, 6410, 4405, 3201, 6149, 3585, 5607, 1088, 1746, 5440, 872, 1203, 5444, 6355, 1108, 2151, 828, 2004, 4743, 2137, 4458, 4438, 2843, 3890, 3994, 2107, 5529, 4447, 5793, 4, 4472, 6261, 5779, 713, 5702, 5950, 4611, 4933, 4983, 2771, 2383, 2125, 941, 1405, 6591, 2176, 6556, 4376, 389, 4334, 806, 87, 4427, 6143, 2201, 2320, 5415, 4483, 5899, 1043, 6686, 693, 6634, 1905, 857, 6961, 173, 983, 6181, 1797, 2312, 2351, 5164, 6864, 6502, 1840, 3129, 2461, 5876, 4037, 92, 3614, 4777, 6301, 3817, 3029, 2687, 195, 6838, 3948, 514, 5521, 4730, 1748, 3998, 6827, 3549, 1139, 5398, 5003, 5964, 3395, 4727, 602, 4946, 3213, 1790, 2270, 2316, 137, 6002, 4019, 3607, 284, 5839, 108, 830, 1291, 5636, 2362, 3785, 1935, 3315, 5661, 1983, 5467, 6347, 409, 1438, 297, 4477, 4133, 3953, 1273, 2410, 1857, 5022, 3146, 5363, 6189, 5796, 272, 3410, 735, 6414, 398, 1642, 1628, 2597, 3885, 4850, 3436, 2701, 1896, 4568, 6636, 5142, 2030, 1015, 6553, 230, 1456, 135, 5569, 1197, 1792, 1822, 5447, 3263, 1817, 6984, 476, 5497, 6158, 1092, 6633, 6405, 6465, 7015, 3399, 5482, 842, 5445, 2595, 612, 5840, 3665, 6094, 6828, 5252, 5986, 1956, 4313, 6032, 5402, 2702, 1740, 5703, 6233, 1619, 1223, 3297, 3616, 1548, 4660, 1698, 4448, 1902, 560, 652, 3030, 1277, 5897, 334, 5646, 4486, 6277, 5074, 481, 5045, 2997, 2014, 2646, 485, 1594, 4380, 7071, 3628, 1490, 2275, 4330, 5032, 7104, 1873, 5530, 716, 5733, 772, 719, 1699, 4920, 1918, 1232, 2880, 3834, 1930, 4905, 2928, 6334, 3587, 4903, 5172, 1686, 1201, 1771, 1708, 2367, 802, 6742, 6601, 1892, 6439, 556, 4385, 1603, 5300, 4694, 120, 2012, 4287, 2087, 873, 4181, 6193, 3641, 5807, 5775, 550, 91, 6579, 6136, 2961, 6524, 2082, 144, 1847, 5510, 4022, 4363, 4647, 55, 1275, 6948, 6028, 613, 5431, 3343, 4406, 2884, 6561, 5590, 4735, 5247, 3417, 2618, 2669, 3532, 4172, 702, 6769, 5102, 6756, 4093, 7, 1612, 2204, 1614, 3542, 4754, 3156, 3790, 997, 7013, 601, 3314, 4679, 96, 5722, 158, 3209, 5422, 4040, 1955, 4226, 5637, 3191, 6124, 6558, 5179, 4254, 1811, 6795, 3525, 6841, 674, 2774, 4571, 5873, 2526, 5174, 5133, 4970, 6705, 758, 131, 6156, 6799, 1613, 6369, 3737, 3921, 5508, 4989, 3203, 1390, 3546, 1888, 5229, 6854, 1214, 5580, 6185, 3550, 6817, 1933, 5367, 1113, 4830, 1415, 6356, 1587, 5647, 3173, 6596, 3698, 1618, 404, 424, 1782, 6046, 4709, 1631, 2135, 850, 3707, 2683, 2430, 739, 6092, 5220, 849, 2298, 726, 1215, 4756, 5953, 5340, 3038, 1281, 3883, 7088, 1757, 4177, 846, 7078, 4302, 1829, 7086, 3493, 6458, 793, 1124, 5451, 251, 4148, 1592, 4866, 6422, 4043, 2741, 5571, 5617, 1784, 705, 4923, 98, 3577, 500, 1192, 6083, 4630, 4578, 6537, 6274, 4722, 3220, 3887, 1683, 5338, 337, 6386, 6091, 2850, 1348, 6768, 2443, 2238, 6920, 4539, 7047, 2853, 2593, 7043, 3611, 1202, 3661, 3, 3366, 1893, 6965, 6616, 732, 4420, 4333, 5516, 4276, 2168, 524, 1921, 1730, 5238, 902, 3789, 924, 2463, 7037, 1510, 6162, 1168, 2864, 3520, 2340, 1872, 2773, 1360, 5582, 3398, 5700, 3796, 176, 4031, 6533, 6517, 3062, 7123, 1985, 6581, 4713, 3830, 4147, 5827, 2916, 498, 5604, 914, 5212, 6563, 5307, 6215, 5663, 5434, 1616, 4634, 2601, 5816, 4241, 2382, 3530, 1989, 79, 3446, 6626, 4299, 6113, 2885, 44, 3824, 7017, 5903, 4171, 5187, 2807, 3128, 3081, 1053, 1187, 4305, 380, 5342, 2511, 4020, 912, 5203, 2520, 4235, 3303, 4252, 3361, 5167, 2375, 583, 6963, 5148, 2058, 2114, 1089, 586, 2069, 6192, 3269, 1696, 2388, 3414, 3900, 1796, 4701, 1352, 1880, 4556, 2487, 3154, 3120, 5782, 1447, 3321, 81, 450, 171, 4117, 291, 4238, 3624, 542, 6196, 936, 5550, 4163, 5472, 6205, 3058, 3448, 3059, 2575, 74, 3970, 5454, 6033, 2673, 648, 5823, 5933, 5721, 6669, 269, 2345, 2954, 3678, 4666, 1133, 2648, 5460, 6464, 1373, 5125, 3420, 2219, 2389, 1303, 5734, 31, 4178, 4622, 5888, 2893, 897, 6902, 5925, 5937, 4968, 3109, 3781, 4564, 6716, 4675, 2728, 5289, 2722, 9, 213, 1418, 3001, 6344, 4975, 4230, 3094, 3134, 493, 4244, 4929, 5577, 5487, 2025, 3584, 2943, 6360, 750, 4205, 4664, 5319, 5627, 2574, 1971, 6515, 6290, 6080, 2268, 4358, 3430, 5288, 5418, 4461, 160, 6423, 3229, 2878], [2752, 7096, 1147, 5321, 4183, 2372, 6108, 165, 4635, 1000, 4440, 4770, 2808, 4248, 4273, 1979, 3667, 3610, 1886, 4412, 1953, 1898, 4041, 6053, 2190, 2913, 2929, 607, 5297, 3693, 4422, 574, 1604, 5728, 1469, 2161, 1684, 6333, 730, 3985, 3017, 1597, 1583, 1099, 3722, 4325, 6987, 2797, 4476, 4533, 5939, 4215, 5993, 483, 4282, 1205, 3825, 2265, 6852, 2423, 1550, 1059, 5584, 2757, 2594, 1361, 6745, 4457, 5858, 4567, 5312, 6901, 2220, 3524, 3042, 2549, 7028, 166, 3957, 6320, 5372, 3950, 6428, 1424, 509, 3152, 2739, 303, 2536, 851, 2678, 1878, 375, 7000, 4810, 2751, 3777, 5750, 1492, 4838, 1286, 5224, 4583, 2485, 3603, 6888, 1874, 6542, 6037, 134, 1717, 20, 5157, 5076, 4398, 2442, 5144, 5740, 2772, 3184, 5914, 5800, 4029, 1535, 2569, 4094, 4339, 1046, 2794, 243, 2008, 4534, 5373, 17, 4652, 2397, 2812, 1856, 5814, 206, 355, 5797, 5349, 6878, 7072, 1338, 1949, 6699, 6530, 5653, 913, 5219, 1602, 3831, 5716, 1129, 3101, 6445, 5134, 549, 3865, 3653, 4653, 1485, 1462, 6176, 1747, 1354, 1675, 1807, 1664, 1070, 164, 6697, 3757, 4146, 6673, 597, 3372, 6918, 1335, 2679, 1419, 5002, 2497, 5824, 6539, 5471, 1553, 871, 5135, 4410, 577, 565, 5293, 1219, 6767, 3161, 2349, 5846, 3626, 2780, 3551, 6342, 6054, 1049, 1155, 3137, 6447, 2693, 1524, 1751, 1014, 2460, 3694, 5185, 6538, 800, 5794, 6621, 6680, 3932, 3427, 3759, 239, 6928, 4101, 3657, 2921, 1801, 2770, 3867, 3638, 935, 2980, 3043, 780, 929, 5544, 4077, 2507, 5496, 2437, 605, 4949, 2999, 1596, 7117, 4842, 2469, 1495, 260, 3471, 1980, 2055, 4760, 5430, 3100, 3295, 3147, 6015, 6168, 4322, 4663, 4026, 2993, 910, 767, 3983, 6923, 6324, 2748, 4131, 387, 457, 4073, 6835, 4641, 3906, 791, 104, 4195, 4837, 847, 4581, 4001, 1996, 5354, 496, 6597, 5501, 2512, 3612, 6228, 4312, 3598, 405, 2860, 2216, 2016, 4271, 3736, 4272, 1869, 6757, 4308, 1910, 1926, 1048, 5242, 3691, 5272, 4111, 2241, 2468, 1316, 6410, 4405, 3201, 6149, 3585, 5607, 1088, 1746, 5440, 872, 1203, 5444, 6355, 1108, 2151, 828, 2004, 4743, 2137, 4458, 4438, 2843, 3890, 3994, 2107, 5529, 4447, 5793, 4, 4472, 6261, 5779, 713, 5702, 5950, 4611, 4933, 4983, 2771, 2383, 2125, 941, 1405, 6591, 2176, 6556, 4376, 389, 4334, 806, 87, 4427, 6143, 2201, 2320, 5415, 4483, 5899, 1043, 6686, 693, 6634, 1905, 857, 6961, 173, 983, 6181, 1797, 2312, 2351, 5164, 6864, 6502, 1840, 3129, 2461, 5876, 4037, 92, 3614, 4777, 6301, 3817, 3029, 2687, 195, 6838, 3948, 514, 5521, 4730, 1748, 3998, 6827, 3549, 1139, 5398, 5003, 5964, 3395, 4727, 602, 4946, 3213, 1790, 2270, 2316, 137, 6002, 4019, 3607, 284, 5839, 108, 830, 1291, 5636, 2362, 3785, 1935, 3315, 5661, 1983, 5467, 6347, 409, 1438, 297, 4477, 4133, 3953, 1273, 2410, 1857, 5022, 3146, 5363, 6189, 5796, 272, 3410, 735, 6414, 398, 1642, 1628, 2597, 3885, 4850, 3436, 2701, 1896, 4568, 6636, 5142, 2030, 1015, 6553, 230, 1456, 135, 5569, 1197, 1792, 1822, 5447, 3263, 1817, 6984, 476, 5497, 6158, 1092, 6633, 6405, 6465, 7015, 3399, 5482, 842, 5445, 2595, 612, 5840, 3665, 6094, 6828, 5252, 5986, 1956, 4313, 6032, 5402, 2702, 1740, 5703, 6233, 1619, 1223, 3297, 3616, 1548, 4660, 1698, 4448, 1902, 560, 652, 3030, 1277, 5897, 334, 5646, 4486, 6277, 5074, 481, 5045, 2997, 2014, 2646, 485, 1594, 4380, 7071, 3628, 1490, 2275, 4330, 5032, 7104, 1873, 5530, 716, 5733, 772, 719, 1699, 4920, 1918, 1232, 2880, 3834, 1930, 4905, 2928, 6334, 3587, 4903, 5172, 1686, 1201, 1771, 1708, 2367, 802, 6742, 6601, 1892, 6439, 556, 4385, 1603, 5300, 4694, 120, 2012, 4287, 2087, 873, 4181, 6193, 3641, 5807, 5775, 550, 91, 6579, 6136, 2961, 6524, 2082, 144, 1847, 5510, 4022, 4363, 4647, 55, 1275, 6948, 6028, 613, 5431, 3343, 4406, 2884, 6561, 5590, 4735, 5247, 3417, 2618, 2669, 3532, 4172, 702, 6769, 5102, 6756, 4093, 7, 1612, 2204, 1614, 3542, 4754, 3156, 3790, 997, 7013, 601, 3314, 4679, 96, 5722, 158, 3209, 5422, 4040, 1955, 4226, 5637, 3191, 6124, 6558, 5179, 4254, 1811, 6795, 3525, 6841, 674, 2774, 4571, 5873, 2526, 5174, 5133, 4970, 6705, 758, 131, 6156, 6799, 1613, 6369, 3737, 3921, 5508, 4989, 3203, 1390, 3546, 1888, 5229, 6854, 1214, 5580, 6185, 3550, 6817, 1933, 5367, 1113, 4830, 1415, 6356, 1587, 5647, 3173, 6596, 3698, 1618, 404, 424, 1782, 6046, 4709, 1631, 2135, 850, 3707, 2683, 2430, 739, 6092, 5220, 849, 2298, 726, 1215, 4756, 5953, 5340, 3038, 1281, 3883, 7088, 1757, 4177, 846, 7078, 4302, 1829, 7086, 3493, 6458, 793, 1124, 5451, 251, 4148, 1592, 4866, 6422, 4043, 2741, 5571, 5617, 1784, 705, 4923, 98, 3577, 500, 1192, 6083, 4630, 4578, 6537, 6274, 4722, 3220, 3887, 1683, 5338, 337, 6386, 6091, 2850, 1348, 6768, 2443, 2238, 6920, 4539, 7047, 2853, 2593, 7043, 3611, 1202, 3661, 3, 3366, 1893, 6965, 6616, 732, 4420, 4333, 5516, 4276, 2168, 524, 1921, 1730, 5238, 902, 3789, 924, 2463, 7037, 1510, 6162, 1168, 2864, 3520, 2340, 1872, 2773, 1360, 5582, 3398, 5700, 3796, 176, 4031, 6533, 6517, 3062, 7123, 1985, 6581, 4713, 3830, 4147, 5827, 2916, 498, 5604, 914, 5212, 6563, 5307, 6215, 5663, 5434, 1616, 4634, 2601, 5816, 4241, 2382, 3530, 1989, 79, 3446, 6626, 4299, 6113, 2885, 44, 3824, 7017, 5903, 4171, 5187, 2807, 3128, 3081, 1053, 1187, 4305, 380, 5342, 2511, 4020, 912, 5203, 2520, 4235, 3303, 4252, 3361, 5167, 2375, 583, 6963, 5148, 2058, 2114, 1089, 586, 2069, 6192, 3269, 1696, 2388, 3414, 3900, 1796, 4701, 1352, 1880, 4556, 2487, 3154, 3120, 5782, 1447, 3321, 81, 450, 171, 4117, 291, 4238, 3624, 542, 6196, 936, 5550, 4163, 5472, 6205, 3058, 3448, 3059, 2575, 74, 3970, 5454, 6033, 2673, 648, 5823, 5933, 5721, 6669, 269, 2345, 2954, 3678, 4666, 1133, 2648, 5460, 6464, 1373, 5125, 3420, 2219, 2389, 1303, 5734, 31, 4178, 4622, 5888, 2893, 897, 6902, 5925, 5937, 4968, 3109, 3781, 4564, 6716, 4675, 2728, 5289, 2722, 9, 213, 1418, 3001, 6344, 4975, 4230, 3094, 3134, 493, 4244, 4929, 5577, 5487, 2025, 3584, 2943, 6360, 750, 4205, 4664, 5319, 5627, 2574, 1971, 6515, 6290, 6080, 2268, 4358, 3430, 5288, 5418, 4461, 160, 6423, 3229, 2878], [2752, 7096, 1147, 5321, 4183, 2372, 6108, 165, 4635, 1000, 4440, 4770, 2808, 4248, 4273, 1979, 3667, 3610, 1886, 4412, 1953, 1898, 4041, 6053, 2190, 2913, 2929, 607, 5297, 3693, 4422, 574, 1604, 5728, 1469, 2161, 1684, 6333, 730, 3985, 3017, 1597, 1583, 1099, 3722, 4325, 6987, 2797, 4476, 4533, 5939, 4215, 5993, 483, 4282, 1205, 3825, 2265, 6852, 2423, 1550, 1059, 5584, 2757, 2594, 1361, 6745, 4457, 5858, 4567, 5312, 6901, 2220, 3524, 3042, 2549, 7028, 166, 3957, 6320, 5372, 3950, 6428, 1424, 509, 3152, 2739, 303, 2536, 851, 2678, 1878, 375, 7000, 4810, 2751, 3777, 5750, 1492, 4838, 1286, 5224, 4583, 2485, 3603, 6888, 1874, 6542, 6037, 134, 1717, 20, 5157, 5076, 4398, 2442, 5144, 5740, 2772, 3184, 5914, 5800, 4029, 1535, 2569, 4094, 4339, 1046, 2794, 243, 2008, 4534, 5373, 17, 4652, 2397, 2812, 1856, 5814, 206, 355, 5797, 5349, 6878, 7072, 1338, 1949, 6699, 6530, 5653, 913, 5219, 1602, 3831, 5716, 1129, 3101, 6445, 5134, 549, 3865, 3653, 4653, 1485, 1462, 6176, 1747, 1354, 1675, 1807, 1664, 1070, 164, 6697, 3757, 4146, 6673, 597, 3372, 6918, 1335, 2679, 1419, 5002, 2497, 5824, 6539, 5471, 1553, 871, 5135, 4410, 577, 565, 5293, 1219, 6767, 3161, 2349, 5846, 3626, 2780, 3551, 6342, 6054, 1049, 1155, 3137, 6447, 2693, 1524, 1751, 1014, 2460, 3694, 5185, 6538, 800, 5794, 6621, 6680, 3932, 3427, 3759, 239, 6928, 4101, 3657, 2921, 1801, 2770, 3867, 3638, 935, 2980, 3043, 780, 929, 5544, 4077, 2507, 5496, 2437, 605, 4949, 2999, 1596, 7117, 4842, 2469, 1495, 260, 3471, 1980, 2055, 4760, 5430, 3100, 3295, 3147, 6015, 6168, 4322, 4663, 4026, 2993, 910, 767, 3983, 6923, 6324, 2748, 4131, 387, 457, 4073, 6835, 4641, 3906, 791, 104, 4195, 4837, 847, 4581, 4001, 1996, 5354, 496, 6597, 5501, 2512, 3612, 6228, 4312, 3598, 405, 2860, 2216, 2016, 4271, 3736, 4272, 1869, 6757, 4308, 1910, 1926, 1048, 5242, 3691, 5272, 4111, 2241, 2468, 1316, 6410, 4405, 3201, 6149, 3585, 5607, 1088, 1746, 5440, 872, 1203, 5444, 6355, 1108, 2151, 828, 2004, 4743, 2137, 4458, 4438, 2843, 3890, 3994, 2107, 5529, 4447, 5793, 4, 4472, 6261, 5779, 713, 5702, 5950, 4611, 4933, 4983, 2771, 2383, 2125, 941, 1405, 6591, 2176, 6556, 4376, 389, 4334, 806, 87, 4427, 6143, 2201, 2320, 5415, 4483, 5899, 1043, 6686, 693, 6634, 1905, 857, 6961, 173, 983, 6181, 1797, 2312, 2351, 5164, 6864, 6502, 1840, 3129, 2461, 5876, 4037, 92, 3614, 4777, 6301, 3817, 3029, 2687, 195, 6838, 3948, 514, 5521, 4730, 1748, 3998, 6827, 3549, 1139, 5398, 5003, 5964, 3395, 4727, 602, 4946, 3213, 1790, 2270, 2316, 137, 6002, 4019, 3607, 284, 5839, 108, 830, 1291, 5636, 2362, 3785, 1935, 3315, 5661, 1983, 5467, 6347, 409, 1438, 297, 4477, 4133, 3953, 1273, 2410, 1857, 5022, 3146, 5363, 6189, 5796, 272, 3410, 735, 6414, 398, 1642, 1628, 2597, 3885, 4850, 3436, 2701, 1896, 4568, 6636, 5142, 2030, 1015, 6553, 230, 1456, 135, 5569, 1197, 1792, 1822, 5447, 3263, 1817, 6984, 476, 5497, 6158, 1092, 6633, 6405, 6465, 7015, 3399, 5482, 842, 5445, 2595, 612, 5840, 3665, 6094, 6828, 5252, 5986, 1956, 4313, 6032, 5402, 2702, 1740, 5703, 6233, 1619, 1223, 3297, 3616, 1548, 4660, 1698, 4448, 1902, 560, 652, 3030, 1277, 5897, 334, 5646, 4486, 6277, 5074, 481, 5045, 2997, 2014, 2646, 485, 1594, 4380, 7071, 3628, 1490, 2275, 4330, 5032, 7104, 1873, 5530, 716, 5733, 772, 719, 1699, 4920, 1918, 1232, 2880, 3834, 1930, 4905, 2928, 6334, 3587, 4903, 5172, 1686, 1201, 1771, 1708, 2367, 802, 6742, 6601, 1892, 6439, 556, 4385, 1603, 5300, 4694, 120, 2012, 4287, 2087, 873, 4181, 6193, 3641, 5807, 5775, 550, 91, 6579, 6136, 2961, 6524, 2082, 144, 1847, 5510, 4022, 4363, 4647, 55, 1275, 6948, 6028, 613, 5431, 3343, 4406, 2884, 6561, 5590, 4735, 5247, 3417, 2618, 2669, 3532, 4172, 702, 6769, 5102, 6756, 4093, 7, 1612, 2204, 1614, 3542, 4754, 3156, 3790, 997, 7013, 601, 3314, 4679, 96, 5722, 158, 3209, 5422, 4040, 1955, 4226, 5637, 3191, 6124, 6558, 5179, 4254, 1811, 6795, 3525, 6841, 674, 2774, 4571, 5873, 2526, 5174, 5133, 4970, 6705, 758, 131, 6156, 6799, 1613, 6369, 3737, 3921, 5508, 4989, 3203, 1390, 3546, 1888, 5229, 6854, 1214, 5580, 6185, 3550, 6817, 1933, 5367, 1113, 4830, 1415, 6356, 1587, 5647, 3173, 6596, 3698, 1618, 404, 424, 1782, 6046, 4709, 1631, 2135, 850, 3707, 2683, 2430, 739, 6092, 5220, 849, 2298, 726, 1215, 4756, 5953, 5340, 3038, 1281, 3883, 7088, 1757, 4177, 846, 7078, 4302, 1829, 7086, 3493, 6458, 793, 1124, 5451, 251, 4148, 1592, 4866, 6422, 4043, 2741, 5571, 5617, 1784, 705, 4923, 98, 3577, 500, 1192, 6083, 4630, 4578, 6537, 6274, 4722, 3220, 3887, 1683, 5338, 337, 6386, 6091, 2850, 1348, 6768, 2443, 2238, 6920, 4539, 7047, 2853, 2593, 7043, 3611, 1202, 3661, 3, 3366, 1893, 6965, 6616, 732, 4420, 4333, 5516, 4276, 2168, 524, 1921, 1730, 5238, 902, 3789, 924, 2463, 7037, 1510, 6162, 1168, 2864, 3520, 2340, 1872, 2773, 1360, 5582, 3398, 5700, 3796, 176, 4031, 6533, 6517, 3062, 7123, 1985, 6581, 4713, 3830, 4147, 5827, 2916, 498, 5604, 914, 5212, 6563, 5307, 6215, 5663, 5434, 1616, 4634, 2601, 5816, 4241, 2382, 3530, 1989, 79, 3446, 6626, 4299, 6113, 2885, 44, 3824, 7017, 5903, 4171, 5187, 2807, 3128, 3081, 1053, 1187, 4305, 380, 5342, 2511, 4020, 912, 5203, 2520, 4235, 3303, 4252, 3361, 5167, 2375, 583, 6963, 5148, 2058, 2114, 1089, 586, 2069, 6192, 3269, 1696, 2388, 3414, 3900, 1796, 4701, 1352, 1880, 4556, 2487, 3154, 3120, 5782, 1447, 3321, 81, 450, 171, 4117, 291, 4238, 3624, 542, 6196, 936, 5550, 4163, 5472, 6205, 3058, 3448, 3059, 2575, 74, 3970, 5454, 6033, 2673, 648, 5823, 5933, 5721, 6669, 269, 2345, 2954, 3678, 4666, 1133, 2648, 5460, 6464, 1373, 5125, 3420, 2219, 2389, 1303, 5734, 31, 4178, 4622, 5888, 2893, 897, 6902, 5925, 5937, 4968, 3109, 3781, 4564, 6716, 4675, 2728, 5289, 2722, 9, 213, 1418, 3001, 6344, 4975, 4230, 3094, 3134, 493, 4244, 4929, 5577, 5487, 2025, 3584, 2943, 6360, 750, 4205, 4664, 5319, 5627, 2574, 1971, 6515, 6290, 6080, 2268, 4358, 3430, 5288, 5418, 4461, 160, 6423, 3229, 2878], [2752, 7096, 1147, 5321, 4183, 2372, 6108, 165, 4635, 1000, 4440, 4770, 2808, 4248, 4273, 1979, 3667, 3610, 1886, 4412, 1953, 1898, 4041, 6053, 2190, 2913, 2929, 607, 5297, 3693, 4422, 574, 1604, 5728, 1469, 2161, 1684, 6333, 730, 3985, 3017, 1597, 1583, 1099, 3722, 4325, 6987, 2797, 4476, 4533, 5939, 4215, 5993, 483, 4282, 1205, 3825, 2265, 6852, 2423, 1550, 1059, 5584, 2757, 2594, 1361, 6745, 4457, 5858, 4567, 5312, 6901, 2220, 3524, 3042, 2549, 7028, 166, 3957, 6320, 5372, 3950, 6428, 1424, 509, 3152, 2739, 303, 2536, 851, 2678, 1878, 375, 7000, 4810, 2751, 3777, 5750, 1492, 4838, 1286, 5224, 4583, 2485, 3603, 6888, 1874, 6542, 6037, 134, 1717, 20, 5157, 5076, 4398, 2442, 5144, 5740, 2772, 3184, 5914, 5800, 4029, 1535, 2569, 4094, 4339, 1046, 2794, 243, 2008, 4534, 5373, 17, 4652, 2397, 2812, 1856, 5814, 206, 355, 5797, 5349, 6878, 7072, 1338, 1949, 6699, 6530, 5653, 913, 5219, 1602, 3831, 5716, 1129, 3101, 6445, 5134, 549, 3865, 3653, 4653, 1485, 1462, 6176, 1747, 1354, 1675, 1807, 1664, 1070, 164, 6697, 3757, 4146, 6673, 597, 3372, 6918, 1335, 2679, 1419, 5002, 2497, 5824, 6539, 5471, 1553, 871, 5135, 4410, 577, 565, 5293, 1219, 6767, 3161, 2349, 5846, 3626, 2780, 3551, 6342, 6054, 1049, 1155, 3137, 6447, 2693, 1524, 1751, 1014, 2460, 3694, 5185, 6538, 800, 5794, 6621, 6680, 3932, 3427, 3759, 239, 6928, 4101, 3657, 2921, 1801, 2770, 3867, 3638, 935, 2980, 3043, 780, 929, 5544, 4077, 2507, 5496, 2437, 605, 4949, 2999, 1596, 7117, 4842, 2469, 1495, 260, 3471, 1980, 2055, 4760, 5430, 3100, 3295, 3147, 6015, 6168, 4322, 4663, 4026, 2993, 910, 767, 3983, 6923, 6324, 2748, 4131, 387, 457, 4073, 6835, 4641, 3906, 791, 104, 4195, 4837, 847, 4581, 4001, 1996, 5354, 496, 6597, 5501, 2512, 3612, 6228, 4312, 3598, 405, 2860, 2216, 2016, 4271, 3736, 4272, 1869, 6757, 4308, 1910, 1926, 1048, 5242, 3691, 5272, 4111, 2241, 2468, 1316, 6410, 4405, 3201, 6149, 3585, 5607, 1088, 1746, 5440, 872, 1203, 5444, 6355, 1108, 2151, 828, 2004, 4743, 2137, 4458, 4438, 2843, 3890, 3994, 2107, 5529, 4447, 5793, 4, 4472, 6261, 5779, 713, 5702, 5950, 4611, 4933, 4983, 2771, 2383, 2125, 941, 1405, 6591, 2176, 6556, 4376, 389, 4334, 806, 87, 4427, 6143, 2201, 2320, 5415, 4483, 5899, 1043, 6686, 693, 6634, 1905, 857, 6961, 173, 983, 6181, 1797, 2312, 2351, 5164, 6864, 6502, 1840, 3129, 2461, 5876, 4037, 92, 3614, 4777, 6301, 3817, 3029, 2687, 195, 6838, 3948, 514, 5521, 4730, 1748, 3998, 6827, 3549, 1139, 5398, 5003, 5964, 3395, 4727, 602, 4946, 3213, 1790, 2270, 2316, 137, 6002, 4019, 3607, 284, 5839, 108, 830, 1291, 5636, 2362, 3785, 1935, 3315, 5661, 1983, 5467, 6347, 409, 1438, 297, 4477, 4133, 3953, 1273, 2410, 1857, 5022, 3146, 5363, 6189, 5796, 272, 3410, 735, 6414, 398, 1642, 1628, 2597, 3885, 4850, 3436, 2701, 1896, 4568, 6636, 5142, 2030, 1015, 6553, 230, 1456, 135, 5569, 1197, 1792, 1822, 5447, 3263, 1817, 6984, 476, 5497, 6158, 1092, 6633, 6405, 6465, 7015, 3399, 5482, 842, 5445, 2595, 612, 5840, 3665, 6094, 6828, 5252, 5986, 1956, 4313, 6032, 5402, 2702, 1740, 5703, 6233, 1619, 1223, 3297, 3616, 1548, 4660, 1698, 4448, 1902, 560, 652, 3030, 1277, 5897, 334, 5646, 4486, 6277, 5074, 481, 5045, 2997, 2014, 2646, 485, 1594, 4380, 7071, 3628, 1490, 2275, 4330, 5032, 7104, 1873, 5530, 716, 5733, 772, 719, 1699, 4920, 1918, 1232, 2880, 3834, 1930, 4905, 2928, 6334, 3587, 4903, 5172, 1686, 1201, 1771, 1708, 2367, 802, 6742, 6601, 1892, 6439, 556, 4385, 1603, 5300, 4694, 120, 2012, 4287, 2087, 873, 4181, 6193, 3641, 5807, 5775, 550, 91, 6579, 6136, 2961, 6524, 2082, 144, 1847, 5510, 4022, 4363, 4647, 55, 1275, 6948, 6028, 613, 5431, 3343, 4406, 2884, 6561, 5590, 4735, 5247, 3417, 2618, 2669, 3532, 4172, 702, 6769, 5102, 6756, 4093, 7, 1612, 2204, 1614, 3542, 4754, 3156, 3790, 997, 7013, 601, 3314, 4679, 96, 5722, 158, 3209, 5422, 4040, 1955, 4226, 5637, 3191, 6124, 6558, 5179, 4254, 1811, 6795, 3525, 6841, 674, 2774, 4571, 5873, 2526, 5174, 5133, 4970, 6705, 758, 131, 6156, 6799, 1613, 6369, 3737, 3921, 5508, 4989, 3203, 1390, 3546, 1888, 5229, 6854, 1214, 5580, 6185, 3550, 6817, 1933, 5367, 1113, 4830, 1415, 6356, 1587, 5647, 3173, 6596, 3698, 1618, 404, 424, 1782, 6046, 4709, 1631, 2135, 850, 3707, 2683, 2430, 739, 6092, 5220, 849, 2298, 726, 1215, 4756, 5953, 5340, 3038, 1281, 3883, 7088, 1757, 4177, 846, 7078, 4302, 1829, 7086, 3493, 6458, 793, 1124, 5451, 251, 4148, 1592, 4866, 6422, 4043, 2741, 5571, 5617, 1784, 705, 4923, 98, 3577, 500, 1192, 6083, 4630, 4578, 6537, 6274, 4722, 3220, 3887, 1683, 5338, 337, 6386, 6091, 2850, 1348, 6768, 2443, 2238, 6920, 4539, 7047, 2853, 2593, 7043, 3611, 1202, 3661, 3, 3366, 1893, 6965, 6616, 732, 4420, 4333, 5516, 4276, 2168, 524, 1921, 1730, 5238, 902, 3789, 924, 2463, 7037, 1510, 6162, 1168, 2864, 3520, 2340, 1872, 2773, 1360, 5582, 3398, 5700, 3796, 176, 4031, 6533, 6517, 3062, 7123, 1985, 6581, 4713, 3830, 4147, 5827, 2916, 498, 5604, 914, 5212, 6563, 5307, 6215, 5663, 5434, 1616, 4634, 2601, 5816, 4241, 2382, 3530, 1989, 79, 3446, 6626, 4299, 6113, 2885, 44, 3824, 7017, 5903, 4171, 5187, 2807, 3128, 3081, 1053, 1187, 4305, 380, 5342, 2511, 4020, 912, 5203, 2520, 4235, 3303, 4252, 3361, 5167, 2375, 583, 6963, 5148, 2058, 2114, 1089, 586, 2069, 6192, 3269, 1696, 2388, 3414, 3900, 1796, 4701, 1352, 1880, 4556, 2487, 3154, 3120, 5782, 1447, 3321, 81, 450, 171, 4117, 291, 4238, 3624, 542, 6196, 936, 5550, 4163, 5472, 6205, 3058, 3448, 3059, 2575, 74, 3970, 5454, 6033, 2673, 648, 5823, 5933, 5721, 6669, 269, 2345, 2954, 3678, 4666, 1133, 2648, 5460, 6464, 1373, 5125, 3420, 2219, 2389, 1303, 5734, 31, 4178, 4622, 5888, 2893, 897, 6902, 5925, 5937, 4968, 3109, 3781, 4564, 6716, 4675, 2728, 5289, 2722, 9, 213, 1418, 3001, 6344, 4975, 4230, 3094, 3134, 493, 4244, 4929, 5577, 5487, 2025, 3584, 2943, 6360, 750, 4205, 4664, 5319, 5627, 2574, 1971, 6515, 6290, 6080, 2268, 4358, 3430, 5288, 5418, 4461, 160, 6423, 3229, 2878], [2752, 7096, 1147, 5321, 4183, 2372, 6108, 165, 4635, 1000, 4440, 4770, 2808, 4248, 4273, 1979, 3667, 3610, 1886, 4412, 1953, 1898, 4041, 6053, 2190, 2913, 2929, 607, 5297, 3693, 4422, 574, 1604, 5728, 1469, 2161, 1684, 6333, 730, 3985, 3017, 1597, 1583, 1099, 3722, 4325, 6987, 2797, 4476, 4533, 5939, 4215, 5993, 483, 4282, 1205, 3825, 2265, 6852, 2423, 1550, 1059, 5584, 2757, 2594, 1361, 6745, 4457, 5858, 4567, 5312, 6901, 2220, 3524, 3042, 2549, 7028, 166, 3957, 6320, 5372, 3950, 6428, 1424, 509, 3152, 2739, 303, 2536, 851, 2678, 1878, 375, 7000, 4810, 2751, 3777, 5750, 1492, 4838, 1286, 5224, 4583, 2485, 3603, 6888, 1874, 6542, 6037, 134, 1717, 20, 5157, 5076, 4398, 2442, 5144, 5740, 2772, 3184, 5914, 5800, 4029, 1535, 2569, 4094, 4339, 1046, 2794, 243, 2008, 4534, 5373, 17, 4652, 2397, 2812, 1856, 5814, 206, 355, 5797, 5349, 6878, 7072, 1338, 1949, 6699, 6530, 5653, 913, 5219, 1602, 3831, 5716, 1129, 3101, 6445, 5134, 549, 3865, 3653, 4653, 1485, 1462, 6176, 1747, 1354, 1675, 1807, 1664, 1070, 164, 6697, 3757, 4146, 6673, 597, 3372, 6918, 1335, 2679, 1419, 5002, 2497, 5824, 6539, 5471, 1553, 871, 5135, 4410, 577, 565, 5293, 1219, 6767, 3161, 2349, 5846, 3626, 2780, 3551, 6342, 6054, 1049, 1155, 3137, 6447, 2693, 1524, 1751, 1014, 2460, 3694, 5185, 6538, 800, 5794, 6621, 6680, 3932, 3427, 3759, 239, 6928, 4101, 3657, 2921, 1801, 2770, 3867, 3638, 935, 2980, 3043, 780, 929, 5544, 4077, 2507, 5496, 2437, 605, 4949, 2999, 1596, 7117, 4842, 2469, 1495, 260, 3471, 1980, 2055, 4760, 5430, 3100, 3295, 3147, 6015, 6168, 4322, 4663, 4026, 2993, 910, 767, 3983, 6923, 6324, 2748, 4131, 387, 457, 4073, 6835, 4641, 3906, 791, 104, 4195, 4837, 847, 4581, 4001, 1996, 5354, 496, 6597, 5501, 2512, 3612, 6228, 4312, 3598, 405, 2860, 2216, 2016, 4271, 3736, 4272, 1869, 6757, 4308, 1910, 1926, 1048, 5242, 3691, 5272, 4111, 2241, 2468, 1316, 6410, 4405, 3201, 6149, 3585, 5607, 1088, 1746, 5440, 872, 1203, 5444, 6355, 1108, 2151, 828, 2004, 4743, 2137, 4458, 4438, 2843, 3890, 3994, 2107, 5529, 4447, 5793, 4, 4472, 6261, 5779, 713, 5702, 5950, 4611, 4933, 4983, 2771, 2383, 2125, 941, 1405, 6591, 2176, 6556, 4376, 389, 4334, 806, 87, 4427, 6143, 2201, 2320, 5415, 4483, 5899, 1043, 6686, 693, 6634, 1905, 857, 6961, 173, 983, 6181, 1797, 2312, 2351, 5164, 6864, 6502, 1840, 3129, 2461, 5876, 4037, 92, 3614, 4777, 6301, 3817, 3029, 2687, 195, 6838, 3948, 514, 5521, 4730, 1748, 3998, 6827, 3549, 1139, 5398, 5003, 5964, 3395, 4727, 602, 4946, 3213, 1790, 2270, 2316, 137, 6002, 4019, 3607, 284, 5839, 108, 830, 1291, 5636, 2362, 3785, 1935, 3315, 5661, 1983, 5467, 6347, 409, 1438, 297, 4477, 4133, 3953, 1273, 2410, 1857, 5022, 3146, 5363, 6189, 5796, 272, 3410, 735, 6414, 398, 1642, 1628, 2597, 3885, 4850, 3436, 2701, 1896, 4568, 6636, 5142, 2030, 1015, 6553, 230, 1456, 135, 5569, 1197, 1792, 1822, 5447, 3263, 1817, 6984, 476, 5497, 6158, 1092, 6633, 6405, 6465, 7015, 3399, 5482, 842, 5445, 2595, 612, 5840, 3665, 6094, 6828, 5252, 5986, 1956, 4313, 6032, 5402, 2702, 1740, 5703, 6233, 1619, 1223, 3297, 3616, 1548, 4660, 1698, 4448, 1902, 560, 652, 3030, 1277, 5897, 334, 5646, 4486, 6277, 5074, 481, 5045, 2997, 2014, 2646, 485, 1594, 4380, 7071, 3628, 1490, 2275, 4330, 5032, 7104, 1873, 5530, 716, 5733, 772, 719, 1699, 4920, 1918, 1232, 2880, 3834, 1930, 4905, 2928, 6334, 3587, 4903, 5172, 1686, 1201, 1771, 1708, 2367, 802, 6742, 6601, 1892, 6439, 556, 4385, 1603, 5300, 4694, 120, 2012, 4287, 2087, 873, 4181, 6193, 3641, 5807, 5775, 550, 91, 6579, 6136, 2961, 6524, 2082, 144, 1847, 5510, 4022, 4363, 4647, 55, 1275, 6948, 6028, 613, 5431, 3343, 4406, 2884, 6561, 5590, 4735, 5247, 3417, 2618, 2669, 3532, 4172, 702, 6769, 5102, 6756, 4093, 7, 1612, 2204, 1614, 3542, 4754, 3156, 3790, 997, 7013, 601, 3314, 4679, 96, 5722, 158, 3209, 5422, 4040, 1955, 4226, 5637, 3191, 6124, 6558, 5179, 4254, 1811, 6795, 3525, 6841, 674, 2774, 4571, 5873, 2526, 5174, 5133, 4970, 6705, 758, 131, 6156, 6799, 1613, 6369, 3737, 3921, 5508, 4989, 3203, 1390, 3546, 1888, 5229, 6854, 1214, 5580, 6185, 3550, 6817, 1933, 5367, 1113, 4830, 1415, 6356, 1587, 5647, 3173, 6596, 3698, 1618, 404, 424, 1782, 6046, 4709, 1631, 2135, 850, 3707, 2683, 2430, 739, 6092, 5220, 849, 2298, 726, 1215, 4756, 5953, 5340, 3038, 1281, 3883, 7088, 1757, 4177, 846, 7078, 4302, 1829, 7086, 3493, 6458, 793, 1124, 5451, 251, 4148, 1592, 4866, 6422, 4043, 2741, 5571, 5617, 1784, 705, 4923, 98, 3577, 500, 1192, 6083, 4630, 4578, 6537, 6274, 4722, 3220, 3887, 1683, 5338, 337, 6386, 6091, 2850, 1348, 6768, 2443, 2238, 6920, 4539, 7047, 2853, 2593, 7043, 3611, 1202, 3661, 3, 3366, 1893, 6965, 6616, 732, 4420, 4333, 5516, 4276, 2168, 524, 1921, 1730, 5238, 902, 3789, 924, 2463, 7037, 1510, 6162, 1168, 2864, 3520, 2340, 1872, 2773, 1360, 5582, 3398, 5700, 3796, 176, 4031, 6533, 6517, 3062, 7123, 1985, 6581, 4713, 3830, 4147, 5827, 2916, 498, 5604, 914, 5212, 6563, 5307, 6215, 5663, 5434, 1616, 4634, 2601, 5816, 4241, 2382, 3530, 1989, 79, 3446, 6626, 4299, 6113, 2885, 44, 3824, 7017, 5903, 4171, 5187, 2807, 3128, 3081, 1053, 1187, 4305, 380, 5342, 2511, 4020, 912, 5203, 2520, 4235, 3303, 4252, 3361, 5167, 2375, 583, 6963, 5148, 2058, 2114, 1089, 586, 2069, 6192, 3269, 1696, 2388, 3414, 3900, 1796, 4701, 1352, 1880, 4556, 2487, 3154, 3120, 5782, 1447, 3321, 81, 450, 171, 4117, 291, 4238, 3624, 542, 6196, 936, 5550, 4163, 5472, 6205, 3058, 3448, 3059, 2575, 74, 3970, 5454, 6033, 2673, 648, 5823, 5933, 5721, 6669, 269, 2345, 2954, 3678, 4666, 1133, 2648, 5460, 6464, 1373, 5125, 3420, 2219, 2389, 1303, 5734, 31, 4178, 4622, 5888, 2893, 897, 6902, 5925, 5937, 4968, 3109, 3781, 4564, 6716, 4675, 2728, 5289, 2722, 9, 213, 1418, 3001, 6344, 4975, 4230, 3094, 3134, 493, 4244, 4929, 5577, 5487, 2025, 3584, 2943, 6360, 750, 4205, 4664, 5319, 5627, 2574, 1971, 6515, 6290, 6080, 2268, 4358, 3430, 5288, 5418, 4461, 160, 6423, 3229, 2878], [2059, 13, 6159, 6170, 4125, 4126, 33, 2088, 6185, 4141, 4145, 6202, 69, 2119, 4171, 78, 2131, 6229, 90, 4196, 2152, 4201, 2155, 6253, 4207, 112, 4209, 4210, 2167, 6266, 2173, 128, 4231, 6291, 6295, 2756, 2202, 4251, 4254, 4255, 165, 6311, 171, 179, 185, 6331, 196, 4293, 5154, 4304, 210, 212, 4321, 230, 721, 237, 241, 2290, 6390, 6391, 251, 252, 256, 263, 4380, 2778, 6430, 2340, 4397, 2351, 2353, 4490, 6464, 6465, 4420, 4421, 2375, 6473, 4426, 2383, 337, 343, 2394, 348, 2399, 2408, 4463, 6514, 380, 6529, 394, 2445, 4496, 403, 2456, 6556, 2468, 431, 4530, 4534, 2491, 2492, 2495, 4554, 473, 475, 2532, 2533, 2544, 503, 510, 85, 2564, 6666, 2571, 525, 4622, 529, 2579, 6682, 6683, 2588, 6686, 2591, 547, 2596, 552, 2603, 2607, 4657, 6712, 2617, 6717, 578, 6726, 583, 2638, 594, 2646, 6749, 4708, 6763, 6764, 2679, 4729, 6780, 2689, 6787, 4740, 4743, 4745, 2698, 2704, 657, 2711, 2715, 2716, 4771, 6820, 677, 2727, 687, 2740, 6839, 708, 6855, 713, 3191, 4817, 729, 6874, 2779, 2781, 735, 2786, 2790, 6887, 746, 6269, 754, 4853, 6902, 4856, 4857, 4876, 4878, 2837, 790, 6937, 798, 6946, 4903, 814, 2868, 4918, 2874, 6971, 6974, 2892, 2898, 855, 7003, 860, 4958, 4581, 5947, 2916, 4965, 7014, 2926, 880, 7038, 7048, 2958, 5016, 928, 5025, 936, 5038, 2991, 3000, 3001, 3002, 956, 5053, 5057, 7109, 966, 967, 7115, 3032, 5081, 995, 167, 5125, 1033, 5136, 5147, 3106, 5160, 1065, 1080, 1097, 3154, 5205, 3173, 5222, 1129, 1130, 1143, 4775, 1155, 5273, 538, 4976, 3239, 5288, 1206, 3258, 1215, 1225, 5330, 4644, 5338, 5339, 1244, 3294, 5346, 5347, 1256, 5354, 1263, 5364, 5377, 3334, 793, 3350, 3366, 6705, 3368, 5419, 3374, 1330, 3386, 1350, 1351, 569, 5466, 3427, 3429, 3436, 5493, 5498, 1423, 1426, 3477, 5528, 5529, 3489, 1445, 1451, 3501, 3507, 3508, 1462, 3514, 1469, 1471, 1482, 3531, 1490, 1493, 5599, 6395, 5606, 3565, 1522, 3575, 5626, 1550, 5655, 5672, 5675, 3638, 5695, 5698, 3683, 5717, 1623, 1624, 3674, 3675, 1629, 1635, 3684, 1641, 5748, 1658, 5772, 3735, 5784, 1690, 1701, 1704, 3759, 5810, 3019, 3786, 1740, 3805, 1318, 4732, 3831, 3836, 3842, 3846, 1804, 5080, 1816, 1823, 5921, 1827, 1838, 3899, 3902, 1861, 5958, 1877, 5976, 3929, 5978, 5434, 3934, 3935, 3943, 6008, 6014, 1930, 1933, 6033, 1940, 3994, 1947, 4763, 4021, 6072, 6474, 1982, 1985, 6085, 6093, 6098, 6819, 2005, 4056, 6109, 6481, 2027, 2028, 6128, 6130, 2045, 715, 84, 6883, 6356, 3143, 3475, 7127, 4689, 1028, 2373, 4716, 3451, 5239, 6466, 1005, 5441, 5013, 5602, 4299, 5107, 3960, 2347, 935, 5455, 3127, 613, 1233, 1139, 2305, 4172, 4147, 1336, 1114, 3885, 4037, 5223, 7071, 6232, 6882, 2025, 6090, 6611, 1161, 4249, 5, 4391, 7015, 1962, 3848, 5837, 1693, 2147, 6615, 6492, 6498, 2409, 5825, 409, 6235, 6620, 6227, 5605, 3800, 4846, 6236, 2368, 7041, 806, 5820, 5087, 96, 2983, 1193, 1979, 6662, 3022, 4107, 4504, 2641, 6357, 1899, 892, 5591, 6966, 5382, 330, 3367, 6732, 187, 5769, 6351, 4863, 1153, 3586, 6164, 6889, 926, 5835, 4627, 3464, 6782, 6081, 2549, 5185, 495, 5446, 5089, 3129, 4460, 6950, 316, 1529, 2077, 5993, 3192, 5253, 5617, 3865, 6399, 3743, 6302, 4353, 5984, 5151, 5578, 4631, 2560, 4813, 253, 4604, 296, 3620, 6821, 7090, 3779, 4511, 6817, 5864, 3170, 2315, 72, 732, 2640, 180, 2594, 1420, 2848, 6550, 5809, 3300, 4382, 5793, 455, 2750, 2151, 3710, 3264, 3867, 5856, 1227, 5619, 1843, 1916, 3552, 5799, 1953, 3527, 3581, 5499, 4945, 6396, 7045, 2406, 4007, 1431, 987, 6180, 3354, 3653, 1688, 1189, 5004, 1590, 831, 1021, 5300, 4115, 1492, 4634, 2741, 4912, 3751, 3602, 3721, 3055, 7012, 964, 6545, 5367, 5450, 719, 6024, 2083, 2330, 4485, 1675, 2700, 3453, 4325, 3309, 6806, 1446, 6648, 1649, 5547, 6870, 6238, 3282, 1464, 5456, 2680, 3423, 1070, 2021, 3439, 6197, 4034, 5562, 6967, 4809, 3591, 7001, 4372, 4960, 3168, 4079, 3890, 1247, 2244, 6539, 6509, 3089, 3577, 4833, 2118, 5227, 4599, 858, 2039, 1209, 850, 104, 2270, 5321, 1014, 5417, 1896, 6143, 938, 6734, 3876, 5872, 586, 6911, 3905, 4468, 4988, 5435, 3131, 6363, 7083, 2117, 5839, 5723, 4154, 4143, 264, 4773, 1796, 3780, 1329, 3133, 1720, 4546, 4749, 3226, 3059, 1390, 4308, 4162, 3649, 5938, 2522, 4412, 3677, 4046, 2870, 3533, 2081, 5746, 4611, 4407, 3784, 2481, 1342, 821, 1815, 4808, 755, 2815, 5826, 191, 753, 5177, 2361, 4885, 6334, 4537, 245, 3478, 5977, 493, 5693, 3937, 5482, 4381, 3549, 4921, 5209, 1124, 4582, 640, 6978, 3594, 5200, 2592, 873, 3462, 4667, 327, 6969, 1356, 5228, 2961, 1633, 2568, 5416, 3861, 6046, 1800, 5214, 4000, 774, 2260, 4443, 1592, 3425, 5030, 3596, 1787, 3989, 7118, 3600, 4302, 557, 1563, 779, 2903, 1533, 3627, 1314, 4825, 2329, 5910, 320, 3525, 4439, 214, 3061, 309, 5718, 6989, 3700, 5119, 3538, 1116, 4017, 4730, 1008, 6228, 2052, 2231, 3343, 5282, 1772, 6520, 3312, 6537, 2714, 4024, 4378, 1682, 1108, 6593, 429, 6699, 2217, 420, 6745, 1311, 6166, 1347, 219, 4262, 6760, 5739, 6220, 5917, 2758, 1337, 6179, 954, 6828, 6845, 5169, 4726, 1497, 17, 4195, 5050, 1793, 3463, 3008, 2415, 4035, 6784, 2209, 5323, 4619, 3397, 2277, 4454, 4395, 62, 4031, 5935, 1994, 3180, 430, 6212, 5882, 3761, 3352, 2878, 2980, 5438, 331, 3202, 5096, 5583, 4259, 3372, 5616, 4641, 6513, 6619, 4886, 472, 1487, 3466, 1096, 454, 3253, 5219, 3768, 6569, 912, 7000, 6584, 2427, 5470, 1185, 4229, 3196, 2037, 156, 3936, 3323, 16, 305, 6352, 295, 3993, 6222, 6511, 2780, 2545, 2938, 4738, 2901, 4673, 3619, 5384, 6651, 3206, 5598, 1037, 5813, 1418, 4001, 6392, 1110, 6776, 5073, 6767, 4571, 133, 298, 1224, 2093, 4841, 3781, 501, 274, 6293, 2663, 5858, 6901, 6030, 3990, 1476, 202, 2719, 6061, 5911, 2317, 5624, 6731, 4189, 3837, 1587, 161, 3259, 847, 1017, 803, 4934, 805, 1537, 5351, 324, 4713, 4118, 2082, 6323, 2605, 6818, 2801, 4307, 1517, 3788, 6316, 5986, 136, 3065, 4493, 3953, 4461, 3219, 4072, 4327, 3573, 1205, 3737, 4697, 6155, 3863, 1784, 1222, 3017, 568, 3401, 1892, 2589, 3884, 479, 1873, 2955, 3189, 2296, 2494, 2693, 1009, 3838, 5992, 6959, 1408, 5197], [2752, 7096, 1147, 5321, 4183, 2372, 6108, 165, 4635, 1000, 4440, 4770, 2808, 4248, 4273, 1979, 3667, 3610, 1886, 4412, 1953, 1898, 4041, 6053, 2190, 2913, 2929, 607, 5297, 3693, 4422, 574, 1604, 5728, 1469, 2161, 1684, 6333, 730, 3985, 3017, 1597, 1583, 1099, 3722, 4325, 6987, 2797, 4476, 4533, 5939, 4215, 5993, 483, 4282, 1205, 3825, 2265, 6852, 2423, 1550, 1059, 5584, 2757, 2594, 1361, 6745, 4457, 5858, 4567, 5312, 6901, 2220, 3524, 3042, 2549, 7028, 166, 3957, 6320, 5372, 3950, 6428, 1424, 509, 3152, 2739, 303, 2536, 851, 2678, 1878, 375, 7000, 4810, 2751, 3777, 5750, 1492, 4838, 1286, 5224, 4583, 2485, 3603, 6888, 1874, 6542, 6037, 134, 1717, 20, 5157, 5076, 4398, 2442, 5144, 5740, 2772, 3184, 5914, 5800, 4029, 1535, 2569, 4094, 4339, 1046, 2794, 243, 2008, 4534, 5373, 17, 4652, 2397, 2812, 1856, 5814, 206, 355, 5797, 5349, 6878, 7072, 1338, 1949, 6699, 6530, 5653, 913, 5219, 1602, 3831, 5716, 1129, 3101, 6445, 5134, 549, 3865, 3653, 4653, 1485, 1462, 6176, 1747, 1354, 1675, 1807, 1664, 1070, 164, 6697, 3757, 4146, 6673, 597, 3372, 6918, 1335, 2679, 1419, 5002, 2497, 5824, 6539, 5471, 1553, 871, 5135, 4410, 577, 565, 5293, 1219, 6767, 3161, 2349, 5846, 3626, 2780, 3551, 6342, 6054, 1049, 1155, 3137, 6447, 2693, 1524, 1751, 1014, 2460, 3694, 5185, 6538, 800, 5794, 6621, 6680, 3932, 3427, 3759, 239, 6928, 4101, 3657, 2921, 1801, 2770, 3867, 3638, 935, 2980, 3043, 780, 929, 5544, 4077, 2507, 5496, 2437, 605, 4949, 2999, 1596, 7117, 4842, 2469, 1495, 260, 3471, 1980, 2055, 4760, 5430, 3100, 3295, 3147, 6015, 6168, 4322, 4663, 4026, 2993, 910, 767, 3983, 6923, 6324, 2748, 4131, 387, 457, 4073, 6835, 4641, 3906, 791, 104, 4195, 4837, 847, 4581, 4001, 1996, 5354, 496, 6597, 5501, 2512, 3612, 6228, 4312, 3598, 405, 2860, 2216, 2016, 4271, 3736, 4272, 1869, 6757, 4308, 1910, 1926, 1048, 5242, 3691, 5272, 4111, 2241, 2468, 1316, 6410, 4405, 3201, 6149, 3585, 5607, 1088, 1746, 5440, 872, 1203, 5444, 6355, 1108, 2151, 828, 2004, 4743, 2137, 4458, 4438, 2843, 3890, 3994, 2107, 5529, 4447, 5793, 4, 4472, 6261, 5779, 713, 5702, 5950, 4611, 4933, 4983, 2771, 2383, 2125, 941, 1405, 6591, 2176, 6556, 4376, 389, 4334, 806, 87, 4427, 6143, 2201, 2320, 5415, 4483, 5899, 1043, 6686, 693, 6634, 1905, 857, 6961, 173, 983, 6181, 1797, 2312, 2351, 5164, 6864, 6502, 1840, 3129, 2461, 5876, 4037, 92, 3614, 4777, 6301, 3817, 3029, 2687, 195, 6838, 3948, 514, 5521, 4730, 1748, 3998, 6827, 3549, 1139, 5398, 5003, 5964, 3395, 4727, 602, 4946, 3213, 1790, 2270, 2316, 137, 6002, 4019, 3607, 284, 5839, 108, 830, 1291, 5636, 2362, 3785, 1935, 3315, 5661, 1983, 5467, 6347, 409, 1438, 297, 4477, 4133, 3953, 1273, 2410, 1857, 5022, 3146, 5363, 6189, 5796, 272, 3410, 735, 6414, 398, 1642, 1628, 2597, 3885, 4850, 3436, 2701, 1896, 4568, 6636, 5142, 2030, 1015, 6553, 230, 1456, 135, 5569, 1197, 1792, 1822, 5447, 3263, 1817, 6984, 476, 5497, 6158, 1092, 6633, 6405, 6465, 7015, 3399, 5482, 842, 5445, 2595, 612, 5840, 3665, 6094, 6828, 5252, 5986, 1956, 4313, 6032, 5402, 2702, 1740, 5703, 6233, 1619, 1223, 3297, 3616, 1548, 4660, 1698, 4448, 1902, 560, 652, 3030, 1277, 5897, 334, 5646, 4486, 6277, 5074, 481, 5045, 2997, 2014, 2646, 485, 1594, 4380, 7071, 3628, 1490, 2275, 4330, 5032, 7104, 1873, 5530, 716, 5733, 772, 719, 1699, 4920, 1918, 1232, 2880, 3834, 1930, 4905, 2928, 6334, 3587, 4903, 5172, 1686, 1201, 1771, 1708, 2367, 802, 6742, 6601, 1892, 6439, 556, 4385, 1603, 5300, 4694, 120, 2012, 4287, 2087, 873, 4181, 6193, 3641, 5807, 5775, 550, 91, 6579, 6136, 2961, 6524, 2082, 144, 1847, 5510, 4022, 4363, 4647, 55, 1275, 6948, 6028, 613, 5431, 3343, 4406, 2884, 6561, 5590, 4735, 5247, 3417, 2618, 2669, 3532, 4172, 702, 6769, 5102, 6756, 4093, 7, 1612, 2204, 1614, 3542, 4754, 3156, 3790, 997, 7013, 601, 3314, 4679, 96, 5722, 158, 3209, 5422, 4040, 1955, 4226, 5637, 3191, 6124, 6558, 5179, 4254, 1811, 6795, 3525, 6841, 674, 2774, 4571, 5873, 2526, 5174, 5133, 4970, 6705, 758, 131, 6156, 6799, 1613, 6369, 3737, 3921, 5508, 4989, 3203, 1390, 3546, 1888, 5229, 6854, 1214, 5580, 6185, 3550, 6817, 1933, 5367, 1113, 4830, 1415, 6356, 1587, 5647, 3173, 6596, 3698, 1618, 404, 424, 1782, 6046, 4709, 1631, 2135, 850, 3707, 2683, 2430, 739, 6092, 5220, 849, 2298, 726, 1215, 4756, 5953, 5340, 3038, 1281, 3883, 7088, 1757, 4177, 846, 7078, 4302, 1829, 7086, 3493, 6458, 793, 1124, 5451, 251, 4148, 1592, 4866, 6422, 4043, 2741, 5571, 5617, 1784, 705, 4923, 98, 3577, 500, 1192, 6083, 4630, 4578, 6537, 6274, 4722, 3220, 3887, 1683, 5338, 337, 6386, 6091, 2850, 1348, 6768, 2443, 2238, 6920, 4539, 7047, 2853, 2593, 7043, 3611, 1202, 3661, 3, 3366, 1893, 6965, 6616, 732, 4420, 4333, 5516, 4276, 2168, 524, 1921, 1730, 5238, 902, 3789, 924, 2463, 7037, 1510, 6162, 1168, 2864, 3520, 2340, 1872, 2773, 1360, 5582, 3398, 5700, 3796, 176, 4031, 6533, 6517, 3062, 7123, 1985, 6581, 4713, 3830, 4147, 5827, 2916, 498, 5604, 914, 5212, 6563, 5307, 6215, 5663, 5434, 1616, 4634, 2601, 5816, 4241, 2382, 3530, 1989, 79, 3446, 6626, 4299, 6113, 2885, 44, 3824, 7017, 5903, 4171, 5187, 2807, 3128, 3081, 1053, 1187, 4305, 380, 5342, 2511, 4020, 912, 5203, 2520, 4235, 3303, 4252, 3361, 5167, 2375, 583, 6963, 5148, 2058, 2114, 1089, 586, 2069, 6192, 3269, 1696, 2388, 3414, 3900, 1796, 4701, 1352, 1880, 4556, 2487, 3154, 3120, 5782, 1447, 3321, 81, 450, 171, 4117, 291, 4238, 3624, 542, 6196, 936, 5550, 4163, 5472, 6205, 3058, 3448, 3059, 2575, 74, 3970, 5454, 6033, 2673, 648, 5823, 5933, 5721, 6669, 269, 2345, 2954, 3678, 4666, 1133, 2648, 5460, 6464, 1373, 5125, 3420, 2219, 2389, 1303, 5734, 31, 4178, 4622, 5888, 2893, 897, 6902, 5925, 5937, 4968, 3109, 3781, 4564, 6716, 4675, 2728, 5289, 2722, 9, 213, 1418, 3001, 6344, 4975, 4230, 3094, 3134, 493, 4244, 4929, 5577, 5487, 2025, 3584, 2943, 6360, 750, 4205, 4664, 5319, 5627, 2574, 1971, 6515, 6290, 6080, 2268, 4358, 3430, 5288, 5418, 4461, 160, 6423, 3229, 2878], [2752, 7096, 1147, 5321, 4183, 2372, 6108, 165, 4635, 1000, 4440, 4770, 2808, 4248, 4273, 1979, 3667, 3610, 1886, 4412, 1953, 1898, 4041, 6053, 2190, 2913, 2929, 607, 5297, 3693, 4422, 574, 1604, 5728, 1469, 2161, 1684, 6333, 730, 3985, 3017, 1597, 1583, 1099, 3722, 4325, 6987, 2797, 4476, 4533, 5939, 4215, 5993, 483, 4282, 1205, 3825, 2265, 6852, 2423, 1550, 1059, 5584, 2757, 2594, 1361, 6745, 4457, 5858, 4567, 5312, 6901, 2220, 3524, 3042, 2549, 7028, 166, 3957, 6320, 5372, 3950, 6428, 1424, 509, 3152, 2739, 303, 2536, 851, 2678, 1878, 375, 7000, 4810, 2751, 3777, 5750, 1492, 4838, 1286, 5224, 4583, 2485, 3603, 6888, 1874, 6542, 6037, 134, 1717, 20, 5157, 5076, 4398, 2442, 5144, 5740, 2772, 3184, 5914, 5800, 4029, 1535, 2569, 4094, 4339, 1046, 2794, 243, 2008, 4534, 5373, 17, 4652, 2397, 2812, 1856, 5814, 206, 355, 5797, 5349, 6878, 7072, 1338, 1949, 6699, 6530, 5653, 913, 5219, 1602, 3831, 5716, 1129, 3101, 6445, 5134, 549, 3865, 3653, 4653, 1485, 1462, 6176, 1747, 1354, 1675, 1807, 1664, 1070, 164, 6697, 3757, 4146, 6673, 597, 3372, 6918, 1335, 2679, 1419, 5002, 2497, 5824, 6539, 5471, 1553, 871, 5135, 4410, 577, 565, 5293, 1219, 6767, 3161, 2349, 5846, 3626, 2780, 3551, 6342, 6054, 1049, 1155, 3137, 6447, 2693, 1524, 1751, 1014, 2460, 3694, 5185, 6538, 800, 5794, 6621, 6680, 3932, 3427, 3759, 239, 6928, 4101, 3657, 2921, 1801, 2770, 3867, 3638, 935, 2980, 3043, 780, 929, 5544, 4077, 2507, 5496, 2437, 605, 4949, 2999, 1596, 7117, 4842, 2469, 1495, 260, 3471, 1980, 2055, 4760, 5430, 3100, 3295, 3147, 6015, 6168, 4322, 4663, 4026, 2993, 910, 767, 3983, 6923, 6324, 2748, 4131, 387, 457, 4073, 6835, 4641, 3906, 791, 104, 4195, 4837, 847, 4581, 4001, 1996, 5354, 496, 6597, 5501, 2512, 3612, 6228, 4312, 3598, 405, 2860, 2216, 2016, 4271, 3736, 4272, 1869, 6757, 4308, 1910, 1926, 1048, 5242, 3691, 5272, 4111, 2241, 2468, 1316, 6410, 4405, 3201, 6149, 3585, 5607, 1088, 1746, 5440, 872, 1203, 5444, 6355, 1108, 2151, 828, 2004, 4743, 2137, 4458, 4438, 2843, 3890, 3994, 2107, 5529, 4447, 5793, 4, 4472, 6261, 5779, 713, 5702, 5950, 4611, 4933, 4983, 2771, 2383, 2125, 941, 1405, 6591, 2176, 6556, 4376, 389, 4334, 806, 87, 4427, 6143, 2201, 2320, 5415, 4483, 5899, 1043, 6686, 693, 6634, 1905, 857, 6961, 173, 983, 6181, 1797, 2312, 2351, 5164, 6864, 6502, 1840, 3129, 2461, 5876, 4037, 92, 3614, 4777, 6301, 3817, 3029, 2687, 195, 6838, 3948, 514, 5521, 4730, 1748, 3998, 6827, 3549, 1139, 5398, 5003, 5964, 3395, 4727, 602, 4946, 3213, 1790, 2270, 2316, 137, 6002, 4019, 3607, 284, 5839, 108, 830, 1291, 5636, 2362, 3785, 1935, 3315, 5661, 1983, 5467, 6347, 409, 1438, 297, 4477, 4133, 3953, 1273, 2410, 1857, 5022, 3146, 5363, 6189, 5796, 272, 3410, 735, 6414, 398, 1642, 1628, 2597, 3885, 4850, 3436, 2701, 1896, 4568, 6636, 5142, 2030, 1015, 6553, 230, 1456, 135, 5569, 1197, 1792, 1822, 5447, 3263, 1817, 6984, 476, 5497, 6158, 1092, 6633, 6405, 6465, 7015, 3399, 5482, 842, 5445, 2595, 612, 5840, 3665, 6094, 6828, 5252, 5986, 1956, 4313, 6032, 5402, 2702, 1740, 5703, 6233, 1619, 1223, 3297, 3616, 1548, 4660, 1698, 4448, 1902, 560, 652, 3030, 1277, 5897, 334, 5646, 4486, 6277, 5074, 481, 5045, 2997, 2014, 2646, 485, 1594, 4380, 7071, 3628, 1490, 2275, 4330, 5032, 7104, 1873, 5530, 716, 5733, 772, 719, 1699, 4920, 1918, 1232, 2880, 3834, 1930, 4905, 2928, 6334, 3587, 4903, 5172, 1686, 1201, 1771, 1708, 2367, 802, 6742, 6601, 1892, 6439, 556, 4385, 1603, 5300, 4694, 120, 2012, 4287, 2087, 873, 4181, 6193, 3641, 5807, 5775, 550, 91, 6579, 6136, 2961, 6524, 2082, 144, 1847, 5510, 4022, 4363, 4647, 55, 1275, 6948, 6028, 613, 5431, 3343, 4406, 2884, 6561, 5590, 4735, 5247, 3417, 2618, 2669, 3532, 4172, 702, 6769, 5102, 6756, 4093, 7, 1612, 2204, 1614, 3542, 4754, 3156, 3790, 997, 7013, 601, 3314, 4679, 96, 5722, 158, 3209, 5422, 4040, 1955, 4226, 5637, 3191, 6124, 6558, 5179, 4254, 1811, 6795, 3525, 6841, 674, 2774, 4571, 5873, 2526, 5174, 5133, 4970, 6705, 758, 131, 6156, 6799, 1613, 6369, 3737, 3921, 5508, 4989, 3203, 1390, 3546, 1888, 5229, 6854, 1214, 5580, 6185, 3550, 6817, 1933, 5367, 1113, 4830, 1415, 6356, 1587, 5647, 3173, 6596, 3698, 1618, 404, 424, 1782, 6046, 4709, 1631, 2135, 850, 3707, 2683, 2430, 739, 6092, 5220, 849, 2298, 726, 1215, 4756, 5953, 5340, 3038, 1281, 3883, 7088, 1757, 4177, 846, 7078, 4302, 1829, 7086, 3493, 6458, 793, 1124, 5451, 251, 4148, 1592, 4866, 6422, 4043, 2741, 5571, 5617, 1784, 705, 4923, 98, 3577, 500, 1192, 6083, 4630, 4578, 6537, 6274, 4722, 3220, 3887, 1683, 5338, 337, 6386, 6091, 2850, 1348, 6768, 2443, 2238, 6920, 4539, 7047, 2853, 2593, 7043, 3611, 1202, 3661, 3, 3366, 1893, 6965, 6616, 732, 4420, 4333, 5516, 4276, 2168, 524, 1921, 1730, 5238, 902, 3789, 924, 2463, 7037, 1510, 6162, 1168, 2864, 3520, 2340, 1872, 2773, 1360, 5582, 3398, 5700, 3796, 176, 4031, 6533, 6517, 3062, 7123, 1985, 6581, 4713, 3830, 4147, 5827, 2916, 498, 5604, 914, 5212, 6563, 5307, 6215, 5663, 5434, 1616, 4634, 2601, 5816, 4241, 2382, 3530, 1989, 79, 3446, 6626, 4299, 6113, 2885, 44, 3824, 7017, 5903, 4171, 5187, 2807, 3128, 3081, 1053, 1187, 4305, 380, 5342, 2511, 4020, 912, 5203, 2520, 4235, 3303, 4252, 3361, 5167, 2375, 583, 6963, 5148, 2058, 2114, 1089, 586, 2069, 6192, 3269, 1696, 2388, 3414, 3900, 1796, 4701, 1352, 1880, 4556, 2487, 3154, 3120, 5782, 1447, 3321, 81, 450, 171, 4117, 291, 4238, 3624, 542, 6196, 936, 5550, 4163, 5472, 6205, 3058, 3448, 3059, 2575, 74, 3970, 5454, 6033, 2673, 648, 5823, 5933, 5721, 6669, 269, 2345, 2954, 3678, 4666, 1133, 2648, 5460, 6464, 1373, 5125, 3420, 2219, 2389, 1303, 5734, 31, 4178, 4622, 5888, 2893, 897, 6902, 5925, 5937, 4968, 3109, 3781, 4564, 6716, 4675, 2728, 5289, 2722, 9, 213, 1418, 3001, 6344, 4975, 4230, 3094, 3134, 493, 4244, 4929, 5577, 5487, 2025, 3584, 2943, 6360, 750, 4205, 4664, 5319, 5627, 2574, 1971, 6515, 6290, 6080, 2268, 4358, 3430, 5288, 5418, 4461, 160, 6423, 3229, 2878], [4097, 4112, 28, 29, 4127, 32, 41, 6193, 4147, 2740, 6202, 66, 6219, 6221, 6222, 79, 4177, 98, 4195, 4196, 4200, 4203, 4206, 6256, 4213, 2168, 4222, 4226, 2180, 6293, 2208, 2210, 4260, 2216, 179, 2233, 4285, 191, 2244, 6344, 2250, 6353, 4309, 215, 218, 231, 2282, 1405, 4338, 244, 2294, 259, 261, 6413, 6415, 274, 4385, 4389, 297, 6465, 2372, 332, 4432, 4437, 5177, 2392, 4442, 2395, 6492, 4453, 4459, 364, 6513, 6517, 4841, 6549, 2432, 4487, 2440, 394, 4495, 4501, 4505, 2460, 415, 2469, 2471, 424, 6556, 6580, 4535, 4543, 2509, 6614, 6615, 6621, 2129, 6642, 6652, 6658, 2563, 2564, 2571, 4624, 6675, 6678, 4633, 6682, 6683, 4638, 4647, 552, 2603, 4656, 2825, 4669, 6722, 6731, 4687, 4690, 597, 6749, 2655, 6755, 6764, 2671, 4722, 2681, 6780, 4740, 6790, 2695, 4747, 656, 6805, 2716, 671, 4770, 6820, 685, 686, 4785, 692, 6837, 6843, 6845, 2751, 712, 4814, 6605, 4816, 4820, 6875, 6881, 4839, 6889, 2794, 752, 2802, 755, 6905, 6910, 2817, 774, 4871, 4872, 777, 783, 6928, 4883, 4886, 2842, 2844, 803, 2852, 806, 4908, 4913, 2868, 2871, 4937, 823, 846, 3370, 7010, 7016, 2924, 890, 7036, 2197, 4992, 897, 902, 5004, 7057, 916, 920, 925, 7071, 5033, 5035, 2991, 5042, 5062, 7116, 5102, 3057, 3064, 3069, 3094, 5143, 3106, 1069, 5173, 3129, 5185, 1091, 3153, 1129, 3181, 3182, 3189, 5244, 5246, 1157, 1166, 1171, 1186, 4635, 5285, 3253, 3257, 5306, 4298, 1214, 3270, 1225, 5322, 1227, 5324, 3277, 3279, 5329, 2938, 5343, 1253, 3281, 3309, 3314, 5364, 1279, 3341, 5399, 6378, 5418, 5419, 1324, 5422, 1336, 5434, 5442, 3397, 1352, 5461, 1370, 2961, 5482, 1388, 234, 5514, 578, 3472, 3484, 5550, 5554, 3514, 3518, 5567, 1474, 5574, 5577, 5579, 5584, 5595, 5607, 1514, 3574, 5627, 5643, 5648, 1554, 4355, 1569, 1577, 1581, 5683, 1591, 3642, 3653, 5702, 3670, 3672, 5729, 3689, 3702, 1641, 1661, 5772, 5780, 5782, 3743, 5799, 3755, 5813, 1718, 3770, 5824, 1735, 1740, 5848, 5851, 5868, 3828, 3833, 1786, 1787, 1322, 1798, 3852, 5906, 3859, 3860, 1822, 5934, 4061, 3897, 5947, 1855, 3927, 5980, 5983, 1890, 6004, 2587, 1927, 1934, 3995, 6045, 1957, 1962, 4018, 4034, 1990, 6087, 4428, 6090, 1226, 6109, 6112, 2021, 6131, 3085, 4413, 1185, 1213, 5263, 2340, 5928, 3276, 4057, 2209, 3725, 1017, 5334, 6156, 6992, 3350, 1489, 1453, 2544, 4043, 2397, 2459, 3591, 3848, 5977, 3135, 4864, 6496, 697, 4525, 184, 382, 3589, 4111, 2330, 6705, 4324, 885, 4558, 3658, 2678, 2862, 6438, 3204, 3595, 4419, 4174, 3421, 2468, 3394, 2735, 5843, 1248, 732, 4469, 2850, 5479, 1672, 4625, 3732, 2643, 4189, 3002, 1538, 165, 7083, 3059, 3338, 4079, 1, 405, 524, 761, 525, 480, 3160, 3102, 3231, 4759, 2993, 86, 3105, 2670, 5638, 2600, 4757, 428, 5975, 1030, 6595, 2334, 6388, 4199, 4391, 4935, 2385, 987, 267, 6092, 2038, 2652, 435, 3054, 7093, 3163, 5732, 831, 3865, 676, 262, 1792, 5557, 4541, 5212, 946, 6763, 5948, 2977, 3156, 1915, 3904, 3789, 695, 5002, 5032, 249, 2178, 5163, 1097, 3411, 2734, 758, 462, 6186, 6908, 2158, 3292, 1471, 5054, 3525, 3398, 3936, 6600, 1482, 4954, 338, 4699, 6826, 3179, 2626, 4126, 6624, 1387, 5271, 6814, 4017, 2320, 5400, 4450, 2310, 5208, 477, 3170, 7028, 2589, 485, 7003, 1659, 273, 4107, 3417, 1228, 1302, 6554, 1330, 5856, 2630, 3183, 3528, 6139, 2594, 6282, 6859, 1224, 1497, 3769, 6640, 5149, 6860, 142, 6276, 3784, 4167, 4263, 4220, 302, 1151, 6566, 3373, 881, 3816, 1579, 3866, 4990, 4629, 2457, 4159, 5475, 1523, 1918, 3969, 1242, 55, 5865, 1468, 3464, 4552, 4742, 5210, 2336, 1809, 6802, 771, 1096, 3032, 222, 2173, 6334, 1544, 2172, 127, 3293, 2892, 3701, 4592, 4000, 3364, 2528, 4180, 884, 2693, 4486, 5075, 5777, 6458, 790, 6487, 6768, 4463, 3930, 2748, 3931, 5810, 187, 448, 128, 4063, 5029, 1334, 6613, 237, 4353, 404, 65, 4514, 988, 6584, 3565, 2045, 6194, 2989, 6662, 2212, 1696, 3192, 6440, 3824, 3692, 2941, 2179, 1335, 2445, 5463, 341, 7112, 4407, 1095, 1273, 6957, 5884, 7035, 2049, 4048, 4056, 2833, 660, 2576, 1880, 6616, 5115, 2251, 5261, 1202, 4491, 5712, 316, 5311, 4383, 6684, 4822, 1953, 3978, 3236, 5964, 464, 1438, 5770, 6014, 1205, 1858, 1965, 4277, 481, 3155, 5828, 5302, 4321, 4868, 5201, 1658, 5132, 1649, 397, 1913, 300, 2458, 4746, 6787, 4869, 5101, 779, 805, 3680, 3519, 584, 4238, 1476, 6752, 1836, 243, 3746, 5403, 1162, 4621, 1955, 2565, 5674, 5649, 603, 4970, 930, 1720, 3145, 5369, 4982, 3720, 5372, 839, 1469, 5095, 665, 1200, 3448, 4849, 687, 1660, 5053, 5270, 1722, 3227, 4888, 6323, 1007, 7106, 1976, 3815, 640, 722, 5229, 6030, 3452, 4775, 1199, 3957, 5508, 3601, 145, 2217, 6185, 3459, 4774, 4534, 1146, 4030, 2260, 7040, 4157, 3604, 3336, 5605, 1297, 4998, 4216, 6611, 6770, 6692, 451, 602, 630, 5536, 1039, 1746, 3326, 5471, 4146, 4903, 3550, 1624, 6589, 3586, 5804, 4418, 4214, 221, 6641, 3915, 5705, 3028, 3650, 5338, 5989, 2059, 3134, 893, 1695, 4007, 6430, 910, 3328, 1972, 2607, 2806, 3801, 5534, 5960, 2790, 334, 3855, 6979, 4706, 4518, 4496, 2285, 2725, 4421, 2382, 4584, 2305, 73, 5959, 2354, 3446, 808, 2132, 4707, 912, 6039, 5023, 5390, 9, 6340, 1424, 5754, 2335, 3130, 6985, 1183, 3686, 2591, 5156, 1401, 7041, 6347, 3168, 6772, 6803, 849, 4599, 561, 618, 3352, 3511, 2582, 1878, 6339, 6245, 1230, 2824, 4451, 1435, 599, 2984, 1161, 7047, 2623, 2822, 6862, 2119, 5996, 6369, 4613, 5146, 3390, 4301, 2679, 6819, 230, 633, 4026, 6999, 3453, 6917, 5450, 3333, 2819, 2447, 1516, 3872, 6063, 3499, 3, 144, 6048, 1602, 1860, 1179, 1782, 4938, 1291, 5465, 4782, 829, 6514, 1639, 3151, 4009, 5355, 2758, 1121, 3141, 1542, 2002, 1818, 1799, 6164, 4717, 68, 3004, 4922, 2976, 2814, 356, 1839, 5838, 4357, 3777, 4751, 2024, 999, 320, 3154, 3147, 1552, 6111, 2718, 1263, 2084, 115, 4016, 3674, 3487, 1075, 2894, 4108, 2687, 3477, 7038, 5072, 251, 6033, 933, 5727, 348, 4851, 955, 5609, 3119, 3112, 4725, 1084, 5248, 1719, 4118, 89, 6490, 3786, 677, 5200, 4909, 860, 4225, 2350, 3118, 6711, 1852, 1662, 611, 5084, 6147, 826, 1899, 4743, 252, 4448, 4993, 6712, 2884, 3121, 1427], [2564, 2571, 6837, 4630, 25, 5146, 6683, 6684, 2591, 1569, 2600, 6185, 2603, 3122, 5173, 55, 5177, 6202, 4669, 3765, 5185, 66, 3653, 6731, 4687, 5712, 3153, 597, 3514, 2655, 5729, 4195, 1641, 4203, 3181, 2671, 6256, 4883, 6780, 5248, 6763, 4740, 3720, 6282, 5772, 1171, 5780, 4633, 6811, 2716, 6682, 3743, 1186, 4635, 6820, 4263, 1192, 28, 2323, 4285, 179, 2740, 5813, 1720, 3484, 2748, 6845, 191, 5824, 552, 1225, 2250, 6347, 6870, 5848, 218, 5851, 6881, 1253, 6889, 3309, 4849, 755, 5364, 4853, 1273, 1787, 6910, 2817, 4310, 6413, 274, 3859, 4886, 2844, 6874, 1822, 806, 808, 297, 3373, 5934, 2353, 823, 5434, 3390, 1858, 2372, 4648, 6711, 2871, 2900, 3129, 3929, 1370, 4453, 6641, 5482, 364, 6513, 884, 1405, 7045, 4489, 394, 1932, 2445, 2961, 2460, 925, 4518, 4007, 6556, 2991, 3002, 4543, 5579, 4770, 6615, 3032, 1497, 2021, 2216, 3574, 6652, 2045, 6143, 4999, 2770, 6447, 3236, 7038, 448, 6822, 4355, 5197, 5855, 1428, 4986, 2715, 3865, 4230, 1137, 6004, 2854, 1957, 2103, 6281, 3202, 4782, 253, 6571, 2686, 196, 1915, 5418, 1939, 1838, 5767, 2848, 1930, 6692, 464, 3674, 4231, 3818, 1968, 2099, 6678, 712, 846, 249, 1166, 4046, 6051, 4146, 1796, 3333, 261, 3700, 5773, 6487, 5270, 5154, 6688, 1069, 6502, 4378, 1168, 6549, 2033, 4859, 3894, 2607, 1741, 363, 1878, 1934, 7003, 5132, 935, 3836, 4424, 3529, 4016, 3278, 4245, 5554, 1846, 1397, 6362, 1014, 5232, 2822, 68, 4367, 5236, 1398, 2477, 6384, 2447, 2178, 3833, 2152, 3596, 4309, 1076, 3935, 4760, 603, 2815, 6179, 3316, 6520, 2938, 4822, 7015, 273, 834, 1477, 1979, 633, 2852, 156, 757, 4321, 6316, 5389, 6974, 4638, 1262, 5474, 5716, 5770, 6139, 2354, 4206, 4319, 4132, 3277, 4200, 2293, 3294, 1023, 2179, 3239, 2455, 6159, 2723, 5960, 816, 5334, 2947, 3459, 5480, 773, 2851, 2336, 499, 614, 1095, 5471, 1516, 5913, 6246, 6772, 6089, 1590, 3386, 6064, 3134, 6658, 3969, 855, 4889, 4722, 4505, 4949, 1213, 3160, 3253, 1705, 3281, 4127, 1352, 852, 6771, 1760, 5219, 2198, 1312, 3126, 4569, 4196, 1700, 2120, 6072, 4940, 2457, 2565, 5072, 493, 4856, 3670, 6416, 7014, 4213, 454, 6946, 6960, 4151, 6764, 2874, 6680, 4277, 5614, 4535, 6957, 6013, 1091, 4501, 4826, 1746, 4839, 7006, 1890, 2760, 3860, 5812, 3895, 6554, 3370, 4963, 2652, 200, 6498, 1336, 794, 6675, 3212, 6600, 296, 1226, 1806, 469, 697, 1230, 4110, 1726, 2294, 3702, 3034, 4970, 3168, 3448, 3936, 1474, 5752, 4209, 6255, 3417, 1792, 3085, 1763, 3995, 868, 5692, 2077, 4162, 6458, 2687, 6080, 3265, 3031, 29, 7035, 466, 6390, 2862, 4751, 1461, 3218, 6875, 4758, 1232, 5475, 215, 2528, 6835, 128, 3348, 2689, 3899, 3020, 4624, 2516, 1608, 2049, 4495, 4114, 5638, 2713, 6620, 1247, 6092, 3793, 4946, 2976, 1800, 3189, 4588, 692, 2681, 3500, 5788, 3182, 5893, 426, 5758, 1999, 2004, 4814, 4172, 3768, 3350, 4796, 1722, 2230, 4211, 5456, 3192, 6399, 5838, 6063, 7044, 3102, 1302, 1913, 3642, 5329, 1681, 1662, 6374, 1553, 5643, 6340, 3330, 6109, 4840, 4021, 5727, 3336, 3446, 1782, 5640, 241, 6293, 3367, 1216, 4214, 5884, 1855, 6614, 1482, 5678, 7106, 826, 5212, 84, 3725, 1476, 1592, 2941, 3638, 1422, 1990, 5054, 5244, 2180, 521, 6075, 3928, 4313, 5271, 7055, 7125, 6229, 6296, 3472, 2626, 5101, 761, 6353, 1381, 2081, 3755, 3083, 6543, 1562, 3338, 3035, 2189, 473, 3452, 6492, 4555, 5975, 4222, 478, 2385, 244, 3036, 7064, 3691, 4238, 1715, 334, 7115, 38, 6855, 4325, 4959, 708, 2139, 2375, 4225, 2960, 6757, 4159, 4614, 5567, 4426, 5837, 2410, 6550, 2596, 4044, 4617, 3019, 259, 2062, 4504, 5403, 2112, 2623, 3179, 4987, 4746, 7052, 6559, 829, 6398, 343, 6816, 4841, 5441, 6642, 4612, 320, 2427, 2087, 1368, 1227, 3094, 2499, 5969, 6279, 4136, 2945, 404, 4941, 6648, 6905, 6595, 4937, 758, 1148, 4993, 1587, 2340, 4437, 6071, 3124, 5070, 5355, 1025, 2842, 6395, 1718, 4442, 2459, 2175, 4306, 5714, 3360, 4042, 5372, 595, 6378, 85, 6181, 5339, 2582, 2924, 1185, 184, 3852, 3581, 4538, 3243, 1037, 3770, 1129, 2556, 6860, 2204, 3759, 2440, 7114, 5087, 5442, 3541, 2570, 338, 1680, 1586, 32, 1116, 4926, 5804, 6612, 4747, 451, 5463, 3931, 2469, 4451, 4419, 6237, 234, 6295, 6529, 1851, 676, 332, 5035, 1634, 1818, 2795, 954, 3626, 2437, 807, 6596, 298, 202, 4219, 3669, 4258, 583, 6388, 5689, 2509, 6033, 686, 6329, 2494, 1786, 6787, 6817, 6215, 1242, 5255, 4918, 4298, 5201, 3692, 6662, 926, 980, 4871, 6048, 5667, 4300, 749, 7093, 4490, 5574, 7116, 5937, 4519, 1205, 2013, 1259, 4029, 5645, 839, 4095, 2504, 4707, 5288, 422, 6118, 3157, 6184, 250, 1030, 3977, 4864, 6294, 3602, 2379, 3343, 4043, 602, 4155, 7046, 245, 4448, 1228, 3445, 2794, 1183, 6859, 1538, 2244, 3362, 1143, 6208, 771, 144, 5208, 4412, 4696, 6908, 2452, 428, 589, 6098, 1564, 3654, 3650, 5417, 1584, 2926, 4799, 5649, 5080, 6344, 1043, 4927, 6219, 5948, 2780, 2273, 6490, 3199, 913, 5498, 6996, 1096, 6430, 5822, 5865, 6523, 2172, 1408, 6188, 3154, 6156, 6443, 6862, 4665, 427, 4545, 6578, 1581, 2719, 2079, 6517, 6566, 4182, 1918, 4346, 6558, 732, 6589, 4774, 6168, 5700, 2251, 4844, 3563, 3855, 5209, 1289, 2491, 613, 2580, 5983, 3732, 4033, 4248, 3082, 685, 5801, 2865, 3983, 599, 4621, 483, 624, 5416, 5042, 5033, 73, 3411, 3668, 3163, 5084, 3171, 6415, 3735, 6044, 5738, 4892, 6396, 3273, 5536, 2260, 5810, 3103, 4111, 752, 4413, 4199, 928, 4972, 1456, 5229, 5095, 1079, 790, 2827, 2725, 6045, 439, 1734, 2677, 4537, 2208, 4460, 2802, 6616, 679, 1646, 3815, 3645, 9, 3151, 4512, 4992, 4629, 7068, 2837, 3611, 3934, 4757, 2063, 3518, 3141, 4558, 1078, 149, 1026, 1075, 6749, 584, 4827, 5615, 6186, 3577, 6928, 4742, 1489, 4459, 2431, 3784, 4112, 1263, 1827, 2489, 1660, 1755, 3601, 275, 5210, 5383, 7108, 2962, 77, 3447, 5868, 5215, 1577, 3147, 6114, 5296, 5390, 6446, 7101, 5528, 6305, 2850, 6357, 1177, 5115, 4035, 6111, 1972, 2059, 958, 4869, 4450, 5377, 6940, 5685, 1432, 1623, 2734, 2994, 4998, 3633, 1658, 3822, 4307, 2122, 3915, 6852, 167, 1579, 6093, 1514, 1557, 2781, 3099, 5986, 2027, 72, 6464, 912, 6630, 2790, 99, 476, 2594, 5246, 1416, 3449, 6291], [2752, 7096, 1147, 5321, 4183, 2372, 6108, 165, 4635, 1000, 4440, 4770, 2808, 4248, 4273, 1979, 3667, 3610, 1886, 4412, 1953, 1898, 4041, 6053, 2190, 2913, 2929, 607, 5297, 3693, 4422, 574, 1604, 5728, 1469, 2161, 1684, 6333, 730, 3985, 3017, 1597, 1583, 1099, 3722, 4325, 6987, 2797, 4476, 4533, 5939, 4215, 5993, 483, 4282, 1205, 3825, 2265, 6852, 2423, 1550, 1059, 5584, 2757, 2594, 1361, 6745, 4457, 5858, 4567, 5312, 6901, 2220, 3524, 3042, 2549, 7028, 166, 3957, 6320, 5372, 3950, 6428, 1424, 509, 3152, 2739, 303, 2536, 851, 2678, 1878, 375, 7000, 4810, 2751, 3777, 5750, 1492, 4838, 1286, 5224, 4583, 2485, 3603, 6888, 1874, 6542, 6037, 134, 1717, 20, 5157, 5076, 4398, 2442, 5144, 5740, 2772, 3184, 5914, 5800, 4029, 1535, 2569, 4094, 4339, 1046, 2794, 243, 2008, 4534, 5373, 17, 4652, 2397, 2812, 1856, 5814, 206, 355, 5797, 5349, 6878, 7072, 1338, 1949, 6699, 6530, 5653, 913, 5219, 1602, 3831, 5716, 1129, 3101, 6445, 5134, 549, 3865, 3653, 4653, 1485, 1462, 6176, 1747, 1354, 1675, 1807, 1664, 1070, 164, 6697, 3757, 4146, 6673, 597, 3372, 6918, 1335, 2679, 1419, 5002, 2497, 5824, 6539, 5471, 1553, 871, 5135, 4410, 577, 565, 5293, 1219, 6767, 3161, 2349, 5846, 3626, 2780, 3551, 6342, 6054, 1049, 1155, 3137, 6447, 2693, 1524, 1751, 1014, 2460, 3694, 5185, 6538, 800, 5794, 6621, 6680, 3932, 3427, 3759, 239, 6928, 4101, 3657, 2921, 1801, 2770, 3867, 3638, 935, 2980, 3043, 780, 929, 5544, 4077, 2507, 5496, 2437, 605, 4949, 2999, 1596, 7117, 4842, 2469, 1495, 260, 3471, 1980, 2055, 4760, 5430, 3100, 3295, 3147, 6015, 6168, 4322, 4663, 4026, 2993, 910, 767, 3983, 6923, 6324, 2748, 4131, 387, 457, 4073, 6835, 4641, 3906, 791, 104, 4195, 4837, 847, 4581, 4001, 1996, 5354, 496, 6597, 5501, 2512, 3612, 6228, 4312, 3598, 405, 2860, 2216, 2016, 4271, 3736, 4272, 1869, 6757, 4308, 1910, 1926, 1048, 5242, 3691, 5272, 4111, 2241, 2468, 1316, 6410, 4405, 3201, 6149, 3585, 5607, 1088, 1746, 5440, 872, 1203, 5444, 6355, 1108, 2151, 828, 2004, 4743, 2137, 4458, 4438, 2843, 3890, 3994, 2107, 5529, 4447, 5793, 4, 4472, 6261, 5779, 713, 5702, 5950, 4611, 4933, 4983, 2771, 2383, 2125, 941, 1405, 6591, 2176, 6556, 4376, 389, 4334, 806, 87, 4427, 6143, 2201, 2320, 5415, 4483, 5899, 1043, 6686, 693, 6634, 1905, 857, 6961, 173, 983, 6181, 1797, 2312, 2351, 5164, 6864, 6502, 1840, 3129, 2461, 5876, 4037, 92, 3614, 4777, 6301, 3817, 3029, 2687, 195, 6838, 3948, 514, 5521, 4730, 1748, 3998, 6827, 3549, 1139, 5398, 5003, 5964, 3395, 4727, 602, 4946, 3213, 1790, 2270, 2316, 137, 6002, 4019, 3607, 284, 5839, 108, 830, 1291, 5636, 2362, 3785, 1935, 3315, 5661, 1983, 5467, 6347, 409, 1438, 297, 4477, 4133, 3953, 1273, 2410, 1857, 5022, 3146, 5363, 6189, 5796, 272, 3410, 735, 6414, 398, 1642, 1628, 2597, 3885, 4850, 3436, 2701, 1896, 4568, 6636, 5142, 2030, 1015, 6553, 230, 1456, 135, 5569, 1197, 1792, 1822, 5447, 3263, 1817, 6984, 476, 5497, 6158, 1092, 6633, 6405, 6465, 7015, 3399, 5482, 842, 5445, 2595, 612, 5840, 3665, 6094, 6828, 5252, 5986, 1956, 4313, 6032, 5402, 2702, 1740, 5703, 6233, 1619, 1223, 3297, 3616, 1548, 4660, 1698, 4448, 1902, 560, 652, 3030, 1277, 5897, 334, 5646, 4486, 6277, 5074, 481, 5045, 2997, 2014, 2646, 485, 1594, 4380, 7071, 3628, 1490, 2275, 4330, 5032, 7104, 1873, 5530, 716, 5733, 772, 719, 1699, 4920, 1918, 1232, 2880, 3834, 1930, 4905, 2928, 6334, 3587, 4903, 5172, 1686, 1201, 1771, 1708, 2367, 802, 6742, 6601, 1892, 6439, 556, 4385, 1603, 5300, 4694, 120, 2012, 4287, 2087, 873, 4181, 6193, 3641, 5807, 5775, 550, 91, 6579, 6136, 2961, 6524, 2082, 144, 1847, 5510, 4022, 4363, 4647, 55, 1275, 6948, 6028, 613, 5431, 3343, 4406, 2884, 6561, 5590, 4735, 5247, 3417, 2618, 2669, 3532, 4172, 702, 6769, 5102, 6756, 4093, 7, 1612, 2204, 1614, 3542, 4754, 3156, 3790, 997, 7013, 601, 3314, 4679, 96, 5722, 158, 3209, 5422, 4040, 1955, 4226, 5637, 3191, 6124, 6558, 5179, 4254, 1811, 6795, 3525, 6841, 674, 2774, 4571, 5873, 2526, 5174, 5133, 4970, 6705, 758, 131, 6156, 6799, 1613, 6369, 3737, 3921, 5508, 4989, 3203, 1390, 3546, 1888, 5229, 6854, 1214, 5580, 6185, 3550, 6817, 1933, 5367, 1113, 4830, 1415, 6356, 1587, 5647, 3173, 6596, 3698, 1618, 404, 424, 1782, 6046, 4709, 1631, 2135, 850, 3707, 2683, 2430, 739, 6092, 5220, 849, 2298, 726, 1215, 4756, 5953, 5340, 3038, 1281, 3883, 7088, 1757, 4177, 846, 7078, 4302, 1829, 7086, 3493, 6458, 793, 1124, 5451, 251, 4148, 1592, 4866, 6422, 4043, 2741, 5571, 5617, 1784, 705, 4923, 98, 3577, 500, 1192, 6083, 4630, 4578, 6537, 6274, 4722, 3220, 3887, 1683, 5338, 337, 6386, 6091, 2850, 1348, 6768, 2443, 2238, 6920, 4539, 7047, 2853, 2593, 7043, 3611, 1202, 3661, 3, 3366, 1893, 6965, 6616, 732, 4420, 4333, 5516, 4276, 2168, 524, 1921, 1730, 5238, 902, 3789, 924, 2463, 7037, 1510, 6162, 1168, 2864, 3520, 2340, 1872, 2773, 1360, 5582, 3398, 5700, 3796, 176, 4031, 6533, 6517, 3062, 7123, 1985, 6581, 4713, 3830, 4147, 5827, 2916, 498, 5604, 914, 5212, 6563, 5307, 6215, 5663, 5434, 1616, 4634, 2601, 5816, 4241, 2382, 3530, 1989, 79, 3446, 6626, 4299, 6113, 2885, 44, 3824, 7017, 5903, 4171, 5187, 2807, 3128, 3081, 1053, 1187, 4305, 380, 5342, 2511, 4020, 912, 5203, 2520, 4235, 3303, 4252, 3361, 5167, 2375, 583, 6963, 5148, 2058, 2114, 1089, 586, 2069, 6192, 3269, 1696, 2388, 3414, 3900, 1796, 4701, 1352, 1880, 4556, 2487, 3154, 3120, 5782, 1447, 3321, 81, 450, 171, 4117, 291, 4238, 3624, 542, 6196, 936, 5550, 4163, 5472, 6205, 3058, 3448, 3059, 2575, 74, 3970, 5454, 6033, 2673, 648, 5823, 5933, 5721, 6669, 269, 2345, 2954, 3678, 4666, 1133, 2648, 5460, 6464, 1373, 5125, 3420, 2219, 2389, 1303, 5734, 31, 4178, 4622, 5888, 2893, 897, 6902, 5925, 5937, 4968, 3109, 3781, 4564, 6716, 4675, 2728, 5289, 2722, 9, 213, 1418, 3001, 6344, 4975, 4230, 3094, 3134, 493, 4244, 4929, 5577, 5487, 2025, 3584, 2943, 6360, 750, 4205, 4664, 5319, 5627, 2574, 1971, 6515, 6290, 6080, 2268, 4358, 3430, 5288, 5418, 4461, 160, 6423, 3229, 2878], [5643, 2304, 3586, 4552, 4097, 1544, 5130, 343, 524, 6658, 4624, 5780, 6675, 3094, 6683, 28, 4127, 3106, 1577, 1069, 6193, 4147, 5173, 4361, 4298, 5177, 3642, 66, 4107, 3653, 5702, 5729, 6731, 6220, 6221, 79, 3153, 4195, 4180, 3670, 3257, 1624, 89, 2655, 4112, 6755, 4196, 5222, 4713, 4203, 6764, 2158, 6845, 4722, 3702, 1129, 2168, 2681, 4543, 6780, 1151, 4740, 2693, 2695, 136, 5770, 5772, 4338, 1171, 660, 2197, 6294, 4759, 3227, 2716, 2206, 671, 2208, 4770, 2587, 4260, 5799, 925, 6517, 4785, 179, 2740, 5813, 1718, 2233, 3770, 4285, 1214, 191, 5824, 5579, 2244, 2678, 3270, 1225, 2250, 1740, 2509, 4816, 6353, 4308, 4309, 215, 6875, 890, 5343, 224, 3519, 1253, 2790, 231, 4841, 6378, 1263, 752, 3314, 755, 5364, 2294, 41, 3833, 1786, 2687, 6396, 5418, 1279, 1792, 2817, 774, 2825, 1405, 6615, 3852, 6413, 6928, 3129, 5399, 1304, 2842, 1157, 5851, 2852, 806, 1186, 5416, 1322, 5934, 816, 5683, 2868, 2358, 823, 1336, 3897, 315, 1855, 1248, 5442, 2372, 3397, 1352, 1591, 332, 5461, 6999, 3928, 4442, 2395, 5980, 5983, 6496, 6498, 5475, 2961, 1388, 4206, 6004, 885, 3746, 4428, 234, 2938, 2794, 6549, 2212, 4487, 3978, 5004, 3650, 910, 3472, 1432, 402, 4501, 920, 4505, 1435, 6045, 415, 4000, 1953, 1955, 1957, 5033, 1962, 5035, 2802, 2991, 5554, 6068, 3514, 5567, 4034, 6219, 1477, 5062, 6087, 6600, 5577, 6090, 3531, 7116, 2210, 5584, 6614, 3065, 1700, 987, 6621, 2469, 1124, 1787, 252, 1514, 5102, 3057, 6642, 6131, 3341, 1014, 5716, 6652, 6492, 2468, 5672, 2119, 4480, 3902, 6989, 3921, 5977, 5810, 5507, 3937, 1411, 677, 7047, 3373, 1899, 3262, 3251, 6416, 5985, 4167, 3074, 2129, 1107, 881, 2563, 3441, 2345, 5865, 4534, 2603, 5286, 2520, 99, 2021, 3019, 2578, 3585, 2522, 6042, 4687, 7124, 5947, 5352, 1493, 6018, 595, 2844, 6871, 2553, 687, 2219, 1091, 3828, 6985, 7066, 3604, 3168, 2049, 2806, 2607, 1123, 1866, 3181, 974, 261, 3773, 466, 849, 462, 6201, 1006, 4963, 669, 68, 5930, 819, 5714, 1438, 1305, 1566, 2494, 783, 3338, 2892, 2758, 225, 2814, 2626, 6227, 2627, 405, 1023, 5595, 2530, 2477, 5023, 5989, 2671, 5650, 3487, 5134, 2460, 4840, 6661, 4875, 2697, 3410, 3446, 1695, 25, 2864, 2417, 2795, 3252, 6910, 1456, 6098, 5193, 4358, 4453, 3677, 4630, 6826, 3477, 4048, 2455, 221, 6624, 6246, 2833, 2310, 4509, 5324, 916, 7049, 305, 1281, 1522, 4114, 484, 2490, 1224, 3130, 6256, 2336, 5734, 6305, 3732, 6960, 1153, 4868, 1874, 4403, 4489, 3696, 5114, 609, 3218, 5514, 1341, 4607, 2458, 1113, 1017, 2353, 5450, 2965, 6202, 6857, 6752, 1413, 4558, 4413, 1291, 1256, 900, 4255, 4626, 364, 6245, 6501, 3134, 428, 1494, 3133, 4177, 648, 1251, 685, 128, 1471, 1370, 1422, 277, 3034, 2884, 4353, 3326, 6787, 2734, 1941, 4226, 4883, 4293, 1752, 3900, 4569, 4355, 5685, 4574, 6369, 1839, 3069, 5054, 2565, 1646, 2004, 2259, 6622, 665, 4496, 5075, 5591, 1934, 3689, 2933, 6768, 4244, 6905, 507, 2652, 1489, 6328, 640, 6843, 144, 5583, 1423, 3064, 6805, 4909, 3145, 4017, 7083, 867, 941, 1334, 4111, 6036, 636, 4081, 6559, 67, 5529, 469, 6908, 5248, 5338, 933, 3511, 5038, 2914, 5069, 1233, 561, 1174, 6581, 5229, 5692, 3656, 2180, 2670, 6829, 4908, 5419, 6774, 382, 3634, 6365, 782, 3859, 1602, 6156, 5817, 2470, 2240, 1015, 4542, 1272, 3042, 3015, 2260, 5843, 3654, 5474, 6005, 3776, 29, 5084, 100, 3186, 4380, 5313, 2392, 1809, 1766, 5815, 2589, 3343, 2573, 4132, 5032, 6682, 3484, 1027, 3672, 3135, 6334, 5480, 4254, 4385, 6554, 4043, 1310, 285, 6255, 2613, 307, 4057, 2674, 2924, 1163, 1262, 1046, 5210, 5087, 1146, 4162, 3354, 3690, 3407, 4517, 4030, 1619, 4495, 1121, 2291, 2926, 2024, 779, 3860, 6459, 2175, 2320, 695, 2084, 316, 1696, 1954, 1581, 5369, 2085, 4277, 3189, 6572, 237, 761, 4573, 1913, 984, 3328, 5258, 969, 2189, 3861, 4219, 5649, 1550, 3891, 1007, 3165, 6293, 5532, 23, 706, 5208, 3959, 1097, 3595, 1892, 6216, 6458, 613, 4871, 4899, 6580, 777, 6367, 5354, 5606, 4930, 3750, 820, 1228, 485, 1626, 6432, 3815, 6381, 1379, 6529, 135, 897, 2581, 1853, 2954, 4835, 259, 7010, 3751, 1537, 1538, 3183, 6646, 3394, 7016, 4775, 2285, 3036, 4272, 3581, 3445, 681, 961, 5468, 326, 5339, 4857, 4945, 1659, 7029, 3752, 2683, 2815, 4433, 1857, 2881, 1116, 2725, 1658, 4686, 771, 6822, 6541, 4465, 2976, 3459, 5777, 5837, 5311, 4611, 4864, 2531, 5948, 3043, 2209, 1139, 1576, 6514, 3151, 5750, 3515, 5347, 3440, 4706, 495, 1213, 1297, 2751, 5201, 2166, 584, 3411, 5614, 4647, 6556, 3786, 6500, 1401, 657, 970, 6032, 6319, 1273, 6640, 733, 2599, 2041, 2440, 4859, 4627, 5328, 4056, 2975, 3636, 1722, 6214, 846, 778, 6483, 955, 4324, 4535, 417, 6571, 5693, 3927, 5096, 5829, 1143, 4993, 2267, 2235, 5609, 1822, 3464, 3010, 2179, 3550, 5233, 483, 5293, 6772, 2246, 3222, 6430, 2665, 633, 1371, 3872, 3276, 2628, 4216, 165, 4707, 5329, 3532, 3400, 5678, 4635, 269, 6440, 1780, 2432, 1880, 5727, 65, 2059, 1641, 3184, 5831, 363, 1603, 1499, 1915, 3658, 6694, 3838, 616, 5610, 7051, 3895, 7112, 356, 1516, 6205, 902, 5616, 5434, 6922, 1464, 2251, 4725, 4723, 5263, 5655, 901, 2296, 2334, 6634, 4612, 7015, 4246, 5965, 4146, 688, 1878, 1720, 7044, 611, 6613, 3, 4555, 4248, 7071, 4849, 5765, 1649, 2360, 142, 2523, 4604, 4183, 6830, 5607, 2147, 3620, 3831, 5569, 5058, 4432, 2173, 3451, 1865, 808, 3059, 140, 5212, 6100, 642, 4609, 4838, 6883, 1554, 6276, 4391, 297, 451, 1811, 3575, 1079, 4935, 4928, 3969, 4079, 3350, 980, 5053, 5317, 5627, 4235, 6889, 6834, 3141, 6226, 262, 5882, 4325, 1398, 2456, 5131, 2623, 594, 5964, 1956, 2866, 2630, 1932, 171, 454, 6790, 2412, 4301, 6820, 805, 5430, 97, 394, 6112, 2112, 3281, 1340, 5215, 6420, 3638, 4396, 4990, 4389, 1416, 1879, 3855, 5906, 4038, 6584, 293, 2414, 5782, 3500, 1486, 5894, 1166, 4418, 1553, 1205, 5642, 3105, 3701, 3518, 6323, 1195, 3566, 1397, 7035, 3784, 2350, 1127, 6143, 6355, 6322, 2862, 6253, 7028, 3362, 5052, 2471, 4486, 6109, 4518, 2495, 5769, 1976, 2686, 1087, 1569, 1179, 4671, 5689, 912, 3293, 749, 1693, 1200, 7007, 5868, 3768, 6819, 4952, 3667, 3336, 997, 3279], [4779, 4107, 6159, 27, 28, 2077, 4126, 33, 6179, 2084, 6492, 4141, 4151, 6201, 6202, 5813, 2117, 2119, 72, 6219, 4172, 4174, 4177, 2131, 6229, 6232, 89, 90, 2139, 94, 4195, 4203, 6253, 2158, 4209, 4210, 119, 2173, 128, 4225, 131, 6276, 4229, 4239, 6289, 6291, 6293, 6294, 4249, 4463, 156, 4254, 4255, 165, 6311, 171, 29, 178, 6323, 185, 4285, 2239, 4293, 5154, 4304, 212, 6357, 2270, 6368, 4321, 4324, 4325, 4775, 2284, 237, 2291, 2296, 252, 256, 4353, 6407, 4361, 267, 2093, 4372, 4379, 4380, 6430, 2340, 295, 2780, 301, 3464, 4407, 2360, 6464, 6465, 4420, 4421, 4426, 2379, 2383, 337, 3811, 343, 2394, 348, 2399, 2400, 6498, 1425, 2408, 365, 367, 6513, 6514, 5468, 2431, 6529, 4485, 394, 4495, 4496, 6556, 6559, 2468, 2471, 4534, 3145, 4537, 2334, 473, 4570, 6620, 477, 2533, 2544, 6648, 6911, 508, 510, 2564, 2571, 4622, 2579, 6677, 6682, 6683, 2588, 2589, 6686, 2591, 2594, 552, 2603, 2605, 2607, 6705, 2614, 6712, 2617, 6716, 4669, 2623, 578, 6723, 583, 586, 6731, 6732, 6737, 594, 595, 2645, 2651, 5562, 613, 614, 4713, 6763, 6764, 4723, 6772, 2677, 2678, 3519, 6780, 6782, 6784, 4738, 6787, 4740, 790, 4751, 2704, 657, 660, 6806, 4759, 4760, 2714, 4763, 2716, 6813, 2719, 6819, 6820, 2725, 2727, 2273, 687, 2737, 6834, 2739, 2740, 6839, 1482, 2751, 708, 4817, 6870, 732, 6881, 2786, 4835, 2790, 4841, 746, 749, 6269, 754, 2806, 4859, 6910, 2815, 6915, 6917, 774, 4880, 4883, 4886, 6937, 801, 4903, 2864, 4918, 5257, 6969, 2874, 6974, 831, 2884, 6966, 4935, 840, 6985, 6989, 2898, 6996, 2903, 7003, 860, 4581, 864, 2914, 7014, 881, 885, 7038, 7041, 7049, 908, 910, 912, 2961, 3781, 5016, 2973, 4251, 7077, 5032, 938, 7083, 5038, 2991, 946, 5046, 955, 5053, 5057, 965, 7112, 3020, 5069, 3028, 3030, 984, 3034, 5352, 5087, 4261, 1397, 5096, 4125, 1008, 5107, 1014, 1017, 1021, 5119, 5124, 5134, 5136, 1046, 516, 3098, 1051, 3105, 3106, 5160, 1067, 5173, 1080, 5177, 3133, 3134, 5184, 3143, 1097, 3147, 1105, 3154, 2233, 5211, 5214, 1124, 5222, 3176, 1129, 5229, 3183, 1143, 3192, 1147, 1148, 1153, 1161, 3236, 5288, 2631, 3247, 1202, 5859, 1215, 1225, 1238, 5338, 1244, 3298, 1251, 1252, 6111, 1256, 5355, 3309, 1263, 5364, 2260, 3323, 3326, 3338, 793, 3343, 5394, 3350, 5399, 3364, 3366, 3367, 3368, 5417, 5419, 3374, 1329, 1334, 3386, 3397, 1350, 3399, 3591, 5450, 5452, 5346, 3417, 5466, 3420, 3425, 1380, 5482, 3436, 6717, 5489, 5493, 1401, 3451, 3457, 1416, 1420, 1424, 7064, 1426, 5528, 5529, 1435, 1438, 3499, 3507, 3511, 3514, 1469, 3518, 1471, 1476, 3530, 3531, 3541, 5606, 1514, 3565, 5617, 1522, 3573, 3574, 3575, 1537, 1538, 2305, 3602, 5655, 1563, 1569, 1572, 3638, 3645, 3650, 3653, 5706, 1618, 5716, 5717, 1623, 1624, 3674, 5723, 1629, 3681, 1639, 1641, 5739, 5745, 5746, 1658, 3718, 3720, 5771, 5772, 1682, 5780, 3743, 5793, 1700, 1701, 3750, 5810, 3765, 5824, 5826, 5829, 5832, 3786, 1740, 3789, 5856, 1763, 4391, 1772, 1795, 1800, 1801, 3855, 3861, 5913, 1818, 3867, 2693, 1827, 1493, 1837, 5938, 3899, 3902, 1856, 5958, 5965, 1872, 1877, 1879, 5977, 655, 5434, 4752, 5986, 3943, 5993, 1899, 4758, 6033, 1940, 3993, 3994, 4000, 4007, 5589, 3059, 6068, 4771, 1982, 4031, 4035, 6098, 2003, 6109, 677, 2027, 4078, 4079, 6128, 1023, 5004, 2147, 4147, 1192, 5609, 3876, 1585, 5253, 4468, 1155, 1842, 455, 2641, 5893, 4913, 5640, 5418, 2129, 2132, 7012, 5365, 2130, 5514, 778, 4259, 409, 3634, 4612, 5947, 1462, 4791, 3860, 4432, 2189, 4966, 802, 1226, 1786, 6972, 4001, 5246, 4095, 1179, 1253, 3471, 5937, 2083, 1564, 2477, 529, 1713, 1176, 3953, 4231, 5524, 6148, 5072, 959, 451, 430, 5612, 4725, 4624, 4965, 3683, 5555, 1544, 1180, 1324, 4597, 1780, 5702, 6113, 3189, 1451, 6156, 3596, 4968, 2136, 1873, 274, 464, 5588, 2551, 2028, 3735, 1048, 6651, 4453, 5693, 4934, 6187, 149, 4096, 5722, 2687, 6695, 3427, 954, 1957, 6861, 4853, 1816, 2122, 980, 4826, 5758, 5607, 6024, 6340, 4730, 6543, 3094, 2456, 2711, 2290, 7021, 2217, 2330, 4182, 1575, 7015, 1054, 5882, 7035, 6797, 2212, 5351, 6630, 1659, 3904, 6826, 427, 2837, 6675, 3118, 6409, 2059, 4443, 2791, 928, 6384, 1121, 926, 2372, 4094, 1033, 3554, 7051, 3212, 2458, 6339, 6080, 1741, 3642, 699, 1394, 6045, 7105, 2375, 5278, 4226, 4635, 1468, 1302, 2781, 298, 1096, 2652, 3732, 3153, 1635, 3239, 5819, 4937, 6155, 2002, 1723, 4825, 2081, 1722, 5498, 3426, 2946, 2916, 230, 1523, 4543, 1974, 1843, 5727, 3934, 1454, 2401, 6130, 5839, 1822, 805, 1368, 1947, 6249, 2842, 3890, 4026, 3501, 4857, 4057, 3891, 4167, 466, 792, 6539, 5297, 5738, 6558, 4515, 2689, 3836, 2377, 4599, 143, 4768, 3844, 1931, 3089, 1855, 4992, 623, 3586, 3308, 2344, 897, 2868, 5474, 1943, 2778, 4308, 503, 2397, 5592, 3186, 4851, 5643, 1242, 3181, 6331, 6942, 537, 3588, 4493, 7028, 5621, 6222, 1893, 4180, 5080, 2844, 4412, 2876, 5868, 5147, 2829, 6158, 3640, 1342, 324, 2613, 3463, 1445, 561, 4584, 568, 1669, 2738, 7016, 2438, 2327, 911, 4211, 685, 3990, 380, 84, 4644, 5391, 3595, 1928, 245, 3253, 4016, 3484, 4742, 1937, 3136, 4505, 1108, 4130, 2219, 1312, 5911, 3815, 4856, 4745, 4631, 4909, 1930, 85, 4852, 1318, 618, 1091, 4946, 6873, 3318, 253, 302, 1258, 5921, 5000, 2376, 3935, 3429, 6816, 4976, 3759, 16, 3672, 5883, 1304, 4142, 6185, 402, 2802, 4816, 6776, 2882, 3628, 752, 5573, 3022, 5712, 1139, 2941, 4034, 5705, 4461, 2145, 5277, 999, 4469, 755, 2251, 5984, 2306, 7045, 4892, 750, 3838, 5833, 2206, 2231, 5599, 5233, 5577, 5285, 1144, 6440, 2304, 2395, 2955, 4727, 6356, 4206, 4787, 4454, 6416, 5444, 6236, 823, 2773, 2435, 6694, 2900, 5770, 310, 847, 104, 2560, 4960, 1268, 634, 647, 1734, 3822, 2037, 1892, 6678, 4222, 418, 3103, 4480, 2515, 956, 3052, 1590, 5248, 1952, 1174, 4370, 4634, 6520, 5025, 3157, 4437, 65, 4145, 7036, 2887, 3440, 4171, 4696, 1383, 1337, 210, 332, 3948, 2924, 2175, 3800, 3533, 3721, 3581, 735, 2151, 2210, 3041, 987, 2939, 4455, 136, 1418, 1492, 6474, 1353, 5081, 6811, 1934, 2473, 1009, 1398, 6740, 6112, 234, 2522, 123, 5583, 25, 5848, 1356, 3001, 5212, 4183], [3, 2059, 17, 28, 29, 4126, 6202, 65, 69, 4172, 89, 2139, 4189, 2147, 4199, 4207, 6256, 4209, 114, 128, 4226, 4231, 6291, 6294, 4248, 4255, 165, 2219, 2230, 2231, 185, 2239, 4294, 203, 4301, 4304, 2273, 4324, 230, 3453, 2296, 2305, 2308, 267, 6413, 4372, 4380, 6430, 4383, 2340, 297, 302, 305, 2354, 315, 6464, 6465, 6469, 6474, 2379, 2393, 2394, 348, 2401, 6498, 4455, 2408, 4463, 6513, 6514, 6517, 2431, 6529, 4485, 6539, 4495, 4496, 402, 6556, 6558, 4515, 2471, 430, 4851, 3145, 6584, 4537, 455, 6220, 4554, 6611, 2516, 6615, 4573, 2532, 4581, 6630, 2335, 4599, 2556, 3499, 516, 517, 2571, 524, 2573, 4622, 508, 4633, 6682, 6683, 552, 2603, 4709, 2613, 2614, 2622, 2623, 578, 583, 6731, 6737, 597, 4696, 613, 623, 6770, 2678, 6776, 2687, 4738, 4886, 4743, 655, 2704, 657, 660, 6806, 5913, 4760, 2719, 112, 6819, 2725, 4210, 2734, 687, 6837, 2751, 2781, 2786, 4835, 749, 755, 4852, 2806, 6911, 4876, 2833, 2837, 790, 2841, 2842, 4892, 2864, 816, 3891, 2868, 4918, 831, 6979, 2884, 840, 6985, 849, 5262, 2903, 860, 2534, 7014, 880, 881, 885, 2938, 7036, 2941, 7038, 4993, 2946, 900, 7045, 7048, 2955, 2961, 2973, 2544, 3227, 936, 5038, 946, 955, 5053, 966, 5072, 980, 3034, 987, 5084, 5087, 4945, 4604, 1014, 3065, 5114, 5130, 5136, 4612, 3106, 3112, 5173, 1080, 5177, 3143, 1097, 3147, 3151, 5212, 5649, 1129, 1139, 1146, 1148, 5257, 1162, 1166, 1171, 5273, 1179, 1186, 3262, 1225, 2564, 3289, 5338, 1244, 3293, 1248, 1251, 1253, 5352, 1263, 5364, 1918, 3326, 3328, 3338, 3343, 3350, 3352, 3366, 5416, 5417, 1324, 3394, 3397, 5452, 5347, 5475, 3440, 5489, 5498, 1405, 1418, 1455, 3477, 5528, 5529, 1435, 1445, 2631, 925, 3515, 1468, 3518, 1471, 5572, 1477, 5579, 5583, 5584, 931, 1492, 1493, 1509, 5606, 1514, 3565, 1522, 5622, 1533, 1537, 1538, 3591, 3600, 1624, 3604, 5655, 5675, 1582, 1583, 3638, 1602, 1603, 5700, 5729, 1623, 3672, 1629, 5727, 3680, 3681, 1641, 5738, 3689, 6420, 5766, 1672, 5770, 5771, 1304, 5780, 1693, 3743, 5793, 3746, 1700, 1704, 4353, 5815, 5819, 5824, 3784, 5833, 3786, 3800, 1772, 1780, 1782, 1799, 1809, 1817, 3866, 1827, 988, 3720, 1843, 1852, 1853, 1879, 3929, 5978, 3936, 5989, 1899, 3953, 3957, 3964, 6014, 1927, 1928, 1947, 6044, 1953, 6068, 4030, 4034, 4048, 4056, 4057, 6112, 6113, 2024, 4078, 5769, 3627, 6732, 3353, 4935, 3176, 3155, 5601, 4164, 237, 4501, 7016, 6766, 4958, 5599, 7006, 5474, 6322, 2304, 2459, 5184, 5693, 6574, 3035, 750, 1352, 6440, 5045, 1793, 6473, 3126, 5714, 403, 4412, 1347, 4655, 3236, 5300, 851, 2369, 309, 1929, 6821, 3507, 5871, 5972, 783, 4817, 3337, 5562, 1208, 3611, 2406, 3586, 4182, 6276, 2171, 2779, 4729, 5702, 4530, 6100, 1446, 4903, 5321, 1890, 3788, 5573, 3533, 7109, 1226, 974, 6901, 167, 4813, 5486, 4639, 5418, 5589, 143, 504, 6739, 5422, 3202, 3277, 3861, 4407, 6222, 5070, 5167, 4638, 938, 920, 4453, 2784, 3538, 6917, 3478, 5705, 6201, 1917, 5429, 1013, 5935, 2873, 5219, 3402, 84, 2522, 7000, 6033, 6651, 2397, 692, 1962, 483, 4779, 6102, 475, 3837, 7049, 6971, 4094, 5717, 6771, 1856, 4017, 1336, 1144, 6131, 2591, 7044, 6559, 6644, 2848, 3511, 2468, 2060, 6902, 3774, 5069, 3754, 7114, 5737, 1599, 7003, 1873, 4236, 3575, 4664, 941, 1105, 6461, 6839, 793, 4000, 2225, 3257, 4079, 4468, 179, 3030, 2679, 2216, 3162, 1591, 5510, 2528, 2477, 2588, 2320, 4370, 85, 5607, 4196, 708, 3913, 274, 296, 4378, 5609, 1390, 4026, 4856, 7112, 3642, 6686, 1397, 6600, 4976, 5391, 1848, 7012, 1422, 6678, 5533, 6813, 3820, 1906, 2491, 6092, 4690, 6802, 2345, 7071, 656, 4908, 3496, 1735, 6311, 3417, 6215, 1046, 31, 6339, 6156, 1322, 390, 6368, 4357, 2210, 1238, 3459, 4940, 2758, 3181, 2180, 498, 2900, 2492, 6246, 2140, 3653, 3765, 1837, 473, 4222, 4171, 6130, 3640, 2175, 3768, 1705, 5434, 6008, 4550, 5000, 79, 7053, 6972, 4204, 2748, 2991, 3111, 1482, 6860, 6593, 5977, 4141, 5288, 3119, 3144, 685, 152, 6664, 4871, 4454, 3852, 1299, 5536, 640, 2152, 2151, 231, 2208, 343, 4254, 6918, 6111, 6314, 3997, 1818, 3750, 3636, 6273, 1658, 4912, 2405, 4988, 1172, 823, 5859, 5772, 2003, 5891, 5102, 2653, 1330, 2876, 847, 3683, 5903, 1270, 2802, 6680, 1726, 5559, 3732, 846, 5343, 3982, 451, 1114, 6082, 6447, 911, 6841, 2021, 4238, 224, 6233, 393, 2133, 1987, 5987, 4880, 5062, 3105, 926, 3519, 1523, 995, 5830, 5185, 791, 2049, 477, 3385, 2237, 2365, 4125, 4167, 4112, 5493, 6549, 3759, 1933, 4872, 4420, 259, 2025, 3098, 3735, 6219, 4448, 1682, 4824, 792, 2281, 5348, 6034, 3298, 5976, 3731, 2671, 5934, 6334, 3079, 6397, 6881, 5473, 322, 4571, 6940, 5854, 4699, 6020, 3103, 3994, 1801, 6193, 3842, 5627, 4999, 855, 1054, 6807, 3970, 6236, 5748, 2716, 470, 4883, 2189, 536, 1137, 5081, 6855, 5160, 3059, 266, 4758, 6966, 2445, 5128, 1737, 6716, 6323, 2495, 2579, 6787, 3943, 1554, 2129, 5988, 1569, 1805, 6046, 4249, 2332, 1701, 2252, 2469, 3550, 3127, 2082, 4310, 3276, 3000, 5119, 5394, 2290, 4635, 568, 1128, 2124, 3315, 5997, 4534, 6853, 6399, 2360, 3094, 2698, 2992, 5986, 6910, 6811, 252, 3756, 1787, 4016, 1096, 6363, 1408, 774, 4487, 5917, 3429, 5228, 6353, 569, 4582, 1350, 1091, 77, 5208, 3854, 7010, 7118, 5829, 6752, 6734, 3885, 2681, 5083, 6237, 7028, 510, 3935, 422, 1842, 3989, 2594, 5856, 4934, 576, 4235, 2844, 2993, 721, 4556, 1424, 3472, 5640, 2871, 1957, 2383, 2084, 6158, 778, 1021, 4338, 4147, 3770, 27, 6499, 2306, 624, 6194, 2438, 699, 3633, 6679, 3100, 4740, 2914, 5882, 1370, 1715, 3073, 5183, 525, 5299, 1792, 6968, 4055, 5330, 6749, 1963, 244, 6805, 4829, 1121, 5156, 1519, 4031, 6883, 6492, 210, 3701, 2924, 2373, 1925, 5080, 5355, 6928, 666, 6875, 2636, 6915, 1931, 1030, 2677, 6391, 6587, 1476, 5809, 367, 4909, 4162, 2417, 4127, 6206, 5911, 4799, 2633, 4265, 5202, 5324, 4593, 349, 2926, 6974, 6501, 2587, 271, 6834, 6024, 4568, 2119, 4151, 1371, 4144, 5419, 4713, 5169, 6740, 3002, 4832, 5107, 7055, 2740, 5209, 6221, 3873, 299, 4580, 1338, 2260, 3341, 1855, 585, 3386, 6547, 3192, 4667, 1804, 6923, 3081, 3316, 1897, 3399, 5905, 3146, 6609, 6166], [2560, 3499, 2564, 5130, 2571, 525, 6159, 5136, 5649, 2579, 171, 4751, 6233, 5712, 6682, 1563, 2077, 6686, 2079, 5154, 2596, 5672, 348, 4141, 1582, 4143, 5171, 5173, 3574, 4151, 6712, 2617, 6202, 3131, 6154, 3765, 3136, 2607, 3653, 2119, 3145, 3147, 6220, 4026, 6464, 4177, 3154, 595, 84, 6229, 4183, 2884, 89, 2139, 3677, 5727, 4195, 1124, 3173, 2152, 5738, 6764, 6253, 6766, 3183, 112, 1137, 4883, 3518, 2679, 6266, 6780, 2173, 2084, 2687, 4737, 5364, 6276, 4229, 4420, 5771, 5772, 655, 6289, 427, 6291, 3735, 2715, 2716, 6170, 4254, 2719, 6817, 1186, 4771, 6820, 2758, 4262, 6513, 5803, 858, 2456, 2734, 687, 5810, 1715, 2740, 1205, 185, 3143, 6334, 1215, 3399, 708, 4993, 4294, 6855, 3784, 713, 1143, 1740, 6033, 4627, 4817, 210, 4307, 6357, 2786, 5338, 5339, 732, 3805, 1248, 4321, 5346, 6950, 230, 5351, 4840, 1148, 4391, 749, 4846, 1263, 2802, 3861, 3318, 4856, 1014, 251, 252, 6399, 1792, 4353, 774, 263, 267, 4876, 3343, 2833, 4673, 4126, 2837, 3350, 4740, 2843, 4380, 5917, 1629, 4485, 5829, 1827, 2340, 1318, 3367, 4306, 3515, 2175, 6445, 6447, 816, 5000, 677, 1337, 3386, 5599, 1418, 6974, 320, 5856, 2955, 2372, 1334, 1350, 6881, 2376, 6985, 1477, 420, 5964, 3298, 337, 4946, 851, 1877, 343, 4952, 3929, 2394, 7003, 860, 2874, 3934, 165, 6498, 2961, 2410, 364, 4463, 1256, 6514, 884, 3390, 6008, 5434, 7038, 2431, 7041, 7045, 6465, 3464, 4490, 4759, 1933, 237, 4725, 913, 1940, 3477, 3478, 5528, 5529, 2973, 1438, 4000, 6390, 4293, 931, 2468, 935, 583, 3501, 430, 2991, 1523, 4534, 3511, 1464, 6073, 3514, 955, 1982, 4635, 3902, 1476, 6085, 5793, 2892, 1994, 3020, 6093, 5147, 3743, 3026, 4056, 1493, 3030, 5080, 1497, 4687, 6620, 5598, 6111, 2233, 5089, 4909, 6630, 4775, 3565, 4078, 2544, 5617, 3059, 4364, 3575, 3577, 6651, 3597, 4095, 3352, 7023, 4480, 3039, 3202, 5820, 3100, 5604, 4697, 5107, 1662, 6087, 2491, 6776, 3417, 5300, 1815, 2145, 1409, 2766, 779, 793, 2737, 2356, 4433, 2594, 76, 806, 597, 1031, 1723, 6528, 5646, 6222, 5205, 928, 1160, 6763, 2662, 613, 5445, 843, 1517, 6118, 5407, 3761, 4631, 5228, 2350, 2551, 6558, 4750, 1388, 5384, 537, 6014, 2132, 2312, 6046, 3633, 3957, 3087, 3683, 2618, 65, 6391, 5365, 1347, 1608, 131, 3750, 6416, 5579, 1704, 5317, 4130, 4372, 6270, 5232, 5652, 1275, 6937, 6405, 5851, 3334, 3118, 3595, 2401, 4577, 4209, 6198, 712, 29, 6915, 3602, 5053, 3811, 6381, 1380, 6100, 910, 6559, 6369, 5234, 1984, 4001, 3586, 3509, 2907, 2334, 2305, 7035, 6506, 3368, 2850, 4225, 4561, 6042, 1312, 4166, 3034, 6839, 3846, 5087, 2425, 2774, 3112, 3089, 418, 1166, 6910, 6619, 302, 6249, 538, 3486, 4034, 2889, 3890, 984, 4231, 27, 6314, 1565, 3180, 3937, 64, 6996, 1587, 6791, 6084, 5764, 5253, 7115, 6228, 6969, 5355, 2260, 5717, 4543, 4554, 3922, 1324, 2838, 1168, 2272, 3634, 2677, 1213, 570, 3440, 3169, 7055, 1843, 880, 582, 3155, 2351, 6311, 3171, 4211, 3008, 6978, 979, 5553, 5383, 4082, 5966, 1252, 2773, 2458, 4863, 167, 241, 1227, 4031, 476, 5416, 73, 1722, 2251, 3028, 5288, 2420, 3253, 1823, 6891, 4007, 4853, 5583, 868, 758, 5124, 5589, 6680, 1471, 3759, 4053, 6413, 4813, 5996, 6648, 4017, 5517, 6779, 2725, 1454, 2839, 6952, 6236, 6509, 926, 6080, 5441, 2471, 4949, 3206, 2589, 5778, 4259, 4114, 6966, 5236, 196, 6323, 1092, 1769, 3326, 888, 5813, 1899, 6815, 4918, 586, 6811, 6356, 6695, 2083, 2791, 4187, 5871, 6705, 1734, 2308, 6161, 6683, 3309, 3681, 6959, 2980, 2158, 5668, 6901, 2542, 4899, 4743, 5069, 3103, 13, 4453, 2938, 4370, 4906, 5524, 6860, 4493, 4604, 616, 657, 1504, 5446, 3, 2655, 1440, 660, 4310, 3004, 2160, 6294, 4358, 1179, 995, 6664, 5330, 3533, 7101, 6041, 2941, 4507, 6543, 1171, 3186, 1514, 1782, 1199, 4609, 5622, 3247, 3573, 310, 6429, 5422, 5714, 5864, 3786, 3455, 1863, 1150, 2847, 2415, 6044, 4184, 1238, 3638, 1796, 5487, 3948, 3689, 462, 5938, 6709, 1223, 6611, 1741, 569, 1951, 3674, 4597, 4308, 6466, 1658, 1353, 1023, 6108, 5876, 7047, 3397, 1425, 3997, 1564, 1804, 7028, 3491, 2472, 3168, 4162, 3394, 992, 6187, 1589, 4392, 6643, 4412, 6130, 3863, 4638, 4826, 1193, 2638, 4223, 3520, 5882, 2732, 2447, 6989, 114, 6841, 2778, 2747, 5389, 3844, 1853, 262, 5116, 5615, 3721, 2473, 4622, 1974, 4324, 4917, 6354, 4301, 5768, 4450, 4079, 6569, 2603, 5297, 623, 451, 6976, 1955, 5016, 3970, 6650, 6396, 4723, 1114, 1537, 5112, 3276, 4760, 4210, 2169, 5498, 6911, 936, 2131, 4167, 2432, 552, 1446, 2335, 6584, 2894, 1435, 1583, 2914, 1214, 6497, 5614, 3610, 4568, 5278, 2779, 5722, 6819, 1097, 5214, 2294, 1669, 524, 5370, 6098, 3106, 5180, 3065, 715, 5812, 4582, 1335, 750, 5840, 6993, 296, 2522, 3649, 3936, 1817, 283, 6772, 1037, 2212, 1398, 1937, 1313, 2399, 444, 3259, 987, 5775, 4849, 2829, 6588, 1893, 5352, 2666, 3999, 5809, 6016, 4168, 5584, 1696, 4486, 3525, 1910, 1522, 1492, 3644, 2353, 5229, 4457, 2652, 3838, 5508, 1947, 3354, 798, 521, 1295, 911, 3909, 4730, 508, 3344, 3865, 4633, 4361, 4825, 6520, 1208, 5203, 818, 4940, 547, 790, 5748, 2841, 3675, 4588, 5729, 6851, 3824, 3720, 2494, 2003, 5322, 4265, 3831, 1416, 3035, 2857, 3420, 6305, 5977, 2358, 1302, 3360, 2723, 7083, 1746, 1861, 6750, 5934, 4107, 7014, 5693, 3150, 4966, 6145, 4426, 4461, 6478, 4538, 3349, 2354, 2383, 3990, 791, 4886, 4991, 2216, 6615, 4200, 3236, 1176, 1918, 2868, 2445, 5187, 1047, 5134, 1130, 5084, 3604, 5334, 1752, 2136, 3807, 5460, 879, 5766, 1224, 5621, 1816, 3181, 2623, 3777, 3427, 2469, 164, 788, 5953, 1705, 5675, 6474, 1046, 4707, 1070, 202, 1096, 4454, 6600, 128, 402, 6155, 2088, 5038, 3105, 6549, 1247, 2397, 6511, 454, 864, 6814, 3905, 3459, 5769, 4255, 2462, 6469, 2189, 5965, 6340, 4732, 1945, 966, 5263, 5700, 422, 3219, 426, 589, 5374, 2877, 2087, 4802, 456, 6109, 6235, 1376, 3718, 2465, 3076, 349, 1962, 1108, 6731, 3815, 4203, 4855, 3951, 1679, 4749, 1174, 7016, 1242, 1161, 4935, 1342, 2693, 6081, 6828, 3129, 1721, 1462, 5347, 2291, 3808, 2239, 4857, 2864, 4880, 2704, 771, 1772, 6164, 3507, 6870, 5394, 1786, 4460, 1509, 6179, 6787, 1849, 6909, 840, 5716], [6154, 6155, 6159, 25, 6170, 28, 2077, 4126, 2079, 33, 6179, 2084, 6492, 4141, 2397, 4144, 4151, 6202, 5813, 2119, 72, 2122, 2131, 84, 6229, 4183, 2136, 89, 90, 2139, 94, 2145, 4195, 2152, 6249, 4203, 6253, 1572, 112, 4209, 4210, 4211, 2167, 6266, 123, 4220, 2173, 4223, 128, 6276, 4229, 4231, 2189, 143, 6289, 6291, 149, 6294, 4249, 156, 4254, 4255, 4259, 165, 6311, 171, 4125, 178, 2231, 185, 4285, 2239, 4293, 203, 5154, 4302, 4304, 210, 4307, 6356, 6357, 2270, 4321, 212, 2284, 237, 2291, 6390, 2296, 252, 256, 4353, 4355, 263, 267, 275, 4372, 2327, 4379, 4380, 6430, 2340, 295, 6445, 2353, 2360, 4412, 4490, 6464, 6465, 4420, 4421, 2375, 2376, 2377, 6882, 2383, 337, 343, 2394, 6543, 348, 4154, 2399, 2400, 6498, 4453, 4454, 365, 4463, 6513, 6514, 6520, 2431, 4480, 6529, 4485, 4486, 394, 2445, 4495, 4496, 402, 405, 409, 4507, 6556, 6559, 418, 4515, 2468, 2471, 6568, 2473, 431, 2481, 4534, 3145, 4537, 2515, 6620, 483, 2532, 2533, 2534, 2544, 6227, 503, 6648, 6651, 2560, 2564, 521, 2571, 2573, 4622, 6232, 2579, 6677, 4631, 4633, 6682, 6683, 2589, 6686, 5893, 547, 552, 2603, 2607, 2614, 1801, 6712, 2617, 578, 6723, 583, 2633, 6731, 594, 595, 6740, 2651, 5562, 613, 4713, 6764, 6766, 623, 104, 4723, 6772, 4725, 2678, 4730, 6780, 6784, 2689, 4738, 6787, 4740, 790, 5911, 4751, 2704, 657, 6802, 660, 6806, 2715, 2716, 6813, 2719, 4768, 4771, 6820, 677, 2273, 687, 2737, 6834, 2739, 2740, 6839, 3530, 708, 6855, 713, 119, 6860, 4817, 6870, 2779, 732, 6881, 2786, 2790, 749, 755, 4852, 4853, 6902, 4856, 4857, 4859, 6910, 2816, 6915, 6917, 4876, 2829, 4883, 2837, 4886, 6937, 2848, 4912, 805, 806, 4903, 1842, 6959, 2864, 4918, 6969, 2874, 6974, 831, 2882, 2884, 6966, 2887, 840, 6985, 6987, 6989, 6993, 4946, 851, 4949, 2903, 4952, 858, 7003, 860, 4581, 864, 6630, 7014, 881, 885, 7035, 2941, 7038, 7040, 7041, 7044, 7045, 5000, 2955, 909, 5006, 911, 912, 2961, 3781, 7064, 2973, 931, 936, 938, 7083, 5038, 2991, 946, 3001, 955, 7101, 959, 7105, 5059, 965, 7112, 3020, 3022, 5072, 3030, 5080, 3034, 5352, 5087, 5096, 3049, 3052, 1008, 1009, 5107, 1014, 1017, 5119, 5136, 5147, 3106, 3591, 3124, 5173, 1080, 5177, 3134, 3253, 3136, 3143, 1097, 3147, 3154, 5212, 4965, 1124, 4966, 5222, 1129, 3183, 1143, 1147, 1148, 1153, 5257, 5263, 5273, 1179, 1180, 5278, 1186, 4635, 3236, 1192, 1202, 4638, 3254, 1208, 3258, 1215, 2251, 1224, 1225, 5322, 5323, 4301, 1238, 3620, 5338, 1244, 3298, 1252, 1253, 5351, 1256, 5354, 3309, 1263, 5364, 3318, 4308, 3323, 4993, 3350, 5399, 3352, 900, 3366, 3367, 3368, 5417, 3374, 1329, 1334, 3386, 1342, 1347, 3855, 1590, 1350, 3399, 5450, 5452, 5346, 3412, 4324, 3420, 3427, 3428, 3429, 1937, 3436, 1390, 5489, 5493, 1401, 3453, 3464, 1418, 1424, 3478, 5528, 5529, 1435, 1446, 3499, 3507, 1462, 3514, 1469, 3518, 1471, 1952, 1476, 1482, 3531, 5588, 1493, 1497, 5598, 3554, 5604, 5609, 3565, 5617, 1522, 3573, 3574, 3575, 3577, 1533, 1537, 1538, 3588, 2305, 5640, 3595, 3596, 3602, 5655, 1563, 1564, 1569, 5668, 1582, 3638, 3640, 3645, 3650, 1603, 5700, 3653, 5705, 5706, 1618, 5032, 5714, 1623, 3674, 1629, 5727, 5731, 1639, 1641, 5739, 5053, 1658, 1659, 5758, 1302, 3718, 5057, 3720, 3721, 5771, 5772, 1304, 5780, 3735, 5784, 3743, 1696, 5793, 1701, 3759, 5810, 3765, 5820, 5824, 5829, 3786, 1740, 1741, 5070, 5856, 3811, 1318, 636, 4391, 1772, 1780, 5882, 2037, 5884, 3837, 3838, 3845, 3846, 1799, 3848, 3849, 1805, 301, 3032, 3861, 3863, 5913, 1818, 1822, 2693, 1827, 3876, 1837, 5934, 3890, 1843, 3899, 3902, 3904, 3909, 5958, 5965, 1877, 1879, 3929, 655, 5434, 3934, 1893, 5996, 3950, 3951, 3953, 3966, 3970, 2369, 1933, 6033, 1940, 3989, 3997, 4000, 1953, 4763, 4016, 3059, 6068, 4024, 1982, 4031, 6085, 2002, 2003, 2725, 2021, 6481, 2027, 4078, 4079, 6128, 6130, 6819, 4095, 6143, 2687, 3262, 1852, 5851, 3594, 4696, 4791, 3800, 4799, 3105, 11, 6014, 1273, 5584, 2495, 3700, 3867, 5921, 1872, 1248, 5051, 4207, 2202, 6331, 6080, 2622, 6828, 4143, 5622, 3326, 5185, 1438, 3994, 3239, 2638, 5339, 3343, 6269, 5977, 4697, 537, 451, 640, 4742, 3247, 1792, 3463, 6340, 3127, 2151, 3338, 1760, 3610, 3041, 5695, 5729, 3515, 597, 5350, 2960, 4745, 5089, 6826, 6889, 297, 3112, 1943, 5935, 6293, 3538, 504, 6461, 4182, 4035, 4225, 4758, 941, 5355, 5253, 1416, 1492, 1368, 196, 6407, 6613, 3489, 2379, 5799, 4968, 430, 4493, 1682, 1693, 4382, 798, 4530, 868, 5746, 6215, 884, 2980, 6972, 5438, 6391, 5803, 2695, 7109, 2060, 3451, 2778, 5969, 298, 4518, 7047, 3364, 5466, 910, 614, 1425, 987, 349, 5976, 3089, 3131, 774, 5134, 980, 6999, 4555, 5859, 3457, 5160, 6732, 5482, 5228, 1517, 5621, 2770, 5984, 3866, 4597, 2939, 1849, 3873, 1873, 4826, 3681, 2781, 4026, 5987, 2926, 6726, 5473, 4094, 2677, 5583, 1700, 1695, 5462, 1575, 5573, 2594, 2219, 2938, 6466, 5592, 466, 2801, 3445, 3750, 1151, 6978, 5468, 5419, 2914, 6420, 880, 6363, 5848, 4846, 1751, 6821, 4370, 5903, 218, 4779, 3133, 6296, 4265, 5229, 1176, 2680, 7127, 6737, 797, 5559, 6092, 2217, 6469, 2806, 2646, 4142, 245, 1589, 6236, 3176, 5589, 2868, 6911, 4096, 4699, 2401, 954, 715, 3186, 3168, 4177, 36, 7051, 4262, 3028, 4611, 6717, 3922, 1550, 2958, 984, 1602, 4588, 1356, 4200, 6887, 3397, 570, 5716, 3501, 4727, 3440, 2758, 27, 4571, 426, 6549, 4673, 6109, 5738, 324, 3581, 6201, 4007, 1994, 274, 7049, 4612, 1048, 1116, 4825, 6008, 6644, 6314, 7115, 1587, 847, 1926, 6098, 5601, 6323, 1146, 4056, 2130, 1928, 3533, 5606, 4239, 4960, 6158, 3508, 1426, 4667, 4514, 1420, 1067, 4841, 2815, 7028, 510, 1423, 3300, 5321, 536, 3805, 3935, 3181, 4878, 7048, 2623, 1383, 2605, 1624, 4130, 4627, 4935, 5016, 1380, 5171, 473, 5416, 508, 1786, 4573, 1312, 665, 4407, 4835, 2494, 2645, 3065, 2233, 2483, 5069, 6558, 422, 4172, 6219, 4238, 2350, 77, 4185, 6611, 3634, 2458, 4669, 2069, 1353, 1856, 2813, 3282, 5675, 5555, 4570, 538, 2158, 3002, 4759, 1838, 4917, 6024, 7012, 477, 4904, 1955, 568, 3227, 3511, 5209, 3675, 3079, 4397, 3628, 3677, 4880, 5394, 3103, 2756], [2563, 2571, 5208, 2580, 4633, 6682, 6683, 28, 29, 31, 32, 1569, 552, 2603, 4655, 2825, 4669, 4277, 6722, 1091, 6219, 6222, 4687, 6739, 597, 3672, 6837, 6749, 2655, 5729, 98, 611, 6246, 4200, 1129, 4203, 3181, 3182, 2671, 6256, 2323, 1641, 2686, 4226, 4740, 364, 4747, 1166, 5777, 6803, 5780, 6293, 3743, 1186, 2216, 4785, 179, 692, 3253, 2233, 6843, 4285, 1726, 191, 3689, 1225, 5322, 1227, 3277, 5848, 6362, 5851, 1253, 6889, 2794, 5868, 755, 2294, 6905, 1786, 1787, 6910, 6845, 4355, 777, 6413, 783, 274, 4883, 2842, 806, 297, 3370, 1324, 5934, 4913, 300, 1336, 6465, 2372, 3397, 4937, 4428, 6114, 3931, 6492, 4448, 4453, 1388, 6517, 1913, 2938, 7036, 1405, 4995, 4487, 66, 2961, 6549, 6556, 925, 2977, 2471, 3484, 1972, 6589, 4543, 4034, 6731, 1999, 6615, 4574, 6112, 4066, 2021, 1514, 4007, 5627, 6652, 3069, 2643, 823, 1755, 6814, 6081, 824, 6826, 3350, 3436, 4460, 2178, 3490, 3292, 6201, 2866, 1822, 5797, 4304, 3653, 4931, 6694, 4043, 3962, 4245, 6487, 5489, 6381, 524, 2780, 4871, 2179, 1718, 234, 4690, 4872, 910, 2406, 332, 315, 6828, 3265, 6438, 2712, 2197, 3441, 6768, 3020, 6490, 3081, 3105, 6658, 15, 4779, 2613, 2831, 7049, 576, 4108, 7055, 2630, 6089, 3100, 4009, 1069, 3262, 3852, 1084, 3755, 6881, 1662, 3861, 299, 656, 4944, 920, 2504, 4555, 5018, 1171, 3163, 4146, 3202, 4851, 3390, 2430, 4496, 5967, 5609, 4056, 5732, 5700, 3932, 6168, 6777, 2045, 1230, 3146, 4094, 6245, 5486, 3371, 3686, 2401, 3936, 1, 2748, 2734, 4246, 3566, 4110, 6024, 3702, 2437, 1202, 4177, 2740, 2786, 4794, 430, 5804, 6045, 4621, 2308, 3034, 4908, 2041, 4541, 4927, 2789, 6985, 1247, 2385, 3761, 4699, 517, 6273, 774, 3225, 6755, 51, 2941, 2534, 1128, 1981, 5677, 4107, 5567, 4922, 4569, 4770, 6232, 1096, 3326, 5200, 1897, 6770, 609, 365, 2234, 329, 2827, 6316, 6356, 2817, 5215, 761, 1591, 4353, 4970, 3816, 6374, 3850, 3982, 6447, 4917, 3859, 630, 5403, 1183, 5714, 3314, 3340, 1719, 1578, 4057, 1890, 5905, 3754, 2622, 4130, 849, 1097, 599, 2469, 2332, 1735, 4157, 6721, 4407, 349, 6614, 618, 2304, 3744, 7116, 6763, 1618, 3915, 2718, 6158, 3385, 1177, 6613, 3099, 3929, 184, 2982, 3514, 5465, 1023, 3515, 243, 6641, 5446, 6627, 830, 1291, 6737, 3157, 3844, 3446, 624, 2964, 3189, 5248, 4029, 2395, 1811, 2596, 1028, 1192, 2784, 4976, 7012, 1962, 6928, 5799, 4097, 2310, 2893, 1537, 405, 2499, 603, 99, 2606, 5429, 2169, 4990, 2005, 6369, 6104, 593, 5201, 2112, 4222, 3894, 1379, 1304, 1839, 6068, 69, 3168, 3601, 1634, 1671, 5828, 5599, 7114, 2873, 334, 1780, 1144, 3276, 1482, 2066, 2834, 27, 5740, 1917, 2038, 1371, 5246, 4324, 3094, 1586, 6523, 3511, 3402, 5029, 2353, 4648, 5577, 6767, 1705, 6616, 1226, 5115, 4535, 3872, 3838, 727, 3421, 3820, 6841, 2024, 2737, 259, 4018, 5906, 1370, 5285, 6870, 5604, 1522, 6378, 3970, 5355, 1276, 7014, 4078, 3082, 1722, 4385, 2237, 435, 1878, 1444, 9, 6165, 3807, 3976, 6040, 470, 6030, 3420, 3227, 4790, 676, 1516, 5236, 685, 5550, 2806, 4859, 4712, 2699, 1763, 584, 5418, 1538, 2003, 4220, 885, 2477, 5959, 988, 5508, 1660, 4159, 3855, 4044, 2515, 2459, 6649, 6611, 2894, 5416, 1553, 5843, 3417, 1030, 619, 4816, 2678, 1602, 3442, 2599, 4063, 3066, 55, 5812, 136, 2147, 3348, 1782, 4338, 7068, 6992, 5143, 1629, 2581, 6852, 1401, 2510, 3445, 4759, 225, 2340, 5883, 2393, 463, 3264, 3232, 4160, 1281, 2955, 4704, 2994, 1381, 916, 1894, 1214, 633, 6584, 4294, 2751, 6677, 1157, 4195, 5991, 6371, 128, 1769, 4982, 3472, 2984, 5270, 2394, 840, 5315, 1335, 1734, 165, 3627, 4868, 4328, 6580, 2797, 4998, 4069, 1562, 4330, 4412, 3077, 6458, 2241, 3170, 5033, 1879, 6802, 3293, 5441, 481, 4231, 2623, 2833, 1554, 1208, 602, 6420, 3411, 4247, 6774, 4864, 3500, 2204, 846, 4945, 5689, 4501, 4017, 2926, 16, 1899, 2687, 5177, 5856, 3036, 3833, 6496, 6609, 1592, 273, 2927, 3706, 5069, 2129, 6636, 3995, 820, 3957, 4158, 3658, 5947, 4490, 683, 5156, 5311, 3235, 5891, 6537, 1251, 4263, 1963, 893, 6734, 2841, 5865, 4815, 4624, 5776, 926, 6237, 1603, 2993, 4055, 6780, 3440, 3035, 5212, 3085, 3, 187, 3565, 999, 1338, 2591, 1174, 1172, 6678, 2291, 4309, 4638, 1727, 5989, 79, 422, 5070, 4320, 2345, 4647, 3978, 469, 4627, 1075, 4061, 3281, 1806, 1858, 6279, 464, 6296, 6592, 448, 2824, 4869, 2035, 1391, 6131, 4822, 3545, 657, 2273, 4507, 829, 5913, 5815, 5442, 1456, 1363, 2460, 6791, 73, 6323, 941, 3128, 1788, 3279, 2514, 6950, 613, 3895, 428, 6145, 1428, 142, 4459, 485, 3153, 3448, 5941, 5996, 4567, 5977, 722, 2060, 6680, 5754, 4880, 6687, 4378, 4275, 4389, 931, 7044, 6621, 1836, 25, 6252, 6070, 5739, 5428, 4118, 6901, 6440, 3880, 1623, 862, 3928, 2991, 1455, 5296, 749, 2723, 805, 421, 5324, 4421, 4992, 912, 3218, 6181, 3616, 1611, 6874, 4599, 1852, 2869, 6008, 1577, 4211, 3130, 1817, 3171, 7045, 1299, 1709, 508, 2652, 6208, 1953, 2587, 3750, 6034, 4016, 6119, 3270, 224, 4738, 7010, 2903, 6844, 5884, 6514, 5054, 712, 18, 2724, 3618, 4310, 6453, 6469, 296, 4845, 2004, 6581, 2735, 1492, 1643, 5149, 5837, 2864, 3121, 5877, 4325, 2814, 2653, 5871, 3600, 686, 1468, 5607, 2492, 1213, 1544, 647, 4429, 4697, 825, 3538, 4593, 2958, 6039, 1779, 4888, 5450, 5873, 2704, 222, 4437, 6860, 5130, 570, 3309, 7106, 6596, 860, 4853, 2811, 4180, 2909, 2948, 2576, 4079, 1827, 634, 2677, 6923, 1460, 5271, 6044, 1289, 2997, 2681, 3668, 4255, 3111, 1965, 5563, 1116, 6559, 3323, 6642, 5510, 424, 2219, 1079, 3731, 4515, 6013, 6004, 2180, 7046, 5813, 1163, 3453, 2725, 6807, 973, 1295, 2458, 4413, 6308, 1042, 6711, 2573, 3642, 5095, 4889, 3977, 3405, 436, 3092, 5083, 5559, 4959, 1659, 2670, 3394, 3308, 6624, 68, 2497, 4743, 1793, 5975, 1179, 2320, 1564, 2673, 4505, 1322, 1519, 3589, 3921, 3496, 1772, 1932, 2397, 4486, 890, 5087, 115, 232, 4225, 1575, 6540, 2049, 2172, 394, 3499, 2175, 6956, 1390, 3179, 84, 2719, 2716, 2848, 2239, 4319, 1427, 5263, 3074, 2368, 4030, 4196, 4380, 3518, 3848, 3579, 516, 1025, 3692, 4121, 338, 816, 6284, 23, 1693, 6999, 2713, 1474, 6185, 6815, 4629, 6679, 1108, 5595], [2752, 7096, 1147, 5321, 4183, 2372, 6108, 165, 4635, 1000, 4440, 4770, 2808, 4248, 4273, 1979, 3667, 3610, 1886, 4412, 1953, 1898, 4041, 6053, 2190, 2913, 2929, 607, 5297, 3693, 4422, 574, 1604, 5728, 1469, 2161, 1684, 6333, 730, 3985, 3017, 1597, 1583, 1099, 3722, 4325, 6987, 2797, 4476, 4533, 5939, 4215, 5993, 483, 4282, 1205, 3825, 2265, 6852, 2423, 1550, 1059, 5584, 2757, 2594, 1361, 6745, 4457, 5858, 4567, 5312, 6901, 2220, 3524, 3042, 2549, 7028, 166, 3957, 6320, 5372, 3950, 6428, 1424, 509, 3152, 2739, 303, 2536, 851, 2678, 1878, 375, 7000, 4810, 2751, 3777, 5750, 1492, 4838, 1286, 5224, 4583, 2485, 3603, 6888, 1874, 6542, 6037, 134, 1717, 20, 5157, 5076, 4398, 2442, 5144, 5740, 2772, 3184, 5914, 5800, 4029, 1535, 2569, 4094, 4339, 1046, 2794, 243, 2008, 4534, 5373, 17, 4652, 2397, 2812, 1856, 5814, 206, 355, 5797, 5349, 6878, 7072, 1338, 1949, 6699, 6530, 5653, 913, 5219, 1602, 3831, 5716, 1129, 3101, 6445, 5134, 549, 3865, 3653, 4653, 1485, 1462, 6176, 1747, 1354, 1675, 1807, 1664, 1070, 164, 6697, 3757, 4146, 6673, 597, 3372, 6918, 1335, 2679, 1419, 5002, 2497, 5824, 6539, 5471, 1553, 871, 5135, 4410, 577, 565, 5293, 1219, 6767, 3161, 2349, 5846, 3626, 2780, 3551, 6342, 6054, 1049, 1155, 3137, 6447, 2693, 1524, 1751, 1014, 2460, 3694, 5185, 6538, 800, 5794, 6621, 6680, 3932, 3427, 3759, 239, 6928, 4101, 3657, 2921, 1801, 2770, 3867, 3638, 935, 2980, 3043, 780, 929, 5544, 4077, 2507, 5496, 2437, 605, 4949, 2999, 1596, 7117, 4842, 2469, 1495, 260, 3471, 1980, 2055, 4760, 5430, 3100, 3295, 3147, 6015, 6168, 4322, 4663, 4026, 2993, 910, 767, 3983, 6923, 6324, 2748, 4131, 387, 457, 4073, 6835, 4641, 3906, 791, 104, 4195, 4837, 847, 4581, 4001, 1996, 5354, 496, 6597, 5501, 2512, 3612, 6228, 4312, 3598, 405, 2860, 2216, 2016, 4271, 3736, 4272, 1869, 6757, 4308, 1910, 1926, 1048, 5242, 3691, 5272, 4111, 2241, 2468, 1316, 6410, 4405, 3201, 6149, 3585, 5607, 1088, 1746, 5440, 872, 1203, 5444, 6355, 1108, 2151, 828, 2004, 4743, 2137, 4458, 4438, 2843, 3890, 3994, 2107, 5529, 4447, 5793, 4, 4472, 6261, 5779, 713, 5702, 5950, 4611, 4933, 4983, 2771, 2383, 2125, 941, 1405, 6591, 2176, 6556, 4376, 389, 4334, 806, 87, 4427, 6143, 2201, 2320, 5415, 4483, 5899, 1043, 6686, 693, 6634, 1905, 857, 6961, 173, 983, 6181, 1797, 2312, 2351, 5164, 6864, 6502, 1840, 3129, 2461, 5876, 4037, 92, 3614, 4777, 6301, 3817, 3029, 2687, 195, 6838, 3948, 514, 5521, 4730, 1748, 3998, 6827, 3549, 1139, 5398, 5003, 5964, 3395, 4727, 602, 4946, 3213, 1790, 2270, 2316, 137, 6002, 4019, 3607, 284, 5839, 108, 830, 1291, 5636, 2362, 3785, 1935, 3315, 5661, 1983, 5467, 6347, 409, 1438, 297, 4477, 4133, 3953, 1273, 2410, 1857, 5022, 3146, 5363, 6189, 5796, 272, 3410, 735, 6414, 398, 1642, 1628, 2597, 3885, 4850, 3436, 2701, 1896, 4568, 6636, 5142, 2030, 1015, 6553, 230, 1456, 135, 5569, 1197, 1792, 1822, 5447, 3263, 1817, 6984, 476, 5497, 6158, 1092, 6633, 6405, 6465, 7015, 3399, 5482, 842, 5445, 2595, 612, 5840, 3665, 6094, 6828, 5252, 5986, 1956, 4313, 6032, 5402, 2702, 1740, 5703, 6233, 1619, 1223, 3297, 3616, 1548, 4660, 1698, 4448, 1902, 560, 652, 3030, 1277, 5897, 334, 5646, 4486, 6277, 5074, 481, 5045, 2997, 2014, 2646, 485, 1594, 4380, 7071, 3628, 1490, 2275, 4330, 5032, 7104, 1873, 5530, 716, 5733, 772, 719, 1699, 4920, 1918, 1232, 2880, 3834, 1930, 4905, 2928, 6334, 3587, 4903, 5172, 1686, 1201, 1771, 1708, 2367, 802, 6742, 6601, 1892, 6439, 556, 4385, 1603, 5300, 4694, 120, 2012, 4287, 2087, 873, 4181, 6193, 3641, 5807, 5775, 550, 91, 6579, 6136, 2961, 6524, 2082, 144, 1847, 5510, 4022, 4363, 4647, 55, 1275, 6948, 6028, 613, 5431, 3343, 4406, 2884, 6561, 5590, 4735, 5247, 3417, 2618, 2669, 3532, 4172, 702, 6769, 5102, 6756, 4093, 7, 1612, 2204, 1614, 3542, 4754, 3156, 3790, 997, 7013, 601, 3314, 4679, 96, 5722, 158, 3209, 5422, 4040, 1955, 4226, 5637, 3191, 6124, 6558, 5179, 4254, 1811, 6795, 3525, 6841, 674, 2774, 4571, 5873, 2526, 5174, 5133, 4970, 6705, 758, 131, 6156, 6799, 1613, 6369, 3737, 3921, 5508, 4989, 3203, 1390, 3546, 1888, 5229, 6854, 1214, 5580, 6185, 3550, 6817, 1933, 5367, 1113, 4830, 1415, 6356, 1587, 5647, 3173, 6596, 3698, 1618, 404, 424, 1782, 6046, 4709, 1631, 2135, 850, 3707, 2683, 2430, 739, 6092, 5220, 849, 2298, 726, 1215, 4756, 5953, 5340, 3038, 1281, 3883, 7088, 1757, 4177, 846, 7078, 4302, 1829, 7086, 3493, 6458, 793, 1124, 5451, 251, 4148, 1592, 4866, 6422, 4043, 2741, 5571, 5617, 1784, 705, 4923, 98, 3577, 500, 1192, 6083, 4630, 4578, 6537, 6274, 4722, 3220, 3887, 1683, 5338, 337, 6386, 6091, 2850, 1348, 6768, 2443, 2238, 6920, 4539, 7047, 2853, 2593, 7043, 3611, 1202, 3661, 3, 3366, 1893, 6965, 6616, 732, 4420, 4333, 5516, 4276, 2168, 524, 1921, 1730, 5238, 902, 3789, 924, 2463, 7037, 1510, 6162, 1168, 2864, 3520, 2340, 1872, 2773, 1360, 5582, 3398, 5700, 3796, 176, 4031, 6533, 6517, 3062, 7123, 1985, 6581, 4713, 3830, 4147, 5827, 2916, 498, 5604, 914, 5212, 6563, 5307, 6215, 5663, 5434, 1616, 4634, 2601, 5816, 4241, 2382, 3530, 1989, 79, 3446, 6626, 4299, 6113, 2885, 44, 3824, 7017, 5903, 4171, 5187, 2807, 3128, 3081, 1053, 1187, 4305, 380, 5342, 2511, 4020, 912, 5203, 2520, 4235, 3303, 4252, 3361, 5167, 2375, 583, 6963, 5148, 2058, 2114, 1089, 586, 2069, 6192, 3269, 1696, 2388, 3414, 3900, 1796, 4701, 1352, 1880, 4556, 2487, 3154, 3120, 5782, 1447, 3321, 81, 450, 171, 4117, 291, 4238, 3624, 542, 6196, 936, 5550, 4163, 5472, 6205, 3058, 3448, 3059, 2575, 74, 3970, 5454, 6033, 2673, 648, 5823, 5933, 5721, 6669, 269, 2345, 2954, 3678, 4666, 1133, 2648, 5460, 6464, 1373, 5125, 3420, 2219, 2389, 1303, 5734, 31, 4178, 4622, 5888, 2893, 897, 6902, 5925, 5937, 4968, 3109, 3781, 4564, 6716, 4675, 2728, 5289, 2722, 9, 213, 1418, 3001, 6344, 4975, 4230, 3094, 3134, 493, 4244, 4929, 5577, 5487, 2025, 3584, 2943, 6360, 750, 4205, 4664, 5319, 5627, 2574, 1971, 6515, 6290, 6080, 2268, 4358, 3430, 5288, 5418, 4461, 160, 6423, 3229, 2878], [1537, 3074, 6147, 4612, 3595, 524, 1538, 4622, 4624, 4611, 6677, 3094, 1048, 4633, 6682, 6683, 3100, 6686, 3106, 2704, 552, 6834, 2607, 3634, 5171, 5173, 2614, 3957, 3129, 6202, 6716, 1087, 4196, 5185, 1602, 1091, 5700, 3653, 583, 5705, 3147, 79, 3153, 3154, 6227, 7112, 597, 4183, 6232, 89, 5212, 3515, 5727, 6928, 98, 4195, 1124, 613, 6215, 1129, 5739, 165, 3182, 3183, 1649, 114, 1139, 1214, 2678, 2679, 4730, 1659, 4220, 2173, 2687, 2335, 4738, 6787, 4740, 3350, 4743, 4713, 3721, 4235, 5772, 4238, 5263, 5528, 1334, 6291, 5780, 6294, 3865, 4760, 2119, 1179, 1818, 2719, 1696, 5793, 4770, 4635, 6820, 677, 5799, 5813, 6314, 171, 1557, 1143, 1199, 6600, 5810, 179, 692, 1205, 185, 187, 2751, 6334, 2239, 5824, 3777, 5317, 3689, 6344, 3314, 5322, 2251, 1740, 719, 4308, 1401, 5338, 732, 1658, 3059, 2273, 5347, 4324, 4325, 2790, 3815, 5352, 4321, 749, 4846, 5672, 2802, 2003, 1782, 5417, 1492, 1786, 4859, 6910, 1792, 3309, 4707, 262, 5038, 3848, 267, 6413, 2320, 4370, 5584, 6420, 5934, 4886, 5399, 1304, 5913, 3866, 4380, 1822, 2693, 2864, 2340, 805, 806, 4903, 296, 297, 6369, 2350, 6447, 816, 4147, 4918, 2360, 1151, 5434, 3902, 831, 6464, 6465, 6466, 2955, 2884, 3638, 6881, 5450, 343, 2892, 5965, 846, 851, 6999, 4952, 1370, 4751, 2397, 1424, 1892, 230, 7015, 5482, 365, 3262, 7028, 885, 3112, 660, 2938, 790, 1405, 7038, 4480, 897, 5782, 4486, 7047, 6537, 7051, 910, 4495, 912, 2961, 405, 5529, 1432, 4420, 6556, 6045, 1438, 1953, 4514, 3227, 2468, 1957, 5617, 5032, 1962, 3499, 941, 5550, 2991, 4016, 3507, 4534, 585, 955, 1469, 6033, 451, 1014, 4226, 4552, 5577, 1482, 3531, 6092, 6989, 5072, 3743, 2002, 6611, 980, 3030, 3034, 987, 476, 5882, 2469, 483, 2021, 3565, 5102, 4079, 2544, 424, 1522, 3155, 5621, 3574, 4094, 4037, 418, 6221, 5536, 5784, 3577, 1752, 2473, 2075, 1297, 477, 4148, 2833, 2471, 1594, 3414, 3119, 3118, 5493, 1108, 3891, 3737, 4353, 2406, 4737, 2139, 5447, 65, 6905, 6037, 2125, 2773, 4851, 6360, 685, 3427, 1215, 925, 537, 6093, 137, 6416, 6415, 3551, 234, 2825, 2939, 3157, 6839, 4254, 3927, 4543, 403, 2219, 2383, 3186, 5136, 149, 1053, 2147, 4759, 4302, 3436, 1477, 1787, 1338, 1554, 6159, 1817, 4934, 4872, 2511, 3364, 5134, 191, 5219, 5461, 259, 6185, 1548, 6356, 2294, 3616, 5346, 6168, 1197, 4968, 7045, 2779, 4704, 4355, 6902, 4310, 3101, 2375, 3863, 671, 5430, 5622, 902, 5716, 256, 1990, 1008, 4785, 6543, 1639, 6428, 3057, 5501, 422, 2868, 6966, 5293, 716, 2004, 3528, 6678, 2630, 2997, 5321, 1226, 1843, 984, 1023, 252, 5069, 3828, 55, 5748, 4820, 4096, 5933, 4181, 3998, 7083, 5472, 2960, 2016, 2238, 5646, 5022, 6194, 605, 66, 2106, 4210, 3253, 6113, 2005, 7101, 4448, 1704, 4383, 4309, 3890, 6396, 1618, 803, 988, 5075, 6817, 6229, 6320, 320, 1069, 1524, 4017, 3542, 4501, 2296, 4294, 5797, 237, 4556, 750, 3921, 2349, 6772, 1329, 5937, 2663, 1348, 6790, 2740, 298, 2055, 3394, 4333, 3064, 7035, 5416, 243, 735, 4454, 1940, 3834, 2291, 1641, 2212, 3993, 4162, 652, 6634, 3269, 5323, 2573, 3399, 5006, 1373, 2345, 849, 2924, 2197, 2549, 3951, 1550, 850, 1829, 462, 5693, 2574, 32, 5722, 2180, 1227, 5130, 5242, 3620, 999, 5154, 5583, 1811, 348, 2641, 800, 5779, 6028, 2447, 4048, 577, 1455, 2082, 5066, 3303, 4117, 394, 4775, 1603, 3293, 2079, 3181, 4735, 7117, 4908, 6087, 1853, 3028, 3448, 6044, 2707, 4171, 27, 1147, 6100, 5964, 1976, 3677, 924, 5441, 1171, 1698, 6205, 1661, 1700, 2752, 16, 1730, 2217, 2190, 215, 3368, 1390, 6549, 2401, 5555, 6166, 4813, 713, 2603, 1523, 2308, 4779, 3, 6274, 3524, 5431, 1849, 1092, 561, 2852, 3370, 6875, 1303, 3161, 3831, 6658, 4581, 4660, 1790, 3761, 6837, 2049, 6889, 1203, 4754, 498, 1695, 5605, 5422, 4041, 6819, 4923, 2270, 3550, 758, 1569, 3674, 1664, 6517, 3352, 823, 4313, 3702, 5363, 4304, 2783, 4421, 2921, 4641, 6293, 3788, 674, 4496, 752, 6405, 560, 6339, 3254, 4664, 4935, 2648, 3626, 2456, 1148, 4647, 6149, 4125, 618, 1616, 1256, 5627, 4656, 578, 4443, 5051, 1982, 1855, 504, 1324, 4144, 4163, 3276, 1279, 219, 81, 6353, 3934, 6513, 2673, 1623, 893, 1121, 1879, 3525, 5177, 2844, 3825, 4427, 3781, 647, 3846, 6136, 712, 1361, 4177, 2358, 5454, 771, 2850, 6530, 1893, 3151, 2741, 3610, 224, 3650, 4701, 5329, 5579, 4241, 6649, 7049, 6770, 2914, 3736, 5220, 3698, 5244, 2926, 4285, 5980, 5609, 2282, 3459, 1302, 2389, 3229, 6829, 5977, 6090, 4447, 913, 6979, 1462, 2169, 4663, 3020, 3642, 4687, 337, 5514, 1187, 2417, 6430, 7078, 3549, 693, 5343, 3279, 2980, 5084, 3518, 5475, 881, 6539, 2353, 166, 7044, 5731, 5758, 1983, 2210, 2393, 2526, 1927, 269, 4437, 6014, 4883, 5367, 4, 3341, 1583, 3844, 6179, 4260, 5167, 1880, 4022, 3446, 5004, 1709, 1581, 3032, 5655, 2024, 508, 5604, 5372, 4126, 5823, 4838, 1447, 6568, 2587, 2813, 1873, 5884, 1096, 128, 156, 1168, 454, 2716, 1931, 341, 6481, 3041, 1330, 4101, 3876, 1591, 135, 4505, 3328, 2739, 6143, 1587, 1166, 538, 6112, 556, 779, 1146, 25, 780, 5893, 4428, 3397, 274, 1956, 1000, 1979, 4111, 4993, 4378, 6498, 2677, 5649, 5703, 5300, 6305, 2362, 2241, 6874, 4391, 3081, 1989, 1322, 4880, 1690, 2903, 2973, 2618, 3953, 4000, 3641, 4638, 3514, 6445, 4095, 3058, 705, 1192, 1684, 2151, 708, 364, 4205, 4992, 7014, 4568, 2671, 847, 4690, 687, 5252, 5668, 3484, 6410, 3372, 3477, 1535, 3004, 6780, 4024, 6734, 5233, 3453, 5848, 1760, 3861, 6642, 4442, 6440, 3257, 4114, 5474, 842, 774, 1418, 5306, 2244, 503, 143, 1398, 3817, 3520, 3049, 5989, 1044, 3833, 5544, 3395, 2533, 91, 4244, 3784, 6860, 6675, 1116, 4376, 1352, 2571, 3657, 6580, 2161, 131, 1747, 6613, 6722, 2014, 5285, 6458, 4330, 777, 2842, 1994, 1464, 85, 5714, 4209, 6749, 2806, 2722, 2808, 5000, 5224, 5096, 1208, 4653, 3611, 5080, 5858, 5958, 7115, 1176, 4675, 1852, 3270, 3519, 4567, 5636, 1926, 6109, 1701, 4385, 2440, 3575, 5846, 5683, 5339, 3852, 4571, 4130, 3759, 6068, 1798, 2595, 3420, 90, 3636, 1886, 41, 4493, 755, 5698, 6605, 4938, 547, 1046, 3750, 6652, 5471, 2816, 5229, 36, 1435, 6814, 3298, 302, 1263], [5643, 2304, 3586, 1544, 1033, 5130, 4107, 6658, 4624, 6675, 3094, 6683, 28, 4127, 3106, 2088, 6492, 4147, 5173, 5177, 3642, 3133, 3134, 3765, 4196, 66, 6723, 3653, 5702, 3425, 72, 6219, 6732, 79, 3153, 594, 4195, 5716, 3157, 3670, 3257, 1624, 89, 90, 3002, 94, 6755, 1124, 5222, 1129, 4203, 6764, 2158, 928, 4723, 3702, 4713, 3192, 4543, 6780, 2687, 5248, 1153, 1155, 4740, 2693, 2695, 136, 5772, 1171, 660, 2197, 6294, 4759, 6811, 2716, 2842, 4254, 2208, 161, 2210, 1700, 5799, 2588, 2740, 5813, 1718, 5577, 2233, 4285, 6517, 5824, 5579, 2244, 2678, 3270, 1225, 1740, 4816, 6353, 4309, 1752, 6875, 5343, 1248, 6883, 4031, 2790, 4817, 4841, 234, 1263, 752, 2802, 5364, 2293, 2294, 332, 5369, 1786, 1151, 252, 2815, 1792, 774, 1800, 4361, 778, 3852, 5567, 5399, 2330, 5418, 816, 2868, 2358, 823, 1336, 3897, 315, 2876, 1855, 324, 3397, 3399, 1352, 5452, 2898, 5461, 343, 3928, 5466, 2395, 3420, 5983, 3937, 6498, 4206, 885, 5468, 5780, 3451, 6529, 1397, 1416, 2711, 5004, 3650, 910, 1423, 2961, 4501, 4505, 1435, 6045, 4000, 4001, 1957, 5033, 1962, 2991, 5554, 6068, 2206, 4537, 3514, 3519, 6816, 4034, 6731, 965, 4428, 3531, 3541, 6614, 2212, 987, 5087, 5606, 6737, 1514, 1014, 4180, 938, 85, 3147, 5023, 4261, 411, 4469, 1801, 2924, 1872, 2529, 3472, 609, 4141, 4695, 4752, 2681, 2737, 6834, 1523, 6871, 2495, 5650, 1658, 5655, 2510, 3410, 3620, 826, 2077, 3441, 1701, 1148, 6716, 5514, 6090, 4096, 7000, 6355, 1262, 5482, 2062, 142, 4954, 5685, 245, 806, 510, 6022, 3786, 3525, 688, 4671, 3042, 7095, 69, 801, 1569, 5704, 3367, 1215, 3199, 5868, 614, 4260, 6686, 5985, 1493, 7089, 6190, 6913, 4146, 2565, 3530, 4145, 4576, 4385, 4568, 1789, 4391, 5321, 6139, 6214, 2092, 1552, 1304, 5528, 6539, 2638, 2666, 3602, 415, 955, 6179, 3844, 5480, 2260, 1192, 1017, 2408, 3333, 5288, 1008, 6600, 2749, 1852, 1244, 3718, 1069, 5801, 2578, 6033, 1471, 5903, 4558, 4442, 5201, 2613, 3575, 6216, 451, 890, 839, 5290, 3141, 2471, 3773, 6805, 2594, 5132, 2235, 2822, 4081, 4389, 6989, 3057, 4177, 5989, 1912, 1114, 9, 2336, 5338, 6480, 6440, 6580, 6208, 896, 1477, 2993, 6036, 2360, 3343, 634, 3130, 5442, 4546, 5038, 97, 6222, 2866, 1553, 2048, 4872, 4569, 3247, 6667, 5069, 212, 4293, 5826, 237, 3672, 3314, 1802, 316, 1116, 6113, 7116, 5474, 5488, 1115, 5689, 5311, 897, 4534, 3028, 464, 933, 3326, 6813, 106, 687, 5398, 6147, 4770, 5500, 4934, 2412, 4824, 5922, 671, 1172, 4509, 3411, 360, 5692, 6822, 5695, 7029, 1340, 6311, 1476, 1143, 2806, 6453, 1144, 5575, 611, 1839, 4738, 2004, 3711, 2579, 2739, 4437, 4813, 4380, 5258, 1195, 7015, 6109, 23, 1892, 1947, 733, 307, 6581, 3019, 4110, 2129, 6960, 1795, 6705, 4358, 6908, 6232, 4644, 5790, 6254, 4225, 5928, 2032, 3041, 7017, 2877, 5921, 1537, 1332, 4172, 2942, 1954, 4927, 1411, 6917, 6430, 2946, 808, 2189, 3252, 2204, 535, 4035, 3065, 3518, 2320, 225, 6999, 424, 2467, 5750, 1472, 4433, 4840, 4418, 6291, 2470, 6004, 5084, 6143, 3055, 1361, 1639, 3487, 1489, 140, 4313, 5592, 5184, 3655, 3634, 7016, 1646, 6996, 5124, 5131, 685, 1849, 648, 6032, 3990, 6830, 540, 2614, 473, 1256, 191, 2732, 3298, 984, 4187, 5562, 1494, 974, 171, 1394, 4057, 3654, 6678, 5938, 2160, 1030, 4351, 6246, 4785, 5489, 4998, 4185, 3833, 1395, 2781, 1816, 4857, 1874, 1808, 708, 1482, 2193, 4928, 5769, 5285, 1380, 1432, 5937, 1381, 2713, 6717, 6377, 2406, 3054, 6270, 568, 1341, 404, 524, 1504, 1745, 228, 4775, 5391, 1979, 1955, 3181, 2725, 2410, 112, 6694, 881, 6795, 3362, 295, 2234, 3471, 3337, 1913, 2353, 427, 3500, 1566, 6446, 4690, 1091, 6253, 4455, 6979, 1827, 163, 5261, 3499, 706, 2414, 6772, 4859, 7124, 1516, 2041, 5246, 1379, 5819, 326, 677, 1783, 448, 5053, 6773, 5119, 3309, 1469, 2284, 6623, 6226, 3902, 114, 5035, 6282, 2786, 1455, 5346, 3696, 2607, 1629, 2394, 755, 6797, 6634, 5328, 2477, 1107, 4136, 2674, 1618, 657, 1096, 516, 2976, 6250, 4246, 2027, 5231, 4112, 5865, 2697, 6584, 6615, 860, 2400, 6376, 2651, 88, 3273, 7038, 7010, 5058, 2399, 2560, 6483, 4115, 926, 2334, 5497, 148, 4489, 4248, 1787, 5843, 583, 5297, 1723, 480, 4078, 814, 1287, 2591, 3226, 6220, 4504, 1932, 2914, 2563, 4626, 5683, 2628, 393, 582, 4420, 3638, 5765, 2748, 1055, 4277, 6093, 1279, 156, 3969, 2699, 5839, 6221, 6819, 732, 867, 6030, 3323, 6276, 4787, 6662, 2209, 864, 2456, 2326, 256, 3770, 4324, 5185, 2670, 5829, 1766, 4657, 6814, 144, 3204, 273, 5306, 251, 2817, 2551, 4495, 3550, 2852, 2372, 3776, 6381, 2151, 3154, 6098, 4353, 3089, 6611, 3165, 2161, 5441, 205, 2599, 3801, 4174, 6608, 165, 1796, 1470, 3484, 1772, 3743, 5993, 5365, 365, 5042, 267, 956, 2005, 3477, 277, 2277, 6514, 29, 2581, 3904, 1692, 980, 367, 4917, 4432, 5793, 3020, 3658, 4496, 6857, 3616, 5934, 1722, 6787, 908, 6677, 761, 495, 2392, 4517, 2724, 2333, 2383, 982, 5057, 230, 1123, 4126, 5462, 616, 3690, 613, 1291, 6966, 2838, 3751, 1497, 1048, 1166, 3943, 1015, 5859, 5591, 1941, 2984, 5913, 6118, 3681, 3636, 6432, 901, 1572, 4209, 4962, 3338, 5244, 1945, 4393, 2814, 2345, 4485, 1113, 6910, 669, 2285, 6624, 4930, 819, 2340, 5367, 6942, 1720, 4264, 4751, 5002, 578, 1329, 224, 5923, 5434, 3303, 2663, 1898, 6331, 2270, 4111, 1837, 1251, 3957, 5848, 5746, 5372, 33, 4338, 2906, 3899, 1438, 4984, 4672, 5081, 1272, 5817, 5706, 4397, 484, 5723, 6595, 1330, 4222, 5233, 3293, 1981, 6570, 2119, 6464, 1915, 5096, 6166, 7036, 4055, 6205, 2267, 3816, 1127, 4732, 5407, 5614, 793, 6541, 1080, 4219, 6128, 2975, 6014, 4463, 1869, 5394, 3464, 1858, 99, 2438, 6985, 1641, 2631, 3135, 6322, 6646, 5052, 797, 51, 4135, 3308, 856, 1252, 5998, 6089, 4132, 2864, 218, 2302, 2156, 1253, 6229, 6459, 2533, 2509, 348, 3565, 5980, 3866, 5930, 5555, 2093, 1751, 460, 2468, 2605, 3009, 6774, 1865, 2884, 4226, 2703, 7077, 1538, 722, 5965, 4570, 1214, 5589, 4722, 6549, 1324, 3725, 1226, 6652, 6407, 2250, 949, 1238, 6782, 5430, 1007, 618, 695, 4367, 4871, 4909, 215, 5894, 5739, 32, 2655, 1682, 5832, 2795, 3341, 402, 2933, 3183, 2035, 4329], [2049, 4107, 6159, 4114, 6164, 4118, 27, 28, 4126, 6179, 2084, 6492, 6202, 65, 4162, 2119, 6220, 6227, 6229, 4183, 6232, 89, 2139, 6237, 2147, 4196, 4203, 6253, 2158, 6256, 4210, 2173, 128, 6291, 4255, 6305, 165, 6311, 171, 179, 185, 2239, 4293, 2251, 5154, 4304, 2260, 4321, 4325, 237, 2291, 2296, 252, 2305, 262, 267, 6420, 4380, 6430, 2335, 2340, 297, 2354, 2358, 2360, 6464, 6465, 4420, 4421, 343, 348, 6498, 2406, 365, 4463, 6513, 6514, 394, 4495, 4496, 402, 6549, 2458, 6556, 6559, 4514, 2468, 2471, 2477, 430, 4534, 3145, 6584, 2494, 4552, 462, 4573, 4584, 2544, 4599, 6649, 4611, 2564, 6664, 2571, 2573, 4622, 4633, 6682, 6683, 2589, 6686, 552, 2603, 2607, 6705, 2613, 2614, 6712, 2617, 6716, 2623, 578, 583, 4535, 595, 597, 4699, 6749, 2655, 613, 2663, 4713, 6764, 6770, 6772, 2677, 2679, 3519, 6780, 2687, 640, 4738, 6787, 4886, 4751, 2704, 657, 660, 5913, 4760, 665, 2716, 1818, 6814, 2719, 6819, 6820, 2725, 4775, 4779, 114, 687, 6600, 6834, 2740, 6845, 2751, 3531, 708, 719, 2773, 732, 6881, 2786, 4835, 2790, 2791, 4841, 749, 2802, 755, 758, 4859, 771, 774, 4880, 4883, 790, 805, 806, 4903, 477, 2864, 2868, 831, 6979, 2884, 6985, 4938, 6989, 849, 851, 6999, 4952, 7003, 860, 2914, 7014, 7015, 4968, 3129, 881, 5609, 2938, 893, 7038, 4993, 7051, 910, 2961, 2973, 3227, 7083, 5038, 2991, 946, 955, 5053, 3574, 7112, 5072, 3028, 3030, 3034, 5084, 5096, 3059, 1014, 1017, 5114, 3069, 3074, 2219, 5130, 516, 3106, 4195, 3591, 5173, 5177, 5185, 1091, 3143, 1096, 1097, 3147, 3151, 3154, 3155, 5212, 3168, 3171, 1124, 5222, 1129, 5228, 3182, 3183, 1139, 1143, 1146, 191, 1148, 5253, 1166, 1171, 1179, 1185, 3253, 3262, 1224, 1225, 5322, 1227, 4301, 1238, 5338, 5339, 1248, 3298, 1251, 1253, 1256, 5355, 3309, 1263, 5364, 6014, 3326, 7041, 3338, 3343, 3350, 5399, 3352, 3364, 5416, 5417, 3370, 5419, 1324, 1334, 5434, 3394, 3397, 2273, 3417, 1370, 5475, 3865, 3440, 1401, 3453, 3464, 1424, 3477, 5528, 5529, 1435, 525, 1438, 3499, 3507, 3511, 1464, 3514, 3515, 1469, 3518, 1471, 1477, 1482, 5579, 5583, 5584, 1493, 6396, 3565, 5617, 1522, 5622, 3575, 3581, 1538, 4353, 1624, 5655, 3677, 3638, 3650, 5700, 3653, 3672, 3674, 1629, 5727, 3681, 1641, 5738, 1658, 5766, 3721, 5770, 5772, 5780, 1693, 3743, 5793, 1700, 1704, 1705, 5810, 1715, 5813, 3768, 5824, 5829, 3784, 3786, 1740, 3789, 5856, 1772, 1780, 980, 5882, 1787, 1792, 5893, 3861, 1817, 3866, 5917, 1827, 3902, 5977, 3936, 1892, 1899, 5996, 3957, 3732, 3962, 3964, 1918, 3978, 6033, 6045, 4000, 1955, 6068, 4026, 4034, 6098, 2005, 4056, 4057, 6109, 677, 6112, 2021, 2025, 4078, 4184, 17, 966, 4487, 5983, 2136, 316, 699, 4177, 6293, 5868, 5716, 4480, 1394, 3927, 1157, 4537, 4035, 5746, 3258, 2681, 4730, 4840, 6131, 5306, 3002, 561, 3559, 6678, 2197, 4809, 3192, 405, 1356, 1763, 1462, 920, 3720, 3367, 7116, 5102, 4937, 4641, 2460, 938, 5184, 341, 4872, 4412, 1121, 4704, 4141, 2417, 3225, 2282, 3628, 1976, 380, 5160, 6613, 5354, 5343, 245, 3876, 1741, 1304, 3837, 6087, 6185, 2432, 6873, 5598, 3670, 3472, 4182, 4378, 1418, 3334, 4696, 3055, 3459, 988, 6469, 6323, 1782, 3105, 3701, 244, 586, 85, 5046, 2534, 6384, 912, 6841, 315, 3928, 4222, 6910, 3620, 6911, 3281, 5851, 5833, 2304, 6821, 2445, 5116, 6194, 4454, 5965, 7089, 5771, 5550, 2815, 1582, 4238, 2410, 4588, 6147, 2898, 3098, 2732, 4235, 4839, 3633, 3538, 4791, 4229, 5350, 2651, 4918, 4849, 367, 6447, 4832, 1713, 4988, 3989, 156, 4147, 880, 3852, 2843, 6987, 916, 2756, 2522, 797, 5317, 3328, 1933, 4664, 4631, 337, 5051, 314, 4742, 3845, 187, 3308, 5285, 4852, 4638, 3019, 3279, 4174, 2216, 3035, 6745, 5604, 4453, 4355, 928, 25, 4207, 2077, 2492, 5826, 2633, 2645, 5442, 5562, 5156, 5069, 3640, 1342, 5330, 5778, 5273, 802, 2945, 4338, 686, 3689, 3318, 305, 2372, 4151, 4154, 1297, 2059, 1054, 7021, 4687, 7101, 712, 1809, 2528, 3157, 7045, 5883, 2350, 7035, 5209, 6156, 5819, 1661, 1523, 6193, 538, 295, 3849, 2027, 1048, 6276, 2368, 510, 3777, 1928, 3700, 1962, 783, 3951, 4206, 1990, 3602, 483, 2320, 7012, 466, 900, 4486, 6889, 4555, 6732, 3750, 4191, 890, 6334, 5675, 1786, 6723, 4846, 2748, 2838, 2003, 3119, 6558, 6143, 941, 1205, 6755, 3134, 5267, 2727, 6782, 6357, 5554, 2401, 3833, 3386, 1252, 897, 6111, 6837, 5739, 2833, 6407, 6615, 2083, 6537, 3735, 6287, 2397, 5705, 3020, 4813, 4669, 5572, 3585, 647, 7109, 777, 3746, 2230, 234, 2806, 5729, 180, 808, 5993, 3929, 1877, 3995, 902, 5461, 5324, 4468, 4213, 6860, 1852, 3645, 6085, 4455, 4940, 5643, 2693, 3079, 1459, 5615, 1752, 3360, 1799, 6797, 1244, 3004, 3466, 4490, 5683, 1336, 6990, 1084, 7036, 2395, 6481, 2758, 2871, 1952, 6413, 231, 6875, 2770, 6972, 261, 547, 7105, 1855, 6466, 4167, 779, 5032, 3323, 2850, 6771, 5075, 6219, 1953, 2599, 5672, 1186, 7118, 4960, 1389, 6399, 4485, 2993, 618, 4851, 2169, 6630, 4142, 3530, 5042, 1024, 1189, 7049, 6520, 5383, 1563, 3683, 2841, 32, 5450, 4917, 4094, 4581, 3457, 4725, 3566, 451, 5080, 3770, 2532, 1023, 2671, 296, 4127, 2167, 6017, 6870, 5607, 2776, 5799, 846, 5514, 4254, 6390, 1108, 3428, 1695, 6092, 1070, 3236, 5906, 1798, 585, 1688, 1174, 1242, 1943, 614, 1476, 1659, 3186, 1215, 6090, 4732, 2231, 1161, 5536, 2373, 7064, 3953, 3596, 1046, 3897, 3337, 4816, 4740, 4428, 5627, 6974, 4308, 6731, 1197, 2900, 420, 791, 2734, 5474, 593, 3891, 3353, 5124, 6790, 1147, 6950, 7016, 6651, 1823, 6501, 6046, 5282, 4432, 5567, 6620, 6353, 5689, 1533, 1577, 203, 4262, 5369, 5277, 4913, 3270, 4143, 3165, 6829, 69, 3257, 3052, 5947, 5352, 1335, 178, 1371, 3402, 7071, 6539, 4612, 5263, 2817, 3604, 1795, 3153, 98, 4357, 3828, 2060, 1398, 5333, 2024, 2152, 524, 5706, 4538, 6680, 5323, 2779, 382, 634, 3718, 149, 1957, 6675, 6611, 2079, 6766, 5171, 7055, 798, 4383, 1380, 2794, 2980, 5731, 3124, 4125, 5605, 4225, 4310, 5722, 5782, 2330, 2175, 3315, 2563, 5441, 2081, 6658, 2469, 2605, 6722, 3844, 569, 33, 2244, 3755, 5452, 1021, 2132, 6605, 4189, 5119, 4501, 4061, 6652, 2250, 5698, 3994, 275, 2093, 2689, 2876], [2752, 7096, 1147, 5321, 4183, 2372, 6108, 165, 4635, 1000, 4440, 4770, 2808, 4248, 4273, 1979, 3667, 3610, 1886, 4412, 1953, 1898, 4041, 6053, 2190, 2913, 2929, 607, 5297, 3693, 4422, 574, 1604, 5728, 1469, 2161, 1684, 6333, 730, 3985, 3017, 1597, 1583, 1099, 3722, 4325, 6987, 2797, 4476, 4533, 5939, 4215, 5993, 483, 4282, 1205, 3825, 2265, 6852, 2423, 1550, 1059, 5584, 2757, 2594, 1361, 6745, 4457, 5858, 4567, 5312, 6901, 2220, 3524, 3042, 2549, 7028, 166, 3957, 6320, 5372, 3950, 6428, 1424, 509, 3152, 2739, 303, 2536, 851, 2678, 1878, 375, 7000, 4810, 2751, 3777, 5750, 1492, 4838, 1286, 5224, 4583, 2485, 3603, 6888, 1874, 6542, 6037, 134, 1717, 20, 5157, 5076, 4398, 2442, 5144, 5740, 2772, 3184, 5914, 5800, 4029, 1535, 2569, 4094, 4339, 1046, 2794, 243, 2008, 4534, 5373, 17, 4652, 2397, 2812, 1856, 5814, 206, 355, 5797, 5349, 6878, 7072, 1338, 1949, 6699, 6530, 5653, 913, 5219, 1602, 3831, 5716, 1129, 3101, 6445, 5134, 549, 3865, 3653, 4653, 1485, 1462, 6176, 1747, 1354, 1675, 1807, 1664, 1070, 164, 6697, 3757, 4146, 6673, 597, 3372, 6918, 1335, 2679, 1419, 5002, 2497, 5824, 6539, 5471, 1553, 871, 5135, 4410, 577, 565, 5293, 1219, 6767, 3161, 2349, 5846, 3626, 2780, 3551, 6342, 6054, 1049, 1155, 3137, 6447, 2693, 1524, 1751, 1014, 2460, 3694, 5185, 6538, 800, 5794, 6621, 6680, 3932, 3427, 3759, 239, 6928, 4101, 3657, 2921, 1801, 2770, 3867, 3638, 935, 2980, 3043, 780, 929, 5544, 4077, 2507, 5496, 2437, 605, 4949, 2999, 1596, 7117, 4842, 2469, 1495, 260, 3471, 1980, 2055, 4760, 5430, 3100, 3295, 3147, 6015, 6168, 4322, 4663, 4026, 2993, 910, 767, 3983, 6923, 6324, 2748, 4131, 387, 457, 4073, 6835, 4641, 3906, 791, 104, 4195, 4837, 847, 4581, 4001, 1996, 5354, 496, 6597, 5501, 2512, 3612, 6228, 4312, 3598, 405, 2860, 2216, 2016, 4271, 3736, 4272, 1869, 6757, 4308, 1910, 1926, 1048, 5242, 3691, 5272, 4111, 2241, 2468, 1316, 6410, 4405, 3201, 6149, 3585, 5607, 1088, 1746, 5440, 872, 1203, 5444, 6355, 1108, 2151, 828, 2004, 4743, 2137, 4458, 4438, 2843, 3890, 3994, 2107, 5529, 4447, 5793, 4, 4472, 6261, 5779, 713, 5702, 5950, 4611, 4933, 4983, 2771, 2383, 2125, 941, 1405, 6591, 2176, 6556, 4376, 389, 4334, 806, 87, 4427, 6143, 2201, 2320, 5415, 4483, 5899, 1043, 6686, 693, 6634, 1905, 857, 6961, 173, 983, 6181, 1797, 2312, 2351, 5164, 6864, 6502, 1840, 3129, 2461, 5876, 4037, 92, 3614, 4777, 6301, 3817, 3029, 2687, 195, 6838, 3948, 514, 5521, 4730, 1748, 3998, 6827, 3549, 1139, 5398, 5003, 5964, 3395, 4727, 602, 4946, 3213, 1790, 2270, 2316, 137, 6002, 4019, 3607, 284, 5839, 108, 830, 1291, 5636, 2362, 3785, 1935, 3315, 5661, 1983, 5467, 6347, 409, 1438, 297, 4477, 4133, 3953, 1273, 2410, 1857, 5022, 3146, 5363, 6189, 5796, 272, 3410, 735, 6414, 398, 1642, 1628, 2597, 3885, 4850, 3436, 2701, 1896, 4568, 6636, 5142, 2030, 1015, 6553, 230, 1456, 135, 5569, 1197, 1792, 1822, 5447, 3263, 1817, 6984, 476, 5497, 6158, 1092, 6633, 6405, 6465, 7015, 3399, 5482, 842, 5445, 2595, 612, 5840, 3665, 6094, 6828, 5252, 5986, 1956, 4313, 6032, 5402, 2702, 1740, 5703, 6233, 1619, 1223, 3297, 3616, 1548, 4660, 1698, 4448, 1902, 560, 652, 3030, 1277, 5897, 334, 5646, 4486, 6277, 5074, 481, 5045, 2997, 2014, 2646, 485, 1594, 4380, 7071, 3628, 1490, 2275, 4330, 5032, 7104, 1873, 5530, 716, 5733, 772, 719, 1699, 4920, 1918, 1232, 2880, 3834, 1930, 4905, 2928, 6334, 3587, 4903, 5172, 1686, 1201, 1771, 1708, 2367, 802, 6742, 6601, 1892, 6439, 556, 4385, 1603, 5300, 4694, 120, 2012, 4287, 2087, 873, 4181, 6193, 3641, 5807, 5775, 550, 91, 6579, 6136, 2961, 6524, 2082, 144, 1847, 5510, 4022, 4363, 4647, 55, 1275, 6948, 6028, 613, 5431, 3343, 4406, 2884, 6561, 5590, 4735, 5247, 3417, 2618, 2669, 3532, 4172, 702, 6769, 5102, 6756, 4093, 7, 1612, 2204, 1614, 3542, 4754, 3156, 3790, 997, 7013, 601, 3314, 4679, 96, 5722, 158, 3209, 5422, 4040, 1955, 4226, 5637, 3191, 6124, 6558, 5179, 4254, 1811, 6795, 3525, 6841, 674, 2774, 4571, 5873, 2526, 5174, 5133, 4970, 6705, 758, 131, 6156, 6799, 1613, 6369, 3737, 3921, 5508, 4989, 3203, 1390, 3546, 1888, 5229, 6854, 1214, 5580, 6185, 3550, 6817, 1933, 5367, 1113, 4830, 1415, 6356, 1587, 5647, 3173, 6596, 3698, 1618, 404, 424, 1782, 6046, 4709, 1631, 2135, 850, 3707, 2683, 2430, 739, 6092, 5220, 849, 2298, 726, 1215, 4756, 5953, 5340, 3038, 1281, 3883, 7088, 1757, 4177, 846, 7078, 4302, 1829, 7086, 3493, 6458, 793, 1124, 5451, 251, 4148, 1592, 4866, 6422, 4043, 2741, 5571, 5617, 1784, 705, 4923, 98, 3577, 500, 1192, 6083, 4630, 4578, 6537, 6274, 4722, 3220, 3887, 1683, 5338, 337, 6386, 6091, 2850, 1348, 6768, 2443, 2238, 6920, 4539, 7047, 2853, 2593, 7043, 3611, 1202, 3661, 3, 3366, 1893, 6965, 6616, 732, 4420, 4333, 5516, 4276, 2168, 524, 1921, 1730, 5238, 902, 3789, 924, 2463, 7037, 1510, 6162, 1168, 2864, 3520, 2340, 1872, 2773, 1360, 5582, 3398, 5700, 3796, 176, 4031, 6533, 6517, 3062, 7123, 1985, 6581, 4713, 3830, 4147, 5827, 2916, 498, 5604, 914, 5212, 6563, 5307, 6215, 5663, 5434, 1616, 4634, 2601, 5816, 4241, 2382, 3530, 1989, 79, 3446, 6626, 4299, 6113, 2885, 44, 3824, 7017, 5903, 4171, 5187, 2807, 3128, 3081, 1053, 1187, 4305, 380, 5342, 2511, 4020, 912, 5203, 2520, 4235, 3303, 4252, 3361, 5167, 2375, 583, 6963, 5148, 2058, 2114, 1089, 586, 2069, 6192, 3269, 1696, 2388, 3414, 3900, 1796, 4701, 1352, 1880, 4556, 2487, 3154, 3120, 5782, 1447, 3321, 81, 450, 171, 4117, 291, 4238, 3624, 542, 6196, 936, 5550, 4163, 5472, 6205, 3058, 3448, 3059, 2575, 74, 3970, 5454, 6033, 2673, 648, 5823, 5933, 5721, 6669, 269, 2345, 2954, 3678, 4666, 1133, 2648, 5460, 6464, 1373, 5125, 3420, 2219, 2389, 1303, 5734, 31, 4178, 4622, 5888, 2893, 897, 6902, 5925, 5937, 4968, 3109, 3781, 4564, 6716, 4675, 2728, 5289, 2722, 9, 213, 1418, 3001, 6344, 4975, 4230, 3094, 3134, 493, 4244, 4929, 5577, 5487, 2025, 3584, 2943, 6360, 750, 4205, 4664, 5319, 5627, 2574, 1971, 6515, 6290, 6080, 2268, 4358, 3430, 5288, 5418, 4461, 160, 6423, 3229, 2878], [6154, 6156, 2069, 4118, 2077, 4126, 2082, 6179, 4151, 5813, 65, 2119, 78, 2131, 84, 6229, 89, 2139, 104, 4203, 2158, 112, 4209, 4210, 6266, 123, 4220, 2173, 2175, 128, 6276, 4229, 143, 6289, 6293, 6294, 3781, 6305, 165, 6311, 2219, 29, 178, 6323, 5934, 185, 4285, 191, 6340, 202, 3106, 4302, 4304, 210, 4308, 6357, 2270, 6368, 237, 6384, 2291, 6390, 2296, 252, 4353, 4355, 263, 4361, 4364, 6416, 275, 4372, 2327, 4380, 2334, 2340, 295, 2780, 6445, 4407, 2360, 6464, 6465, 4420, 4421, 2375, 2377, 2383, 6481, 343, 4443, 6492, 2397, 2399, 6498, 365, 367, 6513, 6514, 6520, 4480, 5184, 2435, 4485, 4490, 2445, 4496, 402, 405, 4507, 6556, 6559, 418, 2468, 2471, 6568, 430, 4534, 4537, 2806, 2522, 477, 4859, 2533, 2534, 4584, 2544, 6648, 2815, 1451, 2571, 2573, 4622, 536, 4633, 6682, 6683, 2589, 6686, 2594, 547, 552, 2603, 2607, 6705, 2614, 6712, 2617, 6717, 6731, 4687, 594, 595, 2645, 4696, 4697, 4713, 6766, 6772, 2677, 2678, 4727, 2680, 4730, 6780, 6782, 4738, 4740, 2693, 4742, 4751, 2704, 657, 2714, 2723, 6820, 2725, 2727, 2734, 687, 6834, 2740, 6839, 2751, 708, 2758, 6855, 6860, 6870, 732, 4829, 6881, 2786, 2790, 2791, 4841, 749, 4853, 758, 4856, 4857, 4223, 6910, 1493, 2816, 6915, 774, 4880, 4883, 2837, 4886, 792, 801, 805, 806, 4573, 4918, 6969, 2874, 6974, 831, 2884, 6966, 840, 6987, 6989, 6993, 4946, 851, 2900, 2903, 4952, 858, 4239, 860, 4960, 2914, 4965, 7014, 4968, 7035, 2941, 7038, 7040, 4993, 900, 7049, 909, 910, 2961, 2973, 938, 5038, 2991, 946, 7101, 7112, 3020, 5069, 5072, 3030, 5080, 3034, 5352, 5087, 5096, 5107, 1014, 1017, 1021, 5134, 5136, 1046, 5977, 5147, 5154, 7003, 1202, 3253, 3136, 3143, 1097, 3147, 1124, 5222, 1129, 5228, 5229, 1143, 5253, 1174, 5273, 5278, 4635, 1192, 3247, 790, 1205, 3258, 1225, 5322, 5346, 1251, 1253, 5351, 1256, 5354, 5355, 3309, 1263, 5364, 2260, 3323, 7041, 3338, 3343, 3344, 5394, 3350, 5399, 3352, 3366, 5417, 5419, 1334, 5434, 3394, 3397, 1350, 3399, 5452, 3298, 3412, 1368, 3417, 5466, 5473, 3427, 6033, 5482, 5489, 3453, 3464, 1424, 5528, 5529, 1435, 1438, 1446, 3499, 3514, 1469, 1952, 1482, 3531, 5583, 1490, 3541, 5598, 5609, 5612, 3565, 6529, 1522, 3574, 1538, 3591, 3595, 3597, 5649, 5655, 1563, 5668, 1575, 3628, 1582, 3677, 3640, 3645, 5700, 3653, 2372, 5706, 5712, 1618, 5716, 5717, 1624, 3674, 1629, 5727, 3681, 1641, 5053, 1658, 1659, 3721, 5770, 5772, 1304, 3732, 1693, 3743, 1696, 5793, 3750, 5810, 3765, 5820, 5824, 5829, 1734, 5832, 3786, 1741, 5851, 5856, 1763, 1772, 3822, 5882, 3836, 5893, 3849, 301, 984, 3861, 5911, 3867, 1822, 1827, 3876, 1838, 3899, 3902, 3909, 5965, 1872, 3922, 1879, 3929, 3935, 5984, 4752, 5986, 5993, 1899, 5996, 3950, 3951, 2369, 1937, 3993, 4000, 1955, 4007, 4016, 3059, 6068, 4771, 4026, 1695, 1982, 2037, 6085, 2002, 2003, 6104, 6109, 677, 4775, 3069, 6819, 2045, 4094, 4866, 1694, 959, 3885, 5739, 3800, 1523, 777, 6937, 4325, 6202, 5438, 1091, 1215, 7044, 4908, 2284, 4657, 589, 2843, 1009, 2631, 4417, 7083, 6740, 5119, 3953, 6985, 4851, 6724, 6256, 1709, 4634, 2376, 6996, 6739, 1497, 5746, 1258, 1569, 1856, 3134, 5016, 179, 640, 2400, 1731, 4724, 2251, 2269, 2773, 1180, 578, 1084, 1149, 2021, 3374, 3005, 1680, 3212, 6630, 1877, 2087, 811, 2230, 256, 1918, 3913, 4935, 5065, 613, 6664, 1705, 4454, 4385, 6333, 2560, 5784, 3569, 1800, 4183, 4255, 2212, 5729, 4174, 2756, 2197, 2345, 2354, 27, 1984, 262, 6395, 5057, 5132, 6430, 2027, 5689, 4554, 965, 3831, 466, 6080, 4236, 3782, 3178, 1817, 5769, 4031, 3813, 4814, 2344, 5616, 5333, 302, 755, 1416, 524, 6253, 462, 5142, 6227, 1028, 5000, 3362, 6231, 6861, 6550, 1380, 4201, 956, 6655, 5601, 5058, 4481, 510, 5622, 6219, 4463, 4667, 2491, 6620, 2456, 931, 2006, 1105, 6201, 3658, 966, 2934, 2231, 1843, 2152, 3834, 1426, 4144, 864, 4652, 6100, 1242, 6322, 4057, 1365, 3052, 4321, 920, 1353, 5037, 4172, 4354, 1420, 3848, 5217, 3582, 1581, 1639, 3663, 784, 2615, 5123, 6092, 787, 6517, 4301, 3511, 2679, 5588, 603, 1572, 4642, 5323, 4453, 3865, 2630, 5745, 1121, 3370, 3221, 2379, 382, 3934, 2147, 1462, 1818, 1974, 6722, 4200, 6449, 5809, 1700, 4495, 4077, 3368, 6802, 2993, 954, 7057, 1054, 1199, 6543, 4599, 2029, 6081, 5869, 6805, 6128, 5567, 2019, 597, 1337, 4673, 6649, 4307, 2591, 2817, 3604, 2719, 4177, 1893, 6470, 5617, 5252, 5062, 5214, 305, 5187, 525, 5244, 1545, 2750, 1114, 5285, 1837, 25, 87, 6014, 4472, 4167, 719, 1726, 636, 2844, 1533, 33, 3079, 6911, 431, 1185, 4182, 583, 483, 7116, 4310, 4056, 4855, 349, 1189, 161, 1805, 982, 3962, 911, 5976, 3776, 516, 5212, 3049, 2417, 4758, 3386, 2440, 218, 3367, 514, 3165, 2739, 1063, 6567, 5043, 6057, 6784, 6723, 6232, 21, 5263, 928, 4570, 2028, 3638, 847, 6159, 4581, 3337, 4382, 3239, 1051, 4274, 1603, 4254, 3300, 3768, 2671, 2958, 3436, 3485, 3105, 3032, 2825, 3720, 634, 2217, 6959, 4505, 6344, 771, 2737, 5599, 3192, 1432, 4759, 4597, 4723, 4791, 4081, 1116, 3276, 1701, 7008, 816, 5102, 4293, 4207, 3588, 6334, 4235, 1008, 6042, 752, 1557, 5723, 3103, 5868, 6111, 4817, 3227, 6887, 3650, 3837, 409, 2081, 4889, 693, 401, 3181, 1682, 4384, 1157, 5177, 1335, 897, 94, 2532, 3773, 4852, 3089, 4261, 6615, 1052, 6605, 7104, 2515, 5903, 4779, 136, 4225, 967, 5338, 4631, 79, 2250, 1318, 5211, 5572, 2871, 3680, 6098, 2282, 6920, 5936, 3546, 4096, 5347, 3620, 348, 3788, 4143, 2132, 2075, 6211, 745, 6356, 2299, 4095, 3838, 2564, 3168, 1505, 6440, 586, 1471, 5714, 5754, 1626, 4409, 5775, 4669, 3538, 3501, 4774, 4743, 1780, 4903, 6407, 28, 3700, 3507, 3906, 3970, 715, 6054, 2305, 5386, 2663, 1080, 5046, 3670, 6971, 4763, 2122, 4024, 5533, 5300, 3684, 5675, 3028, 3989, 3206, 4699, 3789, 1087, 6112, 131, 498, 6339, 1476, 2596, 4448, 1244, 1214, 1024, 1608, 735, 5884, 3402, 1108, 521, 3735, 1979, 4514, 5614, 5540, 3478, 6314, 3586, 6024, 5589, 1179, 5257, 1926, 2393, 3491, 6539, 3420, 6886, 1407, 4211, 2350, 6363, 4107, 2285, 4259, 3904, 3236, 2605, 4048, 5511, 267, 4391, 5006, 6811, 731, 1067, 2579, 3364, 6291], [2752, 7096, 1147, 5321, 4183, 2372, 6108, 165, 4635, 1000, 4440, 4770, 2808, 4248, 4273, 1979, 3667, 3610, 1886, 4412, 1953, 1898, 4041, 6053, 2190, 2913, 2929, 607, 5297, 3693, 4422, 574, 1604, 5728, 1469, 2161, 1684, 6333, 730, 3985, 3017, 1597, 1583, 1099, 3722, 4325, 6987, 2797, 4476, 4533, 5939, 4215, 5993, 483, 4282, 1205, 3825, 2265, 6852, 2423, 1550, 1059, 5584, 2757, 2594, 1361, 6745, 4457, 5858, 4567, 5312, 6901, 2220, 3524, 3042, 2549, 7028, 166, 3957, 6320, 5372, 3950, 6428, 1424, 509, 3152, 2739, 303, 2536, 851, 2678, 1878, 375, 7000, 4810, 2751, 3777, 5750, 1492, 4838, 1286, 5224, 4583, 2485, 3603, 6888, 1874, 6542, 6037, 134, 1717, 20, 5157, 5076, 4398, 2442, 5144, 5740, 2772, 3184, 5914, 5800, 4029, 1535, 2569, 4094, 4339, 1046, 2794, 243, 2008, 4534, 5373, 17, 4652, 2397, 2812, 1856, 5814, 206, 355, 5797, 5349, 6878, 7072, 1338, 1949, 6699, 6530, 5653, 913, 5219, 1602, 3831, 5716, 1129, 3101, 6445, 5134, 549, 3865, 3653, 4653, 1485, 1462, 6176, 1747, 1354, 1675, 1807, 1664, 1070, 164, 6697, 3757, 4146, 6673, 597, 3372, 6918, 1335, 2679, 1419, 5002, 2497, 5824, 6539, 5471, 1553, 871, 5135, 4410, 577, 565, 5293, 1219, 6767, 3161, 2349, 5846, 3626, 2780, 3551, 6342, 6054, 1049, 1155, 3137, 6447, 2693, 1524, 1751, 1014, 2460, 3694, 5185, 6538, 800, 5794, 6621, 6680, 3932, 3427, 3759, 239, 6928, 4101, 3657, 2921, 1801, 2770, 3867, 3638, 935, 2980, 3043, 780, 929, 5544, 4077, 2507, 5496, 2437, 605, 4949, 2999, 1596, 7117, 4842, 2469, 1495, 260, 3471, 1980, 2055, 4760, 5430, 3100, 3295, 3147, 6015, 6168, 4322, 4663, 4026, 2993, 910, 767, 3983, 6923, 6324, 2748, 4131, 387, 457, 4073, 6835, 4641, 3906, 791, 104, 4195, 4837, 847, 4581, 4001, 1996, 5354, 496, 6597, 5501, 2512, 3612, 6228, 4312, 3598, 405, 2860, 2216, 2016, 4271, 3736, 4272, 1869, 6757, 4308, 1910, 1926, 1048, 5242, 3691, 5272, 4111, 2241, 2468, 1316, 6410, 4405, 3201, 6149, 3585, 5607, 1088, 1746, 5440, 872, 1203, 5444, 6355, 1108, 2151, 828, 2004, 4743, 2137, 4458, 4438, 2843, 3890, 3994, 2107, 5529, 4447, 5793, 4, 4472, 6261, 5779, 713, 5702, 5950, 4611, 4933, 4983, 2771, 2383, 2125, 941, 1405, 6591, 2176, 6556, 4376, 389, 4334, 806, 87, 4427, 6143, 2201, 2320, 5415, 4483, 5899, 1043, 6686, 693, 6634, 1905, 857, 6961, 173, 983, 6181, 1797, 2312, 2351, 5164, 6864, 6502, 1840, 3129, 2461, 5876, 4037, 92, 3614, 4777, 6301, 3817, 3029, 2687, 195, 6838, 3948, 514, 5521, 4730, 1748, 3998, 6827, 3549, 1139, 5398, 5003, 5964, 3395, 4727, 602, 4946, 3213, 1790, 2270, 2316, 137, 6002, 4019, 3607, 284, 5839, 108, 830, 1291, 5636, 2362, 3785, 1935, 3315, 5661, 1983, 5467, 6347, 409, 1438, 297, 4477, 4133, 3953, 1273, 2410, 1857, 5022, 3146, 5363, 6189, 5796, 272, 3410, 735, 6414, 398, 1642, 1628, 2597, 3885, 4850, 3436, 2701, 1896, 4568, 6636, 5142, 2030, 1015, 6553, 230, 1456, 135, 5569, 1197, 1792, 1822, 5447, 3263, 1817, 6984, 476, 5497, 6158, 1092, 6633, 6405, 6465, 7015, 3399, 5482, 842, 5445, 2595, 612, 5840, 3665, 6094, 6828, 5252, 5986, 1956, 4313, 6032, 5402, 2702, 1740, 5703, 6233, 1619, 1223, 3297, 3616, 1548, 4660, 1698, 4448, 1902, 560, 652, 3030, 1277, 5897, 334, 5646, 4486, 6277, 5074, 481, 5045, 2997, 2014, 2646, 485, 1594, 4380, 7071, 3628, 1490, 2275, 4330, 5032, 7104, 1873, 5530, 716, 5733, 772, 719, 1699, 4920, 1918, 1232, 2880, 3834, 1930, 4905, 2928, 6334, 3587, 4903, 5172, 1686, 1201, 1771, 1708, 2367, 802, 6742, 6601, 1892, 6439, 556, 4385, 1603, 5300, 4694, 120, 2012, 4287, 2087, 873, 4181, 6193, 3641, 5807, 5775, 550, 91, 6579, 6136, 2961, 6524, 2082, 144, 1847, 5510, 4022, 4363, 4647, 55, 1275, 6948, 6028, 613, 5431, 3343, 4406, 2884, 6561, 5590, 4735, 5247, 3417, 2618, 2669, 3532, 4172, 702, 6769, 5102, 6756, 4093, 7, 1612, 2204, 1614, 3542, 4754, 3156, 3790, 997, 7013, 601, 3314, 4679, 96, 5722, 158, 3209, 5422, 4040, 1955, 4226, 5637, 3191, 6124, 6558, 5179, 4254, 1811, 6795, 3525, 6841, 674, 2774, 4571, 5873, 2526, 5174, 5133, 4970, 6705, 758, 131, 6156, 6799, 1613, 6369, 3737, 3921, 5508, 4989, 3203, 1390, 3546, 1888, 5229, 6854, 1214, 5580, 6185, 3550, 6817, 1933, 5367, 1113, 4830, 1415, 6356, 1587, 5647, 3173, 6596, 3698, 1618, 404, 424, 1782, 6046, 4709, 1631, 2135, 850, 3707, 2683, 2430, 739, 6092, 5220, 849, 2298, 726, 1215, 4756, 5953, 5340, 3038, 1281, 3883, 7088, 1757, 4177, 846, 7078, 4302, 1829, 7086, 3493, 6458, 793, 1124, 5451, 251, 4148, 1592, 4866, 6422, 4043, 2741, 5571, 5617, 1784, 705, 4923, 98, 3577, 500, 1192, 6083, 4630, 4578, 6537, 6274, 4722, 3220, 3887, 1683, 5338, 337, 6386, 6091, 2850, 1348, 6768, 2443, 2238, 6920, 4539, 7047, 2853, 2593, 7043, 3611, 1202, 3661, 3, 3366, 1893, 6965, 6616, 732, 4420, 4333, 5516, 4276, 2168, 524, 1921, 1730, 5238, 902, 3789, 924, 2463, 7037, 1510, 6162, 1168, 2864, 3520, 2340, 1872, 2773, 1360, 5582, 3398, 5700, 3796, 176, 4031, 6533, 6517, 3062, 7123, 1985, 6581, 4713, 3830, 4147, 5827, 2916, 498, 5604, 914, 5212, 6563, 5307, 6215, 5663, 5434, 1616, 4634, 2601, 5816, 4241, 2382, 3530, 1989, 79, 3446, 6626, 4299, 6113, 2885, 44, 3824, 7017, 5903, 4171, 5187, 2807, 3128, 3081, 1053, 1187, 4305, 380, 5342, 2511, 4020, 912, 5203, 2520, 4235, 3303, 4252, 3361, 5167, 2375, 583, 6963, 5148, 2058, 2114, 1089, 586, 2069, 6192, 3269, 1696, 2388, 3414, 3900, 1796, 4701, 1352, 1880, 4556, 2487, 3154, 3120, 5782, 1447, 3321, 81, 450, 171, 4117, 291, 4238, 3624, 542, 6196, 936, 5550, 4163, 5472, 6205, 3058, 3448, 3059, 2575, 74, 3970, 5454, 6033, 2673, 648, 5823, 5933, 5721, 6669, 269, 2345, 2954, 3678, 4666, 1133, 2648, 5460, 6464, 1373, 5125, 3420, 2219, 2389, 1303, 5734, 31, 4178, 4622, 5888, 2893, 897, 6902, 5925, 5937, 4968, 3109, 3781, 4564, 6716, 4675, 2728, 5289, 2722, 9, 213, 1418, 3001, 6344, 4975, 4230, 3094, 3134, 493, 4244, 4929, 5577, 5487, 2025, 3584, 2943, 6360, 750, 4205, 4664, 5319, 5627, 2574, 1971, 6515, 6290, 6080, 2268, 4358, 3430, 5288, 5418, 4461, 160, 6423, 3229, 2878], [2049, 7083, 2564, 3591, 5130, 2571, 6159, 6232, 4611, 4573, 4118, 89, 4633, 6682, 6683, 28, 2589, 4126, 3106, 6179, 2084, 262, 552, 6185, 2603, 3019, 2607, 6705, 6787, 5173, 2614, 5177, 6202, 6716, 5813, 5185, 578, 1091, 5700, 3653, 583, 1096, 3147, 2914, 4174, 3151, 5714, 3171, 7112, 597, 3672, 1124, 3674, 4699, 1116, 6749, 4555, 2655, 3168, 4195, 4196, 1639, 1641, 5738, 4203, 6764, 3182, 3183, 6256, 6770, 1139, 1129, 3192, 6870, 2751, 6780, 3861, 640, 1334, 5253, 5766, 3720, 5228, 5770, 5655, 5772, 1166, 2704, 657, 1171, 5780, 25, 708, 2119, 1179, 2716, 4254, 3743, 6819, 6820, 677, 881, 1704, 1564, 2147, 3155, 3784, 790, 179, 2740, 3253, 3768, 5819, 2748, 6845, 3262, 191, 5824, 2251, 6860, 4293, 6855, 1224, 1225, 5322, 1227, 1740, 4304, 2663, 1746, 3519, 4310, 5338, 5339, 1014, 5856, 6881, 2786, 4835, 1253, 2791, 1256, 4841, 1772, 3309, 4849, 2802, 755, 5364, 758, 2296, 5369, 1787, 253, 4353, 3350, 771, 5072, 5893, 774, 7041, 128, 4880, 1424, 4883, 4107, 4886, 5399, 3352, 4380, 1629, 1827, 806, 4903, 297, 3370, 5419, 1324, 4496, 477, 2864, 1672, 2358, 2868, 5212, 6311, 5434, 3511, 3978, 3902, 831, 6465, 4420, 3397, 6985, 1477, 343, 4938, 3298, 6291, 849, 1251, 660, 3129, 4952, 5977, 1370, 7003, 6492, 4325, 165, 6498, 2679, 7014, 6033, 4968, 1899, 4463, 3440, 6513, 6514, 1700, 2025, 1172, 2938, 2623, 3964, 893, 7038, 4993, 65, 3464, 394, 4162, 910, 4495, 2059, 2961, 402, 6549, 5528, 5529, 1435, 6556, 6045, 4421, 4000, 4514, 2468, 430, 2471, 4599, 3499, 2477, 6237, 2991, 3765, 946, 2291, 6068, 6814, 4535, 6584, 1780, 3514, 2719, 1469, 3518, 1471, 4034, 5622, 4552, 1482, 5579, 462, 6989, 5584, 6098, 237, 3028, 4056, 3236, 851, 5084, 6109, 2725, 6112, 4255, 749, 516, 2021, 5609, 4775, 3154, 4078, 3581, 2544, 1522, 595, 3574, 3575, 3069, 6229, 1337, 6390, 32, 6910, 348, 5134, 3789, 3800, 5784, 1297, 243, 4285, 6745, 2139, 5746, 5823, 224, 6114, 234, 167, 6889, 4635, 6975, 5330, 4647, 1046, 2560, 2176, 6971, 6651, 2494, 4210, 1453, 6080, 3277, 6415, 5953, 7028, 791, 3098, 3977, 4543, 732, 5236, 1849, 4206, 5782, 2695, 2645, 1312, 6621, 1035, 6766, 5937, 2410, 1368, 2551, 5554, 4746, 1394, 1213, 1918, 5832, 665, 2406, 2132, 1238, 3328, 6529, 750, 1795, 5444, 337, 3366, 5921, 4763, 5754, 3153, 2756, 671, 803, 1408, 1817, 7077, 5132, 4809, 6605, 4472, 3059, 802, 647, 3595, 6774, 5727, 5205, 633, 5070, 1618, 1693, 4226, 6438, 5621, 4656, 4412, 1517, 5820, 2083, 3677, 1151, 4937, 3936, 5210, 3670, 2129, 1658, 2060, 6220, 305, 1538, 4960, 4480, 2131, 6384, 2495, 6805, 5606, 1252, 2580, 3453, 2984, 4669, 4571, 4787, 5627, 314, 480, 777, 4928, 2180, 5282, 7006, 4061, 6771, 5480, 1281, 3559, 5323, 884, 3508, 85, 2504, 1405, 3735, 3594, 41, 2643, 6063, 5894, 3683, 2781, 1200, 5350, 5479, 6294, 6905, 4913, 5038, 29, 5947, 6731, 1695, 2837, 5577, 1582, 1279, 4709, 850, 6215, 1105, 405, 4785, 1080, 5248, 5160, 3466, 3157, 1928, 3334, 2686, 5595, 2633, 524, 2516, 3035, 862, 3394, 5837, 4355, 3859, 1174, 5689, 1242, 3501, 6340, 2375, 1226, 6662, 2282, 6843, 6092, 3360, 6684, 2244, 3931, 1456, 3343, 259, 4779, 4211, 6950, 4338, 2197, 6782, 4537, 2027, 6381, 6266, 2024, 6873, 90, 3124, 3589, 1563, 6051, 5351, 980, 2638, 2605, 3989, 3899, 1017, 2862, 6649, 6641, 3815, 3446, 5960, 3373, 5311, 2035, 3105, 6198, 4453, 5906, 6732, 6399, 1709, 6990, 2369, 1422, 3134, 6911, 6752, 1065, 6680, 4612, 4655, 4574, 6686, 5119, 5333, 3675, 6999, 4207, 876, 1591, 3718, 156, 6987, 4641, 4507, 4638, 6837, 5346, 3500, 225, 3633, 4505, 341, 1084, 1943, 4832, 5583, 4021, 4624, 783, 6111, 6648, 2579, 4704, 5285, 4629, 1933, 5617, 3270, 5136, 2570, 4965, 5296, 5508, 1809, 2677, 1858, 2882, 2166, 5156, 3725, 7093, 1715, 880, 4142, 4657, 820, 7118, 4029, 5302, 3770, 3852, 1930, 3079, 3951, 3993, 3181, 5053, 1569, 4260, 5471, 2168, 4031, 1934, 3721, 2980, 3490, 5246, 4949, 2871, 3538, 231, 2492, 6496, 4794, 98, 4872, 3254, 3786, 275, 1718, 274, 5729, 686, 2206, 6928, 6640, 5498, 6642, 3759, 2212, 409, 4588, 3573, 3620, 3281, 712, 7101, 2817, 2152, 2216, 4191, 6362, 6017, 5297, 3604, 3844, 6004, 2081, 3477, 5452, 3145, 3002, 1322, 5306, 5329, 5446, 5463, 1720, 3816, 5244, 5674, 2651, 1390, 5416, 3681, 4026, 3962, 23, 2373, 2852, 2687, 4183, 184, 2617, 6593, 1446, 3628, 966, 4918, 955, 5598, 2838, 4220, 6875, 94, 2440, 6193, 797, 5025, 1228, 2445, 5262, 33, 1972, 2702, 1051, 4459, 779, 2515, 1533, 713, 2483, 5002, 5702, 4912, 6378, 1304, 5051, 5033, 2727, 5980, 4648, 5550, 4931, 890, 2394, 6409, 4740, 4437, 4157, 420, 1602, 185, 5911, 1955, 1705, 5648, 656, 3026, 73, 5569, 3293, 3239, 2681, 6139, 4057, 309, 5771, 6580, 609, 6446, 6616, 2738, 4933, 3314, 5046, 6165, 4154, 1153, 367, 902, 4063, 517, 3348, 4817, 1805, 5801, 3943, 2155, 2106, 178, 7021, 4760, 2372, 3421, 2689, 4725, 3119, 2794, 1939, 4222, 4146, 4876, 5882, 3258, 2379, 5799, 2140, 1248, 6073, 1302, 5928, 5572, 719, 6322, 5833, 4501, 2946, 2306, 634, 5624, 2082, 5712, 364, 4489, 2844, 7016, 3865, 6678, 2874, 5705, 5604, 3927, 4426, 3052, 5793, 7010, 2431, 3645, 6221, 1537, 1462, 6474, 3530, 2790, 3131, 2392, 4238, 1752, 1143, 4454, 1, 6862, 424, 2194, 3038, 1741, 7045, 6652, 99, 5574, 5722, 4321, 3484, 1147, 6087, 1070, 415, 3542, 7012, 4534, 6722, 2724, 4840, 1114, 3064, 4151, 3231, 5057, 469, 2385, 3287, 9, 4110, 1843, 1097, 2712, 2330, 2167, 5154, 3189, 2169, 4813, 3805, 1799, 1329, 3204, 3636, 4328, 6469, 3411, 4177, 6279, 3094, 3807, 4791, 4991, 4383, 5354, 6420, 547, 298, 5391, 3264, 4723, 2233, 4125, 1994, 5567, 4771, 1798, 4413, 6082, 1025, 72, 885, 3235, 6687, 2210, 38, 1379, 4568, 4448, 397, 7125, 1418, 1023, 2613, 4372, 499, 1108, 920, 1990, 4144, 6946, 6245, 1877, 4417, 4816, 267, 2077, 4259, 6148, 4035, 5810, 7105, 2884, 172, 5643, 6046, 2393, 3188, 3034, 2509, 5731, 2776, 1575, 115, 735, 1626, 7055, 3507, 1581, 1197, 1342, 3824, 245, 3082, 2231, 1214, 1454, 798, 525, 5668, 4940, 3165, 4512, 2599, 4490, 4298], [2060, 25, 28, 2077, 4126, 4130, 348, 6187, 4142, 4151, 6202, 4162, 69, 2122, 4172, 2131, 6229, 89, 2139, 6237, 2147, 5868, 2151, 2155, 6253, 112, 4210, 2169, 123, 128, 6276, 4231, 6291, 6294, 4249, 2206, 4255, 165, 4262, 6311, 6314, 171, 179, 180, 2231, 185, 2239, 4301, 4304, 4321, 4324, 237, 2291, 245, 6390, 251, 252, 4353, 2313, 267, 6413, 274, 275, 2327, 4380, 6430, 2340, 296, 297, 298, 6447, 2358, 4407, 4412, 6464, 6465, 4420, 6474, 337, 2394, 6492, 6498, 4455, 4463, 6513, 6514, 6517, 6520, 380, 2431, 6529, 4485, 6537, 4490, 6539, 2445, 4495, 4496, 402, 6549, 4507, 6556, 2468, 2471, 2473, 2477, 1097, 6584, 4537, 451, 6220, 2334, 6611, 4571, 477, 4581, 2534, 4588, 2544, 4604, 85, 2564, 6664, 2571, 2573, 4622, 2579, 6677, 4633, 6682, 6683, 2589, 4641, 552, 2603, 4189, 4657, 2613, 2614, 568, 2617, 6716, 578, 6723, 583, 6731, 6732, 6734, 5219, 2645, 4709, 4713, 618, 6770, 4723, 2678, 6776, 6782, 4203, 790, 4745, 655, 2704, 657, 660, 6806, 665, 2719, 6819, 6820, 2725, 4779, 2732, 114, 687, 2740, 6839, 699, 2748, 2751, 3531, 708, 713, 4817, 2770, 6870, 6881, 2786, 749, 750, 4852, 4856, 4859, 6911, 6915, 774, 2833, 4886, 793, 2848, 4903, 2864, 2868, 4918, 2874, 6974, 831, 6985, 6990, 2898, 4949, 4952, 7003, 860, 2914, 7014, 2926, 880, 7036, 2941, 7038, 7041, 7048, 7051, 910, 2417, 2961, 2973, 936, 5038, 946, 5046, 3002, 955, 5053, 5621, 5057, 5622, 7112, 3020, 1186, 5072, 5080, 3034, 5084, 5087, 3055, 1014, 1017, 5119, 5136, 1054, 5154, 3112, 1065, 5173, 1080, 5177, 1091, 3143, 1096, 3145, 3151, 1105, 3154, 3155, 3157, 1114, 3106, 5222, 1129, 3182, 3183, 1139, 1147, 1148, 5257, 1166, 1171, 1179, 538, 5282, 3236, 1225, 1208, 3262, 1215, 5321, 5333, 1238, 5338, 1244, 1248, 5346, 5347, 5354, 5355, 1263, 2838, 5364, 5365, 1273, 3326, 3334, 4993, 3338, 3343, 3350, 3352, 1312, 3366, 3367, 5417, 5419, 1329, 1334, 1336, 3386, 2955, 3397, 3399, 5452, 569, 3417, 2273, 3426, 1390, 3440, 5498, 1416, 1418, 3650, 3477, 5528, 5529, 1435, 1445, 1446, 3507, 3514, 3515, 1469, 3518, 1471, 1477, 3530, 5579, 3538, 1493, 5609, 3565, 2369, 1522, 1523, 3573, 3574, 597, 1538, 3591, 3596, 1624, 5655, 1564, 1569, 1572, 5672, 3638, 3640, 5698, 1603, 5705, 5714, 1623, 3672, 5723, 1629, 5727, 3684, 1639, 1641, 3720, 5771, 3735, 5784, 3743, 3750, 3759, 3761, 5813, 5819, 5824, 5833, 3786, 1741, 1751, 1318, 3815, 1772, 1780, 1782, 3838, 1792, 1795, 1805, 3855, 5913, 1818, 5917, 5921, 1827, 5935, 3891, 1849, 3902, 1856, 3909, 1877, 1879, 3929, 3934, 3951, 6014, 1927, 1928, 6033, 3993, 3994, 4000, 1953, 6068, 1982, 4034, 4035, 66, 6113, 4078, 613, 6381, 2395, 6034, 4568, 1455, 4310, 5418, 3124, 2663, 4940, 6615, 1918, 503, 3508, 3620, 1800, 1796, 868, 1585, 537, 3948, 2481, 3459, 5997, 5130, 4421, 3340, 2887, 7077, 4428, 6020, 6179, 900, 3499, 1051, 3105, 2225, 6206, 5886, 5606, 5989, 2373, 3633, 3594, 1672, 1618, 2560, 4662, 5211, 3202, 3970, 4279, 938, 3258, 5288, 5577, 6547, 2492, 266, 2780, 966, 4385, 5525, 2093, 1514, 3849, 191, 3323, 4204, 3089, 3719, 409, 405, 5605, 758, 1686, 675, 5262, 5399, 3866, 7045, 2360, 634, 4631, 3466, 4872, 4880, 4057, 5208, 1575, 4912, 579, 6660, 2447, 4707, 885, 324, 6979, 1582, 1843, 4223, 4114, 4832, 7015, 4992, 2208, 5171, 1176, 6499, 2145, 5983, 5799, 2119, 309, 6686, 5648, 4007, 6396, 3464, 5758, 7083, 5285, 4225, 791, 365, 2532, 234, 5000, 4461, 3239, 2397, 1899, 1952, 816, 7035, 5810, 2399, 6044, 4308, 228, 4584, 6558, 3953, 3966, 6966, 4554, 6918, 4760, 3253, 1199, 6073, 368, 4239, 508, 1722, 4355, 1401, 3977, 5832, 2244, 3176, 1185, 2160, 1804, 203, 5546, 6273, 6780, 4325, 4550, 4235, 4158, 1424, 1276, 4118, 438, 6860, 2636, 3718, 1153, 3168, 623, 3172, 3118, 3689, 7115, 6128, 1342, 3127, 5746, 798, 3675, 1423, 2125, 3235, 6946, 2801, 454, 755, 4259, 4913, 1013, 3863, 2551, 2802, 4222, 1563, 3370, 5729, 6989, 3191, 4185, 4472, 1649, 3610, 3294, 4164, 3636, 4638, 3484, 6940, 5592, 5695, 2884, 6791, 3721, 178, 3134, 3462, 525, 3777, 4825, 2059, 5702, 1108, 2892, 5462, 466, 5772, 3019, 3774, 6651, 2335, 4664, 4991, 7049, 3937, 2607, 1959, 3368, 4624, 6968, 4501, 5466, 2829, 2365, 6910, 1314, 5446, 1146, 6100, 3412, 3680, 677, 5619, 4459, 5947, 6800, 926, 4669, 2673, 5615, 4771, 4535, 666, 2084, 315, 1675, 426, 2776, 4364, 143, 1602, 4302, 1343, 4448, 2284, 4196, 5562, 3181, 7016, 6942, 6215, 5429, 2406, 6170, 2716, 6353, 3580, 3674, 6600, 2703, 1584, 3436, 152, 1509, 6283, 3186, 6017, 6352, 4417, 5089, 3457, 17, 6081, 2817, 7127, 6589, 4612, 2556, 4031, 1861, 3611, 1690, 4480, 3041, 2304, 3575, 1930, 1571, 1398, 4437, 1533, 2410, 4226, 6644, 4200, 2938, 6923, 1713, 4229, 1143, 2494, 1046, 5607, 1763, 3460, 6996, 6409, 3140, 1851, 3153, 5202, 912, 2738, 3126, 4171, 1700, 3854, 6784, 2695, 2715, 5584, 2815, 3270, 6648, 2270, 1256, 1162, 1799, 1189, 3595, 3212, 4144, 4240, 4751, 5051, 4180, 5032, 4732, 27, 4704, 1016, 2825, 5851, 3811, 1740, 1268, 4567, 4696, 295, 4163, 6198, 928, 6574, 5993, 2946, 1552, 1640, 5604, 6699, 5489, 3683, 5187, 5339, 517, 806, 13, 6882, 754, 3136, 4125, 4938, 5185, 5081, 984, 430, 3602, 4555, 3261, 1459, 322, 1275, 797, 3420, 6232, 3517, 2876, 6993, 3865, 5826, 1793, 1961, 427, 1517, 6082, 6305, 2718, 5352, 1370, 1974, 1023, 4742, 5903, 858, 7053, 3279, 4937, 1947, 4177, 4593, 4796, 1350, 6929, 4960, 4857, 1253, 5573, 6807, 1227, 6159, 2993, 5769, 4876, 3049, 2964, 4293, 2350, 3653, 5016, 6937, 636, 210, 4024, 2376, 3309, 2088, 6712, 987, 4504, 1021, 1855, 1128, 1587, 5299, 349, 2633, 3374, 1252, 956, 5953, 1026, 6148, 6233, 1400, 5156, 4737, 4193, 4017, 4883, 1940, 4220, 2533, 4154, 219, 6093, 6208, 3429, 332, 5616, 2882, 6287, 2083, 2393, 6331, 1476, 3853, 771, 2233, 5987, 156, 5205, 4016, 5965, 4079, 2104, 3000, 6045, 6249, 3165, 7026, 6587, 6680, 5143, 4209, 3677, 3103, 473, 4276, 5317, 3928, 1766, 4976, 310, 4892, 6109, 1313, 624, 7044, 7012, 90, 1987, 217, 5297, 585, 2779, 514, 5246, 4740, 6469, 4251, 1432, 2427], [6148, 4107, 6159, 4118, 25, 28, 2077, 4126, 33, 2082, 6179, 2084, 348, 4775, 4144, 4151, 4154, 5813, 65, 2059, 2119, 72, 4174, 4177, 2131, 2132, 6229, 4183, 6232, 89, 90, 2139, 94, 4195, 2152, 4203, 4207, 4210, 4211, 2167, 6266, 4220, 128, 6291, 6294, 4463, 156, 4254, 4255, 4259, 165, 6311, 29, 178, 2231, 185, 4285, 191, 6340, 4293, 2081, 2251, 5154, 1741, 4304, 4321, 4325, 237, 6384, 2291, 6390, 2296, 4353, 4355, 6409, 267, 275, 4372, 4380, 4412, 4490, 6465, 4420, 4421, 2375, 4426, 2379, 337, 343, 2394, 6492, 6202, 6498, 4453, 4454, 367, 6513, 6514, 4472, 2431, 4480, 6529, 394, 2445, 4495, 4496, 402, 405, 409, 4507, 6556, 2468, 2471, 430, 4534, 3145, 4537, 462, 2515, 477, 2544, 6648, 6651, 2560, 2564, 2571, 2579, 4633, 6682, 6683, 2589, 4638, 547, 552, 2603, 2605, 2607, 6705, 2614, 2617, 6716, 4669, 2623, 578, 583, 2633, 6731, 6732, 2638, 595, 2645, 2651, 6764, 6766, 4723, 4725, 3519, 6780, 6782, 640, 2689, 6787, 4740, 4886, 2704, 657, 660, 4760, 4763, 2716, 2719, 6819, 6820, 2725, 2727, 4779, 2740, 4791, 6845, 2751, 708, 6855, 713, 6860, 802, 4817, 6870, 6873, 732, 2781, 735, 6881, 2786, 4835, 2790, 2791, 4841, 749, 755, 758, 6910, 6911, 774, 4876, 4880, 4883, 2837, 790, 2844, 798, 4912, 6946, 806, 4903, 4573, 2864, 4918, 2874, 831, 2882, 2884, 6985, 6987, 4940, 6989, 850, 851, 4949, 4952, 7003, 4960, 2914, 7012, 4965, 7014, 4968, 881, 885, 7038, 4993, 7045, 910, 2961, 5025, 7077, 7083, 5038, 2991, 946, 5046, 955, 7101, 7105, 966, 7112, 5070, 5072, 3028, 3034, 4125, 1533, 3059, 1014, 1017, 3069, 5119, 3079, 5134, 5136, 1046, 516, 3098, 1051, 3105, 3106, 5160, 3124, 5173, 1080, 5177, 3131, 3134, 3253, 1097, 3147, 1105, 3154, 1108, 5205, 2233, 5212, 1124, 1129, 5228, 3183, 1143, 3192, 1147, 1153, 5253, 1174, 1179, 4635, 3236, 3239, 6686, 3254, 3258, 1224, 1225, 5322, 5323, 1238, 3620, 5338, 5339, 5346, 1251, 1252, 1253, 6111, 5351, 1256, 5354, 3309, 5364, 7041, 3343, 3350, 5399, 3352, 3366, 5416, 5419, 1329, 1334, 1337, 5434, 1342, 3394, 5444, 3397, 5452, 3298, 1368, 1390, 3453, 3464, 1418, 1424, 5528, 5529, 1435, 1446, 3499, 3501, 1454, 3507, 1462, 3511, 3514, 1469, 3518, 1471, 3530, 5583, 5598, 1482, 5604, 5606, 5609, 5714, 5617, 1522, 3573, 3574, 3575, 3581, 1537, 1538, 3591, 3595, 5655, 1563, 1564, 5722, 1569, 4699, 5668, 1575, 3628, 1582, 3677, 3645, 5700, 3653, 2372, 5705, 5712, 1618, 3674, 1629, 5727, 3681, 5731, 1639, 1641, 5738, 5053, 5746, 1658, 1302, 3718, 5057, 3720, 3721, 5770, 5771, 5772, 1304, 5780, 3735, 5784, 1693, 3743, 5793, 1700, 3759, 5810, 3765, 5820, 2677, 5824, 5832, 3786, 1740, 3789, 1746, 3800, 3805, 5856, 1772, 1780, 5882, 1795, 5893, 1799, 1805, 3861, 5911, 1827, 1843, 3899, 3902, 1877, 5977, 3943, 1899, 3951, 2369, 3052, 1930, 1933, 6033, 3989, 3993, 4000, 1955, 6068, 4026, 1695, 4031, 6080, 4035, 6098, 4771, 4056, 6109, 677, 2021, 2027, 4078, 1023, 2399, 4931, 2591, 7040, 6357, 5572, 1940, 1972, 5311, 6550, 1148, 4443, 6640, 1380, 5482, 6928, 858, 5185, 1879, 3188, 420, 3978, 3867, 6580, 4571, 2260, 2140, 1918, 6128, 6717, 5391, 5296, 1356, 6362, 586, 959, 5584, 636, 3732, 5479, 719, 746, 1263, 5837, 3436, 5754, 314, 1704, 3367, 912, 6737, 4095, 3484, 1990, 6322, 1477, 4935, 9, 6695, 5574, 4448, 2631, 2938, 4021, 6678, 2028, 3824, 1451, 5579, 3863, 5297, 5480, 4191, 3636, 3153, 5640, 3931, 4631, 3428, 3876, 4581, 4859, 6680, 6085, 967, 3950, 3411, 2045, 2714, 3533, 1718, 4856, 2334, 3542, 4515, 2473, 6381, 123, 860, 3189, 6837, 5782, 2504, 3848, 6971, 6593, 3360, 3334, 483, 6800, 1818, 7028, 4391, 210, 3094, 3165, 791, 2393, 5096, 1401, 6882, 1208, 119, 115, 6139, 3904, 614, 23, 4206, 4379, 5693, 5689, 4852, 3399, 2155, 6481, 3002, 4383, 5823, 1856, 4489, 5330, 5826, 5894, 4568, 2168, 78, 750, 1405, 6712, 2383, 4514, 5263, 5032, 3865, 4310, 1705, 2353, 2197, 41, 3815, 3119, 5769, 1675, 2210, 4196, 5033, 4110, 597, 5567, 4182, 5550, 7093, 4872, 6185, 3264, 2158, 1394, 5883, 5282, 1602, 4785, 2492, 1021, 4759, 3859, 2037, 1623, 2838, 954, 909, 6017, 754, 5809, 779, 5851, 5084, 1067, 1200, 3386, 6752, 5498, 3936, 6004, 6193, 245, 6474, 5766, 2955, 5859, 1297, 3589, 4742, 2900, 3770, 2802, 5184, 6677, 3049, 3588, 1801, 1116, 5468, 2715, 415, 4746, 6092, 5350, 6889, 243, 1514, 4308, 3597, 5051, 2573, 5132, 4094, 2175, 2868, 2406, 2663, 4966, 633, 6073, 2136, 3326, 2599, 5466, 2122, 6745, 3429, 6642, 2206, 212, 5333, 5002, 3328, 3247, 2166, 4655, 6215, 5624, 6959, 2282, 5884, 2516, 5702, 2173, 1192, 3032, 5589, 2702, 5329, 4034, 1161, 1591, 2815, 4913, 224, 6740, 2686, 6966, 6356, 73, 3784, 5649, 4704, 1312, 4938, 893, 341, 3559, 671, 980, 7006, 4437, 5493, 3565, 6045, 1752, 876, 686, 3180, 5222, 4301, 4799, 880, 5829, 2069, 3994, 4826, 5612, 6605, 3421, 4624, 4787, 4696, 499, 862, 1279, 4361, 3466, 4063, 3427, 4588, 1121, 6802, 469, 1590, 2477, 4057, 4768, 2758, 503, 1226, 4832, 3554, 3030, 4029, 2794, 6130, 2734, 920, 5273, 4535, 2687, 5984, 5050, 4142, 984, 5674, 4656, 3457, 2534, 2400, 1822, 1589, 6237, 2806, 4298, 2724, 2941, 1420, 3168, 2779, 5976, 2129, 1682, 4829, 5369, 4622, 1213, 2984, 5588, 4647, 3364, 1572, 2169, 5107, 6771, 2613, 525, 665, 2756, 7047, 3873, 6630, 965, 225, 1952, 6323, 4485, 524, 3935, 4552, 5446, 2305, 5921, 5288, 2376, 1438, 6279, 6420, 5236, 2244, 1151, 647, 4555, 4857, 1172, 6839, 1800, 783, 1974, 803, 1872, 2340, 6584, 6469, 7055, 6722, 3103, 5130, 6905, 5229, 5244, 3781, 6749, 3374, 2738, 6843, 3970, 2392, 6112, 521, 3750, 6558, 6620, 1205, 5758, 3318, 687, 4260, 5306, 1171, 4251, 2542, 2360, 4687, 2024, 1734, 2678, 2273, 6543, 4222, 32, 6220, 3964, 820, 1493, 1837, 5463, 911, 3417, 5969, 431, 2216, 1426, 613, 6826, 171, 2646, 2916, 5489, 3133, 3844, 4584, 634, 4162, 1934, 6999, 5016, 5938, 4667, 3837, 262, 5819, 1350, 4738, 5355, 7044, 4407, 4809, 847, 655, 1258, 5128, 1453, 5906, 1422, 5622, 3934, 3477, 3962, 3204, 623, 179, 5958, 3373, 2594, 5739, 6198, 6813, 4713, 3440, 1186, 1939], [2752, 7096, 1147, 5321, 4183, 2372, 6108, 165, 4635, 1000, 4440, 4770, 2808, 4248, 4273, 1979, 3667, 3610, 1886, 4412, 1953, 1898, 4041, 6053, 2190, 2913, 2929, 607, 5297, 3693, 4422, 574, 1604, 5728, 1469, 2161, 1684, 6333, 730, 3985, 3017, 1597, 1583, 1099, 3722, 4325, 6987, 2797, 4476, 4533, 5939, 4215, 5993, 483, 4282, 1205, 3825, 2265, 6852, 2423, 1550, 1059, 5584, 2757, 2594, 1361, 6745, 4457, 5858, 4567, 5312, 6901, 2220, 3524, 3042, 2549, 7028, 166, 3957, 6320, 5372, 3950, 6428, 1424, 509, 3152, 2739, 303, 2536, 851, 2678, 1878, 375, 7000, 4810, 2751, 3777, 5750, 1492, 4838, 1286, 5224, 4583, 2485, 3603, 6888, 1874, 6542, 6037, 134, 1717, 20, 5157, 5076, 4398, 2442, 5144, 5740, 2772, 3184, 5914, 5800, 4029, 1535, 2569, 4094, 4339, 1046, 2794, 243, 2008, 4534, 5373, 17, 4652, 2397, 2812, 1856, 5814, 206, 355, 5797, 5349, 6878, 7072, 1338, 1949, 6699, 6530, 5653, 913, 5219, 1602, 3831, 5716, 1129, 3101, 6445, 5134, 549, 3865, 3653, 4653, 1485, 1462, 6176, 1747, 1354, 1675, 1807, 1664, 1070, 164, 6697, 3757, 4146, 6673, 597, 3372, 6918, 1335, 2679, 1419, 5002, 2497, 5824, 6539, 5471, 1553, 871, 5135, 4410, 577, 565, 5293, 1219, 6767, 3161, 2349, 5846, 3626, 2780, 3551, 6342, 6054, 1049, 1155, 3137, 6447, 2693, 1524, 1751, 1014, 2460, 3694, 5185, 6538, 800, 5794, 6621, 6680, 3932, 3427, 3759, 239, 6928, 4101, 3657, 2921, 1801, 2770, 3867, 3638, 935, 2980, 3043, 780, 929, 5544, 4077, 2507, 5496, 2437, 605, 4949, 2999, 1596, 7117, 4842, 2469, 1495, 260, 3471, 1980, 2055, 4760, 5430, 3100, 3295, 3147, 6015, 6168, 4322, 4663, 4026, 2993, 910, 767, 3983, 6923, 6324, 2748, 4131, 387, 457, 4073, 6835, 4641, 3906, 791, 104, 4195, 4837, 847, 4581, 4001, 1996, 5354, 496, 6597, 5501, 2512, 3612, 6228, 4312, 3598, 405, 2860, 2216, 2016, 4271, 3736, 4272, 1869, 6757, 4308, 1910, 1926, 1048, 5242, 3691, 5272, 4111, 2241, 2468, 1316, 6410, 4405, 3201, 6149, 3585, 5607, 1088, 1746, 5440, 872, 1203, 5444, 6355, 1108, 2151, 828, 2004, 4743, 2137, 4458, 4438, 2843, 3890, 3994, 2107, 5529, 4447, 5793, 4, 4472, 6261, 5779, 713, 5702, 5950, 4611, 4933, 4983, 2771, 2383, 2125, 941, 1405, 6591, 2176, 6556, 4376, 389, 4334, 806, 87, 4427, 6143, 2201, 2320, 5415, 4483, 5899, 1043, 6686, 693, 6634, 1905, 857, 6961, 173, 983, 6181, 1797, 2312, 2351, 5164, 6864, 6502, 1840, 3129, 2461, 5876, 4037, 92, 3614, 4777, 6301, 3817, 3029, 2687, 195, 6838, 3948, 514, 5521, 4730, 1748, 3998, 6827, 3549, 1139, 5398, 5003, 5964, 3395, 4727, 602, 4946, 3213, 1790, 2270, 2316, 137, 6002, 4019, 3607, 284, 5839, 108, 830, 1291, 5636, 2362, 3785, 1935, 3315, 5661, 1983, 5467, 6347, 409, 1438, 297, 4477, 4133, 3953, 1273, 2410, 1857, 5022, 3146, 5363, 6189, 5796, 272, 3410, 735, 6414, 398, 1642, 1628, 2597, 3885, 4850, 3436, 2701, 1896, 4568, 6636, 5142, 2030, 1015, 6553, 230, 1456, 135, 5569, 1197, 1792, 1822, 5447, 3263, 1817, 6984, 476, 5497, 6158, 1092, 6633, 6405, 6465, 7015, 3399, 5482, 842, 5445, 2595, 612, 5840, 3665, 6094, 6828, 5252, 5986, 1956, 4313, 6032, 5402, 2702, 1740, 5703, 6233, 1619, 1223, 3297, 3616, 1548, 4660, 1698, 4448, 1902, 560, 652, 3030, 1277, 5897, 334, 5646, 4486, 6277, 5074, 481, 5045, 2997, 2014, 2646, 485, 1594, 4380, 7071, 3628, 1490, 2275, 4330, 5032, 7104, 1873, 5530, 716, 5733, 772, 719, 1699, 4920, 1918, 1232, 2880, 3834, 1930, 4905, 2928, 6334, 3587, 4903, 5172, 1686, 1201, 1771, 1708, 2367, 802, 6742, 6601, 1892, 6439, 556, 4385, 1603, 5300, 4694, 120, 2012, 4287, 2087, 873, 4181, 6193, 3641, 5807, 5775, 550, 91, 6579, 6136, 2961, 6524, 2082, 144, 1847, 5510, 4022, 4363, 4647, 55, 1275, 6948, 6028, 613, 5431, 3343, 4406, 2884, 6561, 5590, 4735, 5247, 3417, 2618, 2669, 3532, 4172, 702, 6769, 5102, 6756, 4093, 7, 1612, 2204, 1614, 3542, 4754, 3156, 3790, 997, 7013, 601, 3314, 4679, 96, 5722, 158, 3209, 5422, 4040, 1955, 4226, 5637, 3191, 6124, 6558, 5179, 4254, 1811, 6795, 3525, 6841, 674, 2774, 4571, 5873, 2526, 5174, 5133, 4970, 6705, 758, 131, 6156, 6799, 1613, 6369, 3737, 3921, 5508, 4989, 3203, 1390, 3546, 1888, 5229, 6854, 1214, 5580, 6185, 3550, 6817, 1933, 5367, 1113, 4830, 1415, 6356, 1587, 5647, 3173, 6596, 3698, 1618, 404, 424, 1782, 6046, 4709, 1631, 2135, 850, 3707, 2683, 2430, 739, 6092, 5220, 849, 2298, 726, 1215, 4756, 5953, 5340, 3038, 1281, 3883, 7088, 1757, 4177, 846, 7078, 4302, 1829, 7086, 3493, 6458, 793, 1124, 5451, 251, 4148, 1592, 4866, 6422, 4043, 2741, 5571, 5617, 1784, 705, 4923, 98, 3577, 500, 1192, 6083, 4630, 4578, 6537, 6274, 4722, 3220, 3887, 1683, 5338, 337, 6386, 6091, 2850, 1348, 6768, 2443, 2238, 6920, 4539, 7047, 2853, 2593, 7043, 3611, 1202, 3661, 3, 3366, 1893, 6965, 6616, 732, 4420, 4333, 5516, 4276, 2168, 524, 1921, 1730, 5238, 902, 3789, 924, 2463, 7037, 1510, 6162, 1168, 2864, 3520, 2340, 1872, 2773, 1360, 5582, 3398, 5700, 3796, 176, 4031, 6533, 6517, 3062, 7123, 1985, 6581, 4713, 3830, 4147, 5827, 2916, 498, 5604, 914, 5212, 6563, 5307, 6215, 5663, 5434, 1616, 4634, 2601, 5816, 4241, 2382, 3530, 1989, 79, 3446, 6626, 4299, 6113, 2885, 44, 3824, 7017, 5903, 4171, 5187, 2807, 3128, 3081, 1053, 1187, 4305, 380, 5342, 2511, 4020, 912, 5203, 2520, 4235, 3303, 4252, 3361, 5167, 2375, 583, 6963, 5148, 2058, 2114, 1089, 586, 2069, 6192, 3269, 1696, 2388, 3414, 3900, 1796, 4701, 1352, 1880, 4556, 2487, 3154, 3120, 5782, 1447, 3321, 81, 450, 171, 4117, 291, 4238, 3624, 542, 6196, 936, 5550, 4163, 5472, 6205, 3058, 3448, 3059, 2575, 74, 3970, 5454, 6033, 2673, 648, 5823, 5933, 5721, 6669, 269, 2345, 2954, 3678, 4666, 1133, 2648, 5460, 6464, 1373, 5125, 3420, 2219, 2389, 1303, 5734, 31, 4178, 4622, 5888, 2893, 897, 6902, 5925, 5937, 4968, 3109, 3781, 4564, 6716, 4675, 2728, 5289, 2722, 9, 213, 1418, 3001, 6344, 4975, 4230, 3094, 3134, 493, 4244, 4929, 5577, 5487, 2025, 3584, 2943, 6360, 750, 4205, 4664, 5319, 5627, 2574, 1971, 6515, 6290, 6080, 2268, 4358, 3430, 5288, 5418, 4461, 160, 6423, 3229, 2878], [2564, 2565, 1030, 9, 2571, 3094, 6168, 4633, 4442, 4638, 31, 3616, 1740, 552, 3626, 6578, 4042, 4147, 4617, 1592, 5177, 4157, 5813, 6208, 3170, 5200, 3153, 3670, 6749, 4706, 6755, 5851, 4716, 4717, 3188, 3189, 3192, 1310, 5258, 5772, 2718, 6820, 6316, 179, 2740, 4277, 1225, 5816, 3257, 2591, 6845, 4043, 2244, 3270, 6855, 4300, 6860, 5837, 1746, 2851, 2510, 6870, 5848, 6874, 1755, 2938, 1247, 4328, 3824, 3828, 4853, 2294, 5372, 3333, 3852, 3348, 2842, 1822, 3362, 803, 5924, 806, 5418, 5419, 4908, 6960, 2869, 1336, 4922, 6465, 4933, 5959, 4426, 1591, 4429, 2894, 2445, 338, 2900, 1370, 2395, 1381, 1387, 4460, 7027, 1913, 890, 7036, 2112, 5508, 1927, 394, 7052, 397, 7057, 4505, 2459, 6556, 5023, 7068, 1965, 6574, 5042, 7093, 6075, 4541, 958, 965, 6090, 5579, 4558, 463, 2004, 4568, 3236, 480, 4613, 485, 2039, 2045, 5858, 4200, 1166, 4572, 3590, 6678, 1718, 6622, 4258, 151, 4730, 686, 4309, 4225, 2216, 3484, 5841, 4263, 6030, 6630, 3332, 4864, 1553, 2902, 7044, 1487, 6293, 3261, 721, 5035, 3446, 3396, 2749, 73, 3514, 2361, 1961, 808, 4596, 4954, 1936, 273, 1872, 3410, 6554, 1984, 1523, 1798, 4829, 3686, 5812, 79, 6566, 5534, 4707, 6548, 6570, 557, 3886, 2643, 5561, 1788, 5937, 2050, 1974, 6600, 2815, 2582, 1202, 3400, 897, 7016, 3927, 2234, 428, 5799, 6112, 884, 2189, 5081, 4622, 3079, 6246, 3140, 736, 3853, 5947, 3276, 2735, 2461, 3165, 6811, 2854, 6624, 6071, 3402, 4495, 3494, 1571, 6692, 5002, 2767, 2166, 1163, 5533, 3063, 823, 2430, 6495, 1052, 6662, 4833, 2038, 1055, 3421, 6098, 6605, 6334, 2266, 2382, 4066, 4367, 249, 6510, 578, 6089, 6124, 5125, 846, 5192, 6529, 533, 4662, 3976, 2858, 5151, 3352, 3895, 3265, 5436, 1435, 6327, 4560, 4863, 1725, 4431, 6227, 4797, 6018, 29, 6843, 4852, 2799, 4749, 20, 3369, 2980, 4413, 2558, 300, 2469, 1431, 51, 3960, 3894, 5746, 5324, 4472, 2180, 2702, 3338, 7009, 253, 5607, 3601, 462, 678, 172, 3077, 2991, 2225, 1065, 4663, 1488, 5255, 6186, 6748, 6266, 1359, 3434, 1460, 5149, 1558, 6320, 5643, 6894, 4774, 4155, 3377, 5390, 3472, 5622, 6803, 4966, 859, 1999, 4689, 314, 4115, 1972, 5655, 748, 6789, 6621, 5073, 1587, 6362, 4105, 5894, 4310, 472, 2691, 2027, 4647, 2576, 6066, 4519, 4669, 2331, 4938, 6760, 104, 6763, 1453, 6683, 4219, 3833, 1186, 3899, 3596, 6048, 5692, 2141, 3518, 5236, 707, 1183, 5037, 1955, 5381, 3088, 3156, 6626, 5315, 1977, 4491, 1416, 3182, 4808, 761, 4315, 5422, 2983, 1302, 4998, 220, 4555, 3863, 540, 719, 692, 2713, 3702, 3977, 5992, 2503, 2046, 4991, 525, 5018, 2688, 5930, 5866, 7079, 2209, 5350, 2570, 3069, 364, 243, 5115, 603, 6550, 1661, 2385, 1114, 1479, 2475, 3678, 6452, 1840, 5960, 4621, 5648, 4782, 5123, 1581, 6549, 3698, 5369, 1557, 1697, 1765, 6666, 2129, 3642, 5136, 6975, 5516, 368, 6004, 3489, 4741, 2763, 3864, 2529, 5934, 6193, 6881, 3649, 4063, 1, 2695, 4876, 5209, 2712, 5398, 3737, 5621, 3552, 6238, 2663, 6917, 5163, 5798, 7059, 2568, 824, 3668, 876, 5343, 7125, 4381, 1169, 5359, 2780, 2958, 3295, 584, 1749, 4285, 6667, 4507, 3995, 2228, 2693, 1405, 5782, 4747, 2099, 2318, 27, 2787, 4757, 1848, 1700, 1215, 4651, 1213, 6370, 1258, 3859, 2545, 6889, 5269, 6233, 6852, 1951, 60, 4592, 2649, 3969, 297, 1552, 2611, 7040, 1652, 6638, 4870, 4380, 3163, 1554, 6381, 3779, 4619, 5725, 6219, 1004, 513, 4064, 1670, 6948, 5980, 7078, 5520, 4308, 2777, 6288, 1079, 3309, 3121, 464, 6458, 5144, 2176, 5909, 5009, 5271, 4982, 1298, 3610, 5259, 4112, 4240, 5103, 4236, 4775, 873, 3181, 5480, 5873, 2205, 5978, 2106, 346, 4570, 4962, 232, 5442, 2606, 4563, 6008, 3204, 5210, 2592, 6800, 4840, 3885, 3672, 6705, 1193, 1735, 1200, 2168, 6563, 6739, 4722, 6103, 704, 6459, 4874, 4095, 4746, 1704, 5306, 1577, 3160, 1915, 5591, 1116, 115, 512, 4688, 5754, 1986, 1322, 6283, 4886, 7116, 4195, 269, 6675, 430, 3288, 3082, 933, 469, 1858, 3135, 3395, 99, 4106, 6109, 7046, 4496, 1675, 4841, 3931, 4849, 1390, 5101, 6872, 6664, 2229, 55, 3936, 1157, 3137, 993, 1137, 4921, 2939, 1804, 2210, 1045, 825, 6547, 5336, 2021, 69, 3397, 6378, 4131, 6087, 6261, 3307, 231, 2549, 3031, 1968, 6353, 1668, 7007, 171, 1564, 6185, 1823, 5569, 1830, 3789, 3303, 1028, 633, 1781, 24, 5010, 306, 6415, 3253, 6340, 4690, 2866, 1855, 2059, 5428, 5824, 2600, 5112, 1464, 656, 4322, 2460, 6551, 602, 6656, 1959, 4395, 5614, 427, 5557, 3125, 1174, 4687, 1850, 5400, 2856, 4992, 3658, 2748, 6202, 3725, 2699, 2926, 5329, 399, 4913, 1444, 4428, 491, 2727, 5801, 3490, 2802, 1582, 3341, 6113, 6031, 3528, 1747, 1224, 4928, 4127, 1038, 2580, 3273, 6577, 5024, 1069, 913, 6816, 2392, 5550, 330, 6545, 2967, 3211, 1439, 547, 6298, 6280, 3057, 3161, 6802, 611, 32, 1757, 6596, 4352, 6977, 6236, 2677, 6344, 6255, 1890, 1289, 2218, 5884, 3248, 5906, 6723, 4305, 4432, 5642, 396, 5460, 1154, 5166, 3050, 1939, 4450, 4319, 4424, 1836, 3330, 3531, 6743, 670, 1514, 3157, 4253, 2204, 4164, 3500, 6141, 1626, 5827, 4009, 3066, 5855, 4054, 495, 2194, 618, 1209, 1934, 2686, 212, 6297, 4820, 5173, 4937, 3589, 5674, 6070, 2995, 5470, 4262, 2242, 4652, 2589, 5997, 186, 5244, 1119, 183, 5223, 3478, 6438, 6571, 5311, 1569, 6819, 210, 4569, 2088, 6908, 4132, 3693, 4869, 1324, 1192, 2172, 2336, 5219, 4046, 4986, 4203, 2929, 4470, 4926, 1007, 3168, 2847, 5322, 1161, 6523, 1832, 4196, 6686, 2279, 2508, 1516, 4377, 1281, 1598, 1318, 3429, 4355, 4222, 1990, 3780, 2020, 6992, 6119, 4712, 5627, 96, 771, 5270, 3696, 23, 3462, 2437, 1981, 916, 356, 2498, 4642, 923, 5865, 1741, 6859, 4193, 2770, 5096, 599, 98, 1222, 1214, 708, 4658, 6222, 4137, 5777, 3287, 5718, 6396, 3194, 3379, 2814, 4796, 4888, 6790, 4785, 3178, 1662, 1013, 774, 4875, 1134, 2771, 1088, 1242, 1433, 4437, 2076, 1542, 3770, 4378, 4385, 4743, 4316, 4871, 6862, 4824, 5736, 5659, 348, 145, 6039, 856, 3765, 3709, 5448, 6711, 5888, 2144, 4961, 2657, 7106, 4448, 5578, 6849, 676, 2984, 334, 5554, 1129, 5602, 2689, 2455, 2626, 6487, 5364, 936, 2372, 363, 1995, 218, 5146, 4512, 5597, 4420, 2334, 2463, 1878, 1846, 2197, 6291, 6595], [4611, 9, 524, 5134, 1046, 6680, 5354, 4635, 6686, 4641, 2082, 4647, 6185, 3628, 1070, 6193, 4146, 3129, 6716, 3134, 5185, 1602, 5700, 4534, 5702, 6215, 3147, 4177, 1618, 1108, 597, 4183, 2884, 6745, 5722, 5212, 1418, 98, 4195, 1124, 4709, 1129, 1469, 1139, 2679, 2168, 1147, 2687, 2176, 4226, 3094, 4420, 2702, 6705, 5782, 1817, 4760, 4254, 5793, 165, 2216, 1197, 3759, 5297, 2748, 1214, 2751, 5824, 713, 1740, 3789, 719, 1746, 5338, 732, 735, 3366, 4325, 2794, 750, 3824, 3314, 243, 758, 1281, 3343, 6928, 6616, 791, 3865, 4380, 5823, 802, 806, 4903, 297, 4238, 2864, 5937, 5434, 4412, 1930, 6465, 2372, 4933, 2375, 6987, 5444, 849, 850, 851, 4949, 5471, 4448, 6033, 4968, 2410, 1390, 7028, 3446, 4472, 1405, 1918, 5508, 902, 5953, 5002, 1933, 910, 1424, 2961, 405, 5529, 409, 6556, 6046, 3653, 1955, 2468, 424, 583, 5550, 1456, 1462, 4026, 4029, 4031, 6080, 5569, 5577, 3530, 6092, 5584, 3542, 4568, 2980, 4571, 6621, 3038, 5604, 337, 2025, 3154, 5617, 3059, 1014, 5627, 4041, 6841, 3182, 2275, 231, 793, 3287, 2689, 4652, 6245, 1225, 6362, 4940, 6347, 3551, 6539, 2885, 726, 6277, 1200, 4539, 4191, 913, 2774, 6017, 4304, 1752, 1174, 5766, 4426, 5873, 2579, 3119, 2298, 1446, 4094, 6910, 184, 1994, 3201, 5472, 4723, 6002, 3499, 6340, 1151, 1972, 2862, 6390, 686, 6999, 3595, 120, 6168, 5810, 6875, 5906, 1695, 5816, 1708, 6673, 1312, 3581, 3146, 4791, 3927, 5451, 2507, 6083, 4078, 1049, 1316, 164, 6795, 1, 4147, 1591, 3052, 4960, 3295, 2536, 4841, 5637, 3830, 4111, 1168, 2991, 5440, 6764, 3058, 3667, 6605, 3501, 5172, 712, 4722, 3394, 1354, 777, 3471, 4063, 3718, 1614, 3213, 5229, 4210, 7038, 3106, 4151, 2291, 1227, 3019, 5391, 5800, 2516, 1902, 3785, 2574, 3124, 6683, 4876, 6301, 4816, 4101, 1477, 7112, 3350, 3062, 173, 885, 3589, 3781, 4040, 1953, 2241, 5367, 6458, 4813, 2212, 4339, 4918, 6355, 398, 862, 4946, 3694, 1985, 6149, 2397, 4172, 6139, 3490, 7015, 4535, 549, 6626, 6189, 6768, 3328, 6109, 1517, 2593, 4866, 6410, 6878, 6156, 4564, 3514, 846, 5487, 601, 6563, 3633, 6232, 5801, 5340, 245, 3181, 755, 1616, 7003, 6766, 2442, 4754, 4461, 6344, 5727, 6496, 2921, 4701, 3281, 5643, 3559, 3098, 6464, 1286, 5248, 6051, 5135, 5173, 525, 355, 1000, 946, 1275, 5242, 3397, 4983, 5418, 1252, 5346, 3765, 6524, 4142, 4205, 375, 6045, 7077, 3670, 4777, 6514, 160, 5721, 4287, 4880, 4555, 602, 6963, 2756, 1717, 6399, 5045, 1091, 3722, 7043, 2469, 2059, 2595, 2603, 708, 5899, 2802, 6731, 224, 6082, 5312, 6593, 5296, 1597, 2509, 1166, 674, 5160, 1080, 640, 6068, 5203, 6911, 2014, 3620, 1059, 5144, 4410, 2808, 2570, 314, 5174, 2560, 771, 7006, 1171, 4235, 1370, 1926, 1641, 6428, 3264, 3030, 6220, 547, 1360, 2797, 6143, 4421, 3614, 2773, 1492, 2030, 2210, 1693, 5445, 3270, 2431, 2984, 1878, 4019, 5780, 6498, 3757, 5252, 3430, 1219, 1683, 6266, 3043, 3042, 4928, 1203, 5496, 1898, 2004, 6502, 6159, 7041, 6561, 2812, 6405, 6640, 6579, 5914, 1996, 2838, 4259, 6515, 2161, 2645, 2928, 2623, 3674, 5154, 3594, 7, 3017, 1910, 5728, 4501, 5051, 2373, 2268, 1025, 4322, 2219, 4248, 5288, 3026, 7017, 1334, 2594, 2021, 4073, 1805, 5648, 2617, 6261, 3743, 6770, 6414, 4293, 5663, 671, 3931, 652, 1872, 6553, 2430, 1811, 1956, 2613, 2954, 2739, 633, 128, 4118, 6053, 6975, 291, 6642, 6360, 6990, 5136, 213, 2551, 2244, 924, 6596, 5164, 897, 2843, 3493, 2727, 1801, 3638, 3269, 4117, 6682, 4162, 4648, 2389, 1338, 4043, 1510, 1748, 4125, 1215, 4770, 3585, 4271, 2201, 3518, 4355, 2296, 2916, 2463, 4679, 6322, 800, 613, 5372, 5554, 4372, 1322, 4174, 4612, 514, 462, 2270, 73, 780, 2757, 5754, 2544, 5703, 3616, 6923, 5595, 1784, 5621, 3101, 5419, 6108, 5846, 380, 5497, 5993, 3236, 1048, 6819, 2016, 3899, 1696, 5142, 5933, 1251, 4657, 3395, 1277, 4305, 3885, 656, 4148, 2204, 2712, 5986, 2791, 4330, 3681, 6492, 6381, 4989, 4507, 3677, 4629, 647, 2055, 2220, 4573, 3315, 1495, 6517, 4838, 5285, 3511, 4727, 2320, 4735, 4, 7117, 1089, 4273, 2752, 2589, 936, 3985, 5622, 4476, 402, 3192, 2993, 5167, 3735, 2316, 3293, 498, 2345, 2704, 6722, 6037, 3834, 6158, 5579, 4495, 1604, 5501, 6386, 1097, 284, 309, 2693, 5746, 2580, 4422, 648, 872, 2206, 1858, 499, 2139, 4849, 3770, 3398, 798, 3626, 4578, 2638, 1179, 7125, 3, 6237, 3361, 5046, 4110, 2520, 1807, 3184, 5179, 2423, 6662, 3155, 1847, 716, 3549, 586, 5236, 6581, 677, 1113, 7093, 6032, 4178, 1143, 2786, 4163, 2152, 2648, 2382, 5224, 3546, 6342, 5925, 4669, 6769, 5321, 1096, 6320, 5446, 1939, 1888, 1809, 4285, 4282, 4552, 3231, 4157, 1053, 5590, 5130, 2437, 5572, 4244, 2330, 3263, 38, 1629, 4398, 4638, 5119, 4850, 5799, 1569, 303, 1533, 4206, 4533, 3161, 4437, 4260, 3636, 6591, 3157, 2868, 7118, 3532, 3645, 4000, 3861, 5858, 5840, 4298, 5647, 5567, 5750, 5510, 4454, 4993, 3152, 6873, 4937, 4453, 4655, 1979, 2306, 4785, 941, 1394, 3028, 2740, 3191, 3183, 705, 1575, 305, 1772, 191, 1581, 4230, 2461, 2997, 5674, 1187, 3675, 6787, 4463, 1639, 3171, 595, 6652, 7047, 4489, 3399, 6687, 4477, 4810, 5246, 1582, 237, 2943, 4543, 4771, 4920, 5339, 6438, 2511, 3128, 4203, 2190, 1594, 33, 2618, 1023, 1822, 6648, 3034, 6447, 1869, 1718, 389, 6558, 4406, 1223, 5219, 1538, 1368, 496, 4057, 3678, 28, 3900, 2683, 3815, 6291, 3994, 3970, 1757, 3440, 72, 2119, 893, 4313, 2027, 3844, 4743, 3105, 6423, 1482, 2741, 3936, 1133, 108, 6782, 6805, 2564, 6950, 3784, 5289, 4830, 4583, 2485, 6112, 550, 7037, 912, 4405, 5187, 2012, 32, 3611, 6181, 5653, 5819, 6641, 5521, 2860, 739, 5903, 6202, 2140, 1935, 4675, 4837, 2137, 4144, 430, 1827, 1304, 5133, 6651, 6732, 2780, 4730, 5398, 4061, 2772, 5947, 2058, 5323, 5897, 1880, 4666, 341, 2526, 873, 3277, 5607, 4077, 81, 2686, 1524, 4255, 5977, 4376, 1899, 5454, 3466, 3137, 1253, 2069, 5038, 364, 1684, 6888, 4093, 6636, 1471, 4660, 2460, 3082, 5960, 3978, 578, 2487, 5928, 2393, 5779, 4490, 1361, 2340, 239, 4486, 6946, 5807, 260, 5771, 3145, 1043, 5716, 4599, 7016, 2781, 3188, 4171, 6063, 5950, 5342, 6124, 6136, 4653, 2725, 5032, 5102, 7055, 1857, 1587, 3796], [6148, 4107, 6159, 4118, 25, 28, 2077, 4126, 33, 2082, 6179, 2084, 348, 4775, 4144, 4151, 4154, 5813, 65, 2059, 2119, 72, 4174, 4177, 2131, 2132, 6229, 4183, 6232, 89, 90, 2139, 94, 4195, 2152, 4203, 4207, 4210, 4211, 2167, 6266, 4220, 128, 6291, 6294, 4463, 156, 4254, 4255, 4259, 165, 6311, 29, 178, 2231, 185, 4285, 191, 6340, 4293, 2081, 2251, 5154, 1741, 4304, 4321, 4325, 237, 6384, 2291, 6390, 2296, 4353, 4355, 6409, 267, 275, 4372, 4380, 4412, 4490, 6465, 4420, 4421, 2375, 4426, 2379, 337, 343, 2394, 6492, 6202, 6498, 4453, 4454, 367, 6513, 6514, 4472, 2431, 4480, 6529, 394, 2445, 4495, 4496, 402, 405, 409, 4507, 6556, 2468, 2471, 430, 4534, 3145, 4537, 462, 2515, 477, 2544, 6648, 6651, 2560, 2564, 2571, 2579, 4633, 6682, 6683, 2589, 4638, 547, 552, 2603, 2605, 2607, 6705, 2614, 2617, 6716, 4669, 2623, 578, 583, 2633, 6731, 6732, 2638, 595, 2645, 2651, 6764, 6766, 4723, 4725, 3519, 6780, 6782, 640, 2689, 6787, 4740, 4886, 2704, 657, 660, 4760, 4763, 2716, 2719, 6819, 6820, 2725, 2727, 4779, 2740, 4791, 6845, 2751, 708, 6855, 713, 6860, 802, 4817, 6870, 6873, 732, 2781, 735, 6881, 2786, 4835, 2790, 2791, 4841, 749, 755, 758, 6910, 6911, 774, 4876, 4880, 4883, 2837, 790, 2844, 798, 4912, 6946, 806, 4903, 4573, 2864, 4918, 2874, 831, 2882, 2884, 6985, 6987, 4940, 6989, 850, 851, 4949, 4952, 7003, 4960, 2914, 7012, 4965, 7014, 4968, 881, 885, 7038, 4993, 7045, 910, 2961, 5025, 7077, 7083, 5038, 2991, 946, 5046, 955, 7101, 7105, 966, 7112, 5070, 5072, 3028, 3034, 4125, 1533, 3059, 1014, 1017, 3069, 5119, 3079, 5134, 5136, 1046, 516, 3098, 1051, 3105, 3106, 5160, 3124, 5173, 1080, 5177, 3131, 3134, 3253, 1097, 3147, 1105, 3154, 1108, 5205, 2233, 5212, 1124, 1129, 5228, 3183, 1143, 3192, 1147, 1153, 5253, 1174, 1179, 4635, 3236, 3239, 6686, 3254, 3258, 1224, 1225, 5322, 5323, 1238, 3620, 5338, 5339, 5346, 1251, 1252, 1253, 6111, 5351, 1256, 5354, 3309, 5364, 7041, 3343, 3350, 5399, 3352, 3366, 5416, 5419, 1329, 1334, 1337, 5434, 1342, 3394, 5444, 3397, 5452, 3298, 1368, 1390, 3453, 3464, 1418, 1424, 5528, 5529, 1435, 1446, 3499, 3501, 1454, 3507, 1462, 3511, 3514, 1469, 3518, 1471, 3530, 5583, 5598, 1482, 5604, 5606, 5609, 5714, 5617, 1522, 3573, 3574, 3575, 3581, 1537, 1538, 3591, 3595, 5655, 1563, 1564, 5722, 1569, 4699, 5668, 1575, 3628, 1582, 3677, 3645, 5700, 3653, 2372, 5705, 5712, 1618, 3674, 1629, 5727, 3681, 5731, 1639, 1641, 5738, 5053, 5746, 1658, 1302, 3718, 5057, 3720, 3721, 5770, 5771, 5772, 1304, 5780, 3735, 5784, 1693, 3743, 5793, 1700, 3759, 5810, 3765, 5820, 2677, 5824, 5832, 3786, 1740, 3789, 1746, 3800, 3805, 5856, 1772, 1780, 5882, 1795, 5893, 1799, 1805, 3861, 5911, 1827, 1843, 3899, 3902, 1877, 5977, 3943, 1899, 3951, 2369, 3052, 1930, 1933, 6033, 3989, 3993, 4000, 1955, 6068, 4026, 1695, 4031, 6080, 4035, 6098, 4771, 4056, 6109, 677, 2021, 2027, 4078, 1023, 4622, 6087, 2477, 4657, 791, 6974, 6322, 5214, 636, 5937, 1380, 6130, 3837, 6004, 4933, 6193, 4324, 6420, 3594, 6305, 1084, 1180, 6975, 3463, 6971, 5498, 3822, 2377, 5352, 4512, 5579, 5769, 2406, 4007, 5947, 5739, 5087, 5493, 980, 4489, 6772, 5096, 6165, 712, 1734, 4794, 783, 3589, 3531, 4759, 5032, 5958, 9, 6415, 1572, 3188, 6902, 4647, 2473, 1408, 4631, 3784, 6399, 4172, 4697, 671, 5446, 1659, 314, 1787, 5480, 2385, 1849, 6959, 3119, 3082, 78, 5278, 2737, 2532, 4938, 3500, 5706, 4813, 6580, 6473, 4574, 1438, 4991, 123, 1350, 4223, 931, 4146, 793, 2838, 420, 3429, 3446, 5969, 6154, 3143, 3370, 3815, 2542, 2401, 3650, 6378, 6323, 3922, 6279, 4709, 1626, 4872, 6649, 2848, 5257, 1297, 5006, 4379, 6128, 4206, 2580, 1763, 1682, 297, 5282, 1065, 6620, 6481, 4382, 2344, 5599, 2244, 4857, 4142, 6889, 1067, 3001, 6520, 4696, 274, 245, 2509, 4584, 2481, 4931, 4448, 5050, 6905, 1490, 2410, 3670, 4758, 3597, 6253, 7010, 2946, 5000, 6445, 7040, 1114, 6237, 4878, 4307, 6740, 864, 7093, 5569, 2334, 4310, 1709, 1928, 1918, 686, 1752, 2392, 3824, 3425, 3636, 3181, 2591, 5130, 3725, 6722, 4061, 6276, 1591, 3577, 4816, 5329, 2852, 3913, 2570, 3865, 6543, 1172, 5562, 6915, 4853, 3904, 4746, 1718, 1244, 5729, 2212, 920, 5595, 1493, 3157, 4624, 1258, 6593, 4229, 6613, 4826, 2239, 5799, 1070, 849, 5674, 3859, 3300, 1861, 2941, 5649, 243, 2686, 1202, 3964, 6568, 2495, 295, 1263, 3155, 2158, 2758, 256, 2136, 2687, 3811, 4057, 5976, 4799, 6862, 862, 3133, 6763, 99, 32, 3934, 98, 6584, 4079, 5823, 2551, 1208, 5928, 4407, 5466, 614, 6723, 305, 6802, 2695, 3231, 1602, 5643, 7049, 3368, 6046, 3035, 3344, 1879, 3559, 2383, 1242, 4667, 3032, 5296, 7064, 6972, 3373, 3541, 2522, 2273, 908, 4515, 2049, 3953, 4486, 1021, 6695, 2748, 6712, 1934, 112, 3478, 3427, 3421, 2738, 3176, 5723, 397, 5754, 5913, 5621, 5394, 6861, 771, 5588, 3038, 6085, 6296, 2663, 2680, 6201, 3338, 3374, 1822, 6630, 900, 1146, 1451, 5559, 2028, 965, 3477, 6249, 4641, 3386, 634, 6805, 3360, 3277, 2122, 820, 1200, 1281, 5859, 259, 2829, 1856, 777, 1205, 3873, 1589, 4029, 3287, 4437, 3490, 115, 2194, 480, 6717, 309, 1720, 2504, 6621, 1148, 1838, 2588, 2393, 613, 5550, 2973, 4024, 2655, 3440, 2399, 5450, 1801, 301, 6642, 7021, 5717, 4239, 2678, 6112, 959, 3484, 1798, 1008, 2980, 797, 647, 2458, 1514, 2681, 3750, 4485, 3019, 4535, 938, 1214, 298, 3781, 6220, 5132, 5567, 2868, 7028, 4196, 1672, 2197, 6834, 6092, 6969, 2483, 3103, 4016, 4514, 4751, 1705, 5069, 1318, 4249, 4730, 4913, 1858, 2400, 4443, 4704, 6641, 1422, 4849, 6550, 3978, 623, 6814, 6678, 5369, 6293, 6680, 3412, 4673, 2573, 4226, 3640, 1517, 6843, 6771, 212, 936, 2780, 6745, 2816, 3212, 597, 6356, 6446, 4413, 2069, 2210, 912, 1453, 1425, 2631, 23, 1940, 5236, 73, 5263, 1937, 4852, 5819, 4571, 5211, 1590, 4301, 6837, 4568, 1704, 7047, 4231, 4856, 2168, 4302, 5640, 1096, 5934, 1943, 4110, 4570, 4141, 2397, 1972, 3836, 1226, 3165, 1139, 1603, 5479, 2147, 5016, 3428, 1939, 4260, 3293, 5350, 6605, 3247, 2714, 1581, 1401, 6978, 415, 1356, 5809, 1035, 4611, 6416, 2802, 3909, 3588, 5894, 4501], [2752, 7096, 1147, 5321, 4183, 2372, 6108, 165, 4635, 1000, 4440, 4770, 2808, 4248, 4273, 1979, 3667, 3610, 1886, 4412, 1953, 1898, 4041, 6053, 2190, 2913, 2929, 607, 5297, 3693, 4422, 574, 1604, 5728, 1469, 2161, 1684, 6333, 730, 3985, 3017, 1597, 1583, 1099, 3722, 4325, 6987, 2797, 4476, 4533, 5939, 4215, 5993, 483, 4282, 1205, 3825, 2265, 6852, 2423, 1550, 1059, 5584, 2757, 2594, 1361, 6745, 4457, 5858, 4567, 5312, 6901, 2220, 3524, 3042, 2549, 7028, 166, 3957, 6320, 5372, 3950, 6428, 1424, 509, 3152, 2739, 303, 2536, 851, 2678, 1878, 375, 7000, 4810, 2751, 3777, 5750, 1492, 4838, 1286, 5224, 4583, 2485, 3603, 6888, 1874, 6542, 6037, 134, 1717, 20, 5157, 5076, 4398, 2442, 5144, 5740, 2772, 3184, 5914, 5800, 4029, 1535, 2569, 4094, 4339, 1046, 2794, 243, 2008, 4534, 5373, 17, 4652, 2397, 2812, 1856, 5814, 206, 355, 5797, 5349, 6878, 7072, 1338, 1949, 6699, 6530, 5653, 913, 5219, 1602, 3831, 5716, 1129, 3101, 6445, 5134, 549, 3865, 3653, 4653, 1485, 1462, 6176, 1747, 1354, 1675, 1807, 1664, 1070, 164, 6697, 3757, 4146, 6673, 597, 3372, 6918, 1335, 2679, 1419, 5002, 2497, 5824, 6539, 5471, 1553, 871, 5135, 4410, 577, 565, 5293, 1219, 6767, 3161, 2349, 5846, 3626, 2780, 3551, 6342, 6054, 1049, 1155, 3137, 6447, 2693, 1524, 1751, 1014, 2460, 3694, 5185, 6538, 800, 5794, 6621, 6680, 3932, 3427, 3759, 239, 6928, 4101, 3657, 2921, 1801, 2770, 3867, 3638, 935, 2980, 3043, 780, 929, 5544, 4077, 2507, 5496, 2437, 605, 4949, 2999, 1596, 7117, 4842, 2469, 1495, 260, 3471, 1980, 2055, 4760, 5430, 3100, 3295, 3147, 6015, 6168, 4322, 4663, 4026, 2993, 910, 767, 3983, 6923, 6324, 2748, 4131, 387, 457, 4073, 6835, 4641, 3906, 791, 104, 4195, 4837, 847, 4581, 4001, 1996, 5354, 496, 6597, 5501, 2512, 3612, 6228, 4312, 3598, 405, 2860, 2216, 2016, 4271, 3736, 4272, 1869, 6757, 4308, 1910, 1926, 1048, 5242, 3691, 5272, 4111, 2241, 2468, 1316, 6410, 4405, 3201, 6149, 3585, 5607, 1088, 1746, 5440, 872, 1203, 5444, 6355, 1108, 2151, 828, 2004, 4743, 2137, 4458, 4438, 2843, 3890, 3994, 2107, 5529, 4447, 5793, 4, 4472, 6261, 5779, 713, 5702, 5950, 4611, 4933, 4983, 2771, 2383, 2125, 941, 1405, 6591, 2176, 6556, 4376, 389, 4334, 806, 87, 4427, 6143, 2201, 2320, 5415, 4483, 5899, 1043, 6686, 693, 6634, 1905, 857, 6961, 173, 983, 6181, 1797, 2312, 2351, 5164, 6864, 6502, 1840, 3129, 2461, 5876, 4037, 92, 3614, 4777, 6301, 3817, 3029, 2687, 195, 6838, 3948, 514, 5521, 4730, 1748, 3998, 6827, 3549, 1139, 5398, 5003, 5964, 3395, 4727, 602, 4946, 3213, 1790, 2270, 2316, 137, 6002, 4019, 3607, 284, 5839, 108, 830, 1291, 5636, 2362, 3785, 1935, 3315, 5661, 1983, 5467, 6347, 409, 1438, 297, 4477, 4133, 3953, 1273, 2410, 1857, 5022, 3146, 5363, 6189, 5796, 272, 3410, 735, 6414, 398, 1642, 1628, 2597, 3885, 4850, 3436, 2701, 1896, 4568, 6636, 5142, 2030, 1015, 6553, 230, 1456, 135, 5569, 1197, 1792, 1822, 5447, 3263, 1817, 6984, 476, 5497, 6158, 1092, 6633, 6405, 6465, 7015, 3399, 5482, 842, 5445, 2595, 612, 5840, 3665, 6094, 6828, 5252, 5986, 1956, 4313, 6032, 5402, 2702, 1740, 5703, 6233, 1619, 1223, 3297, 3616, 1548, 4660, 1698, 4448, 1902, 560, 652, 3030, 1277, 5897, 334, 5646, 4486, 6277, 5074, 481, 5045, 2997, 2014, 2646, 485, 1594, 4380, 7071, 3628, 1490, 2275, 4330, 5032, 7104, 1873, 5530, 716, 5733, 772, 719, 1699, 4920, 1918, 1232, 2880, 3834, 1930, 4905, 2928, 6334, 3587, 4903, 5172, 1686, 1201, 1771, 1708, 2367, 802, 6742, 6601, 1892, 6439, 556, 4385, 1603, 5300, 4694, 120, 2012, 4287, 2087, 873, 4181, 6193, 3641, 5807, 5775, 550, 91, 6579, 6136, 2961, 6524, 2082, 144, 1847, 5510, 4022, 4363, 4647, 55, 1275, 6948, 6028, 613, 5431, 3343, 4406, 2884, 6561, 5590, 4735, 5247, 3417, 2618, 2669, 3532, 4172, 702, 6769, 5102, 6756, 4093, 7, 1612, 2204, 1614, 3542, 4754, 3156, 3790, 997, 7013, 601, 3314, 4679, 96, 5722, 158, 3209, 5422, 4040, 1955, 4226, 5637, 3191, 6124, 6558, 5179, 4254, 1811, 6795, 3525, 6841, 674, 2774, 4571, 5873, 2526, 5174, 5133, 4970, 6705, 758, 131, 6156, 6799, 1613, 6369, 3737, 3921, 5508, 4989, 3203, 1390, 3546, 1888, 5229, 6854, 1214, 5580, 6185, 3550, 6817, 1933, 5367, 1113, 4830, 1415, 6356, 1587, 5647, 3173, 6596, 3698, 1618, 404, 424, 1782, 6046, 4709, 1631, 2135, 850, 3707, 2683, 2430, 739, 6092, 5220, 849, 2298, 726, 1215, 4756, 5953, 5340, 3038, 1281, 3883, 7088, 1757, 4177, 846, 7078, 4302, 1829, 7086, 3493, 6458, 793, 1124, 5451, 251, 4148, 1592, 4866, 6422, 4043, 2741, 5571, 5617, 1784, 705, 4923, 98, 3577, 500, 1192, 6083, 4630, 4578, 6537, 6274, 4722, 3220, 3887, 1683, 5338, 337, 6386, 6091, 2850, 1348, 6768, 2443, 2238, 6920, 4539, 7047, 2853, 2593, 7043, 3611, 1202, 3661, 3, 3366, 1893, 6965, 6616, 732, 4420, 4333, 5516, 4276, 2168, 524, 1921, 1730, 5238, 902, 3789, 924, 2463, 7037, 1510, 6162, 1168, 2864, 3520, 2340, 1872, 2773, 1360, 5582, 3398, 5700, 3796, 176, 4031, 6533, 6517, 3062, 7123, 1985, 6581, 4713, 3830, 4147, 5827, 2916, 498, 5604, 914, 5212, 6563, 5307, 6215, 5663, 5434, 1616, 4634, 2601, 5816, 4241, 2382, 3530, 1989, 79, 3446, 6626, 4299, 6113, 2885, 44, 3824, 7017, 5903, 4171, 5187, 2807, 3128, 3081, 1053, 1187, 4305, 380, 5342, 2511, 4020, 912, 5203, 2520, 4235, 3303, 4252, 3361, 5167, 2375, 583, 6963, 5148, 2058, 2114, 1089, 586, 2069, 6192, 3269, 1696, 2388, 3414, 3900, 1796, 4701, 1352, 1880, 4556, 2487, 3154, 3120, 5782, 1447, 3321, 81, 450, 171, 4117, 291, 4238, 3624, 542, 6196, 936, 5550, 4163, 5472, 6205, 3058, 3448, 3059, 2575, 74, 3970, 5454, 6033, 2673, 648, 5823, 5933, 5721, 6669, 269, 2345, 2954, 3678, 4666, 1133, 2648, 5460, 6464, 1373, 5125, 3420, 2219, 2389, 1303, 5734, 31, 4178, 4622, 5888, 2893, 897, 6902, 5925, 5937, 4968, 3109, 3781, 4564, 6716, 4675, 2728, 5289, 2722, 9, 213, 1418, 3001, 6344, 4975, 4230, 3094, 3134, 493, 4244, 4929, 5577, 5487, 2025, 3584, 2943, 6360, 750, 4205, 4664, 5319, 5627, 2574, 1971, 6515, 6290, 6080, 2268, 4358, 3430, 5288, 5418, 4461, 160, 6423, 3229, 2878]]

#similarity_matrix = compute_similary_between_subsets(yolo, compare_func=jaccard)
#plt.figure(figsize=(32, 48))
#plot_feature_subsets_matrix(similarity_matrix, range(len(yolo)), title="Jaccard similarity between two feature subsets")